# Notebook to match "by hand" recipes from menus to recipes in the database
- input files : 
    - "../data/Tools/MenuRecipe.xlsx" which contains a list of recipes taken manually from our list of menus
    - '../data/Recipes/recipes_jow.json' (Jow recipes in french)
    - '../data/Recipes/recettes_completes_7tomorrow.xlsx' (complete recipes, ie with Agribalyse ingredients, from the projet 7tomorrow, in english)
- output file : 
    - "../data/Tools/MenuToJowRecipes_v1.xlsx" : matching with JOW
    - "../data/Tools/MenuToJowRecipes_v2.xlsx" : matching with 7tomorrow

In [1]:
import pandas as pd
import re

**Indicate the path where the folder data/ is**

In [2]:
data_path = '../'

### Import input JOW data

In [3]:
menu_data = pd.read_excel(data_path+"data/Tools/MenuRecipes.xlsx", header = [0])
menu_data.head()

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients
0,burgart_205074_c70.jpg,Burgers,Superfin,"Bœuf (VBF Metzger), Comté AOC, Iceberg, Bacon,..."
1,burgart_205074_c70.jpg,Burgers,Bon & Beau,"Bœuf (VBF Metzger), Cheddar, Sucrine, Bacon, T..."
2,burgart_205074_c70.jpg,Burgers,Dandy,"Bœuf (VBF Metzger), Cheddar fumée, Mâche nant..."
3,burgart_205074_c70.jpg,Burgers,Green Chicken,"Filet de poulet, Tomme de Savoie IGP, Roquette..."
4,burgart_205074_c70.jpg,Poke Bowls,Black bowl,"Thon, Riz noir, Avocat, Oignon rouge, Cébette,..."


In [4]:
jow_path = data_path + 'data/Recipes/recipes_jow.json'
jow_data = pd.read_json(jow_path)
jow_data = jow_data.transpose()
pd.set_option('display.max_colwidth', None)
jow_data['recettes'] = jow_data.index
jow_data.head()

,cuisson,ingredients,recettes
Yakitori bœuf fromage,10 minutes,"[[80 g, Bœuf (carpaccio)], [100 g, Emmental], [1 càs, Sauce Teriyaki], [70 g, Riz], [6, Pics à brochette]]",Yakitori bœuf fromage
Gnocchis Epinards Gorgonzola,5 minutes,"[[150 g, Gnocchi], [100 g, Epinard (frais)], [30 g, Gorgonzola], [5 cl, Crème liquide]]",Gnocchis Epinards Gorgonzola
Dahl de lentilles corail,plat froid,"[[50 g, Lentilles corail], [80 g, Tomate (chair)], [5 cl, Lait de coco], [1/2 gou., Ail]]",Dahl de lentilles corail
Veggie lasagna,50 minutes,"[[50 g, Lasagnes (cru)], [150 g, Courgette], [150 g, Tomate (purée)], [80 g, Champignons de Paris (frais)], [1/2, Mozzarella (boule)]]",Veggie lasagna
Pasta alla carbonara,9 minutes,"[[100 g, Pâtes (spaghetti)], [3 tran., Pancetta], [1, Œuf], [40 g, Parmesan (morceaux)]]",Pasta alla carbonara


In [5]:
print('Number of recipes = ', jow_data.shape[0])

Number of recipes =  1406


Add a column giving the list of ingredients

In [6]:
REGEX_REPLACEMENTS = [(r"\u0153", "oe"), \
                      (r"\u0152", "Oe")]

def remove_ligatures(transcript):
    for old, new in REGEX_REPLACEMENTS:
        transcript = re.sub(old, new, transcript, flags=re.IGNORECASE)
    return transcript

In [7]:
# Known from previous notebook '[Genevieve] ingredients_scores__process.ipynb'
meaningless_jow_ingredients = ['1/10 bou.', '1/5 bou.', '1/50 bou.', '6 pinc.', '1 càc', '1/4', '3 pinc.', '2 càs', \
                               '1 cm', '1/20 bou.', '1 càs', '1/2 càc', 'Papier cuisson', 'Pics à brochette']

def extract_ingredients(row):
    ingredients = []
    for x in row:
        if len(x)>1:         
            ingredient = x[-1]    # the ingredient is the last element of the list 'row'  
            if ingredient not in meaningless_jow_ingredients:   # keep only meaningfull ingredients
                ingredient = remove_ligatures(ingredient)
                ingredients.append(ingredient) 

    ingredients = [re.sub("[\(\[].*?[\)\]]", "", ingredient) for ingredient in ingredients]  #remove informations in parenthesis
    # remove blank at the end of the string that remained when parenthesis have been removed
    ingredients = [ing[:-1] if ing[-1]==" " else ing for ing in ingredients]   

    return ingredients

jow_data['only_ings'] = jow_data['ingredients'].apply(extract_ingredients)
jow_data.head()

,cuisson,ingredients,recettes,only_ings
Yakitori bœuf fromage,10 minutes,"[[80 g, Bœuf (carpaccio)], [100 g, Emmental], [1 càs, Sauce Teriyaki], [70 g, Riz], [6, Pics à brochette]]",Yakitori bœuf fromage,"[Boeuf, Emmental, Sauce Teriyaki, Riz]"
Gnocchis Epinards Gorgonzola,5 minutes,"[[150 g, Gnocchi], [100 g, Epinard (frais)], [30 g, Gorgonzola], [5 cl, Crème liquide]]",Gnocchis Epinards Gorgonzola,"[Gnocchi, Epinard, Gorgonzola, Crème liquide]"
Dahl de lentilles corail,plat froid,"[[50 g, Lentilles corail], [80 g, Tomate (chair)], [5 cl, Lait de coco], [1/2 gou., Ail]]",Dahl de lentilles corail,"[Lentilles corail, Tomate, Lait de coco, Ail]"
Veggie lasagna,50 minutes,"[[50 g, Lasagnes (cru)], [150 g, Courgette], [150 g, Tomate (purée)], [80 g, Champignons de Paris (frais)], [1/2, Mozzarella (boule)]]",Veggie lasagna,"[Lasagnes, Courgette, Tomate, Champignons de Paris, Mozzarella]"
Pasta alla carbonara,9 minutes,"[[100 g, Pâtes (spaghetti)], [3 tran., Pancetta], [1, Œuf], [40 g, Parmesan (morceaux)]]",Pasta alla carbonara,"[Pâtes, Pancetta, oeuf, Parmesan]"


In [8]:
jow_data['only_ings'] = jow_data['only_ings'].apply(lambda x: ', '.join([str(s) for s in x]))

In [9]:
jow_data.head()

,cuisson,ingredients,recettes,only_ings
Yakitori bœuf fromage,10 minutes,"[[80 g, Bœuf (carpaccio)], [100 g, Emmental], [1 càs, Sauce Teriyaki], [70 g, Riz], [6, Pics à brochette]]",Yakitori bœuf fromage,"Boeuf, Emmental, Sauce Teriyaki, Riz"
Gnocchis Epinards Gorgonzola,5 minutes,"[[150 g, Gnocchi], [100 g, Epinard (frais)], [30 g, Gorgonzola], [5 cl, Crème liquide]]",Gnocchis Epinards Gorgonzola,"Gnocchi, Epinard, Gorgonzola, Crème liquide"
Dahl de lentilles corail,plat froid,"[[50 g, Lentilles corail], [80 g, Tomate (chair)], [5 cl, Lait de coco], [1/2 gou., Ail]]",Dahl de lentilles corail,"Lentilles corail, Tomate, Lait de coco, Ail"
Veggie lasagna,50 minutes,"[[50 g, Lasagnes (cru)], [150 g, Courgette], [150 g, Tomate (purée)], [80 g, Champignons de Paris (frais)], [1/2, Mozzarella (boule)]]",Veggie lasagna,"Lasagnes, Courgette, Tomate, Champignons de Paris, Mozzarella"
Pasta alla carbonara,9 minutes,"[[100 g, Pâtes (spaghetti)], [3 tran., Pancetta], [1, Œuf], [40 g, Parmesan (morceaux)]]",Pasta alla carbonara,"Pâtes, Pancetta, oeuf, Parmesan"


Let's remove ligatures, e.g. Bœuf -> Boeuf

In [10]:
jow_data['recettes'] = jow_data['recettes'].apply(lambda s: remove_ligatures(str(s)))
jow_data['ingredients'] = jow_data['ingredients'].apply(lambda s: remove_ligatures(str(s)))
jow_data['only_ings'] = jow_data['only_ings'].apply(lambda s: remove_ligatures(str(s)))
jow_data.head()

,cuisson,ingredients,recettes,only_ings
Yakitori bœuf fromage,10 minutes,"[['80 g', 'Boeuf (carpaccio)'], ['100 g', 'Emmental'], ['1 càs', 'Sauce Teriyaki'], ['70 g', 'Riz'], ['6', 'Pics à brochette']]",Yakitori boeuf fromage,"Boeuf, Emmental, Sauce Teriyaki, Riz"
Gnocchis Epinards Gorgonzola,5 minutes,"[['150 g', 'Gnocchi'], ['100 g', 'Epinard (frais)'], ['30 g', 'Gorgonzola'], ['5 cl', 'Crème liquide']]",Gnocchis Epinards Gorgonzola,"Gnocchi, Epinard, Gorgonzola, Crème liquide"
Dahl de lentilles corail,plat froid,"[['50 g', 'Lentilles corail'], ['80 g', 'Tomate (chair)'], ['5 cl', 'Lait de coco'], ['1/2 gou.', 'Ail']]",Dahl de lentilles corail,"Lentilles corail, Tomate, Lait de coco, Ail"
Veggie lasagna,50 minutes,"[['50 g', 'Lasagnes (cru)'], ['150 g', 'Courgette'], ['150 g', 'Tomate (purée)'], ['80 g', 'Champignons de Paris (frais)'], ['1/2', 'Mozzarella (boule)']]",Veggie lasagna,"Lasagnes, Courgette, Tomate, Champignons de Paris, Mozzarella"
Pasta alla carbonara,9 minutes,"[['100 g', 'Pâtes (spaghetti)'], ['3 tran.', 'Pancetta'], ['1', 'oeuf'], ['40 g', 'Parmesan (morceaux)']]",Pasta alla carbonara,"Pâtes, Pancetta, oeuf, Parmesan"


Let's change to lower case

In [11]:
for col in ["ingredients", "recettes", "only_ings"]:
    jow_data[col] = jow_data[col].apply(lambda s: str(s).lower())
    
jow_data.head(2)

,cuisson,ingredients,recettes,only_ings
Yakitori bœuf fromage,10 minutes,"[['80 g', 'boeuf (carpaccio)'], ['100 g', 'emmental'], ['1 càs', 'sauce teriyaki'], ['70 g', 'riz'], ['6', 'pics à brochette']]",yakitori boeuf fromage,"boeuf, emmental, sauce teriyaki, riz"
Gnocchis Epinards Gorgonzola,5 minutes,"[['150 g', 'gnocchi'], ['100 g', 'epinard (frais)'], ['30 g', 'gorgonzola'], ['5 cl', 'crème liquide']]",gnocchis epinards gorgonzola,"gnocchi, epinard, gorgonzola, crème liquide"


### Match menu recipes to Jow recipes

Let's now match menu recipes to Jow recipes

In [12]:
menu_data['Closest JOW recipe by hand'] = 'Not yet handled'

In [13]:
menu_data

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand
0,burgart_205074_c70.jpg,Burgers,Superfin,"Bœuf (VBF Metzger), Comté AOC, Iceberg, Bacon, Compotée d'oignons, Sauce Burgart",Not yet handled
1,burgart_205074_c70.jpg,Burgers,Bon & Beau,"Bœuf (VBF Metzger), Cheddar, Sucrine, Bacon, Tomate Confite, Sauce poivre maison",Not yet handled
2,burgart_205074_c70.jpg,Burgers,Dandy,"Bœuf (VBF Metzger), Cheddar fumée, Mâche nantaise, Bresaola, Aubergine, Oignon rouge, Pickles, Sauce cocktail maison",Not yet handled
3,burgart_205074_c70.jpg,Burgers,Green Chicken,"Filet de poulet, Tomme de Savoie IGP, Roquette, Mortadelle, Parmesan, Sauce verte maison",Not yet handled
4,burgart_205074_c70.jpg,Poke Bowls,Black bowl,"Thon, Riz noir, Avocat, Oignon rouge, Cébette, Concombre, Sésame, Coriandre, Grenade, Sauce douce",Not yet handled
...,...,...,...,...,...
123,IMG_7934.PNG,Plats,Hot-Duck,"Pain brioché, confit de canard, ketchup de pomme, coleslaw, Tomme des Aravis, frites",Not yet handled
124,IMG_7934.PNG,Plats,Tartiflette,"Reblochon, pommes de terre, oignons, poitrine fumée, mesclun (20' de cuisson)",Not yet handled
125,IMG_7934.PNG,Plats,Saucisse - Purée truffée,"Saucisse d'Auvergne, purée à la truffe, crème d'Abondance",Not yet handled
126,IMG_7934.PNG,Desserts,Cake au chocolat,"Sauce au chocolat, copeaux de vieux Gouda et noix",Not yet handled


Let's consider one menu recipe

In [14]:
menu_data[menu_data.index==48]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand
48,in-bocca-al-lupo_42356_293.jpg,Pizza Bambini (pour les petits),Margherita,"sauce tomate, mozzarella",Not yet handled


In [15]:
jow_data["recettes"][jow_data["recettes"].str.contains("pizza")].values.tolist()

['pizza parma',
 'pizza maison 3 fromages',
 'pizza maison chèvre miel',
 'pizza verde à la courgette',
 'pizza verde à la burrata',
 'pizza printanière',
 'pizza saumon avocat',
 'pizza poire gorgonzola',
 'pizza puttanesca',
 'pizza pesto, tomates & burrata',
 'pizza prosciutto, chèvre & miel',
 'pizza cerise, tapenade & tomme',
 'pizza chorizo & artichaut',
 'pizza margherita',
 'pizza champignons & ricotta',
 'pizza aux anchois',
 'pizza au chorizo',
 'pâte à pizza maison',
 'pizza bianca']

In [16]:
jow_data[jow_data["recettes"]=='pizza margherita'].index[0]

'Pizza Margherita'

In [17]:
menu_data.loc[48, 'Closest JOW recipe by hand'] = jow_data[jow_data["recettes"]=='pizza margherita'].index[0]

Next menu recipe

In [18]:
menu_data[menu_data.index==76]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand
76,fabbrezza_202084_665.jpg,Plat au choix,Pizza Margherita,NaN,Not yet handled


In [19]:
menu_data.loc[76, 'Closest JOW recipe by hand'] = jow_data[jow_data["recettes"]=='pizza margherita'].index[0]

Next menu recipe

In [20]:
menu_data[menu_data.index==8]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand
8,burgart_205074_c70.jpg,Desserts,Fondant au chocolat artisan boulanger,NaN,Not yet handled


In [21]:
jow_data["recettes"][jow_data["recettes"].str.contains("fondant")].values.tolist()

['fondant choco marrons',
 'fondant au chocolat',
 "fondant au chocolat à l'huile d'olive",
 'fondant-flan au chocolat & amande']

In [22]:
jow_data[jow_data["recettes"]=='fondant au chocolat']

,cuisson,ingredients,recettes,only_ings
Fondant au chocolat,plat froid,"[['200 g', 'chocolat à cuire'], ['125 g', 'beurre demi-sel'], ['130 g', 'sucre (en poudre)'], ['4', 'oeuf'], ['25 g']]",fondant au chocolat,"chocolat à cuire, beurre demi-sel, sucre, oeuf"


In [23]:
menu_data.loc[8, 'Closest JOW recipe by hand'] = jow_data[jow_data["recettes"]=='fondant au chocolat'].index[0]

Next menu recipe

In [24]:
menu_data[menu_data.index==7]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand
7,burgart_205074_c70.jpg,Desserts,Mousse au chocolat maison,NaN,Not yet handled


In [25]:
jow_data["recettes"][jow_data["recettes"].str.contains("mousse")].values.tolist()

['avocat, pamplemousse & betterave',
 'mousse au chocolat maison',
 "mousse au chocolat à l'orange",
 'salade pamplemousse, avocat & crevettes',
 'asperges & sauce mousseline']

In [26]:
menu_data.loc[7, 'Closest JOW recipe by hand'] = jow_data[jow_data["recettes"]=='mousse au chocolat maison'].index[0]

Next menu recipe

In [27]:
menu_data[menu_data.index==9]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand
9,burgart_205074_c70.jpg,Desserts,Financier artisan boulanger,NaN,Not yet handled


In [28]:
jow_data["recettes"][jow_data["recettes"].str.contains("financier")].values.tolist()

['financier cerises noires & pistache', 'financier à la framboise']

In [29]:
tmp = []
for l in jow_data["recettes"][jow_data["recettes"].str.contains("financier")].values.tolist():
    tmp.append(jow_data[jow_data["recettes"]==l].index[0])

tmp

['Financier cerises noires & pistache', 'Financier à la framboise']

In [30]:
menu_data.loc[9, 'Closest JOW recipe by hand'] = tmp

Next menu recipe

In [31]:
menu_data[menu_data.index==10]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand
10,burgart_205074_c70.jpg,Boissons,Expresso,NaN,Not yet handled


In [32]:
jow_data["recettes"][jow_data["recettes"].str.contains("café")].values.tolist()

[]

In [33]:
jow_data["recettes"][jow_data["recettes"].str.contains("expresso")].values.tolist()

[]

In [34]:
jow_data["recettes"][jow_data["recettes"].str.contains("espresso")].values.tolist()

[]

In [35]:
jow_data[jow_data["ingredients"].str.contains("café")].values.tolist()

[['3 heures',
  "[['100 g', 'sucre (en poudre)'], ['facultatif', '1 sac.', 'sucre vanillé'], ['3', 'oeuf'], ['2 càs', 'cacao (en poudre)'], ['250 g', 'mascarpone'], ['20', 'biscuit cuiller'], ['2 càc', 'café'], ['facultatif', '1 càs', 'amaretto']]",
  'tiramisu',
  'sucre, sucre vanillé, oeuf, cacao, mascarpone, biscuit cuiller, café, amaretto'],
 ['2 minutes',
  "[['15 cl', 'lait'], ['1', 'café caramel (dose)'], ['1 càc', 'caramel (liquide)']]",
  'caramel macchiato',
  'lait, café caramel, caramel'],
 ['plat froid',
  "[['7 g', 'café'], ['60 g', 'crème glacée vanille'], ['20 g', 'amandes (effilées)']]",
  'affogato',
  'café, crème glacée vanille, amandes']]

In [36]:
menu_data.loc[10, 'Closest JOW recipe by hand'] = 'no match'

Next recipe

In [37]:
menu_data[menu_data.index==11]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand
11,burgart_205074_c70.jpg,Boissons,Café long,NaN,Not yet handled


In [38]:
menu_data.loc[11, 'Closest JOW recipe by hand'] = 'no match'

Next recipe

In [39]:
menu_data[menu_data.index==12]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand
12,burgart_205074_c70.jpg,Boissons,Chocolat chaud,NaN,Not yet handled


In [40]:
jow_data["recettes"][jow_data["recettes"].str.contains("chocolat chaud")].values.tolist()

['chocolat chaud', 'chocolat chaud & marshmallows']

In [41]:
jow_data[jow_data["recettes"]=='chocolat chaud']

,cuisson,ingredients,recettes,only_ings
Chocolat chaud,plat froid,"[['30 g', 'chocolat à cuire'], ['25 cl', 'lait'], ['1 càc', 'cacao (en poudre)']]",chocolat chaud,"chocolat à cuire, lait, cacao"


In [42]:
menu_data.loc[12, 'Closest JOW recipe by hand'] = jow_data[jow_data["recettes"]=='chocolat chaud'].index[0]

Next recipe

In [43]:
menu_data[menu_data.index==13]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand
13,burgart_205074_c70.jpg,Boissons,Ice Tea,NaN,Not yet handled


In [44]:
jow_data["recettes"][jow_data["recettes"].str.contains("thé")].values.tolist()

[]

In [45]:
jow_data[jow_data["ingredients"].str.contains("thé")].values.tolist()

[]

In [46]:
jow_data["recettes"][jow_data["recettes"].str.contains("ice tea")].values.tolist()

[]

In [47]:
menu_data.loc[13, 'Closest JOW recipe by hand'] = 'no match'

Next recipe

In [48]:
menu_data[menu_data.index==14]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand
14,burgart_205074_c70.jpg,Boissons,Elixia,Limonade Bio,Not yet handled


In [49]:
jow_data["recettes"][jow_data["recettes"].str.contains("limonade")].values.tolist()

[]

In [50]:
jow_data[jow_data["ingredients"].str.contains("limonade")].values.tolist()

[['1 heure',
  "[['10 cl', 'vin rouge'], ['10 cl', 'limonade'], ['1/4', 'orange'], ['1/4', 'citron jaune']]",
  'tinto de verano',
  'vin rouge, limonade, orange, citron jaune'],
 ['plat froid',
  "[['150 g', 'pastèque (classique)'], ['30 g', 'concombre'], ['5 cl', 'limonade'], ['1/10 bou.', 'menthe (feuilles)'], ['1/2', 'citron jaune']]",
  'watermelon refresher',
  'pastèque, concombre, limonade, menthe, citron jaune'],
 ['plat froid',
  "[['70 g', 'pastèque (classique)'], ['20 g', 'concombre'], ['5 cl', 'limonade'], ['1/10 bou.', 'menthe (feuilles)'], ['1/2', 'citron jaune'], ['4 cl', 'vodka']]",
  'vodka watermelon refresher',
  'pastèque, concombre, limonade, menthe, citron jaune, vodka'],
 ['3 heures',
  "[['75 cl', 'vin blanc'], ['60 cl', 'jus de pêche'], ['40 cl', 'limonade'], ['3', 'pêche'], ['3/10 bou.', 'menthe (feuilles)']]",
  'sangria à la pêche',
  'vin blanc, jus de pêche, limonade, pêche, menthe']]

In [51]:
menu_data.loc[14, 'Closest JOW recipe by hand'] = 'no match'

Next recipe

In [52]:
menu_data[menu_data.index==15]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand
15,burgart_205074_c70.jpg,Boissons,Alain Milliat,"Orange, Mangue, Pomme",Not yet handled


In [53]:
jow_data["recettes"][jow_data["recettes"].str.contains("jus")].values.tolist()

[]

In [54]:
jow_data[jow_data["ingredients"].str.contains("jus")].values.tolist()

[['35 minutes',
  '[[\'1\', \'poulet (cuisse)\'], [\'1\', \'clémentine\'], [\'1/4\', \'oignon jaune\'], [\'200 g\', \'haricots blancs nature (cuits)\'], [\'1 càs\', "moutarde à l\'ancienne"], [\'5 cl\', "jus d\'orange"]]',
  'poulet clémentine & moutarde',
  "poulet, clémentine, oignon jaune, haricots blancs nature, moutarde à l'ancienne, jus d'orange"],
 ['plat froid',
  '[[\'5 cl\', \'rhum (blanc)\'], [\'4 cl\', \'lait de coco\'], [\'12 cl\', "jus d\'ananas"]]',
  'piña colada',
  "rhum, lait de coco, jus d'ananas"],
 ['plat froid',
  "[['4 cl', 'vodka'], ['2 cl', 'jus de cranberry'], ['2 cl', 'cointreau'], ['1/4', 'citron vert']]",
  'cosmopolitan',
  'vodka, jus de cranberry, cointreau, citron vert'],
 ['plat froid',
  "[['4 cl', 'vodka'], ['12 cl', 'jus de tomate'], ['1/4 càc', 'tabasco'], ['1 pinc.', 'sel au céleri']]",
  'bloody mary',
  'vodka, jus de tomate, tabasco, sel au céleri'],
 ['plat froid',
  '[[\'4 cl\', \'lait de coco\'], [\'12 cl\', "jus d\'ananas"], [\'1/2\', \'ci

In [55]:
menu_data.loc[15, 'Closest JOW recipe by hand'] = 'no match'

Next recipe

In [56]:
menu_data[menu_data.index==16]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand
16,burgart_205074_c70.jpg,Boissons,Bière Corona,NaN,Not yet handled


In [57]:
jow_data["recettes"][jow_data["recettes"].str.contains("bière")].values.tolist()

[]

In [58]:
menu_data.loc[16, 'Closest JOW recipe by hand'] = 'no match'

Next recipe

In [59]:
menu_data[menu_data.index==17]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand
17,Carte-in-seoul.png,Anju,Mandoo (6 pcs),Raviolis maison grillés au porc et légumes,Not yet handled


In [60]:
jow_data["recettes"][jow_data["recettes"].str.contains("mandoo")].values.tolist()

[]

In [61]:
jow_data["recettes"][jow_data["recettes"].str.contains("raviolis")].values.tolist()

['raviolis à la crème de champignons',
 'raviolis, tomate & feta',
 'raviolis aux champignons, chèvre frais & noisette']

In [62]:
jow_data[jow_data["recettes"]=='raviolis aux champignons, chèvre frais & noisette']

,cuisson,ingredients,recettes,only_ings
"Raviolis aux champignons, chèvre frais & noisette",2 minutes,"[['125 g', 'ravioli (aux champignons)'], ['50 g', 'chèvre frais'], ['20 g', 'noisette'], ['40 g', 'champignons bruns']]","raviolis aux champignons, chèvre frais & noisette","ravioli, chèvre frais, noisette, champignons bruns"


In [63]:
menu_data.loc[17, 'Closest JOW recipe by hand'] = 'no match'

Next recipe

In [64]:
menu_data[menu_data.index==18]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand
18,Carte-in-seoul.png,Anju,Haemul Jeol,Galette de fruit mer,Not yet handled


In [65]:
jow_data["recettes"][jow_data["recettes"].str.contains("galette")].values.tolist()

['galette forestière',
 'galette saumon',
 'galette complète',
 'galette seguin',
 'galette roquefort',
 'galette des rois maison',
 'galette parma',
 'galette chèvre miel',
 'galette fondue de poireaux & camembert',
 'galette saucisse',
 'galette champignons & oeuf miroir',
 'galette fondue de poireaux & saumon',
 'galette italienne',
 'galette fondue de poireaux & oeuf miroir',
 'galette savoyarde',
 'galette oeuf & chorizo',
 'galette chocolat & amandes express',
 'galette des rois à la pomme',
 'galette de sarrasin',
 'galette épinard chèvre']

In [66]:
jow_data["recettes"][jow_data["recettes"].str.contains("fruit de mer")].values.tolist()

[]

In [67]:
jow_data["recettes"][jow_data["recettes"].str.contains("jeol")].values.tolist()

[]

In [68]:
jow_data[jow_data["ingredients"].str.contains("fruit de mer")].values.tolist()

[]

In [69]:
menu_data.loc[18, 'Closest JOW recipe by hand'] = 'no match'

Next recipe

In [70]:
menu_data[menu_data.index==19]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand
19,Carte-in-seoul.png,Anju,Sewoo kkanpoong (8 pcs),Crevettes frites dans une sauce aigre douce,Not yet handled


In [71]:
jow_data["recettes"][jow_data["recettes"].str.contains("sewoo")].values.tolist()

[]

In [72]:
jow_data[jow_data["recettes"].str.contains("crevette")]

,cuisson,ingredients,recettes,only_ings
Spaghetti ail & crevette,10 minutes,"[['100 g', 'pâtes (spaghetti)'], ['100 g', 'crevette (cuite)'], ['1/2 gou.', 'ail'], ['1 pinc.', ""piment d'espelette""], ['1/10 bou.', 'persil (frais)']]",spaghetti ail & crevette,"pâtes, crevette, ail, piment d'espelette, persil"
Crevettes miel sésame,plat froid,"[['100 g', 'crevette (cuite)'], ['70 g', 'riz'], ['1/2 gou.', 'ail'], ['1 càc', 'graines de sésame']]",crevettes miel sésame,"crevette, riz, ail, graines de sésame"
Gaspacho repas crevette & avocat,plat froid,"[['30 cl', 'gaspacho'], ['100 g', 'crevette (cuite)'], ['1/2', 'avocat'], ['facultatif', '1/5 bou.', 'basilic (frais)'], ['1 tran.', 'pain de mie']]",gaspacho repas crevette & avocat,"gaspacho, crevette, avocat, basilic, pain de mie"
Bouillon coco crevettes,plat froid,"[['3 cl', 'lait de coco'], ['1 càc', 'curry (poudre)'], ['1/4', 'bouillon de volaille (cube)'], ['facultatif', '1/10 bou.', 'basilic (frais)']]",bouillon coco crevettes,"lait de coco, curry, bouillon de volaille, basilic"
Blanquette express aux crevettes,plat froid,"[['100 g', 'crevette (cuite)'], ['1/2', 'carotte (frais)'], ['1/2', 'poireau'], ['1 càc', 'fond de veau']]",blanquette express aux crevettes,"crevette, carotte, poireau, fond de veau"
Ramen aux crevettes,5 minutes,"[['1 càc', 'pâte de curry (rouge)'], ['2 càs', 'lait de coco'], ['1', 'nouilles instantanées (curry)'], ['facultatif', '1/4', 'citron vert'], ['1/10 bou.', 'coriandre (frais)'], ['50 g', 'crevette (cuite)']]",ramen aux crevettes,"pâte de curry, lait de coco, nouilles instantanées, citron vert, coriandre, crevette"
Risotto crevettes & chorizo,plat froid,"[['70 g', 'riz (arborio)'], ['25 g', 'chorizo (entier)'], ['50 g', 'crevette (cuite)'], ['1/4', 'bouillon de volaille (cube)']]",risotto crevettes & chorizo,"riz, chorizo, crevette, bouillon de volaille"
Poêlée de crevettes & polenta,8 minutes,"[['50 g', 'polenta'], ['25 cl', 'lait'], ['1 càs', 'parmesan (râpé)'], ['80 g', 'tomate cerise'], ['50 g'], ['1/4 gou.'], ['facultatif', '1/10 bou.']]",poêlée de crevettes & polenta,"polenta, lait, parmesan, tomate cerise"
Pad Thaï crevette,8 minutes,"[['80 g', 'nouilles de riz'], ['1/2 gou.', 'ail'], ['2 càs', 'sauce soja salée'], ['1/2 càs', 'beurre de cacahuète'], ['1/4', 'citron vert'], ['50 g', 'pousses de soja'], ['50 g', 'crevette (cuite)'], ['facultatif', '1/10 bou.', 'coriandre (frais)']]",pad thaï crevette,"nouilles de riz, ail, sauce soja salée, beurre de cacahuète, citron vert, pousses de soja, crevette, coriandre"
"Salade de riz, crevettes & avocat",plat froid,"[['60 g', 'riz (mélange)'], ['50 g', 'crevette (cuite)'], ['1/2', 'avocat'], ['1 poignée', 'epinard (frais)'], ['facultatif', '1/10 bou.', 'ciboulette'], ['1/4', 'oignon rouge'], ['1/4', 'citron vert']]","salade de riz, crevettes & avocat","riz, crevette, avocat, epinard, ciboulette, oignon rouge, citron vert"


In [73]:
tmp = []
tmp.append(jow_data[jow_data["recettes"]=='crevettes teriyaki'].index[0])
tmp.append(jow_data[jow_data["recettes"]=='crevettes miel sésame'].index[0])
tmp

['Crevettes teriyaki', 'Crevettes miel sésame']

In [74]:
menu_data.loc[19, 'Closest JOW recipe by hand'] = tmp

Next recipe

In [75]:
menu_data[menu_data.index==20]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand
20,Carte-in-seoul.png,Bibimbap,Dolsot bibimbap (en pierrade au choix),Bœuf,Not yet handled


In [76]:
jow_data[jow_data["recettes"].str.contains("bibimbap")]

,cuisson,ingredients,recettes,only_ings


In [77]:
jow_data[jow_data["recettes"].str.contains("boeuf")]

,cuisson,ingredients,recettes,only_ings
Yakitori bœuf fromage,10 minutes,"[['80 g', 'boeuf (carpaccio)'], ['100 g', 'emmental'], ['1 càs', 'sauce teriyaki'], ['70 g', 'riz'], ['6', 'pics à brochette']]",yakitori boeuf fromage,"boeuf, emmental, sauce teriyaki, riz"
Wok de bœuf légumes croquants,12 minutes,"[['1', 'boeuf (rumsteck)'], ['1', 'carotte (frais)'], ['50 g', 'petits pois (frais)'], ['1/2', 'courgette'], ['facultatif', '1/2 càc', 'graines de sésame'], ['1 cm', 'gingembre (frais)']]",wok de boeuf légumes croquants,"boeuf, carotte, petits pois, courgette, graines de sésame, gingembre"
Bœuf sauce roquefort,20 minutes,"[['250 g', 'pomme de terre grenaille (surgelées)'], ['1', 'boeuf (steak)'], ['2 cl', 'crème liquide'], ['10 g', 'roquefort']]",boeuf sauce roquefort,"pomme de terre grenaille, boeuf, crème liquide, roquefort"
Bœuf haricots verts & purée,20 minutes,"[['100 g', 'haricot vert (frais)'], ['250 g', 'pomme de terre'], ['5 cl', 'lait'], ['1', 'boeuf (steak)'], ['20 g', 'beurre']]",boeuf haricots verts & purée,"haricot vert, pomme de terre, lait, boeuf, beurre"
Curry de bœuf thaï,plat froid,"[['1', 'boeuf (steak haché)'], ['1/2', 'carotte (frais)'], ['100 g', 'pomme de terre'], ['1 càc', 'pâte de curry (vert)'], ['10 cl', 'lait de coco'], ['1 càs', 'sauce soja salée'], ['1/4', 'bouillon de volaille (cube)'], ['facultatif', '1/4', 'citron vert']]",curry de boeuf thaï,"boeuf, carotte, pomme de terre, pâte de curry, lait de coco, sauce soja salée, bouillon de volaille, citron vert"
Ramen au bœuf,7 minutes,"[['1', 'nouilles instantanées (boeuf)'], ['1', 'oeuf'], ['50 g', 'boeuf (carpaccio)'], ['1 càs', 'sauce soja salée'], ['facultatif', '1/10 bou.', 'ciboulette']]",ramen au boeuf,"nouilles instantanées, oeuf, boeuf, sauce soja salée, ciboulette"
Enchiladas au bœuf,25 minutes,"[['2', 'tortilla (maïs)'], ['1/2', 'poivron rouge'], ['50 g', 'tomate (chair)'], ['1', 'boeuf (steak haché)'], ['1 càs', 'crème fraîche épaisse'], ['20 g', 'cheddar (râpé)'], ['1/2', 'oignon jaune']]",enchiladas au boeuf,"tortilla, poivron rouge, tomate, boeuf, crème fraîche épaisse, cheddar, oignon jaune"
Bœuf à la mexicaine,plat froid,"[['1', 'boeuf (steak)'], ['1 càc', 'mélange épices mexicaines'], ['1/4', 'oignon jaune'], ['100 g', 'tomate (chair)'], ['facultatif']]",boeuf à la mexicaine,"boeuf, mélange épices mexicaines, oignon jaune, tomate"
Bœuf & crème persillée,30 minutes,"[['1', 'boeuf (steak)'], ['1', 'carotte (frais)'], ['200 g', 'pomme de terre'], ['1 càs', 'crème fraîche épaisse'], ['1 càc', 'vinaigre de vin blanc'], ['1/10 bou.', 'persil (frais)'], ['1/4', 'citron jaune'], ['facultatif', '1/2 càc', 'raifort (crème)']]",boeuf & crème persillée,"boeuf, carotte, pomme de terre, crème fraîche épaisse, vinaigre de vin blanc, persil, citron jaune, raifort"
Tagliata de bœuf & courgettes,4 minutes,"[['240 g', 'courgette'], ['1', 'boeuf (rumsteck)'], ['1 càs', 'vinaigre de vin rouge'], ['1/2 càc', 'origan (séché)']]",tagliata de boeuf & courgettes,"courgette, boeuf, vinaigre de vin rouge, origan"


In [78]:
menu_data.loc[20, 'Closest JOW recipe by hand'] = 'no match'

Next recipe

In [79]:
menu_data[menu_data.index==21]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand
21,Carte-in-seoul.png,Chicken,"YangNyeom chicken (5pcs, 12pcs, 24pcs)",Poulet frit enrobé de sauce + salade,Not yet handled


In [80]:
jow_data[jow_data["recettes"].str.contains("poulet")]

,cuisson,ingredients,recettes,only_ings
Poulet au miel & haricots,13 minutes,"[['1', 'poulet (escalope)'], ['1 càs', 'miel (liquide)'], ['1/2', 'citron jaune'], ['250 g', 'haricot vert (frais)']]",poulet au miel & haricots,"poulet, miel, citron jaune, haricot vert"
Poulet cacahuète légumes croquants,10 minutes,"[['1 càs', 'beurre de cacahuète'], ['1', 'poulet (escalope)'], ['150 g', 'brocoli (frais)'], ['1 càs', 'sauce soja salée']]",poulet cacahuète légumes croquants,"beurre de cacahuète, poulet, brocoli, sauce soja salée"
Poulet & poêlée légumes au curry,15 minutes,"[['1', 'poulet (escalope)'], ['1/2', 'courgette'], ['2', 'carotte (frais)'], ['1 càc', 'curry (poudre)'], ['1/2 gou.', 'ail'], ['facultatif', '1/5 bou.', 'coriandre (frais)']]",poulet & poêlée légumes au curry,"poulet, courgette, carotte, curry, ail, coriandre"
Poulet satay,plat froid,"[['70 g', 'riz'], ['1', 'poulet (escalope)'], ['1 càs', 'beurre de cacahuète'], ['10 cl', 'lait de coco'], ['1/4', 'oignon rouge'], ['1 pinc.', 'piment de cayenne'], ['1', 'pics à brochette']]",poulet satay,"riz, poulet, beurre de cacahuète, lait de coco, oignon rouge, piment de cayenne"
Brochettes de poulet Tsukune,10 minutes,"[['1', 'poulet (escalope)'], ['2 càs', 'sauce soja salée'], ['70 g', 'riz'], ['1/4', 'oeuf'], ['1/4', 'oignon jaune'], ['2 càs', 'chapelure'], ['facultatif', '2', 'pics à brochette'], ['1 càs', 'sucre (en poudre)']]",brochettes de poulet tsukune,"poulet, sauce soja salée, riz, oeuf, oignon jaune, chapelure, sucre"
...,...,...,...,...
Poulet basquaise,1 heure,"[['150 g', 'tomate'], ['120 g', 'poivron rouge'], ['1', 'poulet (cuisse)'], ['4 cl', 'vin blanc'], ['1/4', 'oignon jaune'], ['1/10', 'bouquet garni'], ['1/2', 'ail']]",poulet basquaise,"tomate, poivron rouge, poulet, vin blanc, oignon jaune, bouquet garni, ail"
Salade au poulet pané,6 minutes,"[['1', 'poulet (pané)'], ['2 poignées', 'salade (mélange)'], ['100 g', 'tomate cerise'], ['15 g', 'parmesan (morceaux)'], ['1 càc', 'crème balsamique']]",salade au poulet pané,"poulet, salade, tomate cerise, parmesan, crème balsamique"
Enchiladas poulet & courgettes,34 minutes,"[['2', 'tortilla (blé)'], ['1', 'poulet (escalope)'], ['125 g', 'courgette'], ['8 cl', 'crème liquide'], ['40 g', 'mozzarella (rapé)'], ['1/4', 'oignon jaune']]",enchiladas poulet & courgettes,"tortilla, poulet, courgette, crème liquide, mozzarella, oignon jaune"
"Poulet, semoule & ratatouille",10 minutes,"[['1', 'poulet (escalope)'], ['100 g', 'ratatouille'], ['70 g', 'semoule']]","poulet, semoule & ratatouille","poulet, ratatouille, semoule"


In [81]:
menu_data.loc[21, 'Closest JOW recipe by hand'] = 'no match'

Next recipe

In [82]:
menu_data[menu_data.index==22]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand
22,Carte-in-seoul.png,Classiques,Bulgogi,"Bœuf émincé à la sauce soja, légumes, riz",Not yet handled


In [83]:
jow_data[jow_data["recettes"].str.contains("boeuf")]

,cuisson,ingredients,recettes,only_ings
Yakitori bœuf fromage,10 minutes,"[['80 g', 'boeuf (carpaccio)'], ['100 g', 'emmental'], ['1 càs', 'sauce teriyaki'], ['70 g', 'riz'], ['6', 'pics à brochette']]",yakitori boeuf fromage,"boeuf, emmental, sauce teriyaki, riz"
Wok de bœuf légumes croquants,12 minutes,"[['1', 'boeuf (rumsteck)'], ['1', 'carotte (frais)'], ['50 g', 'petits pois (frais)'], ['1/2', 'courgette'], ['facultatif', '1/2 càc', 'graines de sésame'], ['1 cm', 'gingembre (frais)']]",wok de boeuf légumes croquants,"boeuf, carotte, petits pois, courgette, graines de sésame, gingembre"
Bœuf sauce roquefort,20 minutes,"[['250 g', 'pomme de terre grenaille (surgelées)'], ['1', 'boeuf (steak)'], ['2 cl', 'crème liquide'], ['10 g', 'roquefort']]",boeuf sauce roquefort,"pomme de terre grenaille, boeuf, crème liquide, roquefort"
Bœuf haricots verts & purée,20 minutes,"[['100 g', 'haricot vert (frais)'], ['250 g', 'pomme de terre'], ['5 cl', 'lait'], ['1', 'boeuf (steak)'], ['20 g', 'beurre']]",boeuf haricots verts & purée,"haricot vert, pomme de terre, lait, boeuf, beurre"
Curry de bœuf thaï,plat froid,"[['1', 'boeuf (steak haché)'], ['1/2', 'carotte (frais)'], ['100 g', 'pomme de terre'], ['1 càc', 'pâte de curry (vert)'], ['10 cl', 'lait de coco'], ['1 càs', 'sauce soja salée'], ['1/4', 'bouillon de volaille (cube)'], ['facultatif', '1/4', 'citron vert']]",curry de boeuf thaï,"boeuf, carotte, pomme de terre, pâte de curry, lait de coco, sauce soja salée, bouillon de volaille, citron vert"
Ramen au bœuf,7 minutes,"[['1', 'nouilles instantanées (boeuf)'], ['1', 'oeuf'], ['50 g', 'boeuf (carpaccio)'], ['1 càs', 'sauce soja salée'], ['facultatif', '1/10 bou.', 'ciboulette']]",ramen au boeuf,"nouilles instantanées, oeuf, boeuf, sauce soja salée, ciboulette"
Enchiladas au bœuf,25 minutes,"[['2', 'tortilla (maïs)'], ['1/2', 'poivron rouge'], ['50 g', 'tomate (chair)'], ['1', 'boeuf (steak haché)'], ['1 càs', 'crème fraîche épaisse'], ['20 g', 'cheddar (râpé)'], ['1/2', 'oignon jaune']]",enchiladas au boeuf,"tortilla, poivron rouge, tomate, boeuf, crème fraîche épaisse, cheddar, oignon jaune"
Bœuf à la mexicaine,plat froid,"[['1', 'boeuf (steak)'], ['1 càc', 'mélange épices mexicaines'], ['1/4', 'oignon jaune'], ['100 g', 'tomate (chair)'], ['facultatif']]",boeuf à la mexicaine,"boeuf, mélange épices mexicaines, oignon jaune, tomate"
Bœuf & crème persillée,30 minutes,"[['1', 'boeuf (steak)'], ['1', 'carotte (frais)'], ['200 g', 'pomme de terre'], ['1 càs', 'crème fraîche épaisse'], ['1 càc', 'vinaigre de vin blanc'], ['1/10 bou.', 'persil (frais)'], ['1/4', 'citron jaune'], ['facultatif', '1/2 càc', 'raifort (crème)']]",boeuf & crème persillée,"boeuf, carotte, pomme de terre, crème fraîche épaisse, vinaigre de vin blanc, persil, citron jaune, raifort"
Tagliata de bœuf & courgettes,4 minutes,"[['240 g', 'courgette'], ['1', 'boeuf (rumsteck)'], ['1 càs', 'vinaigre de vin rouge'], ['1/2 càc', 'origan (séché)']]",tagliata de boeuf & courgettes,"courgette, boeuf, vinaigre de vin rouge, origan"


In [84]:
tmp = []
tmp.append(jow_data[jow_data["recettes"]=='wok de boeuf légumes croquants'].index[0])
tmp.append(jow_data[jow_data["recettes"]=='boeuf thaï'].index[0])
tmp.append(jow_data[jow_data["recettes"]=='boeuf sauté au chou'].index[0])
tmp.append(jow_data[jow_data["recettes"]=='aiguillettes de boeuf & légumes poêlés'].index[0])
tmp

['Wok de bœuf légumes croquants',
 'Bœuf thaï',
 'Bœuf sauté au chou',
 'Aiguillettes de bœuf & légumes poêlés']

In [85]:
menu_data.loc[22, 'Closest JOW recipe by hand'] = tmp

Next recipe

In [86]:
menu_data[menu_data.index==23]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand
23,Carte-in-seoul.png,Classiques,Jeyuk,"Porc sauté, légumes, riz - soja ou piquant",Not yet handled


In [87]:
jow_data[jow_data["recettes"].str.contains("jeyuk")]

,cuisson,ingredients,recettes,only_ings


In [88]:
jow_data[jow_data["recettes"].str.contains("porc")]

,cuisson,ingredients,recettes,only_ings
Sauté de porc à l'ananas,30 minutes,"[['1/4', 'porc (filet mignon)'], ['1 tran.', 'ananas au sirop (tranches)'], ['1/4', 'échalote'], ['1/4 gou.', 'ail'], ['70 g', 'riz'], ['1/2 càc', 'curry (poudre)'], ['2 cl', 'vin blanc'], ['1/4', 'bouillon de volaille (cube)']]",sauté de porc à l'ananas,"porc, ananas au sirop, échalote, ail, riz, curry, vin blanc, bouillon de volaille"
Porc au caramel,plat froid,"[['1/4', 'porc (filet mignon)'], ['70 g', 'riz'], ['2 càs', 'sauce soja salée'], ['1/2 càs', 'vinaigre de xérès'], ['1 càc', 'sucre (en poudre)'], ['1 cm', 'gingembre (frais)']]",porc au caramel,"porc, riz, sauce soja salée, vinaigre de xérès, sucre, gingembre"
Nouilles de riz & porc sauté,7 minutes,"[['80 g', 'nouilles de riz'], ['80 g', 'chair à saucisse'], ['50 g', 'concombre'], ['20 g', 'cacahuète'], ['1 pinc.', ""piment d'espelette""], ['2 càs', 'sauce soja salée'], ['facultatif', '1/10 bou.', 'menthe (feuilles)']]",nouilles de riz & porc sauté,"nouilles de riz, chair à saucisse, concombre, cacahuète, piment d'espelette, sauce soja salée, menthe"
Côtes de porc au curcuma & tomates,40 minutes,"[['1', 'côte de porc'], ['1/2', 'citron vert'], ['3', 'tomate'], ['1/10 bou.', 'persil (frais)'], ['1/4 càc'], ['1/4 càs']]",côtes de porc au curcuma & tomates,"côte de porc, citron vert, tomate, persil"
Aubergine braisée au porc,15 minutes,"[['150 g', 'aubergine'], ['60 g', 'chair à saucisse'], ['1 gou.', 'ail'], ['5 g', 'gingembre (frais)'], ['1 càc', 'sucre (en poudre)'], ['1 càs', 'sauce soja salée'], ['70 g', 'riz'], ['facultatif', '1/10 bou.', 'ciboulette']]",aubergine braisée au porc,"aubergine, chair à saucisse, ail, gingembre, sucre, sauce soja salée, riz, ciboulette"
Côte de porc miel curry & patate douce,25 minutes,"[['1', 'côte de porc'], ['250 g', 'patate douce'], ['1 càc', 'miel (liquide)'], ['1/2 càc', 'curry (poudre)'], ['1/4'], ['facultatif', '1/2 càc']]",côte de porc miel curry & patate douce,"côte de porc, patate douce, miel, curry"
Côte de porc & tomates cerises rôties,10 minutes,"[['1', 'côte de porc'], ['100 g', 'tomate cerise'], ['70 g', 'riz'], ['1 pinc.', 'herbes de provence'], ['facultatif', '1/10 bou.']]",côte de porc & tomates cerises rôties,"côte de porc, tomate cerise, riz, herbes de provence"


In [89]:
menu_data.loc[23, 'Closest JOW recipe by hand'] = jow_data[jow_data["recettes"]=='porc au caramel'].index[0]

Next recipe

In [90]:
menu_data[menu_data.index==24]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand
24,Carte-in-seoul.png,Classiques,Ojing-oe,"Calamar sauté, légumes, riz - soja ou piquant )",Not yet handled


In [91]:
jow_data[jow_data["recettes"].str.contains("ojing")]

,cuisson,ingredients,recettes,only_ings


In [92]:
jow_data[jow_data["recettes"].str.contains("calamar")]

,cuisson,ingredients,recettes,only_ings


In [93]:
jow_data[jow_data["ingredients"].str.contains("calamar")]

,cuisson,ingredients,recettes,only_ings


In [94]:
menu_data.loc[24, 'Closest JOW recipe by hand'] = 'no match'

Next recipe

In [95]:
menu_data[menu_data.index==27]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand
27,Carte-Restaurant-Nelsons-Paris-Jour-2-1024x724.jpg,Salades,La César,"poulet grillé, tomates cerise, œuf, romaine, parmesan",Not yet handled


In [96]:
jow_data[jow_data["recettes"].str.contains("césar")]

,cuisson,ingredients,recettes,only_ings
Salade César,13 minutes,"[['1', 'poulet (escalope)'], ['2 poignées', 'salade (coeur de laitue)'], ['20 g', 'parmesan (morceaux)'], ['1 tran.', 'pain de campagne (tranché)'], ['1/4', 'oeuf'], ['1/4 càs', 'moutarde'], ['1/4 gou.', 'ail'], ['4 cl', 'huile de tournesol']]",salade césar,"poulet, salade, parmesan, pain de campagne, oeuf, moutarde, ail, huile de tournesol"


In [97]:
menu_data.loc[27, 'Closest JOW recipe by hand'] = jow_data[jow_data["recettes"]=='salade césar'].index[0]

Next recipe

In [98]:
menu_data[menu_data.index==28]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand
28,Carte-Restaurant-Nelsons-Paris-Jour-2-1024x724.jpg,Salades,La Biquette de Rocamadour,mielleuse et craquante,Not yet handled


In [99]:
jow_data[jow_data["recettes"].str.contains("chèvre")]

,cuisson,ingredients,recettes,only_ings
Tartine chèvre figue jambon,10 minutes,"[['2', 'figue'], ['2 tran.', 'pain de campagne (tranché)'], ['2 càs', 'chèvre frais'], ['2 tran.', 'jambon cru'], ['1 càc', 'miel (liquide)'], ['10 g', 'noix'], ['1 poignée', 'salade (mélange)']]",tartine chèvre figue jambon,"figue, pain de campagne, chèvre frais, jambon cru, miel, noix, salade"
Toasts chèvre miel,10 minutes,"[['2', 'chèvre (crottin)'], ['2 tran.', 'pain de campagne (tranché)'], ['10 g', 'noix'], ['2 càc', 'miel (liquide)'], ['1 poignée', 'salade (mélange)']]",toasts chèvre miel,"chèvre, pain de campagne, noix, miel, salade"
Tourte chèvre épinards,40 minutes,"[['1 gou.', 'ail'], ['2', 'pâte feuilletée'], ['250 g', 'chèvre frais'], ['3', 'oeuf'], ['250 g', 'epinards (surgelés)'], ['60 g', 'parmesan (râpé)'], ['facultatif', '2 càs', 'pignons de pin'], ['2 càs', 'sauce pesto']]",tourte chèvre épinards,"ail, pâte feuilletée, chèvre frais, oeuf, epinards, parmesan, pignons de pin, sauce pesto"
"Salade de betterave, chèvre & noix",10 minutes,"[['150 g', 'betterave (cuite)'], ['50 g', 'quinoa (cru)'], ['15 g', 'noix'], ['30 g', 'chèvre frais'], ['2 poignées', 'salade (mélange)'], ['2 càc', 'vinaigre de cidre']]","salade de betterave, chèvre & noix","betterave, quinoa, noix, chèvre frais, salade, vinaigre de cidre"
Pizza maison chèvre miel,10 minutes,"[['1', 'pâte à pizza'], ['2 g', 'romarin (séché)'], ['2 càc', 'miel (liquide)'], ['100 g', 'chèvre frais']]",pizza maison chèvre miel,"pâte à pizza, romarin, miel, chèvre frais"
"Quiche oignons, chèvre & miel",40 minutes,"[['120 g', 'chèvre (bûche)'], ['2 càc', 'miel (liquide)'], ['3', 'oignon jaune'], ['3', 'oeuf'], ['100 g'], ['1'], ['10 cl'], ['facultatif', '6 pinc.']]","quiche oignons, chèvre & miel","chèvre, miel, oignon jaune, oeuf"
Quiche brocoli saumon & chèvre,35 minutes,"[['1', 'pâte brisée'], ['1', 'brocoli (frais)'], ['4 tran.', 'saumon (fumé)'], ['100 g', 'chèvre frais'], ['100 g'], ['10 cl'], ['3']]",quiche brocoli saumon & chèvre,"pâte brisée, brocoli, saumon, chèvre frais"
Wrap courgette chèvre,plat froid,"[['2 càs', 'chèvre frais'], ['1/2', 'courgette'], ['2 poignées', 'salade (mélange mâche-roquette)'], ['1', 'tortilla (blé)']]",wrap courgette chèvre,"chèvre frais, courgette, salade, tortilla"
Quiche betterave chèvre noix,50 minutes,"[['3', 'oeuf'], ['17 cl', 'crème liquide'], ['1', 'pâte brisée'], ['100 g', 'chèvre frais'], ['400 g', 'betterave (cuite)'], ['20 g', 'noix'], ['facultatif', '4 càc', ""confits d'oignons""]]",quiche betterave chèvre noix,"oeuf, crème liquide, pâte brisée, chèvre frais, betterave, noix, confits d'oignons"
Brick chèvre poire,15 minutes,"[['1', 'feuille de brick'], ['2 poignées', 'salade (mélange)'], ['1/2', 'poire'], ['15 g', 'pistaches (grillées)'], ['1/2 càc', 'miel (liquide)'], ['40 g', 'chèvre frais'], ['facultatif', '1 pinc.', 'romarin (séché)']]",brick chèvre poire,"feuille de brick, salade, poire, pistaches, miel, chèvre frais, romarin"


In [100]:
tmp = []
tmp.append(jow_data[jow_data["recettes"]=='salade de chèvre chaud'].index[0])
tmp.append(jow_data[jow_data["recettes"]=='salade chèvre, miel & pignons'].index[0])
tmp.append(jow_data[jow_data["recettes"]=='salade aux croustillants de chèvre'].index[0])
tmp

['Salade de chèvre chaud',
 'Salade chèvre, miel & pignons',
 'Salade aux croustillants de chèvre']

In [101]:
menu_data.loc[28, 'Closest JOW recipe by hand'] = tmp

Next recipe

In [102]:
menu_data[menu_data.index==29]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand
29,Carte-Restaurant-Nelsons-Paris-Jour-2-1024x724.jpg,Salades,Végé Bowl,"avocat, choux kale frit, concombre, edamame, radis, tomate et tofu mariné aux sésames",Not yet handled


In [103]:
jow_data[jow_data["recettes"].str.contains("bowl")]

,cuisson,ingredients,recettes,only_ings
Primavera Poke bowl,10 minutes,"[['1', 'saumon (cru)'], ['1/2', 'avocat'], ['70 g', 'riz'], ['2 càs', 'sauce soja salée'], ['2 càs', 'vinaigre de riz'], ['10 g', 'radis'], ['facultatif', '1/5 bou.', 'coriandre (frais)'], ['20 g', 'concombre']]",primavera poke bowl,"saumon, avocat, riz, sauce soja salée, vinaigre de riz, radis, coriandre, concombre"
Rainbowl,plat froid,"[['1/2', 'avocat'], ['100 g', 'maïs (conserve)'], ['1/5 bou.', 'coriandre (frais)'], ['1/5', 'oignon rouge'], ['100 g', 'tomate cerise'], ['1/4', 'citron jaune']]",rainbowl,"avocat, maïs, coriandre, oignon rouge, tomate cerise, citron jaune"
Comfort Bowl,15 minutes,"[['150 g', 'patate douce'], ['1', 'oeuf'], ['1 poignée', 'salade (roquette)'], ['70 g', 'riz (mélange)'], ['1/4', 'citron jaune']]",comfort bowl,"patate douce, oeuf, salade, riz, citron jaune"
Rice Power Bowl,10 minutes,"[['70 g', 'riz'], ['1/2', 'avocat'], ['1', 'oeuf'], ['facultatif', '1/10 bou.', 'ciboulette'], ['1 càc', 'vinaigre de vin rouge']]",rice power bowl,"riz, avocat, oeuf, ciboulette, vinaigre de vin rouge"
Cuban rice bowl,10 minutes,"[['70 g', 'riz'], ['1', 'poulet (escalope)'], ['1/2 gou.', 'ail'], ['1 càs', 'sauce soja salée'], ['facultatif', '1/4', 'citron vert'], ['facultatif', '1/10 bou.', 'coriandre (frais)'], ['1/2 càc', 'cumin (moulu)'], ['100 g', 'haricots rouges (cuits)']]",cuban rice bowl,"riz, poulet, ail, sauce soja salée, citron vert, coriandre, cumin, haricots rouges"
Sésame bowl,15 minutes,"[['40 g', 'quinoa (cru)'], ['1/4', 'aubergine'], ['1/4', 'brocoli (frais)'], ['1/2 càc', 'graines de sésame'], ['1/2 càs', 'beurre de cacahuète'], ['1/4', 'citron jaune'], ['70 g', 'pois chiches (cuits)']]",sésame bowl,"quinoa, aubergine, brocoli, graines de sésame, beurre de cacahuète, citron jaune, pois chiches"
Bowl poire & chocolat,5 minutes,"[['1', 'poire'], ['2 càs', 'fromage blanc'], ['50 g', 'chocolat à cuire'], ['5 cl', 'crème liquide'], ['20 g', 'palet breton']]",bowl poire & chocolat,"poire, fromage blanc, chocolat à cuire, crème liquide, palet breton"
Sashimi bowl,10 minutes,"[['1', 'saumon (cru)'], ['1/4', 'oignon rouge'], ['1/4', 'citron jaune'], ['1/2 càs', 'câpres'], ['1/2 càs', ""huile d'olive""], ['70 g', 'riz (à sushi)']]",sashimi bowl,"saumon, oignon rouge, citron jaune, câpres, huile d'olive, riz"
Vegan poke bowl,10 minutes,"[['70 g', 'riz (complet)'], ['70 g', 'tofu (nature)'], ['1 càs', 'sauce soja salée'], ['1/2', 'avocat'], ['50 g', 'concombre'], ['1 càs', 'yaourt végétal (coco)']]",vegan poke bowl,"riz, tofu, sauce soja salée, avocat, concombre, yaourt végétal"
Mexican bowl,10 minutes,"[['1', 'poulet (escalope)'], ['1 càc', 'mélange épices mexicaines'], ['1/4', 'avocat'], ['60 g', 'maïs (conserve)'], ['60 g', 'haricots rouges (cuits)'], ['70 g', 'riz'], ['facultatif', '1/10 bou.', 'coriandre (frais)']]",mexican bowl,"poulet, mélange épices mexicaines, avocat, maïs, haricots rouges, riz, coriandre"


In [104]:
menu_data.loc[29, 'Closest JOW recipe by hand'] = jow_data[jow_data["recettes"]=='vegan poke bowl'].index[0]

Next recipe

In [105]:
menu_data[menu_data.index==30]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand
30,Carte-Restaurant-Nelsons-Paris-Jour-2-1024x724.jpg,Végé pour tous,Moussaka végétarienne,aux gombos,Not yet handled


In [106]:
jow_data[jow_data["recettes"].str.contains("moussaka")]

,cuisson,ingredients,recettes,only_ings


In [598]:
jow_data[jow_data["ingredients"].str.contains("gombo")]

,cuisson,ingredients,recettes,only_ings


In [108]:
menu_data.loc[30, 'Closest JOW recipe by hand'] = 'no match'

Next recipe

In [109]:
menu_data[menu_data.index==32]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand
32,Carte-Restaurant-Nelsons-Paris-Jour-2-1024x724.jpg,Végé pour tous,Risotto primavera,"asperges, artichaut et petit pois",Not yet handled


In [110]:
jow_data[jow_data["recettes"].str.contains("risotto")]

,cuisson,ingredients,recettes,only_ings
Risotto aux Saint-Jacques & poireaux,plat froid,"[['70 g', 'riz (arborio)'], ['1', 'poireau'], ['1/4', 'bouillon de légumes (cube)'], ['10 g', 'beurre']]",risotto aux saint-jacques & poireaux,"riz, poireau, bouillon de légumes, beurre"
Risotto aux poireaux,plat froid,"[['70 g', 'riz (arborio)'], ['1', 'poireau'], ['1/4', 'bouillon de légumes (cube)'], ['10 g', 'beurre']]",risotto aux poireaux,"riz, poireau, bouillon de légumes, beurre"
Risotto au pesto,plat froid,"[['70 g', 'riz (arborio)'], ['1/5 bou.', 'basilic (frais)'], ['1 càs', 'parmesan (râpé)'], ['15 g', 'noix de cajou'], ['1/4', 'échalote'], ['2 cl', 'vin blanc'], ['1/4', 'bouillon de volaille (cube)'], ['1/4', 'ail']]",risotto au pesto,"riz, basilic, parmesan, noix de cajou, échalote, vin blanc, bouillon de volaille, ail"
Risotto de coquillettes,plat froid,"[['90 g', 'pâtes (coquillettes)'], ['1 tran.', 'jambon blanc'], ['1/4', 'échalote'], ['20 g', 'comté'], ['1/2 càs'], ['1/4'], ['2 cl']]",risotto de coquillettes,"pâtes, jambon blanc, échalote, comté"
Risotto crevettes & chorizo,plat froid,"[['70 g', 'riz (arborio)'], ['25 g', 'chorizo (entier)'], ['50 g', 'crevette (cuite)'], ['1/4', 'bouillon de volaille (cube)']]",risotto crevettes & chorizo,"riz, chorizo, crevette, bouillon de volaille"
Risotto de quinoa au brocoli,25 minutes,"[['1', 'poulet (escalope)'], ['50 g', 'quinoa (cru)'], ['1/4', 'bouillon de volaille (cube)'], ['150 g', 'brocoli (frais)'], ['1/4 gou.', 'ail'], ['1 càs', 'parmesan (râpé)'], ['1/4', 'oignon jaune'], ['20 g', 'noix de cajou']]",risotto de quinoa au brocoli,"poulet, quinoa, bouillon de volaille, brocoli, ail, parmesan, oignon jaune, noix de cajou"
Risotto aux asperges,plat froid,"[['70 g', 'riz (arborio)'], ['100 g', 'asperges vertes (fraîches)'], ['1/4', 'bouillon de légumes (cube)'], ['1 càs', 'parmesan (râpé)']]",risotto aux asperges,"riz, asperges vertes, bouillon de légumes, parmesan"
Risotto aux courgettes,plat froid,"[['70 g', 'riz (arborio)'], ['100 g', 'courgette'], ['1/4', 'bouillon de légumes (cube)'], ['1 càs', 'parmesan (râpé)']]",risotto aux courgettes,"riz, courgette, bouillon de légumes, parmesan"
Risotto comté & saucisse à l'ancienne,plat froid,"[['70 g', 'riz (arborio)'], ['1/4', 'échalote'], ['20 g', 'comté'], ['100 g', 'saucisse (fumée)']]",risotto comté & saucisse à l'ancienne,"riz, échalote, comté, saucisse"
Risotto aux poivrons,plat froid,"[['70 g', 'riz (arborio)'], ['1/2', 'poivron rouge'], ['1/4', 'bouillon de légumes (cube)'], ['1 càs', 'parmesan (râpé)']]",risotto aux poivrons,"riz, poivron rouge, bouillon de légumes, parmesan"


In [111]:
menu_data.loc[32, 'Closest JOW recipe by hand'] = jow_data[jow_data["recettes"]=='risotto aux asperges'].index[0]

Next recipe

In [112]:
menu_data[menu_data.index==33]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand
33,Carte-Restaurant-Nelsons-Paris-Jour-2-1024x724.jpg,Végé pour tous,Truffade des montagnes,salade,Not yet handled


In [113]:
jow_data[jow_data["recettes"].str.contains("truffade")]

,cuisson,ingredients,recettes,only_ings


In [114]:
jow_data[jow_data["recettes"].str.contains("tartiflette")]

,cuisson,ingredients,recettes,only_ings
Tartiflette express,35 minutes,"[['100 g', 'reblochon'], ['250 g', 'pomme de terre'], ['50 g', 'lardons'], ['1/2', 'oignon jaune'], ['2 cl', 'vin blanc'], ['1 càs', 'crème fraîche épaisse']]",tartiflette express,"reblochon, pomme de terre, lardons, oignon jaune, vin blanc, crème fraîche épaisse"


In [115]:
menu_data.loc[33, 'Closest JOW recipe by hand'] = jow_data[jow_data["recettes"]=='tartiflette express'].index[0]

Next recipe

In [116]:
menu_data[menu_data.index==37]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand
37,feaa6c7b-9b16-4d51-b71d-75518f16cb08.jpg,Starters,"Gaspacho tomate, pastèque et crabe","Tomato, watermelon, coriander and chive oil",Not yet handled


In [117]:
jow_data[jow_data["recettes"].str.contains("gaspacho")]

,cuisson,ingredients,recettes,only_ings
Gaspacho de petits pois à la menthe,12 minutes,"[['200 g', 'petits pois (surgelés)'], ['1/5 bou.', 'menthe (feuilles)'], ['1/2', 'échalote'], ['1/2 pinc.', ""piment d'espelette""], ['2 càs', 'lait de coco']]",gaspacho de petits pois à la menthe,"petits pois, menthe, échalote, piment d'espelette, lait de coco"
Gaspacho tomate fraise,plat froid,"[['3', 'tomate'], ['4', 'fraise'], ['1/10 bou.', 'basilic (frais)'], ['1/2', 'burrata'], ['1/4 gou.'], ['1/4'], ['1']]",gaspacho tomate fraise,"tomate, fraise, basilic, burrata"
Gaspacho repas crevette & avocat,plat froid,"[['30 cl', 'gaspacho'], ['100 g', 'crevette (cuite)'], ['1/2', 'avocat'], ['facultatif', '1/5 bou.', 'basilic (frais)'], ['1 tran.', 'pain de mie']]",gaspacho repas crevette & avocat,"gaspacho, crevette, avocat, basilic, pain de mie"
Gaspacho de betterave,plat froid,"[['150 g', 'betterave (cuite)'], ['120 g', 'concombre'], ['1/4', 'citron jaune'], ['40 g', 'chèvre frais'], ['facultatif']]",gaspacho de betterave,"betterave, concombre, citron jaune, chèvre frais"
Gaspacho andalou,plat froid,"[['150 g', 'tomate'], ['1/2', 'poivron rouge'], ['1/4', 'concombre'], ['1/4', 'oignon rouge'], ['1/4 gou.', 'ail']]",gaspacho andalou,"tomate, poivron rouge, concombre, oignon rouge, ail"
Gaspacho verde,plat froid,"[['200 g', 'courgette'], ['1/4', 'oignon jaune'], ['2 càs', 'yaourt grec'], ['1/4 gou.', 'ail'], ['1/10 bou.', 'menthe (feuilles)'], ['1/2', 'avocat']]",gaspacho verde,"courgette, oignon jaune, yaourt grec, ail, menthe, avocat"
Gaspacho citron & avocat,plat froid,"[['30 cl', 'gaspacho'], ['1/2', 'avocat'], ['facultatif', '1/5 bou.', 'basilic (frais)'], ['30 g', 'sorbet citron vert']]",gaspacho citron & avocat,"gaspacho, avocat, basilic, sorbet citron vert"


In [118]:
menu_data.loc[37, 'Closest JOW recipe by hand'] = jow_data[jow_data["recettes"]=='gaspacho repas crevette & avocat'].index[0]

Next recipe

In [119]:
menu_data[menu_data.index==38]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand
38,feaa6c7b-9b16-4d51-b71d-75518f16cb08.jpg,Starters,Carpaccio Thon Sésame,"Tuna carpaccio, miso, mango and lime salsa, chilli",Not yet handled


In [120]:
jow_data[jow_data["recettes"].str.contains("thon")]

,cuisson,ingredients,recettes,only_ings
Brandade de thon,plat froid,"[['200 g', 'pomme de terre'], ['50 g', 'thon (conserve)'], ['1/4 gou.', 'ail'], ['facultatif', '1/10 bou.', 'persil (frais)'], ['8 cl', 'lait'], ['2 càs', 'chapelure']]",brandade de thon,"pomme de terre, thon, ail, persil, lait, chapelure"
Muffins thon & avocat,plat froid,"[['1', 'english muffin'], ['40 g', 'thon (conserve)'], ['40 g', 'fromage frais'], ['1/2', 'avocat'], ['1/2 poignée', 'epinard (frais)']]",muffins thon & avocat,"english muffin, thon, fromage frais, avocat, epinard"
"Pâtes thon, citron & ricotta",12 minutes,"[['100 g', 'pâtes (linguine)'], ['1 càs', 'ricotta'], ['50 g', 'thon au citron'], ['1/4', 'citron jaune'], ['1 càs', 'crème liquide']]","pâtes thon, citron & ricotta","pâtes, ricotta, thon au citron, citron jaune, crème liquide"
Salade au thon & œuf mollet,7 minutes,"[['50 g', 'thon (conserve)'], ['1', 'oeuf'], ['1 càc', ""moutarde à l'ancienne""], ['2 poignées', 'salade (mâche)'], ['1 càc', 'vinaigre de cidre']]",salade au thon & oeuf mollet,"thon, oeuf, moutarde à l'ancienne, salade, vinaigre de cidre"
"Salade semoule, tomate & thon",5 minutes,"[['50 g', 'semoule'], ['100 g', 'tomate cerise'], ['60 g', 'poivron vert'], ['1/4', 'citron jaune'], ['facultatif', '1/10 bou.', 'persil (frais)'], ['50 g', 'thon (conserve)']]","salade semoule, tomate & thon","semoule, tomate cerise, poivron vert, citron jaune, persil, thon"
Salade de riz thon & coriandre,10 minutes,"[['70 g', 'riz'], ['50 g', 'thon (conserve)'], ['1/10 bou.', 'coriandre (frais)'], ['1/8', 'oignon rouge']]",salade de riz thon & coriandre,"riz, thon, coriandre, oignon rouge"
Gratin de pâtes au thon,30 minutes,"[['100 g', 'pâtes (fusilli)'], ['5 cl', 'crème liquide'], ['80 g', 'thon (conserve)'], ['15 g', 'fromage râpé']]",gratin de pâtes au thon,"pâtes, crème liquide, thon, fromage râpé"
"Bruschetta thon, tomate & œuf",9 minutes,"[['1', 'oeuf'], ['2 tran.', 'pain de campagne (tranché)'], ['70 g', 'tomate'], ['70 g', 'thon (conserve)'], ['10 g', 'olive noire'], ['1 càs', 'mayonnaise']]","bruschetta thon, tomate & oeuf","oeuf, pain de campagne, tomate, thon, olive noire, mayonnaise"
Salade de thon & haricots blancs,10 minutes,"[['80 g', 'thon (conserve)'], ['1 poignée', 'salade (roquette)'], ['60 g', 'mélange céréales'], ['50 g', 'haricots blancs nature (cuits)'], ['1/8'], ['1/2']]",salade de thon & haricots blancs,"thon, salade, mélange céréales, haricots blancs nature"
"Quiche thon, tomate & moutarde",50 minutes,"[['2 càs', 'moutarde'], ['150 g', 'thon (conserve)'], ['3', 'oeuf'], ['10 cl', 'lait'], ['10 cl', 'crème liquide'], ['300 g', 'tomate'], ['1', 'pâte brisée']]","quiche thon, tomate & moutarde","moutarde, thon, oeuf, lait, crème liquide, tomate, pâte brisée"


In [121]:
menu_data.loc[38, 'Closest JOW recipe by hand'] = 'no match'

Next recipe

In [122]:
menu_data[menu_data.index==40]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand
40,feaa6c7b-9b16-4d51-b71d-75518f16cb08.jpg,Mains,Paris London Rumsteak,"(240g) French beef, onion rings, café de Paris butter, pepper sauce",Not yet handled


In [123]:
jow_data[jow_data["recettes"].str.contains("steak")]

,cuisson,ingredients,recettes,only_ings
Rumsteak à l'italienne,4 minutes,"[['1', 'boeuf (rumsteck)'], ['2 poignées', 'salade (roquette)'], ['30 g', 'parmesan (morceaux)'], ['1 càc', 'sauce pesto'], ['1 càs', 'pignons de pin']]",rumsteak à l'italienne,"boeuf, salade, parmesan, sauce pesto, pignons de pin"
Steak & cheddar polenta,15 minutes,"[['50 g', 'polenta'], ['10 g', 'cheddar (râpé)'], ['25 cl', 'lait'], ['1', 'boeuf (steak)'], ['1/2', 'oignon jaune'], ['50 g', 'tomate (chair)'], ['1/2', 'poivron rouge'], ['1/4 gou.', 'ail']]",steak & cheddar polenta,"polenta, cheddar, lait, boeuf, oignon jaune, tomate, poivron rouge, ail"
Steak de chou-fleur,25 minutes,"[['1/2', 'chou-fleur (frais)'], ['1/4 càc', 'harissa (pâte)'], ['1 càs', ""sirop d'érable""], ['1/4 gou.', 'ail'], ['1/2 càc', 'paprika'], ['1/8', 'citron jaune'], ['facultatif', '1/10 bou.', 'persil (frais)']]",steak de chou-fleur,"chou-fleur, harissa, sirop d'érable, ail, paprika, citron jaune, persil"
Steak haché & pommes dauphines,15 minutes,"[['120 g', 'pomme dauphine'], ['1', 'boeuf (steak haché)']]",steak haché & pommes dauphines,"pomme dauphine, boeuf"
Steak haché & pommes noisettes,15 minutes,"[['120 g', 'pomme noisette'], ['1', 'boeuf (steak haché)']]",steak haché & pommes noisettes,"pomme noisette, boeuf"
Steak haché sauce au roquefort & purée,plat froid,"[['1', 'boeuf (steak haché)'], ['250 g', 'pomme de terre'], ['5 cl', 'lait'], ['20 g', 'beurre']]",steak haché sauce au roquefort & purée,"boeuf, pomme de terre, lait, beurre"
"Steak haché, purée & haricots verts",plat froid,"[['1', 'boeuf (steak haché)'], ['150 g', 'pomme de terre'], ['1/2', 'échalote'], ['100 g', 'haricot vert (frais)'], ['3 cl'], ['20 g']]","steak haché, purée & haricots verts","boeuf, pomme de terre, échalote, haricot vert"
Steak haché aux échalotes & pommes sautées,17 minutes,"[['1', 'boeuf (steak haché)'], ['200 g', 'pommes de terre rissolées'], ['20 g', 'beurre'], ['1/2', 'échalote']]",steak haché aux échalotes & pommes sautées,"boeuf, pommes de terre rissolées, beurre, échalote"
Steak haché à l'échalote & purée,plat froid,"[['1', 'boeuf (steak haché)'], ['250 g', 'pomme de terre'], ['1/2', 'échalote'], ['3 cl', 'lait']]",steak haché à l'échalote & purée,"boeuf, pomme de terre, échalote, lait"
Steak de chou-fleur & haricots verts,24 minutes,"[['1/2', 'chou-fleur (frais)'], ['1 càs', ""sirop d'érable""], ['1/4 gou.', 'ail'], ['1/8', 'citron jaune'], ['100 g', 'haricot vert (frais)'], ['1 càc', 'paprika']]",steak de chou-fleur & haricots verts,"chou-fleur, sirop d'érable, ail, citron jaune, haricot vert, paprika"


In [124]:
menu_data.loc[40, 'Closest JOW recipe by hand'] = 'no match'

Next recipe

In [125]:
menu_data[menu_data.index==41]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand
41,feaa6c7b-9b16-4d51-b71d-75518f16cb08.jpg,Mains,Miso Salmon,"Raw and cooked white asparagus, artichoke mousse",Not yet handled


In [126]:
jow_data[jow_data["recettes"].str.contains("miso")]

,cuisson,ingredients,recettes,only_ings


In [127]:
jow_data[jow_data["recettes"].str.contains("saumon")]

,cuisson,ingredients,recettes,only_ings
Saumon mi-cuit & riz,10 minutes,"[['1', 'saumon (frais)'], ['90 g', 'riz']]",saumon mi-cuit & riz,"saumon, riz"
Salade saumon mangue avocat,plat froid,"[['2 tran.', 'saumon (fumé)'], ['1/4', 'mangue'], ['2 poignées', 'epinard (frais)'], ['1 cm', 'gingembre (frais)'], ['1 càs', 'sauce soja salée'], ['1/2', 'avocat'], ['1/2', 'citron vert'], ['facultatif', '1/2 càc', 'graines de sésame']]",salade saumon mangue avocat,"saumon, mangue, epinard, gingembre, sauce soja salée, avocat, citron vert, graines de sésame"
Wrap Saumon,plat froid,"[['2 tran.', 'saumon (fumé)'], ['1', 'tortilla (blé)'], ['1/10', 'concombre'], ['40 g', 'fromage frais'], ['1/2 poignée', 'epinard (frais)'], ['1/8', 'citron jaune']]",wrap saumon,"saumon, tortilla, concombre, fromage frais, epinard, citron jaune"
Papillote saumon couscous,17 minutes,"[['50 g', 'couscous aux épices'], ['1', 'saumon (frais)'], ['30 g', 'tomate séchée'], ['1', 'oignon nouveau']]",papillote saumon couscous,"couscous aux épices, saumon, tomate séchée, oignon nouveau"
Saumon mi-cuit & salade citronnée,10 minutes,"[['40 g', 'concombre'], ['1', 'saumon (frais)'], ['80 g', 'mélange céréales'], ['1/4', 'citron jaune'], ['3/10 bou.']]",saumon mi-cuit & salade citronnée,"concombre, saumon, mélange céréales, citron jaune"
...,...,...,...,...
Salade de couscous & saumon grillé,9 minutes,"[['1', 'saumon (frais)'], ['70 g', 'semoule'], ['1/4', 'concombre'], ['30 g', 'feta'], ['1/10 bou.', 'menthe (feuilles)'], ['facultatif', '1/4', 'citron jaune']]",salade de couscous & saumon grillé,"saumon, semoule, concombre, feta, menthe, citron jaune"
Papillote saumon pesto & légumes d'été,30 minutes,"[['1', 'saumon (frais)'], ['120 g', 'haricot vert (surgelé)'], ['100 g', 'tomate cerise'], ['1 càs', 'sauce pesto']]",papillote saumon pesto & légumes d'été,"saumon, haricot vert, tomate cerise, sauce pesto"
"Filet de saumon en croûte, tomates & courgettes",30 minutes,"[['600 g', 'filet de saumon'], ['600 g', 'courgette'], ['400 g', 'tomate cerise'], ['80 g', 'parmesan (râpé)'], ['60 g', 'beurre'], ['40 g', 'chapelure']]","filet de saumon en croûte, tomates & courgettes","filet de saumon, courgette, tomate cerise, parmesan, beurre, chapelure"
"Saumon, boulgour & ratatouille",10 minutes,"[['1', 'saumon (frais)'], ['100 g', 'ratatouille'], ['70 g', 'boulgour']]","saumon, boulgour & ratatouille","saumon, ratatouille, boulgour"


In [128]:
jow_data[jow_data["recettes"].str.contains("asperge")]

,cuisson,ingredients,recettes,only_ings
Œuf à la coque & asperges vertes,7 minutes,"[['1', 'oeuf'], ['100 g', 'asperges vertes (fraîches)']]",oeuf à la coque & asperges vertes,"oeuf, asperges vertes"
Œuf cocotte asperges et parmesan,15 minutes,"[['1', 'oeuf'], ['1 càs', 'crème fraîche épaisse'], ['100 g', 'asperges vertes (fraîches)'], ['15 g', 'parmesan (râpé)']]",oeuf cocotte asperges et parmesan,"oeuf, crème fraîche épaisse, asperges vertes, parmesan"
Risotto aux asperges,plat froid,"[['70 g', 'riz (arborio)'], ['100 g', 'asperges vertes (fraîches)'], ['1/4', 'bouillon de légumes (cube)'], ['1 càs', 'parmesan (râpé)']]",risotto aux asperges,"riz, asperges vertes, bouillon de légumes, parmesan"
Asperges & vinaigrette à l'estragon,7 minutes,"[['250 g', 'asperges vertes (fraîches)'], ['1', 'oeuf'], ['1 càc', 'moutarde'], ['1/4', 'échalote'], ['2 tran.'], ['1 càc'], ['1 g']]",asperges & vinaigrette à l'estragon,"asperges vertes, oeuf, moutarde, échalote"
Saumon & asperges rôtis,27 minutes,"[['1', 'saumon (frais)'], ['250 g', 'asperges vertes (fraîches)'], ['1/8', 'citron jaune']]",saumon & asperges rôtis,"saumon, asperges vertes, citron jaune"
Tarte fine aux asperges,plat froid,"[['1 bou.', 'asperges vertes (fraîches)'], ['1', 'pâte feuilletée'], ['200 g', 'chèvre frais'], ['5 cl', 'crème liquide']]",tarte fine aux asperges,"asperges vertes, pâte feuilletée, chèvre frais, crème liquide"
Pâtes au citron & asperges,12 minutes,"[['100 g', 'pâtes (orecchiette)'], ['1/4', 'citron jaune'], ['20 g', 'parmesan (râpé)'], ['15 g', 'beurre'], ['1/4 bou.', 'asperges vertes (fraîches)']]",pâtes au citron & asperges,"pâtes, citron jaune, parmesan, beurre, asperges vertes"
"Asperges, ricotta & pistaches",plat froid,"[['80 g', 'asperges vertes (fraîches)'], ['20 g', 'pistaches (grillées)'], ['1/4', 'citron jaune'], ['3 càs', 'ricotta'], ['facultatif', '1/4 bou.', 'menthe (feuilles)'], ['1 càs', 'vinaigre balsamique']]","asperges, ricotta & pistaches","asperges vertes, pistaches, citron jaune, ricotta, menthe, vinaigre balsamique"
Asperges & sauce mousseline,6 minutes,"[['200 g', 'asperge blanche (fraîche)'], ['1', 'oeuf'], ['1 càs', 'moutarde'], ['5 cl', 'huile de colza'], ['1 càs', 'vinaigre de cidre']]",asperges & sauce mousseline,"asperge blanche, oeuf, moutarde, huile de colza, vinaigre de cidre"
Salade d’asperges & œuf mollet,22 minutes,"[['1/2 bou.', 'asperges vertes (fraîches)'], ['40 g', 'petits pois (frais)'], ['10 g', 'parmesan (morceaux)'], ['1', 'oeuf'], ['30 g', 'pain (baguette)'], ['1/2', 'ail'], ['1 càs', ""huile d'olive""], ['1/4', 'citron jaune']]",salade d’asperges & oeuf mollet,"asperges vertes, petits pois, parmesan, oeuf, pain, ail, huile d'olive, citron jaune"


In [129]:
jow_data[jow_data["recettes"].str.contains("artichaut")]

,cuisson,ingredients,recettes,only_ings
Salade artichauts & noisettes,plat froid,"[['120 g', 'artichauts marinés'], ['2 poignées', 'salade (roquette)'], ['40 g', 'parmesan (morceaux)'], ['10 g', 'noisette']]",salade artichauts & noisettes,"artichauts marinés, salade, parmesan, noisette"
Artichaut vinaigrette,40 minutes,"[['1', 'artichaut (gros)'], ['1/2 càs', ""huile d'olive""], ['1 càc', 'vinaigre de vin rouge'], ['1 càc', ""moutarde à l'ancienne""]]",artichaut vinaigrette,"artichaut, huile d'olive, vinaigre de vin rouge, moutarde à l'ancienne"
Salade de pâtes & artichauts,10 minutes,"[['100 g', 'pâtes (conchiglioni)'], ['80 g', 'artichauts marinés'], ['50 g', 'olive noire'], ['1 poignée', 'salade (roquette)']]",salade de pâtes & artichauts,"pâtes, artichauts marinés, olive noire, salade"
"Artichauts sautés, parmesan & citron",15 minutes,"[['250 g', 'artichaut (petit)'], ['15 g', 'parmesan (morceaux)'], ['1/4', 'citron jaune'], ['facultatif', '1 pinc.', 'thym (branches)']]","artichauts sautés, parmesan & citron","artichaut, parmesan, citron jaune, thym"
Quiche artichaut & parma,50 minutes,"[['1', 'pâte brisée'], ['1', 'mozzarella (boule)'], ['160 g', 'artichauts marinés'], ['4 tran.', 'jambon cru (prosciutto)'], ['3', 'oeuf'], ['10 cl', 'lait'], ['10 cl', 'crème liquide']]",quiche artichaut & parma,"pâte brisée, mozzarella, artichauts marinés, jambon cru, oeuf, lait, crème liquide"
Pizza chorizo & artichaut,10 minutes,"[['1', 'pâte à pizza'], ['60 g', 'tomate (purée)'], ['1', 'mozzarella (boule)'], ['6 tran.', 'chorizo (tranches)'], ['80 g', 'artichauts marinés'], ['facultatif', '1/5 bou.', 'basilic (frais)']]",pizza chorizo & artichaut,"pâte à pizza, tomate, mozzarella, chorizo, artichauts marinés, basilic"
Artichauts braisés & aïoli au citron confit,30 minutes,"[['2', 'artichaut (petit)'], ['20 g', 'citron confit'], ['1/2 gou.', 'ail'], ['1/4', 'orange'], ['1/4', 'oeuf'], ['5 cl', 'huile de colza'], ['1/2 càc', 'origan (séché)'], ['1/2 càc', 'moutarde']]",artichauts braisés & aïoli au citron confit,"artichaut, citron confit, ail, orange, oeuf, huile de colza, origan, moutarde"
One pot pasta artichauts & tomates séchées,13 minutes,"[['100 g', 'pâtes (trofie)'], ['1/4', 'échalote'], ['50 g', 'tomate séchée'], ['80 g', 'artichauts marinés'], ['20 g'], ['facultatif', '1/10 bou.']]",one pot pasta artichauts & tomates séchées,"pâtes, échalote, tomate séchée, artichauts marinés"


In [130]:
menu_data.loc[41, 'Closest JOW recipe by hand'] = jow_data[jow_data["recettes"]=='saumon & asperges rôtis'].index[0]

Next recipe

In [131]:
menu_data[menu_data.index==42]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand
42,feaa6c7b-9b16-4d51-b71d-75518f16cb08.jpg,Hox Signatures,Hox Cheeseburger,"Spanish beef, cheddar, caramelized onions, fries, salad",Not yet handled


In [132]:
jow_data[jow_data["recettes"].str.contains("cheeseburger")]

,cuisson,ingredients,recettes,only_ings
Classic Cheeseburger,40 minutes,"[['1/2 càc', 'yellow mustard'], ['1 càc', 'ketchup'], ['1/8', 'oignon rouge'], ['150 g', 'pomme de terre'], ['10 g', 'salade (coeur de laitue)'], ['1', 'pain burger'], ['1 tran.', 'cheddar (tranches)'], ['1', 'boeuf (steak haché)']]",classic cheeseburger,"yellow mustard, ketchup, oignon rouge, pomme de terre, salade, pain burger, cheddar, boeuf"
Cheeseburger veggie,4 minutes,"[['1', 'pain burger'], ['1', 'steak végétal'], ['1/2 poignée', 'salade (coeur de laitue)'], ['1 càc', 'yellow mustard'], ['1 càs', 'ketchup'], ['1/8', 'oignon rouge'], ['1 tran.', 'cheddar (tranches)']]",cheeseburger veggie,"pain burger, steak végétal, salade, yellow mustard, ketchup, oignon rouge, cheddar"


In [133]:
jow_data[jow_data["recettes"].str.contains("burger")]

,cuisson,ingredients,recettes,only_ings
Blue Burger,40 minutes,"[['1 càc', ""confits d'oignons""], ['30 g', 'bleu'], ['1', 'boeuf (steak haché)'], ['150 g', 'patate douce'], ['1 poignée', 'salade (roquette)'], ['1', 'pain burger'], ['1/2 càc', ""moutarde à l'ancienne""]]",blue burger,"confits d'oignons, bleu, boeuf, patate douce, salade, pain burger, moutarde à l'ancienne"
French Burger,40 minutes,"[['2 càc', ""confits d'oignons""], ['30 g', 'comté'], ['1', 'boeuf (steak haché)'], ['150 g', 'pomme de terre'], ['10 g', 'salade (coeur de laitue)'], ['1', 'pain burger'], ['1/2 càc', ""moutarde à l'ancienne""]]",french burger,"confits d'oignons, comté, boeuf, pomme de terre, salade, pain burger, moutarde à l'ancienne"
Italian burger,40 minutes,"[['1/2 càs', 'sauce pesto'], ['1/2', 'mozzarella (boule)'], ['1', 'boeuf (steak haché)'], ['150 g', 'pomme de terre'], ['1', 'pain burger'], ['1/4', 'tomate'], ['facultatif', '1 pinc.', 'origan (séché)'], ['1 poignée', 'salade (roquette)']]",italian burger,"sauce pesto, mozzarella, boeuf, pomme de terre, pain burger, tomate, origan, salade"
Salmon burger,40 minutes,"[['1/2 càs', 'crème fraîche épaisse'], ['1/8', 'citron jaune'], ['1 poignée', 'epinard (frais)'], ['1/2', 'avocat'], ['1', 'saumon (frais)'], ['1', 'pain burger'], ['150 g', 'pomme de terre'], ['facultatif', '1/10 bou.', 'ciboulette']]",salmon burger,"crème fraîche épaisse, citron jaune, epinard, avocat, saumon, pain burger, pomme de terre, ciboulette"
Classic Cheeseburger,40 minutes,"[['1/2 càc', 'yellow mustard'], ['1 càc', 'ketchup'], ['1/8', 'oignon rouge'], ['150 g', 'pomme de terre'], ['10 g', 'salade (coeur de laitue)'], ['1', 'pain burger'], ['1 tran.', 'cheddar (tranches)'], ['1', 'boeuf (steak haché)']]",classic cheeseburger,"yellow mustard, ketchup, oignon rouge, pomme de terre, salade, pain burger, cheddar, boeuf"
Super Bacon Burger,40 minutes,"[['1/8', 'oignon rouge'], ['1 tran.', 'cheddar (tranches)'], ['1', 'boeuf (steak haché)'], ['150 g', 'pomme de terre'], ['10 g', 'salade (coeur de laitue)'], ['1', 'pain burger'], ['1 càc', 'moutarde'], ['1 càs', ""sirop d'érable""]]",super bacon burger,"oignon rouge, cheddar, boeuf, pomme de terre, salade, pain burger, moutarde, sirop d'érable"
Tom burger,40 minutes,"[['1 càc', ""confits d'oignons""], ['1/2 càs', 'crème fraîche épaisse'], ['1/4', 'échalote'], ['150 g', 'pomme de terre'], ['1', 'pain burger'], ['30 g', 'tomme'], ['1', 'boeuf (steak haché)'], ['1/10 bou.', 'persil (frais)']]",tom burger,"confits d'oignons, crème fraîche épaisse, échalote, pomme de terre, pain burger, tomme, boeuf, persil"
Brunch Burger,3 minutes,"[['1', 'pain burger'], ['2', 'oeuf'], ['15 g', 'beurre'], ['facultatif', '1/10 bou.', 'ciboulette'], ['1 tran.', 'jambon blanc']]",brunch burger,"pain burger, oeuf, beurre, ciboulette, jambon blanc"
Cheeseburger veggie,4 minutes,"[['1', 'pain burger'], ['1', 'steak végétal'], ['1/2 poignée', 'salade (coeur de laitue)'], ['1 càc', 'yellow mustard'], ['1 càs', 'ketchup'], ['1/8', 'oignon rouge'], ['1 tran.', 'cheddar (tranches)']]",cheeseburger veggie,"pain burger, steak végétal, salade, yellow mustard, ketchup, oignon rouge, cheddar"
Burger raclette,5 minutes,"[['1', 'pain burger (brasserie)'], ['1', 'boeuf (steak haché)'], ['2 càc', ""confits d'oignons""], ['1 càc', ""moutarde à l'ancienne""], ['20 g', 'raclette']]",burger raclette,"pain burger, boeuf, confits d'oignons, moutarde à l'ancienne, raclette"


In [134]:
menu_data.loc[42, 'Closest JOW recipe by hand'] = jow_data[jow_data["recettes"]=='classic cheeseburger'].index[0]

Next recipe

In [135]:
menu_data[menu_data.index==43]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand
43,feaa6c7b-9b16-4d51-b71d-75518f16cb08.jpg,Hox Signatures,Beyond meat veggie burger,"Beyond Meat pattie, charcoal bun, avocado, fries, salad",Not yet handled


In [136]:
menu_data.loc[43, 'Closest JOW recipe by hand'] = jow_data[jow_data["recettes"]=='cheeseburger veggie'].index[0]

Next recipe

In [137]:
menu_data[menu_data.index==44]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand
44,feaa6c7b-9b16-4d51-b71d-75518f16cb08.jpg,Sides,Frites,(v),Not yet handled


In [138]:
jow_data[jow_data["recettes"].str.contains("frite")]

,cuisson,ingredients,recettes,only_ings


In [139]:
jow_data[jow_data["ingredients"].str.contains("frite")]

,cuisson,ingredients,recettes,only_ings


In [140]:
menu_data.loc[44, 'Closest JOW recipe by hand'] = 'no match'

Next recipe

In [141]:
menu_data[menu_data.index==45]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand
45,feaa6c7b-9b16-4d51-b71d-75518f16cb08.jpg,Sides,Frites patate douce,"Parmesan cheese, rosemary, spicy mayo",Not yet handled


In [142]:
menu_data.loc[45, 'Closest JOW recipe by hand'] = 'no match'

Next recipe

In [143]:
menu_data[menu_data.index==46]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand
46,in-bocca-al-lupo_42356_293.jpg,Calzoni,Calzone parigino,"mozzarella, jambon, champignons, œuf, sauce tomates au dessus",Not yet handled


In [144]:
jow_data[jow_data["recettes"].str.contains("calzone")]

,cuisson,ingredients,recettes,only_ings


In [145]:
jow_data[jow_data["recettes"].str.contains("pizza")]

,cuisson,ingredients,recettes,only_ings
Pizza Parma,10 minutes,"[['1', 'pâte à pizza'], ['60 g', 'tomate (purée)'], ['3 tran.', 'jambon cru'], ['1', 'mozzarella (boule)'], ['1 poignée', 'salade (roquette)']]",pizza parma,"pâte à pizza, tomate, jambon cru, mozzarella, salade"
Pizza maison 3 fromages,10 minutes,"[['1', 'pâte à pizza'], ['50 g', 'roquefort'], ['50 g', 'tomate (concentré)'], ['1', 'mozzarella (boule)'], ['100 g', 'chèvre frais']]",pizza maison 3 fromages,"pâte à pizza, roquefort, tomate, mozzarella, chèvre frais"
Pizza maison chèvre miel,10 minutes,"[['1', 'pâte à pizza'], ['2 g', 'romarin (séché)'], ['2 càc', 'miel (liquide)'], ['100 g', 'chèvre frais']]",pizza maison chèvre miel,"pâte à pizza, romarin, miel, chèvre frais"
Pizza verde à la courgette,15 minutes,"[['1', 'pâte à pizza'], ['3 càs', 'sauce pesto'], ['1', 'courgette'], ['1', 'mozzarella (boule)'], ['2 poignées', 'epinard (frais)']]",pizza verde à la courgette,"pâte à pizza, sauce pesto, courgette, mozzarella, epinard"
Pizza verde à la burrata,10 minutes,"[['1', 'pâte à pizza'], ['1', 'burrata'], ['100 g', 'artichauts marinés'], ['1 poignée', 'salade (roquette)'], ['facultatif', '1 càs', 'pignons de pin'], ['2 càs', 'sauce pesto'], ['1 càs', 'olives vertes']]",pizza verde à la burrata,"pâte à pizza, burrata, artichauts marinés, salade, pignons de pin, sauce pesto, olives vertes"
Pizza printanière,10 minutes,"[['1', 'pâte à pizza'], ['140 g', 'petits pois (surgelés)'], ['4 tran.', 'jambon cru (prosciutto)'], ['1', 'oeuf'], ['1 càs', 'sauce pesto'], ['1', 'mozzarella (boule)'], ['facultatif', '2 poignées', 'salade (roquette)']]",pizza printanière,"pâte à pizza, petits pois, jambon cru, oeuf, sauce pesto, mozzarella, salade"
Pizza saumon avocat,8 minutes,"[['1', 'pâte à pizza'], ['1', 'avocat'], ['4 tran.', 'saumon (fumé)'], ['80 g', 'fromage frais'], ['2 poignées', 'salade (roquette)'], ['1/2', 'citron jaune']]",pizza saumon avocat,"pâte à pizza, avocat, saumon, fromage frais, salade, citron jaune"
Pizza poire gorgonzola,10 minutes,"[['1', 'poire'], ['1', 'pâte à pizza'], ['100 g', 'gorgonzola'], ['facultatif', '20 g', 'noisette'], ['2 càs', 'crème fraîche épaisse']]",pizza poire gorgonzola,"poire, pâte à pizza, gorgonzola, noisette, crème fraîche épaisse"
Pizza puttanesca,10 minutes,"[['1', 'pâte à pizza'], ['60 g', 'tomate (purée)'], ['60 g', 'anchois'], ['60 g', 'olive noire'], ['1 càs', 'câpres'], ['1', 'mozzarella (boule)']]",pizza puttanesca,"pâte à pizza, tomate, anchois, olive noire, câpres, mozzarella"
"Pizza pesto, tomates & burrata",10 minutes,"[['1', 'pâte à pizza'], ['60 g', 'sauce pesto'], ['200 g', 'tomate cerise'], ['1', 'burrata'], ['facultatif', '1/5 bou.', 'basilic (frais)']]","pizza pesto, tomates & burrata","pâte à pizza, sauce pesto, tomate cerise, burrata, basilic"


In [146]:
menu_data.loc[46, 'Closest JOW recipe by hand'] = 'no match'

Next recipe

In [147]:
menu_data[menu_data.index==49]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand
49,in-bocca-al-lupo_42356_293.jpg,Pizza Bambini (pour les petits),Prosciutto,"sauce tomate, mozzarella, jambon",Not yet handled


In [148]:
menu_data.loc[49, 'Closest JOW recipe by hand'] = jow_data[jow_data["recettes"]=='pizza parma'].index[0]

Next recipe

In [149]:
menu_data[menu_data.index==50]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand
50,in-bocca-al-lupo_42356_293.jpg,Planches,Salumi misti,assortiment de charcuteries italiennes,Not yet handled


In [150]:
jow_data[jow_data["recettes"].str.contains("charcuterie")]

,cuisson,ingredients,recettes,only_ings
Planche de charcuterie,plat froid,"[['1 tran.', 'jambon blanc'], ['1 tran.', 'jambon cru'], ['2 tran.', 'chorizo (tranches)'], ['2 tran.', 'rosette'], ['20 g', 'rillettes'], ['10 g', 'cornichon']]",planche de charcuterie,"jambon blanc, jambon cru, chorizo, rosette, rillettes, cornichon"


In [151]:
menu_data.loc[50, 'Closest JOW recipe by hand'] = jow_data[jow_data["recettes"]=='planche de charcuterie'].index[0]

Next recipe

In [152]:
menu_data[menu_data.index==51]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand
51,in-bocca-al-lupo_42356_293.jpg,Insalata,Mista,"roquette, mozzarella di bufala, artichauts, jambon de parme, tomates cerises, parmesan",Not yet handled


In [153]:
jow_data[jow_data["recettes"].str.contains("jambon")]

,cuisson,ingredients,recettes,only_ings
Les classiques coquillettes au jambon,7 minutes,"[['100 g', 'pâtes (coquillettes)'], ['1 tran.', 'jambon blanc']]",les classiques coquillettes au jambon,"pâtes, jambon blanc"
Tartine chèvre figue jambon,10 minutes,"[['2', 'figue'], ['2 tran.', 'pain de campagne (tranché)'], ['2 càs', 'chèvre frais'], ['2 tran.', 'jambon cru'], ['1 càc', 'miel (liquide)'], ['10 g', 'noix'], ['1 poignée', 'salade (mélange)']]",tartine chèvre figue jambon,"figue, pain de campagne, chèvre frais, jambon cru, miel, noix, salade"
Gratin de pâtes au jambon,25 minutes,"[['100 g', 'pâtes (macaroni)'], ['40 g', 'fromage râpé'], ['1 tran.', 'jambon blanc'], ['1 càs', 'crème fraîche épaisse']]",gratin de pâtes au jambon,"pâtes, fromage râpé, jambon blanc, crème fraîche épaisse"
Poireaux gratinés au jambon,plat froid,"[['1', 'poireau'], ['1 tran.', 'jambon blanc'], ['10 cl', 'crème liquide'], ['25 g', 'fromage râpé']]",poireaux gratinés au jambon,"poireau, jambon blanc, crème liquide, fromage râpé"
Brochette melon & jambon,plat froid,"[['facultatif', '4', 'pics à brochette'], ['1/4', 'melon'], ['2 tran.', 'jambon cru (prosciutto)'], ['1/10 bou.', 'basilic (frais)'], ['60 g', 'mozzarella (mini)']]",brochette melon & jambon,"melon, jambon cru, basilic, mozzarella"
Melon jambon 2.0,plat froid,"[['1/2', 'melon'], ['80 g', 'ricotta'], ['3 tran.', 'jambon cru'], ['1/10 bou.', 'basilic (frais)']]",melon jambon 2.0,"melon, ricotta, jambon cru, basilic"
Soupe potimarron & jambon,plat froid,"[['1/2', 'potimarron'], ['5 cl', 'crème liquide'], ['1 tran.', 'jambon cru'], ['1/2', 'échalote']]",soupe potimarron & jambon,"potimarron, crème liquide, jambon cru, échalote"
Endives caramélisées au jambon,20 minutes,"[['250 g', 'endives'], ['2 tran.', 'jambon cru'], ['1 càs', 'crème fraîche épaisse'], ['1 càs', 'parmesan (râpé)']]",endives caramélisées au jambon,"endives, jambon cru, crème fraîche épaisse, parmesan"
Pâtes au beaufort & jambon,10 minutes,"[['130 g', 'pâtes (fusilli)'], ['1 tran.', 'jambon blanc'], ['5 cl', 'crème liquide'], ['40 g', 'beaufort']]",pâtes au beaufort & jambon,"pâtes, jambon blanc, crème liquide, beaufort"
Purée maison & jambon,plat froid,"[['1 tran.', 'jambon blanc'], ['250 g', 'pomme de terre'], ['5 cl', 'lait'], ['20 g', 'beurre'], ['1 pinc.']]",purée maison & jambon,"jambon blanc, pomme de terre, lait, beurre"


In [154]:
jow_data[jow_data["recettes"].str.contains("parma")]

,cuisson,ingredients,recettes,only_ings
Pizza Parma,10 minutes,"[['1', 'pâte à pizza'], ['60 g', 'tomate (purée)'], ['3 tran.', 'jambon cru'], ['1', 'mozzarella (boule)'], ['1 poignée', 'salade (roquette)']]",pizza parma,"pâte à pizza, tomate, jambon cru, mozzarella, salade"
Insalata di parma,plat froid,"[['2 poignées', 'salade (mélange)'], ['1/4', 'melon'], ['100 g', 'concombre'], ['1 tran.', 'jambon cru (prosciutto)'], ['facultatif', '1/10 bou.', 'ciboulette']]",insalata di parma,"salade, melon, concombre, jambon cru, ciboulette"
Galette parma,3 minutes,"[['15 g', 'tomate séchée'], ['1', 'galette bretonne'], ['1/2', 'mozzarella (boule)'], ['1/4 càs', 'sauce pesto'], ['1 tran.', 'jambon cru']]",galette parma,"tomate séchée, galette bretonne, mozzarella, sauce pesto, jambon cru"
Bruschetta parma,plat froid,"[['2 tran.', 'pain de campagne (tranché)'], ['50 g', 'mozzarella (boule)'], ['150 g', 'tomate'], ['facultatif', '1/10 bou.', 'basilic (frais)'], ['2 tran.', 'jambon cru']]",bruschetta parma,"pain de campagne, mozzarella, tomate, basilic, jambon cru"
Quiche artichaut & parma,50 minutes,"[['1', 'pâte brisée'], ['1', 'mozzarella (boule)'], ['160 g', 'artichauts marinés'], ['4 tran.', 'jambon cru (prosciutto)'], ['3', 'oeuf'], ['10 cl', 'lait'], ['10 cl', 'crème liquide']]",quiche artichaut & parma,"pâte brisée, mozzarella, artichauts marinés, jambon cru, oeuf, lait, crème liquide"
"Salade nectarine, parma & mozzarella",plat froid,"[['1', 'nectarine'], ['1/2', 'avocat'], ['1 tran.', 'jambon cru'], ['2 poignées', 'salade (roquette)'], ['1 càs'], ['1/2']]","salade nectarine, parma & mozzarella","nectarine, avocat, jambon cru, salade"
Poulet parma & salade,15 minutes,"[['1', 'poulet (pané)'], ['50 g', 'sauce tomate (basilic)'], ['1 càs', 'parmesan (râpé)'], ['40 g', 'mozzarella (à cuire)'], ['2 poignées', 'salade (mélange)'], ['facultatif', '1/10 bou.', 'basilic (frais)']]",poulet parma & salade,"poulet, sauce tomate, parmesan, mozzarella, salade, basilic"
"Salade lentilles, brocolis & parma",10 minutes,"[['130 g', 'lentilles (cuites)'], ['100 g', 'brocoli (frais)'], ['2 tran.', 'jambon cru'], ['1/4', 'échalote'], ['1/2 càc', ""moutarde à l'ancienne""]]","salade lentilles, brocolis & parma","lentilles, brocoli, jambon cru, échalote, moutarde à l'ancienne"
Aubergine parma & salade,35 minutes,"[['1/2', 'aubergine'], ['50 g', 'sauce tomate (basilic)'], ['40 g', 'mozzarella (boule)'], ['2 poignées', 'salade (mélange)'], ['1 càs', 'parmesan (râpé)'], ['facultatif', '1/10 bou.', 'basilic (frais)']]",aubergine parma & salade,"aubergine, sauce tomate, mozzarella, salade, parmesan, basilic"


There is only one salad with ham

In [155]:
tmp = []
tmp.append(jow_data[jow_data["recettes"]=='salade jambon cru, mozza & pistaches'].index[0])
tmp.append(jow_data[jow_data["recettes"]=='insalata di parma'].index[0])

menu_data.loc[51, 'Closest JOW recipe by hand'] = tmp

Next recipe

In [156]:
menu_data[menu_data.index==52]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand
52,in-bocca-al-lupo_42356_293.jpg,Desserts,Tiramisù,NaN,Not yet handled


In [157]:
jow_data[jow_data["recettes"].str.contains("tiramis")]

,cuisson,ingredients,recettes,only_ings
Tiramisu,3 heures,"[['100 g', 'sucre (en poudre)'], ['facultatif', '1 sac.', 'sucre vanillé'], ['3', 'oeuf'], ['2 càs', 'cacao (en poudre)'], ['250 g', 'mascarpone'], ['20', 'biscuit cuiller'], ['2 càc', 'café'], ['facultatif', '1 càs', 'amaretto']]",tiramisu,"sucre, sucre vanillé, oeuf, cacao, mascarpone, biscuit cuiller, café, amaretto"
Roulé façon tiramisu,12 minutes,"[['4', 'oeuf'], ['125 g', 'farine de blé'], ['125 g', 'sucre (en poudre)'], ['1 càc', 'levure chimique'], ['35 g'], ['25 cl'], ['120 g'], ['15 g']]",roulé façon tiramisu,"oeuf, farine de blé, sucre, levure chimique"


In [158]:
menu_data.loc[52, 'Closest JOW recipe by hand'] = jow_data[jow_data["recettes"]=='tiramisu'].index[0]

Next recipe

In [159]:
menu_data[menu_data.index==53]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand
53,in-bocca-al-lupo_42356_293.jpg,Desserts,Panna cotta (avec gousses de vanille) et coulis de fruits de frais,NaN,Not yet handled


In [160]:
jow_data[jow_data["recettes"].str.contains("panna cotta")]

,cuisson,ingredients,recettes,only_ings
Panna cotta à la fraise,3 minutes,"[['80 g', 'fraise'], ['10 cl', 'crème liquide'], ['2 càs', 'sucre (en poudre)'], ['1/4', 'vanille (gousse)'], ['1/2', 'gélatine']]",panna cotta à la fraise,"fraise, crème liquide, sucre, vanille, gélatine"


In [161]:
menu_data.loc[53, 'Closest JOW recipe by hand'] = jow_data[jow_data["recettes"]=='panna cotta à la fraise'].index[0]

Next recipe

In [162]:
menu_data[menu_data.index==55]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand
55,menu_bobby.jpg,Petites assiettes à partager,Gros raviolo,aux champis d'une ferme urbaine du 18è siècle,Not yet handled


In [163]:
jow_data[jow_data["recettes"].str.contains("raviol")]

,cuisson,ingredients,recettes,only_ings
Ravioles sautées,4 minutes,"[['120 g', 'ravioles au fromage'], ['2 poignées', 'epinard (frais)'], ['10 g', 'noisette']]",ravioles sautées,"ravioles au fromage, epinard, noisette"
Bouillon de ravioles du Dauphiné,plat froid,"[['120 g', 'ravioles au fromage'], ['150 g', 'champignons (mélange)'], ['1/2', 'bouillon de volaille (cube)'], ['1/2', 'échalote']]",bouillon de ravioles du dauphiné,"ravioles au fromage, champignons, bouillon de volaille, échalote"
Raviolis à la crème de champignons,14 minutes,"[['125 g', 'ravioli (aux champignons)'], ['1 càs', 'crème fraîche épaisse'], ['100 g', 'champignons de paris (frais)'], ['1/4 gou.', 'ail'], ['facultatif', '1/2 càs', 'cognac'], ['1/2 càs', 'parmesan (râpé)']]",raviolis à la crème de champignons,"ravioli, crème fraîche épaisse, champignons de paris, ail, cognac, parmesan"
Canard & ravioles forestières,15 minutes,"[['1/2', 'canard (magret)'], ['5 ml', 'vin blanc'], ['10 g', 'parmesan (morceaux)'], ['125 g', 'ravioli (aux champignons)']]",canard & ravioles forestières,"canard, vin blanc, parmesan, ravioli"
"Ravioli, tomates séchées & câpres",10 minutes,"[['125 g', 'ravioli (ricotta-épinards)'], ['40 g', 'tomate séchée'], ['1 poignée', 'epinard (frais)'], ['1/2 càs', 'câpres'], ['50 g', 'artichauts marinés'], ['10 g', 'parmesan (morceaux)']]","ravioli, tomates séchées & câpres","ravioli, tomate séchée, epinard, câpres, artichauts marinés, parmesan"
Ravioles fromage & poire,5 minutes,"[['125 g', 'ravioli (au gorgonzola)'], ['10 cl', 'crème liquide'], ['5 g', 'noix'], ['1/2 càs', 'parmesan (râpé)'], ['1/3', 'poire']]",ravioles fromage & poire,"ravioli, crème liquide, noix, parmesan, poire"
Gratin de ravioles au saumon,30 minutes,"[['120 g', 'ravioles au fromage'], ['1', 'poireau'], ['1 tran.', 'saumon (fumé)'], ['10 cl', 'crème liquide'], ['20 g', 'comté']]",gratin de ravioles au saumon,"ravioles au fromage, poireau, saumon, crème liquide, comté"
Salade de ravioli ricotta et épinards,2 minutes,"[['125 g', 'ravioli (ricotta-épinards)'], ['2 poignées', 'salade (mélange)'], ['15 g', 'comté'], ['10 g', 'pignons de pin'], ['1 càc', 'vinaigre balsamique']]",salade de ravioli ricotta et épinards,"ravioli, salade, comté, pignons de pin, vinaigre balsamique"
Ravioles à la toscane,7 minutes,"[['125 g', 'ravioli (ricotta-épinards)'], ['50 g', 'tomate séchée'], ['1 càs', 'crème fraîche épaisse'], ['2 poignées', 'epinard (frais)'], ['1 càs', 'parmesan (râpé)'], ['1/4 gou.', 'ail']]",ravioles à la toscane,"ravioli, tomate séchée, crème fraîche épaisse, epinard, parmesan, ail"
Gratin de ravioles à la courgette,25 minutes,"[['150 g', 'ravioles au fromage'], ['90 g', 'crème fraîche épaisse'], ['25 g', 'comté (râpé)'], ['100 g', 'courgette']]",gratin de ravioles à la courgette,"ravioles au fromage, crème fraîche épaisse, comté, courgette"


In [164]:
tmp = []
tmp.append(jow_data[jow_data["recettes"]=='raviolis à la crème de champignons'].index[0])
tmp.append(jow_data[jow_data["recettes"]=='raviolis aux champignons, chèvre frais & noisette'].index[0])

menu_data.loc[55, 'Closest JOW recipe by hand'] = tmp

Next recipe

In [165]:
menu_data[menu_data.index==57]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand
57,menu_bobby.jpg,Pizz',Quattro,"Mozza fior di latte et scamorza IDF, gorgonzola, parmigiano, basilico",Not yet handled


In [166]:
jow_data[jow_data["recettes"].str.contains("pizza")]

,cuisson,ingredients,recettes,only_ings
Pizza Parma,10 minutes,"[['1', 'pâte à pizza'], ['60 g', 'tomate (purée)'], ['3 tran.', 'jambon cru'], ['1', 'mozzarella (boule)'], ['1 poignée', 'salade (roquette)']]",pizza parma,"pâte à pizza, tomate, jambon cru, mozzarella, salade"
Pizza maison 3 fromages,10 minutes,"[['1', 'pâte à pizza'], ['50 g', 'roquefort'], ['50 g', 'tomate (concentré)'], ['1', 'mozzarella (boule)'], ['100 g', 'chèvre frais']]",pizza maison 3 fromages,"pâte à pizza, roquefort, tomate, mozzarella, chèvre frais"
Pizza maison chèvre miel,10 minutes,"[['1', 'pâte à pizza'], ['2 g', 'romarin (séché)'], ['2 càc', 'miel (liquide)'], ['100 g', 'chèvre frais']]",pizza maison chèvre miel,"pâte à pizza, romarin, miel, chèvre frais"
Pizza verde à la courgette,15 minutes,"[['1', 'pâte à pizza'], ['3 càs', 'sauce pesto'], ['1', 'courgette'], ['1', 'mozzarella (boule)'], ['2 poignées', 'epinard (frais)']]",pizza verde à la courgette,"pâte à pizza, sauce pesto, courgette, mozzarella, epinard"
Pizza verde à la burrata,10 minutes,"[['1', 'pâte à pizza'], ['1', 'burrata'], ['100 g', 'artichauts marinés'], ['1 poignée', 'salade (roquette)'], ['facultatif', '1 càs', 'pignons de pin'], ['2 càs', 'sauce pesto'], ['1 càs', 'olives vertes']]",pizza verde à la burrata,"pâte à pizza, burrata, artichauts marinés, salade, pignons de pin, sauce pesto, olives vertes"
Pizza printanière,10 minutes,"[['1', 'pâte à pizza'], ['140 g', 'petits pois (surgelés)'], ['4 tran.', 'jambon cru (prosciutto)'], ['1', 'oeuf'], ['1 càs', 'sauce pesto'], ['1', 'mozzarella (boule)'], ['facultatif', '2 poignées', 'salade (roquette)']]",pizza printanière,"pâte à pizza, petits pois, jambon cru, oeuf, sauce pesto, mozzarella, salade"
Pizza saumon avocat,8 minutes,"[['1', 'pâte à pizza'], ['1', 'avocat'], ['4 tran.', 'saumon (fumé)'], ['80 g', 'fromage frais'], ['2 poignées', 'salade (roquette)'], ['1/2', 'citron jaune']]",pizza saumon avocat,"pâte à pizza, avocat, saumon, fromage frais, salade, citron jaune"
Pizza poire gorgonzola,10 minutes,"[['1', 'poire'], ['1', 'pâte à pizza'], ['100 g', 'gorgonzola'], ['facultatif', '20 g', 'noisette'], ['2 càs', 'crème fraîche épaisse']]",pizza poire gorgonzola,"poire, pâte à pizza, gorgonzola, noisette, crème fraîche épaisse"
Pizza puttanesca,10 minutes,"[['1', 'pâte à pizza'], ['60 g', 'tomate (purée)'], ['60 g', 'anchois'], ['60 g', 'olive noire'], ['1 càs', 'câpres'], ['1', 'mozzarella (boule)']]",pizza puttanesca,"pâte à pizza, tomate, anchois, olive noire, câpres, mozzarella"
"Pizza pesto, tomates & burrata",10 minutes,"[['1', 'pâte à pizza'], ['60 g', 'sauce pesto'], ['200 g', 'tomate cerise'], ['1', 'burrata'], ['facultatif', '1/5 bou.', 'basilic (frais)']]","pizza pesto, tomates & burrata","pâte à pizza, sauce pesto, tomate cerise, burrata, basilic"


In [167]:
menu_data.loc[57, 'Closest JOW recipe by hand'] = jow_data[jow_data["recettes"]=='pizza maison 3 fromages'].index[0]

Next recipe

In [168]:
menu_data[menu_data.index==58]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand
58,menu_bobby.jpg,Pizz',Zucchine,"Crème de courgette, mozza fior di latte IDF, provolone del monaco, chips de courgette, basilico",Not yet handled


In [169]:
menu_data.loc[58, 'Closest JOW recipe by hand'] = jow_data[jow_data["recettes"]=='pizza verde à la courgette'].index[0]

Next recipe

In [170]:
menu_data[menu_data.index==59]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand
59,menu_bobby.jpg,Pasta Fresca,Limone,"Trenette, sauce au citron, piment (ou non si vous aimez pas ), basilico",Not yet handled


In [171]:
jow_data[jow_data["recettes"].str.contains("pâte")]

,cuisson,ingredients,recettes,only_ings
Pâtes aux aubergines & parmesan,10 minutes,"[['100 g', 'pâtes (linguine)'], ['1/4', 'aubergine'], ['2 tran.', 'jambon cru'], ['20 g', 'parmesan (morceaux)']]",pâtes aux aubergines & parmesan,"pâtes, aubergine, jambon cru, parmesan"
Gratin de pâtes au jambon,25 minutes,"[['100 g', 'pâtes (macaroni)'], ['40 g', 'fromage râpé'], ['1 tran.', 'jambon blanc'], ['1 càs', 'crème fraîche épaisse']]",gratin de pâtes au jambon,"pâtes, fromage râpé, jambon blanc, crème fraîche épaisse"
Pâtes à la Sicilienne,9 minutes,"[['100 g', 'pâtes (conchiglioni)'], ['1 càs', 'câpres'], ['100 g', 'tomate cerise'], ['90 g', 'thon (conserve)'], ['1 càc']]",pâtes à la sicilienne,"pâtes, câpres, tomate cerise, thon"
Salade de pâtes printanière,10 minutes,"[['2 tran.', 'jambon cru'], ['80 g', 'pâtes (pipe rigate)'], ['1/2', 'salade (sucrine)'], ['70 g', 'petits pois (frais)'], ['1/4', 'citron jaune'], ['15 g', 'parmesan (morceaux)']]",salade de pâtes printanière,"jambon cru, pâtes, salade, petits pois, citron jaune, parmesan"
Pâtes au pesto de coriandre,8 minutes,"[['120 g', 'pâtes (pipe rigate)'], ['1/5 bou.', 'coriandre (frais)'], ['15 g', 'amandes (entières)'], ['1/4 gou.', 'ail'], ['20 g', 'parmesan (râpé)']]",pâtes au pesto de coriandre,"pâtes, coriandre, amandes, ail, parmesan"
Pâte à tartiner aux speculoos,plat froid,"[['200 g', 'speculoos'], ['1 càc', 'cannelle (en poudre)'], ['1 càc', 'miel (liquide)'], ['300 g', 'lait concentré sucré']]",pâte à tartiner aux speculoos,"speculoos, cannelle, miel, lait concentré sucré"
Pâte à tartiner au chocolat,plat froid,"[['50 g', 'noisette'], ['50 g', 'chocolat à cuire'], ['7 cl', 'crème liquide'], ['8 g', 'sucre (en poudre)'], ['5 ml', 'huile de tournesol']]",pâte à tartiner au chocolat,"noisette, chocolat à cuire, crème liquide, sucre, huile de tournesol"
Pâtes au brocoli,20 minutes,"[['100 g', 'brocoli (frais)'], ['100 g', 'pâtes (orecchiette)'], ['2', 'anchois'], ['1/4 gou.', 'ail'], ['10 g', 'parmesan (morceaux)']]",pâtes au brocoli,"brocoli, pâtes, anchois, ail, parmesan"
Pâtes au beaufort & jambon,10 minutes,"[['130 g', 'pâtes (fusilli)'], ['1 tran.', 'jambon blanc'], ['5 cl', 'crème liquide'], ['40 g', 'beaufort']]",pâtes au beaufort & jambon,"pâtes, jambon blanc, crème liquide, beaufort"
"Pâtes thon, citron & ricotta",12 minutes,"[['100 g', 'pâtes (linguine)'], ['1 càs', 'ricotta'], ['50 g', 'thon au citron'], ['1/4', 'citron jaune'], ['1 càs', 'crème liquide']]","pâtes thon, citron & ricotta","pâtes, ricotta, thon au citron, citron jaune, crème liquide"


In [172]:
jow_data[jow_data["recettes"].str.contains("spaghetti")]

,cuisson,ingredients,recettes,only_ings
Spaghetti ail & crevette,10 minutes,"[['100 g', 'pâtes (spaghetti)'], ['100 g', 'crevette (cuite)'], ['1/2 gou.', 'ail'], ['1 pinc.', ""piment d'espelette""], ['1/10 bou.', 'persil (frais)']]",spaghetti ail & crevette,"pâtes, crevette, ail, piment d'espelette, persil"
Spaghetti alla puttanesca,plat froid,"[['100 g', 'pâtes (spaghetti)'], ['100 g', 'tomate (chair)'], ['20 g', 'olive noire'], ['1/2 càs', 'câpres']]",spaghetti alla puttanesca,"pâtes, tomate, olive noire, câpres"
Spaghetti endives & prosciutto,10 minutes,"[['100 g', 'pâtes (spaghetti)'], ['100 g', 'endives'], ['1 tran.', 'jambon cru (prosciutto)'], ['1/2 càs', 'parmesan (râpé)'], ['1 càc', 'sucre (en poudre)']]",spaghetti endives & prosciutto,"pâtes, endives, jambon cru, parmesan, sucre"
Spaghetti alla bolognese express,plat froid,"[['1/4', 'oignon jaune'], ['1/2', 'carotte (frais)'], ['100 g', 'tomate (purée)'], ['1/4 gou.', 'ail']]",spaghetti alla bolognese express,"oignon jaune, carotte, tomate, ail"
Cherry tomatoes spaghetti,10 minutes,"[['120 g', 'tomate cerise'], ['100 g', 'pâtes (spaghetti)'], ['15 g', 'parmesan (morceaux)'], ['1/10 bou.', 'basilic (frais)'], ['1/4 gou.', 'ail']]",cherry tomatoes spaghetti,"tomate cerise, pâtes, parmesan, basilic, ail"
Spaghettis nordiques,13 minutes,"[['150 g', 'pâtes (spaghetti)'], ['1 càs', 'mascarpone'], ['1/4', 'citron jaune'], ['facultatif', '1/10 bou.', 'aneth (frais)'], ['10 g', 'oeufs de poisson']]",spaghettis nordiques,"pâtes, mascarpone, citron jaune, aneth, oeufs de poisson"
Spaghetti bolognese à la ratatouille,10 minutes,"[['150 g', 'ratatouille'], ['100 g', 'boeuf (haché)'], ['1 pinc.', 'origan (séché)'], ['100 g', 'pâtes (spaghetti)'], ['1/2 càs']]",spaghetti bolognese à la ratatouille,"ratatouille, boeuf, origan, pâtes"


In [173]:
jow_data[jow_data["recettes"].str.contains("tagliatelle")]

,cuisson,ingredients,recettes,only_ings
Tagliatelles crevettes & chorizo,10 minutes,"[['70 g', 'pâtes (tagliatelles)'], ['80 g', 'crevette (cuite)'], ['1 càs', 'crème fraîche épaisse'], ['25 g', 'chorizo (entier)'], ['60 g', 'petits pois (surgelés)']]",tagliatelles crevettes & chorizo,"pâtes, crevette, crème fraîche épaisse, chorizo, petits pois"
Tagliatelles poulet & champignons,10 minutes,"[['1', 'poulet (escalope)'], ['100 g', 'champignons de paris (frais)'], ['70 g', 'pâtes (tagliatelles)'], ['1 càs', 'crème fraîche épaisse'], ['1/4', 'oignon jaune']]",tagliatelles poulet & champignons,"poulet, champignons de paris, pâtes, crème fraîche épaisse, oignon jaune"
Tagliatelles crémeuses & saumon,13 minutes,"[['100 g', 'pâtes fraîches'], ['1', 'saumon (frais)'], ['50 g', 'fromage ail & fines herbes'], ['1/4', 'citron jaune'], ['facultatif', '1 càs']]",tagliatelles crémeuses & saumon,"pâtes fraîches, saumon, fromage ail & fines herbes, citron jaune"
Tagliatelles saumon épinards,10 minutes,"[['100 g', 'pâtes (tagliatelles)'], ['1 tran.', 'saumon (fumé)'], ['1 poignée', 'epinard (frais)'], ['1 càs', 'crème fraîche épaisse'], ['1/4', 'citron jaune']]",tagliatelles saumon épinards,"pâtes, saumon, epinard, crème fraîche épaisse, citron jaune"


In [174]:
jow_data[jow_data["recettes"].str.contains("pasta")]

,cuisson,ingredients,recettes,only_ings
Pasta alla carbonara,9 minutes,"[['100 g', 'pâtes (spaghetti)'], ['3 tran.', 'pancetta'], ['1', 'oeuf'], ['40 g', 'parmesan (morceaux)']]",pasta alla carbonara,"pâtes, pancetta, oeuf, parmesan"
Pasta Alfredo,12 minutes,"[['100 g', 'pâtes (tagliatelles)'], ['1', 'poulet (escalope)'], ['10 cl', 'crème liquide'], ['1/2 gou.', 'ail'], ['1 càs', 'parmesan (râpé)']]",pasta alfredo,"pâtes, poulet, crème liquide, ail, parmesan"
Creamy avocado pasta,8 minutes,"[['120 g', 'pâtes (linguine)'], ['1/4', 'avocat'], ['1/5 bou.', 'basilic (frais)'], ['10 g', 'amandes (effilées)'], ['15 g', 'parmesan (morceaux)']]",creamy avocado pasta,"pâtes, avocat, basilic, amandes, parmesan"
Pasta petits pois lardons,10 minutes,"[['100 g', 'pâtes (pipe rigate)'], ['100 g', 'petits pois (frais)'], ['75 g', 'lardons'], ['2 cl', 'crème liquide'], ['2 càs'], ['facultatif', '1/50 bou.']]",pasta petits pois lardons,"pâtes, petits pois, lardons, crème liquide"
Pasta Rocchini,15 minutes,"[['100 g', 'pâtes (trofie)'], ['2', 'saucisse (chipolata)'], ['1 poignée', 'salade (roquette)'], ['1 càs', 'parmesan (râpé)'], ['1/2', 'oignon rouge'], ['1/2', 'fenouil']]",pasta rocchini,"pâtes, saucisse, salade, parmesan, oignon rouge, fenouil"
Pasta poulet citron,10 minutes,"[['1', 'poulet (escalope)'], ['100 g', 'pâtes (spaghetti)'], ['8 cl', 'crème liquide'], ['1 poignée', 'epinard (frais)'], ['1 càs', 'parmesan (râpé)'], ['1/2', 'citron jaune']]",pasta poulet citron,"poulet, pâtes, crème liquide, epinard, parmesan, citron jaune"
Creamy Eggs & Prosciutto pasta,8 minutes,"[['100 g', 'pâtes (pappardelle)'], ['2 tran.', 'jambon cru (prosciutto)'], ['5 cl', 'crème liquide'], ['1/2 càs', 'ricotta'], ['1', 'oeuf'], ['facultatif', '1/10 bou.', 'basilic (frais)']]",creamy eggs & prosciutto pasta,"pâtes, jambon cru, crème liquide, ricotta, oeuf, basilic"
Greek pasta salad,8 minutes,"[['70 g', 'pâtes (fusilli)'], ['1', 'tomate'], ['15 g', 'olive noire'], ['130 g', 'concombre'], ['30 g', 'feta'], ['facultatif', '1/8', 'oignon rouge'], ['1 pinc.', 'origan (séché)']]",greek pasta salad,"pâtes, tomate, olive noire, concombre, feta, oignon rouge, origan"
Pasta pistacchio e olive,plat froid,"[['15 g', 'pistaches (grillées)'], ['100 g', 'pâtes (pipe rigate)'], ['15 g', 'olive noire'], ['1/4 gou.', 'ail']]",pasta pistacchio e olive,"pistaches, pâtes, olive noire, ail"
Pasta al ragu d'anatra,plat froid,"[['100 g', 'pâtes (pappardelle)'], ['1/2', 'canard (confit)'], ['70 g', 'tomate pelée'], ['1/2', 'échalote']]",pasta al ragu d'anatra,"pâtes, canard, tomate pelée, échalote"


In [175]:
menu_data.loc[59, 'Closest JOW recipe by hand'] = jow_data[jow_data["recettes"]=='pasta al limone'].index[0]

Next recipe

In [176]:
menu_data[menu_data.index==60]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand
60,menu_bobby.jpg,Pasta Fresca,Norma,"Tagliatelle, sauce tomate et aubergines frites, ricotta salée fumée",Not yet handled


In [177]:
menu_data.loc[60, 'Closest JOW recipe by hand'] = jow_data[jow_data["recettes"]=='pasta alla norma'].index[0]

Next recipe

In [178]:
menu_data[menu_data.index==61]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand
61,menu_bobby.jpg,Pasta Fresca,Carbobby,"Spaghetti, crème d'œuf bio et de pecorino, guanciale, poivre",Not yet handled


In [179]:
jow_data[jow_data["recettes"].str.contains("carbonara")]

,cuisson,ingredients,recettes,only_ings
Pasta alla carbonara,9 minutes,"[['100 g', 'pâtes (spaghetti)'], ['3 tran.', 'pancetta'], ['1', 'oeuf'], ['40 g', 'parmesan (morceaux)']]",pasta alla carbonara,"pâtes, pancetta, oeuf, parmesan"


In [180]:
menu_data.loc[61, 'Closest JOW recipe by hand'] = jow_data[jow_data["recettes"]=='pasta alla carbonara'].index[0]

Next recipe

In [181]:
menu_data[menu_data.index==62]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand
62,menu_bobby.jpg,Dolci,Affogato al caffè,Boule de glace à la vanille + shot d'espresso,Not yet handled


In [182]:
jow_data[jow_data["recettes"].str.contains("glace")]

,cuisson,ingredients,recettes,only_ings
"Ananas rôti, glace coco",9 minutes,"[['1/4', 'ananas (entier)'], ['1/4', 'citron vert'], ['20 g', 'sucre (en poudre)'], ['20 g', 'beurre'], ['60 g', 'glace coco']]","ananas rôti, glace coco","ananas, citron vert, sucre, beurre, glace coco"
Banane caramélisée miel & glace vanille,7 minutes,"[['1', 'banane'], ['1 càs', 'miel (liquide)'], ['15 g', 'noix'], ['40 g', 'crème glacée vanille'], ['10 g']]",banane caramélisée miel & glace vanille,"banane, miel, noix, crème glacée vanille"
Gaufres au sucre glace,3 minutes,"[['50 g', 'farine de blé'], ['20 g', 'sucre (en poudre)'], ['20 g', 'beurre'], ['6 cl', 'lait'], ['1', 'oeuf'], ['2 g', 'levure chimique'], ['1 càs', 'sucre glace']]",gaufres au sucre glace,"farine de blé, sucre, beurre, lait, oeuf, levure chimique, sucre glace"
"Glace au yaourt, pêche & granola",3 heures,"[['1', 'yaourt nature'], ['1', 'pêche'], ['30 g', 'granola']]","glace au yaourt, pêche & granola","yaourt nature, pêche, granola"
Nectarine rôtie & glace vanille,4 minutes,"[['1 boule', 'crème glacée vanille'], ['1', 'nectarine'], ['15 g', 'amandes (effilées)'], ['20 g', 'beurre'], ['1 càc', 'sucre de canne']]",nectarine rôtie & glace vanille,"crème glacée vanille, nectarine, amandes, beurre, sucre de canne"


In [183]:
menu_data.loc[62, 'Closest JOW recipe by hand'] = 'no match'

Next recipe

In [184]:
menu_data[menu_data.index==63]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand
63,menu_bobby.jpg,Dolci,Boozy tiramisù,Tiramisù classique + Amaretto (liqueur d'amande),Not yet handled


In [185]:
menu_data.loc[63, 'Closest JOW recipe by hand'] = jow_data[jow_data["recettes"]=='tiramisu'].index[0]

### To be continued with more recipes. Now let's save the file

In [186]:
menu_data[menu_data['Closest JOW recipe by hand']!='Not yet handled'].to_excel(data_path+"data/Tools/MenuToJowRecipes_v1.xlsx")

In [187]:
print("Number of menu recipes with no found match in Jow = ")
menu_data[menu_data['Closest JOW recipe by hand']=='no match']['Closest JOW recipe by hand'].count()

Number of menu recipes with no found match in Jow = 


18

In [188]:
print("Number of menu recipes that have not yet been handled = ")
menu_data[menu_data['Closest JOW recipe by hand']=='Not yet handled']['Closest JOW recipe by hand'].count()

Number of menu recipes that have not yet been handled = 


80

In [189]:
print("Number of menu recipes with found matches in Jow = ")
menu_data[(menu_data['Closest JOW recipe by hand']!='Not yet handled') & (menu_data['Closest JOW recipe by hand']!='no match')]['Closest JOW recipe by hand'].count()

Number of menu recipes with found matches in Jow = 


30

## Repeat the same job for the database of 7tomorrow   
### Let's first import the database    
We import only data corresponding to recipes whose all ingredients can be matched with Agribalyse ingredients.

In [190]:
seven_data = pd.read_excel(data_path+"data/Recipes/recettes_completes_7tomorrow.xlsx", header = [0])
seven_data.head()

,Unnamed: 0,Unnamed: 0.1,title_raw,id,url,unit,quantity_raw,ingredient_raw,weight_per_ingr,ingredient_clean_comma,matched_ingredient,food_clean_comma,est_impact,title_clean,source_website
0,0,36,natural peanut butter chocolate bon bons,0006ca31f4,http://www.food.com/recipe/natural-peanut-butter-chocolate-bon-bons-449265,cup,12,"cocoa, dry powder, unsweetened",1032.0,"cocoa , dry powder , unsweetened","cocoa powder , sugar , powder , instant","cocoa powder , sugar , powder , instant",933.0,natural peanut butter chocolate bon bons,http://www.food.com
1,1,37,natural peanut butter chocolate bon bons,0006ca31f4,http://www.food.com/recipe/natural-peanut-butter-chocolate-bon-bons-449265,cup,12,honey,4068.0,honey,honey,honey,38.0,natural peanut butter chocolate bon bons,http://www.food.com
2,2,38,natural peanut butter chocolate bon bons,0006ca31f4,http://www.food.com/recipe/natural-peanut-butter-chocolate-bon-bons-449265,cup,1,"peanut butter, smooth style, without salt",256.0,"peanut butter , style , salt",peanut butter peanut paste,peanut butter peanut paste,316.7,natural peanut butter chocolate bon bons,http://www.food.com
3,3,170,cilantro-mustard mayo (dip or sauce),00185a60a8,http://www.food.com/recipe/cilantro-mustard-mayo-dip-or-sauce-136524,cup,12,"mustard, prepared, yellow",2880.0,"mustard , , yellow",mustard,mustard,120.0,cilantro mustard mayo,http://www.food.com
4,4,171,cilantro-mustard mayo (dip or sauce),00185a60a8,http://www.food.com/recipe/cilantro-mustard-mayo-dip-or-sauce-136524,cup,12,"salad dressing, mayonnaise, regular",2649.6,"salad dressing , mayonnaise ,","salad dressing ,","salad dressing ,",120.0,cilantro mustard mayo,http://www.food.com


In [191]:
print('Number of recipes = ', seven_data['title_raw'].unique().shape[0])

Number of recipes =  7418


In [192]:
seven_data['matched_ingredient'].equals(seven_data['food_clean_comma'])

True

-> the columns 'matched_ingredient' and 'food_clean_comma' are equal.

In [193]:
seven_data[seven_data['title_raw']!=seven_data['title_clean']][['title_raw', 'title_clean']]

,title_raw,title_clean
3,cilantro-mustard mayo (dip or sauce),cilantro mustard mayo
4,cilantro-mustard mayo (dip or sauce),cilantro mustard mayo
5,cilantro-mustard mayo (dip or sauce),cilantro mustard mayo
6,cilantro-mustard mayo (dip or sauce),cilantro mustard mayo
7,cilantro-mustard mayo (dip or sauce),cilantro mustard mayo
...,...,...
35283,maple-cream-cinnamon smoothie,maple cream cinnamon smoothie
35284,maple-cream-cinnamon smoothie,maple cream cinnamon smoothie
35285,maple-cream-cinnamon smoothie,maple cream cinnamon smoothie
35286,maple-cream-cinnamon smoothie,maple cream cinnamon smoothie


Let's define another version of seven_data that's more convenient for doing matching (I've realized it afterwards, so it's used only at the end of this notebook)

In [194]:
seven_data2 = seven_data.groupby('title_raw')['ingredient_raw'].agg(list).reset_index()
seven_data2.head()

,title_raw,ingredient_raw
0,""" child's play"" sour pops","[lime juice, raw, lemon juice, raw]"
1,""" fried egg sundaes""","[ice creams, vanilla, cream, whipped, cream topping, pressurized, spices, pumpkin pie spice]"
2,""" world's best ""( and easiest ) teriyaki chicken wings","[chicken, broilers or fryers, wing, meat and skin, raw, soy sauce made from soy (tamari), sugars, brown]"
3,"""apple crisp"" peanut butter snack bites","[peanut butter, smooth style, without salt, honey, oats, nuts, walnuts, english, apples, raw, with skin, spices, cinnamon, ground]"
4,"""berry good"" smoothie","[apple juice, canned or bottled, unsweetened, without added ascorbic acid, strawberries, raw, raspberries, raw, blackberries, raw, blueberries, raw]"


In [195]:
seven_data2.loc[0, 'ingredient_raw']

['lime juice, raw', 'lemon juice, raw']

In [196]:
seven_data2['ingredient_raw'] = seven_data2['ingredient_raw'].apply(lambda x: ", ".join([str(s) for s in list(set(x))]))
seven_data2.head()

,title_raw,ingredient_raw
0,""" child's play"" sour pops","lemon juice, raw, lime juice, raw"
1,""" fried egg sundaes""","ice creams, vanilla, spices, pumpkin pie spice, cream, whipped, cream topping, pressurized"
2,""" world's best ""( and easiest ) teriyaki chicken wings","chicken, broilers or fryers, wing, meat and skin, raw, soy sauce made from soy (tamari), sugars, brown"
3,"""apple crisp"" peanut butter snack bites","peanut butter, smooth style, without salt, honey, oats, apples, raw, with skin, nuts, walnuts, english, spices, cinnamon, ground"
4,"""berry good"" smoothie","apple juice, canned or bottled, unsweetened, without added ascorbic acid, raspberries, raw, blackberries, raw, strawberries, raw, blueberries, raw"


### Let's now proceed with the matching (long and boring but must be done ...)

In [197]:
menu_data['Closest 7t recipe by hand'] = 'Not yet handled'
menu_data.head(12)

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand,Closest 7t recipe by hand
0,burgart_205074_c70.jpg,Burgers,Superfin,"Bœuf (VBF Metzger), Comté AOC, Iceberg, Bacon, Compotée d'oignons, Sauce Burgart",Not yet handled,Not yet handled
1,burgart_205074_c70.jpg,Burgers,Bon & Beau,"Bœuf (VBF Metzger), Cheddar, Sucrine, Bacon, Tomate Confite, Sauce poivre maison",Not yet handled,Not yet handled
2,burgart_205074_c70.jpg,Burgers,Dandy,"Bœuf (VBF Metzger), Cheddar fumée, Mâche nantaise, Bresaola, Aubergine, Oignon rouge, Pickles, Sauce cocktail maison",Not yet handled,Not yet handled
3,burgart_205074_c70.jpg,Burgers,Green Chicken,"Filet de poulet, Tomme de Savoie IGP, Roquette, Mortadelle, Parmesan, Sauce verte maison",Not yet handled,Not yet handled
4,burgart_205074_c70.jpg,Poke Bowls,Black bowl,"Thon, Riz noir, Avocat, Oignon rouge, Cébette, Concombre, Sésame, Coriandre, Grenade, Sauce douce",Not yet handled,Not yet handled
5,burgart_205074_c70.jpg,Salades,Crispy Caesar,"Croquettes de poulet, Parmesan, Champignons bruns, Salade Romaine, Ciboulette",Not yet handled,Not yet handled
6,burgart_205074_c70.jpg,Desserts,Cheesecake maison,NaN,Not yet handled,Not yet handled
7,burgart_205074_c70.jpg,Desserts,Mousse au chocolat maison,NaN,Mousse au chocolat maison,Not yet handled
8,burgart_205074_c70.jpg,Desserts,Fondant au chocolat artisan boulanger,NaN,Fondant au chocolat,Not yet handled
9,burgart_205074_c70.jpg,Desserts,Financier artisan boulanger,NaN,"[Financier cerises noires & pistache, Financier à la framboise]",Not yet handled


Let's condiser one menu recipe

In [198]:
menu_data[menu_data.index==48]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand,Closest 7t recipe by hand
48,in-bocca-al-lupo_42356_293.jpg,Pizza Bambini (pour les petits),Margherita,"sauce tomate, mozzarella",Pizza Margherita,Not yet handled


In [199]:
seven_data["title_raw"][seven_data["title_raw"].str.contains("pizza")].values.tolist()

['buffalo style chicken pizza recipe',
 'buffalo style chicken pizza recipe',
 'buffalo style chicken pizza recipe',
 'buffalo style chicken pizza recipe',
 'buffalo style chicken pizza recipe',
 'buffalo style chicken pizza recipe',
 'buffalo style chicken pizza recipe',
 "papa john's garlic butter for pizza dipping",
 "papa john's garlic butter for pizza dipping",
 'homemade pizza sauce',
 'homemade pizza sauce',
 'homemade pizza sauce',
 'homemade pizza sauce',
 'homemade pizza sauce',
 'homemade pizza sauce',
 'spicy adobo for pizza',
 'spicy adobo for pizza',
 'spicy adobo for pizza',
 'spicy adobo for pizza',
 '2-ingredient pizza dough',
 'pizza spice blend',
 'pizza spice blend',
 'pizza spice blend',
 'pizza spice blend',
 'pizza spice blend',
 'pizza spice blend',
 'pizza spices',
 'pizza spices',
 'pizza spices',
 'pizza spices',
 'pizza popcorn',
 'pizza popcorn',
 'pizza popcorn',
 'pizza popcorn',
 'pizza popcorn',
 'pizza popcorn',
 'pizza popcorn',
 'simple pizza sauce',

In [200]:
seven_data["title_raw"][seven_data["title_raw"].str.contains("margherita")].values.tolist()

[]

In [201]:
seven_data["ingredient_clean_comma"][seven_data["ingredient_clean_comma"].str.contains("pizza")].values.tolist()

[]

In [202]:
seven_data[seven_data['title_raw'].isin(['5 minute pizza base. no yeast!', 'pizza cheese', '2 ingredient pizza crust (the slow roasted italian)'])]

,Unnamed: 0,Unnamed: 0.1,title_raw,id,url,unit,quantity_raw,ingredient_raw,weight_per_ingr,ingredient_clean_comma,matched_ingredient,food_clean_comma,est_impact,title_clean,source_website
23978,23978,250494,pizza cheese,ae65e4d58c,http://www.food.com/recipe/pizza-cheese-346970,cup,14,"cheese, parmesan, hard",2081.886576,"cheese , parmesan , hard","parmesan cheese , cow milk","parmesan cheese , cow milk",1110.0,pizza cheese,http://www.food.com
23979,23979,250495,pizza cheese,ae65e4d58c,http://www.food.com/recipe/pizza-cheese-346970,cup,14,"cheese, mozzarella, low sodium",1848.000000,"cheese , mozzarella , low sodium","mozzarella cheese , cow milk","mozzarella cheese , cow milk",1110.0,pizza cheese,http://www.food.com
26952,26952,282034,2 ingredient pizza crust (the slow roasted italian),c4db91233f,http://www.food.com/recipe/2-ingredient-pizza-crust-the-slow-roasted-italian-523628,cup,1,"yogurt, greek, plain, nonfat",245.000000,"yogurt , greek , plain ,","yogurt , greek style , plain","yogurt , greek style , plain",240.0,2 ingredient pizza crust,http://www.food.com
26953,26953,282035,2 ingredient pizza crust (the slow roasted italian),c4db91233f,http://www.food.com/recipe/2-ingredient-pizza-crust-the-slow-roasted-italian-523628,cup,1.5,"wheat flour, white, all-purpose, unenriched",187.500000,"wheat flour , white , ,","wheat flour ,","wheat flour ,",-1.0,2 ingredient pizza crust,http://www.food.com
34072,34072,354095,5 minute pizza base. no yeast!,f73c68bcba,https://cookpad.com/us/recipes/364529-5-minute-pizza-base-no-yeast,g,500,"wheat flour, white, all-purpose, unenriched",500.000000,"wheat flour , white , ,","wheat flour ,","wheat flour ,",-1.0,5 minute pizza base yeast,NaN
34073,34073,354099,5 minute pizza base. no yeast!,f73c68bcba,https://cookpad.com/us/recipes/364529-5-minute-pizza-base-no-yeast,g,500,"yogurt, greek, plain, nonfat",500.000000,"yogurt , greek , plain ,","yogurt , greek style , plain","yogurt , greek style , plain",240.0,5 minute pizza base yeast,NaN


In [203]:
menu_data.loc[48, 'Closest 7t recipe by hand'] = 'no match'

Next recipe

In [204]:
menu_data[menu_data.index==8]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand,Closest 7t recipe by hand
8,burgart_205074_c70.jpg,Desserts,Fondant au chocolat artisan boulanger,NaN,Fondant au chocolat,Not yet handled


In [205]:
seven_data["title_raw"][seven_data["title_raw"].str.contains("chocolate fondant")].values.tolist()

[]

In [206]:
seven_data["title_raw"][seven_data["title_raw"].str.contains("fondant")].values.tolist()

[]

In [207]:
seven_data["title_raw"][seven_data["title_raw"].str.contains("chocolate")].values.tolist()

['natural peanut butter chocolate bon bons',
 'natural peanut butter chocolate bon bons',
 'natural peanut butter chocolate bon bons',
 'chocolate krispy bars recipe',
 'chocolate krispy bars recipe',
 'chocolate krispy bars recipe',
 "the fairway cafe's mitchell london hot chocolate",
 "the fairway cafe's mitchell london hot chocolate",
 "the fairway cafe's mitchell london hot chocolate",
 'chocolate sauce (raw)',
 'chocolate sauce (raw)',
 'chocolate sauce (raw)',
 'chocolate sauce (raw)',
 'homemade ice milk. -vanilla, chocolate, almond, fruit choices',
 'homemade ice milk. -vanilla, chocolate, almond, fruit choices',
 'homemade ice milk. -vanilla, chocolate, almond, fruit choices',
 'chocolate coconut milk ice cream',
 'chocolate coconut milk ice cream',
 'chocolate coconut milk ice cream',
 'chocolate coconut milk ice cream',
 'chocolate orange truffles recipe elanaspantry',
 'chocolate orange truffles recipe elanaspantry',
 'chocolate orange truffles recipe elanaspantry',
 'choco

In [208]:
seven_data[seven_data['title_raw'].isin(['chocolate fudge'])]

,Unnamed: 0,Unnamed: 0.1,title_raw,id,url,unit,quantity_raw,ingredient_raw,weight_per_ingr,ingredient_clean_comma,matched_ingredient,food_clean_comma,est_impact,title_clean,source_website
15672,15672,163745,chocolate fudge,7354fa9af7,http://www.food.com/recipe/chocolate-fudge-423634,cup,1,"milk, canned, condensed, sweetened",305.600,"milk , , condensed ,","condensed milk , sugar , whole","condensed milk , sugar , whole",158.4,chocolate fudge,http://www.food.com
15673,15673,163747,chocolate fudge,7354fa9af7,http://www.food.com/recipe/chocolate-fudge-423634,pound,1,"candies, semisweet chocolate",453.592,"candy , semisweet chocolate",chocolate sauce,chocolate sauce,120.0,chocolate fudge,http://www.food.com
15674,15674,163748,chocolate fudge,7354fa9af7,http://www.food.com/recipe/chocolate-fudge-423634,tablespoon,1,vanilla extract,12.600,vanilla extract,"vanilla , alcoholic extract","vanilla , alcoholic extract",-1.0,chocolate fudge,http://www.food.com
15675,15675,163749,chocolate fudge,7354fa9af7,http://www.food.com/recipe/chocolate-fudge-423634,cup,2,"nuts, pecans",218.000,"nut , pecan",pecan nut,pecan nut,300.0,chocolate fudge,http://www.food.com


In [209]:
seven_data["title_raw"][seven_data["title_raw"].str.contains("fudge")].values.tolist()

['three minute fudge',
 'three minute fudge',
 'made-in-minutes no-cook fudge',
 'made-in-minutes no-cook fudge',
 'made-in-minutes no-cook fudge',
 'toffee fudge',
 'toffee fudge',
 'toffee fudge',
 'toffee fudge',
 "fudge sauce `21' club",
 "fudge sauce `21' club",
 'super healthy fudge',
 'super healthy fudge',
 'super healthy fudge',
 'super healthy fudge',
 'ladybirds choc fudge sauce , hot or cold or as a pudding .',
 'ladybirds choc fudge sauce , hot or cold or as a pudding .',
 'ladybirds choc fudge sauce , hot or cold or as a pudding .',
 'almond butter fudge',
 'almond butter fudge',
 'almond butter fudge',
 'almond butter fudge',
 'almond butter fudge',
 'sanders hot fudge sauce',
 'sanders hot fudge sauce',
 'sanders hot fudge sauce',
 'sanders hot fudge sauce',
 'sanders hot fudge sauce',
 'classic hot fudge sundae sauce',
 'classic hot fudge sundae sauce',
 'famous fudge',
 'famous fudge',
 'famous fudge',
 'famous fudge',
 'famous fudge',
 'famous fudge',
 'christmas fud

In [210]:
menu_data.loc[8, 'Closest 7t recipe by hand'] = 'no match'

Next recipe

In [211]:
menu_data[menu_data.index==7]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand,Closest 7t recipe by hand
7,burgart_205074_c70.jpg,Desserts,Mousse au chocolat maison,NaN,Mousse au chocolat maison,Not yet handled


In [212]:
seven_data["title_raw"][seven_data["title_raw"].str.contains("chocolate mousse")].values.tolist()

['chocolate mousse',
 'chocolate mousse',
 'marshmallow fluff chocolate mousse',
 'marshmallow fluff chocolate mousse',
 'marshmallow fluff chocolate mousse',
 'marshmallow fluff chocolate mousse',
 'raw chocolate mousse, lactose free',
 'raw chocolate mousse, lactose free',
 'raw chocolate mousse, lactose free',
 'raw chocolate mousse, lactose free']

In [213]:
set(seven_data["title_raw"][seven_data["title_raw"].str.contains("chocolate") & seven_data["title_raw"].str.contains("mousse")].values.tolist())

{'chocolate cake batter mousse',
 'chocolate cheesecake mousse',
 'chocolate dream mousse',
 'chocolate mousse',
 'chocolate peanut butter mousse',
 'chocolate soup with brownie croutons and mousse dessert',
 'marshmallow fluff chocolate mousse',
 'milk chocolate peanut butter mousse',
 'raw chocolate mousse, lactose free'}

In [214]:
seven_data[seven_data['title_raw'].isin(['chocolate cake batter mousse', 'chocolate mousse', 'chocolate dream mousse', 'raw chocolate mousse, lactose free'])]

,Unnamed: 0,Unnamed: 0.1,title_raw,id,url,unit,quantity_raw,ingredient_raw,weight_per_ingr,ingredient_clean_comma,matched_ingredient,food_clean_comma,est_impact,title_clean,source_website
1506,1506,15792,chocolate cake batter mousse,0acc517647,http://www.food.com/recipe/chocolate-cake-batter-mousse-365928,cup,1,"cream, fluid, heavy whipping",120.000000,"cream , ,",cream sauce,cream sauce,670.0,chocolate cake batter mousse,http://www.food.com
2222,2222,24355,chocolate mousse,10f98f9775,http://www.food.com/recipe/chocolate-mousse-203014,ounce,16,"tofu, raw, regular, prepared with calcium sulfate",453.592000,"tofu , , , calcium sulfate","tofu , plain","tofu , plain",-1.0,chocolate mousse,http://www.food.com
2223,2223,24356,chocolate mousse,10f98f9775,http://www.food.com/recipe/chocolate-mousse-203014,cup,34,"candies, semisweet chocolate",6188.000000,"candy , semisweet chocolate",chocolate sauce,chocolate sauce,120.0,chocolate mousse,http://www.food.com
26372,26372,275704,chocolate dream mousse,c091579f08,http://www.food.com/recipe/chocolate-dream-mousse-177667,ounce,12.333333,"tofu, raw, regular, prepared with calcium sulfate",349.643833,"tofu , , , calcium sulfate","tofu , plain","tofu , plain",-1.0,chocolate dream mousse,http://www.food.com
26373,26373,275705,chocolate dream mousse,c091579f08,http://www.food.com/recipe/chocolate-dream-mousse-177667,cup,14,"cocoa, dry powder, unsweetened",1204.000000,"cocoa , dry powder , unsweetened","cocoa powder , sugar , powder , instant","cocoa powder , sugar , powder , instant",933.0,chocolate dream mousse,http://www.food.com
26374,26374,275706,chocolate dream mousse,c091579f08,http://www.food.com/recipe/chocolate-dream-mousse-177667,cup,14,honey,4746.000000,honey,honey,honey,38.0,chocolate dream mousse,http://www.food.com
26375,26375,275707,chocolate dream mousse,c091579f08,http://www.food.com/recipe/chocolate-dream-mousse-177667,teaspoon,1,vanilla extract,4.200000,vanilla extract,"vanilla , alcoholic extract","vanilla , alcoholic extract",-1.0,chocolate dream mousse,http://www.food.com
31239,31239,324253,"raw chocolate mousse, lactose free",e244f2a20a,http://www.food.com/recipe/raw-chocolate-mousse-lactose-free-522050,cup,34,"nuts, cashew nuts, raw",5100.000000,"nut , cashew nut ,","cashew nut , grilled , salted","cashew nut , grilled , salted",-1.0,raw chocolate mousse lactose,NaN
31240,31240,324254,"raw chocolate mousse, lactose free",e244f2a20a,http://www.food.com/recipe/raw-chocolate-mousse-lactose-free-522050,tablespoon,2,"cocoa, dry powder, unsweetened",10.750000,"cocoa , dry powder , unsweetened","cocoa powder , sugar , powder , instant","cocoa powder , sugar , powder , instant",933.0,raw chocolate mousse lactose,NaN
31241,31241,324255,"raw chocolate mousse, lactose free",e244f2a20a,http://www.food.com/recipe/raw-chocolate-mousse-lactose-free-522050,tablespoon,3,"sweetener, syrup, agave",62.100000,"sweetener , syrup , agave",syrup fruit syrup,syrup fruit syrup,120.0,raw chocolate mousse lactose,NaN


In [215]:
menu_data.loc[7, 'Closest 7t recipe by hand'] = ["chocolate mousse", "chocolate dream mousse", "raw chocolate mousse, lactose free"]

In [216]:
menu_data.head(8)

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand,Closest 7t recipe by hand
0,burgart_205074_c70.jpg,Burgers,Superfin,"Bœuf (VBF Metzger), Comté AOC, Iceberg, Bacon, Compotée d'oignons, Sauce Burgart",Not yet handled,Not yet handled
1,burgart_205074_c70.jpg,Burgers,Bon & Beau,"Bœuf (VBF Metzger), Cheddar, Sucrine, Bacon, Tomate Confite, Sauce poivre maison",Not yet handled,Not yet handled
2,burgart_205074_c70.jpg,Burgers,Dandy,"Bœuf (VBF Metzger), Cheddar fumée, Mâche nantaise, Bresaola, Aubergine, Oignon rouge, Pickles, Sauce cocktail maison",Not yet handled,Not yet handled
3,burgart_205074_c70.jpg,Burgers,Green Chicken,"Filet de poulet, Tomme de Savoie IGP, Roquette, Mortadelle, Parmesan, Sauce verte maison",Not yet handled,Not yet handled
4,burgart_205074_c70.jpg,Poke Bowls,Black bowl,"Thon, Riz noir, Avocat, Oignon rouge, Cébette, Concombre, Sésame, Coriandre, Grenade, Sauce douce",Not yet handled,Not yet handled
5,burgart_205074_c70.jpg,Salades,Crispy Caesar,"Croquettes de poulet, Parmesan, Champignons bruns, Salade Romaine, Ciboulette",Not yet handled,Not yet handled
6,burgart_205074_c70.jpg,Desserts,Cheesecake maison,NaN,Not yet handled,Not yet handled
7,burgart_205074_c70.jpg,Desserts,Mousse au chocolat maison,NaN,Mousse au chocolat maison,"[chocolate mousse, chocolate dream mousse, raw chocolate mousse, lactose free]"


Next recipe

In [217]:
menu_data[menu_data.index==17]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand,Closest 7t recipe by hand
17,Carte-in-seoul.png,Anju,Mandoo (6 pcs),Raviolis maison grillés au porc et légumes,no match,Not yet handled


In [218]:
set(seven_data["title_raw"][seven_data["title_raw"].str.contains("mandoo")].values.tolist())

set()

In [219]:
set(seven_data["title_raw"][seven_data["title_raw"].str.contains("mandow")].values.tolist())

set()

In [220]:
set(seven_data["title_raw"][seven_data["title_raw"].str.contains("mandu")].values.tolist())

set()

In [221]:
set(seven_data["title_raw"][seven_data["title_raw"].str.contains("raviol")].values.tolist())

{'gluten free ravioli filling',
 'potato and ricotta ravioli #5fix',
 'ravioli primavera'}

In [222]:
set(seven_data["ingredient_raw"][seven_data["ingredient_raw"].str.contains("raviol")].values.tolist())

set()

In [223]:
seven_data[seven_data['title_raw'].isin(['gluten free ravioli filling', 'ravioli primavera'])]

,Unnamed: 0,Unnamed: 0.1,title_raw,id,url,unit,quantity_raw,ingredient_raw,weight_per_ingr,ingredient_clean_comma,matched_ingredient,food_clean_comma,est_impact,title_clean,source_website
9378,9378,97833,ravioli primavera,448a000e76,http://www.food.com/recipe/ravioli-primavera-360597,pound,1,"cheese, parmesan, hard",453.592000,"cheese , parmesan , hard","parmesan cheese , cow milk","parmesan cheese , cow milk",1110.0,ravioli primavera,http://www.food.com
9379,9379,97834,ravioli primavera,448a000e76,http://www.food.com/recipe/ravioli-primavera-360597,pound,1,"shortening, vegetable, household, composite",453.592000,", vegetable , household ,",vegetable fritter,vegetable fritter,-1.0,ravioli primavera,http://www.food.com
9380,9380,97835,ravioli primavera,448a000e76,http://www.food.com/recipe/ravioli-primavera-360597,ounce,16,"sauce, pasta, spaghetti/marinara, ready-to-serve",453.592000,"sauce , pasta , spaghetti marinara ,","pasta cheese sauce spaghetti , tagliatelleâ€¦","pasta cheese sauce spaghetti , tagliatelleâ€¦",120.0,ravioli primavera,http://www.food.com
9381,9381,97836,ravioli primavera,448a000e76,http://www.food.com/recipe/ravioli-primavera-360597,cup,12,"cheese, parmesan, hard",1784.474208,"cheese , parmesan , hard","parmesan cheese , cow milk","parmesan cheese , cow milk",1110.0,ravioli primavera,http://www.food.com
24842,24842,260481,gluten free ravioli filling,b598be154e,http://www.food.com/recipe/gluten-free-ravioli-filling-263927,cup,23,"cheese, ricotta, whole milk",5704.000000,"cheese , ricotta , whole milk",ricotta cheese,ricotta cheese,1110.0,gluten ravioli filling,http://www.food.com
24843,24843,260482,gluten free ravioli filling,b598be154e,http://www.food.com/recipe/gluten-free-ravioli-filling-263927,cup,12,"leeks, (bulb and lower leaf-portion), raw",1068.000000,"leek , ,","leek ,","leek ,",-1.0,gluten ravioli filling,http://www.food.com
24844,24844,260483,gluten free ravioli filling,b598be154e,http://www.food.com/recipe/gluten-free-ravioli-filling-263927,cup,12,"tomatoes, sun-dried",648.000000,"tomato , sun dried","tomato , dried","tomato , dried",150.0,gluten ravioli filling,http://www.food.com
24845,24845,260484,gluten free ravioli filling,b598be154e,http://www.food.com/recipe/gluten-free-ravioli-filling-263927,tablespoon,1,"butter, without salt",14.200000,"butter , salt",cocoa butter,cocoa butter,-1.0,gluten ravioli filling,http://www.food.com


In [224]:
menu_data.loc[17, 'Closest 7t recipe by hand'] = 'no match'

Next recipe

In [225]:
menu_data[menu_data.index==18]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand,Closest 7t recipe by hand
18,Carte-in-seoul.png,Anju,Haemul Jeol,Galette de fruit mer,no match,Not yet handled


In [226]:
set(seven_data["title_raw"][seven_data["title_raw"].str.contains("haemul")].values.tolist())

set()

In [227]:
set(seven_data["title_raw"][seven_data["title_raw"].str.contains("jeol")].values.tolist())

set()

In [228]:
set(seven_data["title_raw"][seven_data["title_raw"].str.contains("seafood")].values.tolist())

{'basic seafood spice blend ii',
 'healthy cucumber sauce for seafood',
 'lemon garlic butter sauce for crab (or seafood)',
 'seafood sauce',
 'seafood seasoning',
 'seafood seasoning 2',
 'seafood seasoning mix'}

In [229]:
seven_data[seven_data['title_raw'].isin(['basic seafood spice blend ii'])]

,Unnamed: 0,Unnamed: 0.1,title_raw,id,url,unit,quantity_raw,ingredient_raw,weight_per_ingr,ingredient_clean_comma,matched_ingredient,food_clean_comma,est_impact,title_clean,source_website
12909,12909,134980,basic seafood spice blend ii,5e95065d5d,http://online-cookbook.com/goto/cook/rpage/000E74,tablespoon,2,"seeds, sesame seeds, whole, dried",18.0000,"seed , sesame seed , whole , dried",sesame seed,sesame seed,473.0,basic seafood spice blend ii,http://online-cookbook.com
12910,12910,134981,basic seafood spice blend ii,5e95065d5d,http://online-cookbook.com/goto/cook/rpage/000E74,tablespoon,2,"dill weed, fresh",1.1125,"dill weed , fresh","dill , fresh","dill , fresh",120.0,basic seafood spice blend ii,http://online-cookbook.com
12911,12911,134982,basic seafood spice blend ii,5e95065d5d,http://online-cookbook.com/goto/cook/rpage/000E74,tablespoon,2,"spices, paprika",13.8000,"spice , paprika",paprika,paprika,120.0,basic seafood spice blend ii,http://online-cookbook.com
12912,12912,134983,basic seafood spice blend ii,5e95065d5d,http://online-cookbook.com/goto/cook/rpage/000E74,tablespoon,1,"spices, garlic powder",9.3000,"spice , garlic powder","garlic , powder , dried","garlic , powder , dried",123.2,basic seafood spice blend ii,http://online-cookbook.com


In [230]:
menu_data.loc[18, 'Closest 7t recipe by hand'] = 'no match'

Next recipe

In [231]:
menu_data[menu_data.index==19]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand,Closest 7t recipe by hand
19,Carte-in-seoul.png,Anju,Sewoo kkanpoong (8 pcs),Crevettes frites dans une sauce aigre douce,"[Crevettes teriyaki, Crevettes miel sésame]",Not yet handled


In [232]:
set(seven_data["title_raw"][seven_data["title_raw"].str.contains("sewoo")].values.tolist())

set()

In [233]:
set(seven_data["title_raw"][seven_data["title_raw"].str.contains("kkanpoong")].values.tolist())

set()

In [234]:
set(seven_data["title_raw"][seven_data["title_raw"].str.contains("kkanp")].values.tolist())

set()

In [235]:
set(seven_data["title_raw"][seven_data["title_raw"].str.contains("shrimp")].values.tolist())

{'5-minute shrimp and pea salad',
 'baby shrimp and tarragon mustard',
 'barbecue shrimp dry seasoning mix recipe',
 'barbecued shrimp',
 'barbecued shrimp - nuwave oven',
 'barbecued shrimp and chicken',
 'barbecued shrimp and chicken recipe',
 'barbequed shrimp and chicken',
 'barbequed shrimp and chicken recipe',
 'basic deep fried shrimp recipe',
 'bbq shrimp',
 'black pepper shrimp',
 'black pepper shrimp (paula deen)',
 "boiling crab's cajun shrimp",
 'broiled shrimp',
 'broiled shrimp with spicy ginger lime butter recipe',
 'butterfly shrimp, easy how-to',
 'buttery shrimp pate',
 'cajun shrimp',
 'cheesy shrimp dip',
 'chicken, beef, pork or shrimp lo mein recipe',
 'citrus basil sauce for chicken or shrimp',
 'coleslaw with bay shrimp and tarragon',
 'deep fried shrimp recipe',
 'dipping sauce for coconut shrimp',
 "donna's shrimp scampi recipe",
 'easy shrimp creole recipe',
 'easy shrimp fried rice',
 'easy shrimp scampi bake',
 'easy shrimp tapas',
 'easy spicy barbecued sh

In [236]:
set(seven_data["title_raw"][seven_data["title_raw"].str.contains("shrimp") & seven_data["title_raw"].str.contains("fried")].values.tolist())

{'basic deep fried shrimp recipe',
 'deep fried shrimp recipe',
 'easy shrimp fried rice'}

In [237]:
set(seven_data["title_raw"][seven_data["title_raw"].str.contains("shrimp") & seven_data["title_raw"].str.contains("sour")].values.tolist())

{'sweet and sour shrimp (flat belly diet)'}

In [238]:
set(seven_data["title_raw"][seven_data["title_raw"].str.contains("prawn")].values.tolist())

{'grilled prawns with fresh coriander', 'spicy glazed prawns'}

In [239]:
seven_data[seven_data['title_raw'].isin(['sweet and sour shrimp (flat belly diet)', 'basic deep fried shrimp recipe', \
                                         'deep fried shrimp recipe'])]

,Unnamed: 0,Unnamed: 0.1,title_raw,id,url,unit,quantity_raw,ingredient_raw,weight_per_ingr,ingredient_clean_comma,matched_ingredient,food_clean_comma,est_impact,title_clean,source_website
13881,13881,145047,sweet and sour shrimp (flat belly diet),663f46975f,http://www.food.com/recipe/sweet-and-sour-shrimp-flat-belly-diet-390561,teaspoon,1,"oil, olive, salad or cooking",4.500000,"oil , olive , salad cooking","albacore , olive oil , , drained","albacore , olive oil , , drained",329.411765,sweet sour shrimp,http://www.food.com
13882,13882,145048,sweet and sour shrimp (flat belly diet),663f46975f,http://www.food.com/recipe/sweet-and-sour-shrimp-flat-belly-diet-390561,ounce,16,"shortening, vegetable, household, composite",453.592000,", vegetable , household ,",vegetable fritter,vegetable fritter,-1.000000,sweet sour shrimp,http://www.food.com
13883,13883,145049,sweet and sour shrimp (flat belly diet),663f46975f,http://www.food.com/recipe/sweet-and-sour-shrimp-flat-belly-diet-390561,cup,23,"jams and preserves, apricot",7360.000000,"jam preserve , apricot","jam , apricot","jam , apricot",-1.000000,sweet sour shrimp,http://www.food.com
13884,13884,145050,sweet and sour shrimp (flat belly diet),663f46975f,http://www.food.com/recipe/sweet-and-sour-shrimp-flat-belly-diet-390561,teaspoon,4,"vinegar, red wine",19.866667,"vinegar , red wine","wine , red , Â°","wine , red , Â°",120.000000,sweet sour shrimp,http://www.food.com
13885,13885,145054,sweet and sour shrimp (flat belly diet),663f46975f,http://www.food.com/recipe/sweet-and-sour-shrimp-flat-belly-diet-390561,ounce,12,"crustaceans, shrimp, raw (not previously frozen)",340.194000,"crustacean , shrimp ,","shrimp , frozen ,","shrimp , frozen ,",1270.000000,sweet sour shrimp,http://www.food.com
13886,13886,145055,sweet and sour shrimp (flat belly diet),663f46975f,http://www.food.com/recipe/sweet-and-sour-shrimp-flat-belly-diet-390561,cup,12,"oil, corn, peanut, and olive",2688.000000,"oil , corn , peanut , olive",maize corn oil,maize corn oil,-1.000000,sweet sour shrimp,http://www.food.com
22858,22858,238990,deep fried shrimp recipe,a67a04830d,http://cookeatshare.com/recipes/deep-fried-shrimp-52701,pound,2,"crustaceans, shrimp, raw (not previously frozen)",907.184000,"crustacean , shrimp ,","shrimp , frozen ,","shrimp , frozen ,",1270.000000,deep fried shrimp recipe,http://cookeatshare.com
23834,23834,248805,basic deep fried shrimp recipe,ad4c4ef87b,http://cookeatshare.com/recipes/basic-deep-fried-shrimp-79463,pound,1,"oil, olive, salad or cooking",453.592000,"oil , olive , salad cooking","albacore , olive oil , , drained","albacore , olive oil , , drained",329.411765,basic deep fried shrimp recipe,http://cookeatshare.com


There are again recipes with only one ingredient while there should be more than one ingredient (e.g. in 'basic deep fried shrimp recipe' : it looks like an error in the database!    
Let's have a closer look

In [240]:
seven_data3 = seven_data.groupby('title_raw')['ingredient_raw'].count().reset_index()
seven_data3[seven_data3['ingredient_raw']==1]

,title_raw,ingredient_raw
15,"""winter is coming"" ribs",1
32,2-ingredient pizza dough,1
143,alton brown's easy clotted cream recipe jdavist,1
145,amaretto cranberry juice recipe,1
147,amaretto grapefruit juice recipe,1
...,...,...
7157,"vickys homemade coconut spread/butter or creamed coconut, gluten, dairy, egg & soy-free",1
7223,watermelon ice cubes,1
7224,watermelon juice,1
7293,"white, milk, and dark chocolate dessert cups",1


-> There are (only) 229 recipes over 7418 with only one ingredient.

In [241]:
menu_data.loc[19, 'Closest 7t recipe by hand'] = 'no match'

Next recipe

In [242]:
menu_data[menu_data.index==20]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand,Closest 7t recipe by hand
20,Carte-in-seoul.png,Bibimbap,Dolsot bibimbap (en pierrade au choix),Bœuf,no match,Not yet handled


In [243]:
set(seven_data["title_raw"][seven_data["title_raw"].str.contains("bib")].values.tolist())

{'bibingka (coconut rice desert) recipe'}

In [244]:
set(seven_data["title_raw"][seven_data["title_raw"].str.contains("dolsot")].values.tolist())

set()

In [245]:
menu_data.loc[20, 'Closest 7t recipe by hand'] = 'no match'

Next recipe

In [246]:
menu_data[menu_data.index==21]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand,Closest 7t recipe by hand
21,Carte-in-seoul.png,Chicken,"YangNyeom chicken (5pcs, 12pcs, 24pcs)",Poulet frit enrobé de sauce + salade,no match,Not yet handled


In [247]:
set(seven_data["title_raw"][seven_data["title_raw"].str.contains("yang")].values.tolist())

{'broiled or grilled marinated chicken (gai yang)'}

In [248]:
set(seven_data["title_raw"][seven_data["title_raw"].str.contains("korea")].values.tolist())

{'beef bulgogi(korean marinated beef)',
 'dak bulgogi (korean chicken bulgogi)',
 'korean bbq chicken',
 'korean dipping sauce',
 'korean greens',
 'korean sesame seed cookies',
 'korean spinach',
 'simple spicy korean hot red pepper paste dip for cucumbers'}

In [249]:
seven_data[seven_data['title_raw'].isin(['korean bbq chicken'])]

,Unnamed: 0,Unnamed: 0.1,title_raw,id,url,unit,quantity_raw,ingredient_raw,weight_per_ingr,ingredient_clean_comma,matched_ingredient,food_clean_comma,est_impact,title_clean,source_website
19100,19100,199545,korean bbq chicken,8b4a345dce,http://www.food.com/recipe/korean-bbq-chicken-503076,pound,1,"chicken, broiler or fryers, breast, skinless, boneless, meat only, raw",453.59200,"chicken , broiler fryer , breast , skinless , boneless , meat ,","chicken , meat ,","chicken , meat ,",580.0,korean bbq chicken,http://www.food.com
19101,19101,199546,korean bbq chicken,8b4a345dce,http://www.food.com/recipe/korean-bbq-chicken-503076,tablespoon,3,soy sauce made from soy (tamari),54.00000,soy sauce soy,"soy sauce ,","soy sauce ,",120.0,korean bbq chicken,http://www.food.com
19102,19102,199547,korean bbq chicken,8b4a345dce,http://www.food.com/recipe/korean-bbq-chicken-503076,tablespoon,1,"lemon juice, raw",15.25000,"lemon juice ,","lemon , pulp ,","lemon , pulp ,",460.0,korean bbq chicken,http://www.food.com
19103,19103,199548,korean bbq chicken,8b4a345dce,http://www.food.com/recipe/korean-bbq-chicken-503076,tablespoon,1,"sugars, brown",9.00000,"sugar , brown","sugar , brown","sugar , brown",38.0,korean bbq chicken,http://www.food.com
19104,19104,199549,korean bbq chicken,8b4a345dce,http://www.food.com/recipe/korean-bbq-chicken-503076,tablespoon,1,honey,21.18750,honey,honey,honey,38.0,korean bbq chicken,http://www.food.com
19105,19105,199550,korean bbq chicken,8b4a345dce,http://www.food.com/recipe/korean-bbq-chicken-503076,tablespoon,1,"rice, white, long-grain, regular, unenriched, cooked without salt",9.87500,"rice , white , long grain , , , cooked salt","white rice , cooked , chicken","white rice , cooked , chicken",467.0,korean bbq chicken,http://www.food.com
19106,19106,199551,korean bbq chicken,8b4a345dce,http://www.food.com/recipe/korean-bbq-chicken-503076,tablespoon,1,"spices, garlic powder",9.30000,"spice , garlic powder","garlic , powder , dried","garlic , powder , dried",123.2,korean bbq chicken,http://www.food.com
19107,19107,199552,korean bbq chicken,8b4a345dce,http://www.food.com/recipe/korean-bbq-chicken-503076,teaspoon,1,"spices, ginger, ground",1.80000,"spice , ginger , ground","ginger ,","ginger ,",80.0,korean bbq chicken,http://www.food.com
19108,19108,199553,korean bbq chicken,8b4a345dce,http://www.food.com/recipe/korean-bbq-chicken-503076,tablespoon,1,"oil, sesame, salad or cooking",13.60000,"oil , sesame , salad cooking",sesame oil,sesame oil,-1.0,korean bbq chicken,http://www.food.com
19109,19109,199554,korean bbq chicken,8b4a345dce,http://www.food.com/recipe/korean-bbq-chicken-503076,pinch,1,"spices, pepper, black",0.14352,"spice , pepper , black","black pepper , powder","black pepper , powder",-1.0,korean bbq chicken,http://www.food.com


In [250]:
menu_data.loc[21, 'Closest 7t recipe by hand'] = 'korean bbq chicken'

Next recipe

In [251]:
menu_data[menu_data.index==22]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand,Closest 7t recipe by hand
22,Carte-in-seoul.png,Classiques,Bulgogi,"Bœuf émincé à la sauce soja, légumes, riz","[Wok de bœuf légumes croquants, Bœuf thaï, Bœuf sauté au chou, Aiguillettes de bœuf & légumes poêlés]",Not yet handled


In [252]:
set(seven_data["title_raw"][seven_data["title_raw"].str.contains("bulgog")].values.tolist())

{'beef bulgogi(korean marinated beef)', 'dak bulgogi (korean chicken bulgogi)'}

In [253]:
menu_data.loc[22, 'Closest 7t recipe by hand'] = 'beef bulgogi(korean marinated beef)'

Next recipe

In [254]:
menu_data[menu_data.index==23]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand,Closest 7t recipe by hand
23,Carte-in-seoul.png,Classiques,Jeyuk,"Porc sauté, légumes, riz - soja ou piquant",Porc au caramel,Not yet handled


In [255]:
set(seven_data["title_raw"][seven_data["title_raw"].str.contains("eyuk")].values.tolist())

set()

In [256]:
set(seven_data["title_raw"][seven_data["title_raw"].str.contains("korea")].values.tolist())

{'beef bulgogi(korean marinated beef)',
 'dak bulgogi (korean chicken bulgogi)',
 'korean bbq chicken',
 'korean dipping sauce',
 'korean greens',
 'korean sesame seed cookies',
 'korean spinach',
 'simple spicy korean hot red pepper paste dip for cucumbers'}

In [257]:
seven_data2[(seven_data2["title_raw"].str.contains("thai")) & (seven_data2["ingredient_raw"].str.contains("pork"))]

,title_raw,ingredient_raw
5434,pork with garlic & black pepper (thai) recipe,"spices, garlic powder, sauce, fish, ready-to-serve, oil, corn, peanut, and olive, spices, pepper, black, pork, fresh, loin, tenderloin, separable lean only, raw, parsley, fresh"
6855,thai pork stir fry,"pork, fresh, loin, tenderloin, separable lean only, raw, spices, garlic powder, spices, cardamom, spices, ginger, ground, spices, curry powder, spices, pepper, black, broccoli, raw, oil, olive, salad or cooking, spices, chili powder, carrots, raw, vinegar, distilled, soy sauce made from soy (tamari), cauliflower, raw, rice, white, long-grain, regular, unenriched, cooked without salt"
6856,thai pork with garlic and black pepper recipe,"spices, garlic powder, soy sauce made from soy (tamari), spices, coriander seed, spices, pepper, black, oil, olive, salad or cooking, pork, fresh, loin, tenderloin, separable lean only, raw"


In [258]:
menu_data.loc[23, 'Closest 7t recipe by hand'] = 'thai pork stir fry'

Next recipe

In [259]:
menu_data[menu_data.index==24]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand,Closest 7t recipe by hand
24,Carte-in-seoul.png,Classiques,Ojing-oe,"Calamar sauté, légumes, riz - soja ou piquant )",no match,Not yet handled


In [260]:
set(seven_data["title_raw"][seven_data["title_raw"].str.contains("ojing")].values.tolist())

set()

In [261]:
set(seven_data["title_raw"][seven_data["title_raw"].str.contains("squid")].values.tolist())

{'barbecued squid recipe', 'fried squid with ginger soy sauce'}

In [262]:
seven_data[seven_data['title_raw'].isin(['fried squid with ginger soy sauce'])]

,Unnamed: 0,Unnamed: 0.1,title_raw,id,url,unit,quantity_raw,ingredient_raw,weight_per_ingr,ingredient_clean_comma,matched_ingredient,food_clean_comma,est_impact,title_clean,source_website
24600,24600,256893,fried squid with ginger soy sauce,b2e9844c6f,https://cookpad.com/us/recipes/355739-fried-squid-with-ginger-soy-sauce,tablespoon,2,"oil, olive, salad or cooking",27.00000,"oil , olive , salad cooking","albacore , olive oil , , drained","albacore , olive oil , , drained",329.411765,fried squid ginger soy sauce,https://cookpad.com
24601,24601,256894,fried squid with ginger soy sauce,b2e9844c6f,https://cookpad.com/us/recipes/355739-fried-squid-with-ginger-soy-sauce,pinch,1,"spices, paprika",0.14352,"spice , paprika",paprika,paprika,120.000000,fried squid ginger soy sauce,https://cookpad.com
24602,24602,256895,fried squid with ginger soy sauce,b2e9844c6f,https://cookpad.com/us/recipes/355739-fried-squid-with-ginger-soy-sauce,teaspoon,1,"spices, coriander seed",1.80000,"spice , coriander seed","coriander , seed","coriander , seed",-1.000000,fried squid ginger soy sauce,https://cookpad.com


-> Again some problems in the database here: only 3 ingredients are given while there are 6 ingredients on the website (including the main one ie squid)

In [263]:
menu_data.loc[24, 'Closest 7t recipe by hand'] = 'no match'

Next recipe

In [264]:
menu_data[menu_data.index==25]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand,Closest 7t recipe by hand
25,Carte-in-seoul.png,Nouilles,Japchae - bœuf ou tofu -,"Nouilles de papate douce, légumes, boeuf ou tofu, accompagné de 3 banchan",Not yet handled,Not yet handled


In [265]:
set(seven_data["title_raw"][seven_data["title_raw"].str.contains("jap")].values.tolist())

{'japanese asparagus dengaku',
 'japanese potsticker dip',
 'japanese spinach with sweet sesame seeds',
 'japanese-style sesame chicken',
 'japanese-style spinach, with sesame seeds',
 'miso, carrot, and sesame dressing - japanese'}

In [266]:
set(seven_data["title_raw"][seven_data["title_raw"].str.contains("dangmyeon")].values.tolist())

set()

In [267]:
set(seven_data["title_raw"][seven_data["ingredient_raw"].str.contains("dangmyeon")].values.tolist())

set()

In [268]:
menu_data.loc[25, 'Closest 7t recipe by hand'] = 'no match'

Next recipe

In [269]:
menu_data[menu_data.index==27]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand,Closest 7t recipe by hand
27,Carte-Restaurant-Nelsons-Paris-Jour-2-1024x724.jpg,Salades,La César,"poulet grillé, tomates cerise, œuf, romaine, parmesan",Salade César,Not yet handled


In [270]:
set(seven_data["title_raw"][seven_data["title_raw"].str.contains("caesar")].values.tolist())

{"acadia's super easy caesar dill sauce",
 'bbq caesar pork ribs',
 'better-for-you grilled chicken caesar salad',
 'caesar salad dressing',
 'caesar veggie dip',
 'chicken caesar tortellini with toasted almonds',
 'classic caesar dressing',
 'favorite grilled chicken caesar salad',
 'flower power caesar salad',
 'grilled chicken a la caesar',
 'grilled chicken caesar salad',
 'mock caesar salad',
 'quick caesar salad',
 'tangy caesar salad'}

In [271]:
seven_data[seven_data['title_raw'].isin(['better-for-you grilled chicken caesar salad', \
                                        'favorite grilled chicken caesar salad', \
                                        'flower power caesar salad', 'grilled chicken a la caesar', \
                                         'grilled chicken caesar salad', \
                                        'mock caesar salad', \
                                         'quick caesar salad', 'tangy caesar salad'])]

,Unnamed: 0,Unnamed: 0.1,title_raw,id,url,unit,quantity_raw,ingredient_raw,weight_per_ingr,ingredient_clean_comma,matched_ingredient,food_clean_comma,est_impact,title_clean,source_website
4476,4476,46486,mock caesar salad,1fdce606c8,http://www.food.com/recipe/mock-caesar-salad-39347,tablespoon,3,"vinegar, cider",44.700000,"vinegar , cider","vinegar , cider","vinegar , cider",167.000000,mock caesar salad,http://www.food.com
4477,4477,46487,mock caesar salad,1fdce606c8,http://www.food.com/recipe/mock-caesar-salad-39347,cup,12,"cheese, parmesan, hard",1784.474208,"cheese , parmesan , hard","parmesan cheese , cow milk","parmesan cheese , cow milk",1110.000000,mock caesar salad,http://www.food.com
4478,4478,46488,mock caesar salad,1fdce606c8,http://www.food.com/recipe/mock-caesar-salad-39347,teaspoon,12,"spices, garlic powder",37.200000,"spice , garlic powder","garlic , powder , dried","garlic , powder , dried",123.200000,mock caesar salad,http://www.food.com
4479,4479,46489,mock caesar salad,1fdce606c8,http://www.food.com/recipe/mock-caesar-salad-39347,cup,13,"oil, olive, salad or cooking",2808.000000,"oil , olive , salad cooking","albacore , olive oil , , drained","albacore , olive oil , , drained",329.411765,mock caesar salad,http://www.food.com
4480,4480,46490,mock caesar salad,1fdce606c8,http://www.food.com/recipe/mock-caesar-salad-39347,cup,7,"lettuce, cos or romaine, raw",329.000000,"lettuce , co romaine ,","co romaine lettuce ,","co romaine lettuce ,",80.000000,mock caesar salad,http://www.food.com
8396,8396,86904,grilled chicken a la caesar,3c5b87408b,http://www.kraftrecipes.com/recipes/grilled-chicken-a-la-caesar-58102.aspx,cup,8,"lettuce, cos or romaine, raw",376.000000,"lettuce , co romaine ,","co romaine lettuce ,","co romaine lettuce ,",80.000000,grilled chicken la caesar,http://www.kraftrecipes.com
8397,8397,86905,grilled chicken a la caesar,3c5b87408b,http://www.kraftrecipes.com/recipes/grilled-chicken-a-la-caesar-58102.aspx,pound,1,"chicken, broiler or fryers, breast, skinless, boneless, meat only, raw",453.592000,"chicken , broiler fryer , breast , skinless , boneless , meat ,","chicken , meat ,","chicken , meat ,",580.000000,grilled chicken la caesar,http://www.kraftrecipes.com
8398,8398,86906,grilled chicken a la caesar,3c5b87408b,http://www.kraftrecipes.com/recipes/grilled-chicken-a-la-caesar-58102.aspx,cup,1,"croutons, plain",30.000000,"crouton , plain","crouton , plain ,","crouton , plain ,",-1.000000,grilled chicken la caesar,http://www.kraftrecipes.com
8399,8399,86907,grilled chicken a la caesar,3c5b87408b,http://www.kraftrecipes.com/recipes/grilled-chicken-a-la-caesar-58102.aspx,cup,01-fŽvr,"cheese, parmesan, hard",74.353092,"cheese , parmesan , hard","parmesan cheese , cow milk","parmesan cheese , cow milk",1110.000000,grilled chicken la caesar,http://www.kraftrecipes.com
8400,8400,86908,grilled chicken a la caesar,3c5b87408b,http://www.kraftrecipes.com/recipes/grilled-chicken-a-la-caesar-58102.aspx,cup,2022-04-03 00:00:00,"salad dressing, italian dressing, commercial, regular",176.400000,"salad dressing , italian dressing , commercial ,","salad dressing ,","salad dressing ,",120.000000,grilled chicken la caesar,http://www.kraftrecipes.com


In [272]:
menu_data.loc[27, 'Closest 7t recipe by hand'] = ['better-for-you grilled chicken caesar salad', \
                                                  'favorite grilled chicken caesar salad', 'grilled chicken a la caesar', \
                                                  'grilled chicken caesar salad']

Next recipe

In [273]:
menu_data[menu_data.index==28]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand,Closest 7t recipe by hand
28,Carte-Restaurant-Nelsons-Paris-Jour-2-1024x724.jpg,Salades,La Biquette de Rocamadour,mielleuse et craquante,"[Salade de chèvre chaud, Salade chèvre, miel & pignons, Salade aux croustillants de chèvre]",Not yet handled


In [274]:
set(seven_data["title_raw"][seven_data["title_raw"].str.contains("goat cheese salad")].values.tolist())

{'waldorf goat cheese salad'}

In [275]:
set(seven_data["title_raw"][seven_data["title_raw"].str.contains("goat cheese") & seven_data["title_raw"].str.contains("salad")].values.tolist())

{'goat cheese cakes (to top your salad)',
 'radish salad with goat cheese',
 'salad with blueberries, almonds and goat cheese',
 'waldorf goat cheese salad'}

In [276]:
set(seven_data["title_raw"][seven_data["title_raw"].str.contains("goat cheese")].values.tolist())

{'asparagus with goat cheese dressing',
 'black pepper and goat cheese dip',
 'cherry compote over goat cheese',
 'chive goat cheese dressing',
 'fresh figs with rosemary goat cheese',
 'goat cheese butter with parsley',
 'goat cheese cakes (to top your salad)',
 'goat cheese log rolled in dried cranberries',
 'goat cheese mashed potatoes',
 'goat cheese potato cakes with smoked salmon',
 'goat cheese spread',
 'goat cheese spread (from thornton winery)',
 'goat cheese spread with fresh spinach and basil',
 'goat cheese spread with rosemary and fig',
 'goat cheese sun-dried tomato and pine nuts',
 'goat cheese torte',
 'goat cheese with cranberries and pistachios',
 'grapes rolled in goat cheese with a pistachio crust',
 'homemade goat cheese',
 'pesto goat cheese spread',
 'pistachio-goat cheese bites',
 'potato, caramelized onion, and goat cheese gratin',
 'radish salad with goat cheese',
 'salad with blueberries, almonds and goat cheese',
 'waldorf goat cheese salad'}

In [277]:
seven_data[seven_data['title_raw'].isin(['waldorf goat cheese salad',  'salad with blueberries, almonds and goat cheese'])]

,Unnamed: 0,Unnamed: 0.1,title_raw,id,url,unit,quantity_raw,ingredient_raw,weight_per_ingr,ingredient_clean_comma,matched_ingredient,food_clean_comma,est_impact,title_clean,source_website
5242,5242,54847,waldorf goat cheese salad,25d5fe849f,http://www.food.com/recipe/waldorf-goat-cheese-salad-495294,cup,2,"lettuce, cos or romaine, raw",94.000,"lettuce , co romaine ,","co romaine lettuce ,","co romaine lettuce ,",80.000000,waldorf goat cheese salad,http://www.food.com
5243,5243,54848,waldorf goat cheese salad,25d5fe849f,http://www.food.com/recipe/waldorf-goat-cheese-salad-495294,tablespoon,2,"raspberries, raw",15.375,"raspberry ,","raspberry ,","raspberry ,",-1.000000,waldorf goat cheese salad,http://www.food.com
5244,5244,54849,waldorf goat cheese salad,25d5fe849f,http://www.food.com/recipe/waldorf-goat-cheese-salad-495294,tablespoon,2,"nuts, walnuts, english",14.625,"nut , walnut , english",walnut oil,walnut oil,-1.000000,waldorf goat cheese salad,http://www.food.com
5245,5245,54850,waldorf goat cheese salad,25d5fe849f,http://www.food.com/recipe/waldorf-goat-cheese-salad-495294,cup,12,"grapes, red or green (european type, such as thompson seedless), raw",1812.000,"grape , red green ,","grape , red ,","grape , red ,",467.000000,waldorf goat cheese salad,http://www.food.com
5246,5246,54851,waldorf goat cheese salad,25d5fe849f,http://www.food.com/recipe/waldorf-goat-cheese-salad-495294,tablespoon,2,"cheese, goat, soft type",15.625,"cheese , goat , soft","cheese , goat milk","cheese , goat milk",220.000000,waldorf goat cheese salad,http://www.food.com
5247,5247,54852,waldorf goat cheese salad,25d5fe849f,http://www.food.com/recipe/waldorf-goat-cheese-salad-495294,tablespoon,2,"nuts, pecans",13.625,"nut , pecan",pecan nut,pecan nut,300.000000,waldorf goat cheese salad,http://www.food.com
18657,18657,195181,waldorf goat cheese salad,884e273dc9,http://allrecipes.com/recipe/waldorf-goat-cheese-salad/,cup,2,"lettuce, cos or romaine, raw",94.000,"lettuce , co romaine ,","co romaine lettuce ,","co romaine lettuce ,",80.000000,waldorf goat cheese salad,http://allrecipes.com
18658,18658,195182,waldorf goat cheese salad,884e273dc9,http://allrecipes.com/recipe/waldorf-goat-cheese-salad/,tablespoon,2,"raspberries, raw",15.375,"raspberry ,","raspberry ,","raspberry ,",-1.000000,waldorf goat cheese salad,http://allrecipes.com
18659,18659,195183,waldorf goat cheese salad,884e273dc9,http://allrecipes.com/recipe/waldorf-goat-cheese-salad/,cup,01-fŽvr,"grapes, red or green (european type, such as thompson seedless), raw",75.500,"grape , red green ,","grape , red ,","grape , red ,",467.000000,waldorf goat cheese salad,http://allrecipes.com
18660,18660,195184,waldorf goat cheese salad,884e273dc9,http://allrecipes.com/recipe/waldorf-goat-cheese-salad/,tablespoon,2,"cheese, goat, soft type",15.625,"cheese , goat , soft","cheese , goat milk","cheese , goat milk",220.000000,waldorf goat cheese salad,http://allrecipes.com


In [278]:
menu_data.loc[28, 'Closest 7t recipe by hand'] = ['waldorf goat cheese salad']

Next recipe

In [279]:
menu_data[menu_data.index==30]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand,Closest 7t recipe by hand
30,Carte-Restaurant-Nelsons-Paris-Jour-2-1024x724.jpg,Végé pour tous,Moussaka végétarienne,aux gombos,no match,Not yet handled


In [280]:
set(seven_data["title_raw"][seven_data["title_raw"].str.contains("moussak")].values.tolist())

set()

In [599]:
set(seven_data["title_raw"][seven_data["title_raw"].str.contains("gumb")].values.tolist())

{'baked okra fries', 'okra with szechuan peppercorn salt'}

In [600]:
set(seven_data["title_raw"][seven_data["title_raw"].str.contains("okra")].values.tolist())

{'baked okra fries', 'okra with szechuan peppercorn salt'}

In [282]:
seven_data[seven_data['title_raw'].isin(['gumbo base (aka roux)', 'gumbo pages creole seasoning blend'])]

,Unnamed: 0,Unnamed: 0.1,title_raw,id,url,unit,quantity_raw,ingredient_raw,weight_per_ingr,ingredient_clean_comma,matched_ingredient,food_clean_comma,est_impact,title_clean,source_website
10281,10281,106909,gumbo pages creole seasoning blend,4b1458b3e3,http://www.food.com/recipe/gumbo-pages-creole-seasoning-blend-385641,tablespoon,2,"spices, onion powder",14.4000,"spice , onion powder","onion ,","onion ,",1270.0,gumbo pages creole seasoning blend,http://www.food.com
10282,10282,106910,gumbo pages creole seasoning blend,4b1458b3e3,http://www.food.com/recipe/gumbo-pages-creole-seasoning-blend-385641,tablespoon,2,"spices, garlic powder",18.6000,"spice , garlic powder","garlic , powder , dried","garlic , powder , dried",123.2,gumbo pages creole seasoning blend,http://www.food.com
10283,10283,106911,gumbo pages creole seasoning blend,4b1458b3e3,http://www.food.com/recipe/gumbo-pages-creole-seasoning-blend-385641,tablespoon,2,"spices, oregano, dried",6.0000,"spice , oregano , dried","oregano , dried","oregano , dried",-1.0,gumbo pages creole seasoning blend,http://www.food.com
10284,10284,106912,gumbo pages creole seasoning blend,4b1458b3e3,http://www.food.com/recipe/gumbo-pages-creole-seasoning-blend-385641,tablespoon,2,"spices, basil, dried",4.2000,"spice , basil , dried","basil , dried","basil , dried",120.0,gumbo pages creole seasoning blend,http://www.food.com
10285,10285,106913,gumbo pages creole seasoning blend,4b1458b3e3,http://www.food.com/recipe/gumbo-pages-creole-seasoning-blend-385641,tablespoon,1,"spices, thyme, dried",3.0000,"spice , thyme , dried","thyme , dried","thyme , dried",120.0,gumbo pages creole seasoning blend,http://www.food.com
10286,10286,106914,gumbo pages creole seasoning blend,4b1458b3e3,http://www.food.com/recipe/gumbo-pages-creole-seasoning-blend-385641,tablespoon,1,"spices, pepper, black",6.9000,"spice , pepper , black","black pepper , powder","black pepper , powder",-1.0,gumbo pages creole seasoning blend,http://www.food.com
10287,10287,106916,gumbo pages creole seasoning blend,4b1458b3e3,http://www.food.com/recipe/gumbo-pages-creole-seasoning-blend-385641,tablespoon,1,"spices, pepper, red or cayenne",5.4000,"spice , pepper , red cayenne",cayenne pepper,cayenne pepper,120.0,gumbo pages creole seasoning blend,http://www.food.com
10288,10288,106917,gumbo pages creole seasoning blend,4b1458b3e3,http://www.food.com/recipe/gumbo-pages-creole-seasoning-blend-385641,tablespoon,1,"celery, raw",6.3125,"celery ,",celery salt,celery salt,-1.0,gumbo pages creole seasoning blend,http://www.food.com
10289,10289,106918,gumbo pages creole seasoning blend,4b1458b3e3,http://www.food.com/recipe/gumbo-pages-creole-seasoning-blend-385641,tablespoon,5,"spices, paprika",34.5000,"spice , paprika",paprika,paprika,120.0,gumbo pages creole seasoning blend,http://www.food.com
20110,20110,210093,gumbo base (aka roux),927b30583f,http://www.food.com/recipe/gumbo-base-aka-roux-47651,cup,34,"wheat flour, white, all-purpose, unenriched",4250.0000,"wheat flour , white , ,","wheat flour ,","wheat flour ,",-1.0,gumbo base,http://www.food.com


In [283]:
menu_data.loc[30, 'Closest 7t recipe by hand'] = 'no match'

Next recipe

In [284]:
menu_data[menu_data.index==32]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand,Closest 7t recipe by hand
32,Carte-Restaurant-Nelsons-Paris-Jour-2-1024x724.jpg,Végé pour tous,Risotto primavera,"asperges, artichaut et petit pois",Risotto aux asperges,Not yet handled


In [285]:
set(seven_data["title_raw"][seven_data["title_raw"].str.contains("risotto")].values.tolist())

{'family dinner travels to italy: risotto recipe familydinnertravels'}

In [286]:
set(seven_data["title_raw"][seven_data["title_raw"].str.contains("aspara") & seven_data["title_raw"].str.contains("rice")].values.tolist())

set()

In [287]:
seven_data[seven_data['title_raw'].isin(['family dinner travels to italy: risotto recipe familydinnertravels'])]

,Unnamed: 0,Unnamed: 0.1,title_raw,id,url,unit,quantity_raw,ingredient_raw,weight_per_ingr,ingredient_clean_comma,matched_ingredient,food_clean_comma,est_impact,title_clean,source_website
5031,5031,52346,family dinner travels to italy: risotto recipe familydinnertravels,2429f7acfb,http://www.chowhound.com/recipes/family-dinner-travels-italy-risotto-30201,cup,3,"lemon juice, raw",732.0,"lemon juice ,","lemon , pulp ,","lemon , pulp ,",460.0,family dinner travels italy risotto recipe familydinnertravels,http://www.chowhound.com


-> One recipe with only one ingredient!

In [288]:
menu_data.loc[32, 'Closest 7t recipe by hand'] = 'no match'

Next recipe

In [289]:
menu_data[menu_data.index==42]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand,Closest 7t recipe by hand
42,feaa6c7b-9b16-4d51-b71d-75518f16cb08.jpg,Hox Signatures,Hox Cheeseburger,"Spanish beef, cheddar, caramelized onions, fries, salad",Classic Cheeseburger,Not yet handled


In [290]:
set(seven_data["title_raw"][seven_data["title_raw"].str.contains("cheeseburger")].values.tolist())

set()

In [291]:
set(seven_data["title_raw"][seven_data["title_raw"].str.contains("cheese burger")].values.tolist())

set()

In [292]:
set(seven_data["title_raw"][seven_data["title_raw"].str.contains("burger")].values.tolist())

{'amara veggie burgers',
 'asian turkey burgers',
 'awesome turkey burgers',
 'barbecue burger topping recipe',
 'blue cheese stuffed turkey burgers',
 'chicken & broccoli burgers',
 'copycat burger king creamy apple cider vinaigrette',
 'greek burgers',
 "honey slaw from harold cohen's ultimate burger",
 'peanut burgers',
 'red, white & blue burger topper',
 'spicy turkey burgers',
 'spinach feta turkey burgers',
 'the woodward veggie burger',
 'tofu burgers',
 'turkey curry burgers',
 'turkey gyro burgers'}

In [293]:
set(seven_data["title_raw"][seven_data["title_raw"].str.contains("burger") & seven_data["ingredient_raw"].str.contains("beef")].values.tolist())

set()

In [294]:
set(seven_data["title_raw"][seven_data["title_raw"].str.contains("burger") & seven_data["ingredient_raw"].str.contains("steak")].values.tolist())

set()

In [295]:
menu_data.loc[42, 'Closest 7t recipe by hand'] = 'no match'

Next recipe

In [296]:
menu_data[menu_data.index==43]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand,Closest 7t recipe by hand
43,feaa6c7b-9b16-4d51-b71d-75518f16cb08.jpg,Hox Signatures,Beyond meat veggie burger,"Beyond Meat pattie, charcoal bun, avocado, fries, salad",Cheeseburger veggie,Not yet handled


In [297]:
set(seven_data["title_raw"][seven_data["title_raw"].str.contains("burger") & seven_data["ingredient_raw"].str.contains("avocado")].values.tolist())

set()

In [298]:
seven_data[seven_data['title_raw'].isin(['amara veggie burgers', 'the woodward veggie burger'])]

,Unnamed: 0,Unnamed: 0.1,title_raw,id,url,unit,quantity_raw,ingredient_raw,weight_per_ingr,ingredient_clean_comma,matched_ingredient,food_clean_comma,est_impact,title_clean,source_website
5185,5185,54214,amara veggie burgers,2570470bf3,http://www.food.com/recipe/amara-veggie-burgers-287258,cup,2,"rice, white, long-grain, regular, unenriched, cooked without salt",316.0,"rice , white , long grain , , , cooked salt","white rice , cooked , chicken","white rice , cooked , chicken",467.000000,amara veggie burgers,http://www.food.com
5186,5186,54215,amara veggie burgers,2570470bf3,http://www.food.com/recipe/amara-veggie-burgers-287258,tablespoon,2,"peanut butter, smooth style, without salt",32.0,"peanut butter , style , salt",peanut butter peanut paste,peanut butter peanut paste,316.700000,amara veggie burgers,http://www.food.com
5187,5187,54216,amara veggie burgers,2570470bf3,http://www.food.com/recipe/amara-veggie-burgers-287258,cup,12,"wheat flour, white, all-purpose, unenriched",1500.0,"wheat flour , white , ,","wheat flour ,","wheat flour ,",-1.000000,amara veggie burgers,http://www.food.com
5188,5188,54220,amara veggie burgers,2570470bf3,http://www.food.com/recipe/amara-veggie-burgers-287258,cup,14,"seeds, sesame seeds, whole, dried",2016.0,"seed , sesame seed , whole , dried",sesame seed,sesame seed,473.000000,amara veggie burgers,http://www.food.com
5189,5189,54221,amara veggie burgers,2570470bf3,http://www.food.com/recipe/amara-veggie-burgers-287258,cup,14,"seeds, sunflower seed kernels, dried",644.0,"seed , sunflower seed kernel , dried",sunflower seed,sunflower seed,-1.000000,amara veggie burgers,http://www.food.com
5190,5190,54222,amara veggie burgers,2570470bf3,http://www.food.com/recipe/amara-veggie-burgers-287258,cup,14,"carrots, raw",1792.0,"carrot ,","carrot ,","carrot ,",-1.000000,amara veggie burgers,http://www.food.com
5191,5191,54223,amara veggie burgers,2570470bf3,http://www.food.com/recipe/amara-veggie-burgers-287258,cup,14,"peppers, sweet, green, raw",2086.0,"pepper , sweet , green ,","sweet pepper , green ,","sweet pepper , green ,",80.000000,amara veggie burgers,http://www.food.com
22572,22572,236148,the woodward veggie burger,a46cfb3f64,http://www.food.com/recipe/the-woodward-veggie-burger-84235,cup,1,"onions, raw",160.0,"onion ,","onion ,","onion ,",1270.000000,woodward veggie burger,http://www.food.com
22573,22573,236149,the woodward veggie burger,a46cfb3f64,http://www.food.com/recipe/the-woodward-veggie-burger-84235,tablespoon,2,"spices, garlic powder",18.6,"spice , garlic powder","garlic , powder , dried","garlic , powder , dried",123.200000,woodward veggie burger,http://www.food.com
22574,22574,236150,the woodward veggie burger,a46cfb3f64,http://www.food.com/recipe/the-woodward-veggie-burger-84235,tablespoon,1,"oil, olive, salad or cooking",13.5,"oil , olive , salad cooking","albacore , olive oil , , drained","albacore , olive oil , , drained",329.411765,woodward veggie burger,http://www.food.com


In [299]:
menu_data.loc[43, 'Closest 7t recipe by hand'] = ['amara veggie burgers', 'the woodward veggie burger']

Next recipe

In [300]:
menu_data[menu_data.index==44]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand,Closest 7t recipe by hand
44,feaa6c7b-9b16-4d51-b71d-75518f16cb08.jpg,Sides,Frites,(v),no match,Not yet handled


In [301]:
set(seven_data["title_raw"][seven_data["title_raw"].str.contains("fries")].values.tolist())

{'baked okra fries',
 'carrot fries',
 'creamy maple dip for sweet potato fries',
 "crispy turnip 'fries'",
 'dip for sweet potato fries',
 'dutch peanutsauce for french fries',
 'garlic & herb sweet potato fries with rouge-ranch dip',
 'home fries with wild mushrooms',
 'maple mayo for sweet potato fries',
 'perfectly simple french fries',
 'sweet potato fries',
 'sweet potato fries with bbq mayo',
 'sweet-hot bbq tater fries',
 'un-fried french fries',
 'unfried french fries',
 'unfried french fries - lindsay jackson'}

In [302]:
seven_data[seven_data['title_raw'].isin(['baked okra fries', "crispy turnip 'fries'", 'perfectly simple french fries'])]

,Unnamed: 0,Unnamed: 0.1,title_raw,id,url,unit,quantity_raw,ingredient_raw,weight_per_ingr,ingredient_clean_comma,matched_ingredient,food_clean_comma,est_impact,title_clean,source_website
15922,15922,166402,baked okra fries,750d6de036,http://tastykitchen.com/recipes/special-dietary-needs/gluten-free/baked-okra-fries/,pound,1,"okra, raw",453.592000,"okra ,","okra , cooked , salt","okra , cooked , salt",-1.000000,baked okra fries,http://tastykitchen.com
15923,15923,166403,baked okra fries,750d6de036,http://tastykitchen.com/recipes/special-dietary-needs/gluten-free/baked-okra-fries/,tablespoon,3,"shortening confectionery, coconut (hydrogenated) and or palm kernel (hydrogenated)",38.400000,"confectionery , coconut palm kernel","coconut , kernel , dried","coconut , kernel , dried",300.000000,baked okra fries,http://tastykitchen.com
15924,15924,166404,baked okra fries,750d6de036,http://tastykitchen.com/recipes/special-dietary-needs/gluten-free/baked-okra-fries/,tablespoon,3,"rice flour, white, unenriched",29.625000,"rice flour , white ,",rice flour,rice flour,120.000000,baked okra fries,http://tastykitchen.com
15925,15925,166405,baked okra fries,750d6de036,http://tastykitchen.com/recipes/special-dietary-needs/gluten-free/baked-okra-fries/,teaspoon,1,"spices, cumin seed",2.100000,"spice , cumin seed","cumin , seed","cumin , seed",-1.000000,baked okra fries,http://tastykitchen.com
15926,15926,166406,baked okra fries,750d6de036,http://tastykitchen.com/recipes/special-dietary-needs/gluten-free/baked-okra-fries/,teaspoon,01-fŽvr,"spices, garlic powder",1.550000,"spice , garlic powder","garlic , powder , dried","garlic , powder , dried",123.200000,baked okra fries,http://tastykitchen.com
17179,17179,179781,crispy turnip 'fries',7dca33c3ce,http://allrecipes.com/recipe/crispy-turnip-fries/,pound,3,"turnips, raw",1360.776000,"turnip ,","turnip , cooked","turnip , cooked",-1.000000,crispy turnip fries,http://allrecipes.com
17180,17180,179782,crispy turnip 'fries',7dca33c3ce,http://allrecipes.com/recipe/crispy-turnip-fries/,tablespoon,1,"oil, olive, salad or cooking",13.500000,"oil , olive , salad cooking","albacore , olive oil , , drained","albacore , olive oil , , drained",329.411765,crispy turnip fries,http://allrecipes.com
17181,17181,179783,crispy turnip 'fries',7dca33c3ce,http://allrecipes.com/recipe/crispy-turnip-fries/,cup,2022-03-01 00:00:00,"cheese, parmesan, hard",49.568728,"cheese , parmesan , hard","parmesan cheese , cow milk","parmesan cheese , cow milk",1110.000000,crispy turnip fries,http://allrecipes.com
17182,17182,179784,crispy turnip 'fries',7dca33c3ce,http://allrecipes.com/recipe/crispy-turnip-fries/,teaspoon,1,"spices, garlic powder",3.100000,"spice , garlic powder","garlic , powder , dried","garlic , powder , dried",123.200000,crispy turnip fries,http://allrecipes.com
17183,17183,179785,crispy turnip 'fries',7dca33c3ce,http://allrecipes.com/recipe/crispy-turnip-fries/,teaspoon,1,"spices, paprika",2.300000,"spice , paprika",paprika,paprika,120.000000,crispy turnip fries,http://allrecipes.com


In [303]:
menu_data.loc[44, 'Closest 7t recipe by hand'] = 'perfectly simple french fries'

Next recipe

In [304]:
menu_data[menu_data.index==45]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand,Closest 7t recipe by hand
45,feaa6c7b-9b16-4d51-b71d-75518f16cb08.jpg,Sides,Frites patate douce,"Parmesan cheese, rosemary, spicy mayo",no match,Not yet handled


In [305]:
seven_data[seven_data['title_raw'].isin(['sweet potato fries', 'sweet potato fries with bbq mayo', \
                                         'sweet-hot bbq tater fries', 'garlic & herb sweet potato fries with rouge-ranch dip'])]

,Unnamed: 0,Unnamed: 0.1,title_raw,id,url,unit,quantity_raw,ingredient_raw,weight_per_ingr,ingredient_clean_comma,matched_ingredient,food_clean_comma,est_impact,title_clean,source_website
1028,1028,10685,sweet potato fries,0752df932c,http://www.kraftrecipes.com/recipes/sweet-potato-fries-155844.aspx,pound,1,"sweet potato, raw, unprepared",453.592000,"sweet potato , , unprepared","sweet potato ,","sweet potato ,",116.000000,sweet potato fries,http://www.kraftrecipes.com
1029,1029,10686,sweet potato fries,0752df932c,http://www.kraftrecipes.com/recipes/sweet-potato-fries-155844.aspx,cup,2022-04-01 00:00:00,"salad dressing, italian dressing, commercial, regular",58.800000,"salad dressing , italian dressing , commercial ,","salad dressing ,","salad dressing ,",120.000000,sweet potato fries,http://www.kraftrecipes.com
1030,1030,10687,sweet potato fries,0752df932c,http://www.kraftrecipes.com/recipes/sweet-potato-fries-155844.aspx,tablespoon,2,"cheese, parmesan, hard",18.588273,"cheese , parmesan , hard","parmesan cheese , cow milk","parmesan cheese , cow milk",1110.000000,sweet potato fries,http://www.kraftrecipes.com
4631,4631,47807,sweet-hot bbq tater fries,20ece4944a,http://www.epicurious.com/recipes/food/views/sweet-hot-bbq-tater-fries-235358,pound,2,"sweet potato, raw, unprepared",907.184000,"sweet potato , , unprepared","sweet potato ,","sweet potato ,",116.000000,sweet hot bbq tater fries,http://www.epicurious.com
4632,4632,47808,sweet-hot bbq tater fries,20ece4944a,http://www.epicurious.com/recipes/food/views/sweet-hot-bbq-tater-fries-235358,cup,2022-04-01 00:00:00,"oil, olive, salad or cooking",54.000000,"oil , olive , salad cooking","albacore , olive oil , , drained","albacore , olive oil , , drained",329.411765,sweet hot bbq tater fries,http://www.epicurious.com
4633,4633,47809,sweet-hot bbq tater fries,20ece4944a,http://www.epicurious.com/recipes/food/views/sweet-hot-bbq-tater-fries-235358,tablespoon,1,"spices, rosemary, dried",3.600000,"spice , rosemary , dried","rosemary , dried","rosemary , dried",-1.000000,sweet hot bbq tater fries,http://www.epicurious.com
4634,4634,47810,sweet-hot bbq tater fries,20ece4944a,http://www.epicurious.com/recipes/food/views/sweet-hot-bbq-tater-fries-235358,tablespoon,1,"sugars, brown",9.000000,"sugar , brown","sugar , brown","sugar , brown",38.000000,sweet hot bbq tater fries,http://www.epicurious.com
4635,4635,47811,sweet-hot bbq tater fries,20ece4944a,http://www.epicurious.com/recipes/food/views/sweet-hot-bbq-tater-fries-235358,teaspoon,1,"spices, garlic powder",3.100000,"spice , garlic powder","garlic , powder , dried","garlic , powder , dried",123.200000,sweet hot bbq tater fries,http://www.epicurious.com
4636,4636,47812,sweet-hot bbq tater fries,20ece4944a,http://www.epicurious.com/recipes/food/views/sweet-hot-bbq-tater-fries-235358,teaspoon,2022-04-01 00:00:00,"spices, pepper, red or cayenne",0.450000,"spice , pepper , red cayenne",cayenne pepper,cayenne pepper,120.000000,sweet hot bbq tater fries,http://www.epicurious.com
13155,13155,137752,garlic & herb sweet potato fries with rouge-ranch dip,60bf49b5f7,http://www.kraftrecipes.com/recipes/garlic-herb-sweet-potato-fries-rouge-ranch-dip-180174.aspx,cup,1,"salad dressing, ranch dressing, regular",240.000000,"salad dressing , ranch dressing ,","salad dressing ,","salad dressing ,",120.000000,garlic herb sweet potato fries rouge ranch dip,http://www.kraftrecipes.com


In [306]:
menu_data.loc[45, 'Closest 7t recipe by hand'] = ['sweet potato fries', 'sweet-hot bbq tater fries', 'sweet potato fries with bbq mayo']

Next recipe

In [307]:
menu_data[menu_data.index==46]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand,Closest 7t recipe by hand
46,in-bocca-al-lupo_42356_293.jpg,Calzoni,Calzone parigino,"mozzarella, jambon, champignons, œuf, sauce tomates au dessus",no match,Not yet handled


In [308]:
set(seven_data["title_raw"][seven_data["title_raw"].str.contains("calzon")].values.tolist())

set()

In [309]:
set(seven_data["title_raw"][seven_data["title_raw"].str.contains("pizz")].values.tolist())

{'10 minute pizza dough!',
 '2 ingredient pizza crust (the slow roasted italian)',
 '2-ingredient pizza dough',
 '5 minute pizza base. no yeast!',
 'basic pizza sauce',
 'buffalo style chicken pizza recipe',
 "california chef's pizza blend recipe",
 "copycat: boston pizza's cactus cut potatoes dip",
 'homemade pizza sauce',
 "papa john's garlic butter for pizza dipping",
 'pizza cheese',
 'pizza popcorn',
 'pizza spice blend',
 'pizza spices',
 'simple pizza sauce',
 'spicy adobo for pizza',
 'ww 2 points - pizza nachos'}

In [310]:
set(seven_data["title_raw"][seven_data["title_raw"].str.contains("pizz") & seven_data["ingredient_raw"].str.contains("ham")].values.tolist())

set()

In [311]:
set(seven_data["title_raw"][seven_data["title_raw"].str.contains("pizz") & seven_data["ingredient_raw"].str.contains("egg")].values.tolist())

set()

In [312]:
menu_data.loc[46, 'Closest 7t recipe by hand'] = 'no match'

Next recipe

In [313]:
menu_data[menu_data.index==50]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand,Closest 7t recipe by hand
50,in-bocca-al-lupo_42356_293.jpg,Planches,Salumi misti,assortiment de charcuteries italiennes,Planche de charcuterie,Not yet handled


In [314]:
set(seven_data["title_raw"][seven_data["title_raw"].str.contains("charcuterie")].values.tolist())

set()

In [315]:
set(seven_data["title_raw"][seven_data["title_raw"].str.contains("platter")].values.tolist())

{'antipasto platter with prosciutto and melon recipe',
 'roasted grapes for cheese platter',
 'veggie dip for veggie platters'}

In [316]:
set(seven_data["title_raw"][seven_data["title_raw"].str.contains("cold")].values.tolist())

{'cold and flu soothing drink',
 'cold breakfast oatmeal',
 'cold brigadeiros',
 'cold fruit explosion soup',
 'cold melon soup',
 'cold spinach dip',
 'cold strawberry soup',
 'herbal hot oil rub for cold joints and muscle spasm',
 'ladybirds choc fudge sauce , hot or cold or as a pudding .',
 'libbies hot orange juice cold remedy',
 'milk and honey cold remedy',
 'soothes coughs, colds,& indigestion',
 'tasty and cold yogurt shake'}

In [317]:
set(seven_data["title_raw"][seven_data["title_raw"].str.contains("deli meat")].values.tolist())

set()

In [318]:
set(seven_data["title_raw"][seven_data["title_raw"].str.contains("itali")].values.tolist())

{'2 ingredient pizza crust (the slow roasted italian)',
 'brown rice italian medley',
 'creamy italian chicken',
 'creamy italian dressing',
 'creamy parmesan italian dressing (weight watcher friendly)',
 'easy cheesy asparagus (italian style)',
 'easy italian popcorn',
 "emeril's italian blend",
 'homemade linen and sweater protector & cedar chest revitaliser',
 'hot italian tomato and cream cheese dip',
 'italian balsamic salad dressing',
 'italian blend spice mix',
 'italian butter',
 'italian cheese bread',
 'italian chicken',
 'italian chicken by chef #739656',
 'italian dressing croutons',
 'italian flank steak',
 'italian grilled shrimp kabobs',
 'italian herb chicken',
 'italian mustard marinade',
 'italian omelet',
 'italian parmesian croutons for soups',
 'italian porketta',
 'italian pudding',
 'italian salad dressing mix',
 'italian seasoning',
 'italian seasoning blend',
 'italian seasoning mix',
 'italian tofu',
 'italian vegetable dip',
 'protein packed italian dip',
 'q

In [319]:
set(seven_data["title_raw"][seven_data["title_raw"].str.contains("cooked meat")].values.tolist())

set()

In [320]:
seven_data[seven_data['title_raw'].isin(['antipasto platter with prosciutto and melon recipe'])]

,Unnamed: 0,Unnamed: 0.1,title_raw,id,url,unit,quantity_raw,ingredient_raw,weight_per_ingr,ingredient_clean_comma,matched_ingredient,food_clean_comma,est_impact,title_clean,source_website
7158,7158,75190,antipasto platter with prosciutto and melon recipe,342f7aa931,http://cookeatshare.com/recipes/antipasto-platter-with-prosciutto-and-melon-65787,ounce,8,"lettuce, cos or romaine, raw",226.796000,"lettuce , co romaine ,","co romaine lettuce ,","co romaine lettuce ,",80.000000,antipasto platter prosciutto melon recipe,http://cookeatshare.com
7159,7159,75191,antipasto platter with prosciutto and melon recipe,342f7aa931,http://cookeatshare.com/recipes/antipasto-platter-with-prosciutto-and-melon-65787,cup,2,"melons, cantaloupe, raw",354.000000,"melon , cantaloupe ,","melon , cantaloupe ex cavaillon charentais melon , pulp ,","melon , cantaloupe ex cavaillon charentais melon , pulp ,",-1.000000,antipasto platter prosciutto melon recipe,http://cookeatshare.com
7160,7160,75192,antipasto platter with prosciutto and melon recipe,342f7aa931,http://cookeatshare.com/recipes/antipasto-platter-with-prosciutto-and-melon-65787,cup,1,"pineapple, raw, all varieties",165.000000,"pineapple , , variety","pineapple , pulp ,","pineapple , pulp ,",-1.000000,antipasto platter prosciutto melon recipe,http://cookeatshare.com
7161,7161,75193,antipasto platter with prosciutto and melon recipe,342f7aa931,http://cookeatshare.com/recipes/antipasto-platter-with-prosciutto-and-melon-65787,cup,2022-04-01 00:00:00,"nuts, almonds",35.750000,"nut , almond",almond cake,almond cake,-1.000000,antipasto platter prosciutto melon recipe,http://cookeatshare.com
7162,7162,75195,antipasto platter with prosciutto and melon recipe,342f7aa931,http://cookeatshare.com/recipes/antipasto-platter-with-prosciutto-and-melon-65787,tablespoon,2,"oil, olive, salad or cooking",27.000000,"oil , olive , salad cooking","albacore , olive oil , , drained","albacore , olive oil , , drained",329.411765,antipasto platter prosciutto melon recipe,http://cookeatshare.com
7163,7163,75196,antipasto platter with prosciutto and melon recipe,342f7aa931,http://cookeatshare.com/recipes/antipasto-platter-with-prosciutto-and-melon-65787,tablespoon,2,"vinegar, balsamic",32.000000,"vinegar , balsamic","vinegar , balsamic","vinegar , balsamic",167.000000,antipasto platter prosciutto melon recipe,http://cookeatshare.com
7164,7164,75197,antipasto platter with prosciutto and melon recipe,342f7aa931,http://cookeatshare.com/recipes/antipasto-platter-with-prosciutto-and-melon-65787,tablespoon,2,"cheese, blue",30.679674,"cheese , blue","roquefort blue cheese sauce ,","roquefort blue cheese sauce ,",-1.000000,antipasto platter prosciutto melon recipe,http://cookeatshare.com


In [321]:
menu_data.loc[50, 'Closest 7t recipe by hand'] = 'no match'

Next recipe

In [322]:
menu_data[menu_data.index==51]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand,Closest 7t recipe by hand
51,in-bocca-al-lupo_42356_293.jpg,Insalata,Mista,"roquette, mozzarella di bufala, artichauts, jambon de parme, tomates cerises, parmesan","[Salade jambon cru, mozza & pistaches, Insalata di parma]",Not yet handled


In [323]:
set(seven_data["title_raw"][seven_data["title_raw"].str.contains("salad") & seven_data["ingredient_raw"].str.contains("ham")].values.tolist())

set()

In [324]:
set(seven_data["title_raw"][seven_data["title_raw"].str.contains("insalat")].values.tolist())

set()

In [325]:
set(seven_data["title_raw"][seven_data["title_raw"].str.contains("salad") & seven_data["ingredient_raw"].str.contains("artichoke")].values.tolist())

{'artichoke rice salad recipe',
 'asparagus and roasted red pepper salad recipe'}

In [326]:
seven_data[seven_data['title_raw'].isin(['asparagus and roasted red pepper salad recipe'])]

,Unnamed: 0,Unnamed: 0.1,title_raw,id,url,unit,quantity_raw,ingredient_raw,weight_per_ingr,ingredient_clean_comma,matched_ingredient,food_clean_comma,est_impact,title_clean,source_website
6000,6000,63217,asparagus and roasted red pepper salad recipe,2bdc69c1f3,http://cookeatshare.com/recipes/asparagus-and-roasted-red-pepper-salad-70744,cup,2,"asparagus, raw",268.000000,"asparagus ,","asparagus , , drained","asparagus , , drained",200.000000,asparagus roasted red pepper salad recipe,http://cookeatshare.com
6001,6001,63218,asparagus and roasted red pepper salad recipe,2bdc69c1f3,http://cookeatshare.com/recipes/asparagus-and-roasted-red-pepper-salad-70744,cup,01-fŽvr,"spices, pepper, red or cayenne",43.200000,"spice , pepper , red cayenne",cayenne pepper,cayenne pepper,120.000000,asparagus roasted red pepper salad recipe,http://cookeatshare.com
6002,6002,63219,asparagus and roasted red pepper salad recipe,2bdc69c1f3,http://cookeatshare.com/recipes/asparagus-and-roasted-red-pepper-salad-70744,cup,01-fŽvr,"artichokes, (globe or french), raw",75.000000,"artichoke , ,","jerusalem artichoke ,","jerusalem artichoke ,",160.000000,asparagus roasted red pepper salad recipe,http://cookeatshare.com
6003,6003,63220,asparagus and roasted red pepper salad recipe,2bdc69c1f3,http://cookeatshare.com/recipes/asparagus-and-roasted-red-pepper-salad-70744,tablespoon,2,"oil, olive, salad or cooking",27.000000,"oil , olive , salad cooking","albacore , olive oil , , drained","albacore , olive oil , , drained",329.411765,asparagus roasted red pepper salad recipe,http://cookeatshare.com
6004,6004,63221,asparagus and roasted red pepper salad recipe,2bdc69c1f3,http://cookeatshare.com/recipes/asparagus-and-roasted-red-pepper-salad-70744,tablespoon,3,"lemon juice, raw",45.750000,"lemon juice ,","lemon , pulp ,","lemon , pulp ,",460.000000,asparagus roasted red pepper salad recipe,http://cookeatshare.com
6005,6005,63222,asparagus and roasted red pepper salad recipe,2bdc69c1f3,http://cookeatshare.com/recipes/asparagus-and-roasted-red-pepper-salad-70744,tablespoon,2,"vinegar, red wine",29.800000,"vinegar , red wine","wine , red , Â°","wine , red , Â°",120.000000,asparagus roasted red pepper salad recipe,http://cookeatshare.com
6006,6006,63226,asparagus and roasted red pepper salad recipe,2bdc69c1f3,http://cookeatshare.com/recipes/asparagus-and-roasted-red-pepper-salad-70744,teaspoon,1,"dill weed, fresh",0.185417,"dill weed , fresh","dill , fresh","dill , fresh",120.000000,asparagus roasted red pepper salad recipe,http://cookeatshare.com
6007,6007,63227,asparagus and roasted red pepper salad recipe,2bdc69c1f3,http://cookeatshare.com/recipes/asparagus-and-roasted-red-pepper-salad-70744,tablespoon,1,"parsley, fresh",3.750000,"parsley , fresh","parsley , fresh","parsley , fresh",-1.000000,asparagus roasted red pepper salad recipe,http://cookeatshare.com


In [327]:
menu_data.loc[51, 'Closest 7t recipe by hand'] = 'no match'

Next recipe

In [328]:
menu_data[menu_data.index==52]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand,Closest 7t recipe by hand
52,in-bocca-al-lupo_42356_293.jpg,Desserts,Tiramisù,NaN,Tiramisu,Not yet handled


In [329]:
set(seven_data["title_raw"][seven_data["title_raw"].str.contains("tirami")].values.tolist())

set()

In [330]:
set(seven_data["title_raw"][seven_data["title_raw"].str.contains("italia")].values.tolist())

{'2 ingredient pizza crust (the slow roasted italian)',
 'brown rice italian medley',
 'creamy italian chicken',
 'creamy italian dressing',
 'creamy parmesan italian dressing (weight watcher friendly)',
 'easy cheesy asparagus (italian style)',
 'easy italian popcorn',
 "emeril's italian blend",
 'hot italian tomato and cream cheese dip',
 'italian balsamic salad dressing',
 'italian blend spice mix',
 'italian butter',
 'italian cheese bread',
 'italian chicken',
 'italian chicken by chef #739656',
 'italian dressing croutons',
 'italian flank steak',
 'italian grilled shrimp kabobs',
 'italian herb chicken',
 'italian mustard marinade',
 'italian omelet',
 'italian parmesian croutons for soups',
 'italian porketta',
 'italian pudding',
 'italian salad dressing mix',
 'italian seasoning',
 'italian seasoning blend',
 'italian seasoning mix',
 'italian tofu',
 'italian vegetable dip',
 'protein packed italian dip',
 'quick & easy italian cheese bread',
 'roasted italian asparagus with

In [331]:
menu_data.loc[52, 'Closest 7t recipe by hand'] = 'no match'

Next recipe

In [332]:
menu_data[menu_data.index==53]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand,Closest 7t recipe by hand
53,in-bocca-al-lupo_42356_293.jpg,Desserts,Panna cotta (avec gousses de vanille) et coulis de fruits de frais,NaN,Panna cotta à la fraise,Not yet handled


In [333]:
set(seven_data["title_raw"][seven_data["title_raw"].str.contains("panna")].values.tolist())

set()

In [334]:
set(seven_data["title_raw"][seven_data["title_raw"].str.contains("coulis")].values.tolist())

set()

In [335]:
set(seven_data["title_raw"][seven_data["title_raw"].str.contains("fruit sauce")].values.tolist())

{'autumn fruit sauce recipe',
 'quick fresh fruit sauces for yogurt, pancakes, and waffles'}

In [336]:
set(seven_data["title_raw"][seven_data["title_raw"].str.contains("cottage cheese")].values.tolist())

{'cottage cheese',
 'dry curd cottage cheese',
 'homemade cottage cheese',
 'homemade paneer (indian cottage cheese)'}

In [337]:
menu_data.loc[53, 'Closest 7t recipe by hand'] = 'no match'

Next recipe

In [338]:
menu_data[menu_data.index==57]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand,Closest 7t recipe by hand
57,menu_bobby.jpg,Pizz',Quattro,"Mozza fior di latte et scamorza IDF, gorgonzola, parmigiano, basilico",Pizza maison 3 fromages,Not yet handled


In [339]:
set(seven_data["title_raw"][seven_data["title_raw"].str.contains("pizz")].values.tolist())

{'10 minute pizza dough!',
 '2 ingredient pizza crust (the slow roasted italian)',
 '2-ingredient pizza dough',
 '5 minute pizza base. no yeast!',
 'basic pizza sauce',
 'buffalo style chicken pizza recipe',
 "california chef's pizza blend recipe",
 "copycat: boston pizza's cactus cut potatoes dip",
 'homemade pizza sauce',
 "papa john's garlic butter for pizza dipping",
 'pizza cheese',
 'pizza popcorn',
 'pizza spice blend',
 'pizza spices',
 'simple pizza sauce',
 'spicy adobo for pizza',
 'ww 2 points - pizza nachos'}

In [340]:
seven_data[seven_data['title_raw'].isin(['pizza cheese', "california chef's pizza blend recipe"])]

,Unnamed: 0,Unnamed: 0.1,title_raw,id,url,unit,quantity_raw,ingredient_raw,weight_per_ingr,ingredient_clean_comma,matched_ingredient,food_clean_comma,est_impact,title_clean,source_website
23978,23978,250494,pizza cheese,ae65e4d58c,http://www.food.com/recipe/pizza-cheese-346970,cup,14,"cheese, parmesan, hard",2081.886576,"cheese , parmesan , hard","parmesan cheese , cow milk","parmesan cheese , cow milk",1110.0,pizza cheese,http://www.food.com
23979,23979,250495,pizza cheese,ae65e4d58c,http://www.food.com/recipe/pizza-cheese-346970,cup,14,"cheese, mozzarella, low sodium",1848.000000,"cheese , mozzarella , low sodium","mozzarella cheese , cow milk","mozzarella cheese , cow milk",1110.0,pizza cheese,http://www.food.com
27501,27501,287431,california chef's pizza blend recipe,c8bed06eff,http://cookeatshare.com/recipes/california-chef-s-pizza-blend-97934,tablespoon,1,"onions, raw",10.000000,"onion ,","onion ,","onion ,",1270.0,california chef pizza blend recipe,http://cookeatshare.com
27502,27502,287432,california chef's pizza blend recipe,c8bed06eff,http://cookeatshare.com/recipes/california-chef-s-pizza-blend-97934,tablespoon,1,"seeds, sesame seeds, whole, dried",9.000000,"seed , sesame seed , whole , dried",sesame seed,sesame seed,473.0,california chef pizza blend recipe,http://cookeatshare.com
27503,27503,287433,california chef's pizza blend recipe,c8bed06eff,http://cookeatshare.com/recipes/california-chef-s-pizza-blend-97934,tablespoon,1,"spices, rosemary, dried",3.600000,"spice , rosemary , dried","rosemary , dried","rosemary , dried",-1.0,california chef pizza blend recipe,http://cookeatshare.com
27504,27504,287434,california chef's pizza blend recipe,c8bed06eff,http://cookeatshare.com/recipes/california-chef-s-pizza-blend-97934,tablespoon,1,"spices, basil, dried",2.100000,"spice , basil , dried","basil , dried","basil , dried",120.0,california chef pizza blend recipe,http://cookeatshare.com
27505,27505,287435,california chef's pizza blend recipe,c8bed06eff,http://cookeatshare.com/recipes/california-chef-s-pizza-blend-97934,tablespoon,1,"spices, garlic powder",9.300000,"spice , garlic powder","garlic , powder , dried","garlic , powder , dried",123.2,california chef pizza blend recipe,http://cookeatshare.com
27506,27506,287436,california chef's pizza blend recipe,c8bed06eff,http://cookeatshare.com/recipes/california-chef-s-pizza-blend-97934,tablespoon,01-fŽvr,"spices, marjoram, dried",0.900000,"spice , marjoram , dried","marjoram , dried","marjoram , dried",120.0,california chef pizza blend recipe,http://cookeatshare.com
27507,27507,287437,california chef's pizza blend recipe,c8bed06eff,http://cookeatshare.com/recipes/california-chef-s-pizza-blend-97934,tablespoon,01-fŽvr,"spices, oregano, dried",1.500000,"spice , oregano , dried","oregano , dried","oregano , dried",-1.0,california chef pizza blend recipe,http://cookeatshare.com
27508,27508,287438,california chef's pizza blend recipe,c8bed06eff,http://cookeatshare.com/recipes/california-chef-s-pizza-blend-97934,tablespoon,01-fŽvr,"spices, ginger, ground",2.700000,"spice , ginger , ground","ginger ,","ginger ,",80.0,california chef pizza blend recipe,http://cookeatshare.com


In [341]:
menu_data.loc[57, 'Closest 7t recipe by hand'] = 'no match'

Next recipe

In [342]:
menu_data[menu_data.index==59]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand,Closest 7t recipe by hand
59,menu_bobby.jpg,Pasta Fresca,Limone,"Trenette, sauce au citron, piment (ou non si vous aimez pas ), basilico",Pasta al limone,Not yet handled


In [343]:
set(seven_data["title_raw"][seven_data["title_raw"].str.contains("pasta")].values.tolist())

{"brigitte sealing's sausage pasta supper recipe",
 'carrot nutmeg pasta',
 'chicken pesto pasta',
 'cream cheese pasta sauce',
 'creamy bacon pasta salad',
 'creamy spinach pasta',
 'easy creamy pesto pasta',
 'homemade (fat free) pasta',
 'pasta blend',
 'pasta florentine',
 'pasta salad dressing',
 'pasta seasoning mix',
 'pasta shrimp parmesan',
 'pesto butter for vegetables and pasta',
 'roquefort pasta',
 'shrimp and pasta primavera',
 'simple cheese pasta sauce'}

In [344]:
set(seven_data["title_raw"][seven_data["title_raw"].str.contains("pasta") & seven_data["ingredient_raw"].str.contains("lemon")].values.tolist())

{'pasta salad dressing'}

In [345]:
set(seven_data["title_raw"][seven_data["title_raw"].str.contains("spaghet")].values.tolist())

{'chicken spaghetti for a crowd recipe', 'lemon basil spaghetti'}

In [346]:
set(seven_data["title_raw"][seven_data["title_raw"].str.contains("taglia")].values.tolist())

set()

In [347]:
set(seven_data["title_raw"][seven_data["title_raw"].str.contains("lemon")].values.tolist())

{'alternative to lemon meringue topping',
 'angelic lemony leeks & sprouts',
 'apple, lemon, ginger drink-sugar free',
 'asparagus with lemon butter recipe',
 'asparagus with lemon zest butter recipe',
 'asparagus with lemon, dill butter sauce',
 'asparagus with olive oil and lemon recipe',
 'barbecued lemon chicken recipe',
 'basil lemon gremolata dip',
 'best-ever lemon parmesan asparagus',
 'broccoli and chicken salad with lemon dressing',
 'broccoli and chicken salad with lemon dressing recipe',
 'brussels sprouts with lemon and mustard sauce recipe',
 'carrot salad with creamy lemon, yogurt, and chive dressing',
 'cauliflower with lemon',
 'cilantro lemon cream',
 'creamy lemon lite dessert recipe',
 'easiest lemon pepper fish',
 'easy lemon and dill salmon',
 'easy lemon pepper chicken wings',
 'easy lemon sauce for veggies recipe',
 'fresh lemon basil salad dressing',
 'frozen lemon cheesecake ice cream',
 'frozen lemonade shake',
 'honey lemon chicken marinade',
 'honey lemon h

In [348]:
seven_data[seven_data['title_raw'].isin(['lemon basil spaghetti', 'lemon butter parmesan linguini'])]

,Unnamed: 0,Unnamed: 0.1,title_raw,id,url,unit,quantity_raw,ingredient_raw,weight_per_ingr,ingredient_clean_comma,matched_ingredient,food_clean_comma,est_impact,title_clean,source_website
4654,4654,47985,lemon basil spaghetti,2115699d75,http://www.food.com/recipe/lemon-basil-spaghetti-330129,ounce,8,"noodles, egg, dry, unenriched",226.796000,"noodle , egg , dry ,","egg ,","egg ,",320.000000,lemon basil spaghetti,http://www.food.com
4655,4655,47986,lemon basil spaghetti,2115699d75,http://www.food.com/recipe/lemon-basil-spaghetti-330129,tablespoon,2,"oil, olive, salad or cooking",27.000000,"oil , olive , salad cooking","albacore , olive oil , , drained","albacore , olive oil , , drained",329.411765,lemon basil spaghetti,http://www.food.com
4656,4656,47987,lemon basil spaghetti,2115699d75,http://www.food.com/recipe/lemon-basil-spaghetti-330129,tablespoon,1,"lemon juice, raw",15.250000,"lemon juice ,","lemon , pulp ,","lemon , pulp ,",460.000000,lemon basil spaghetti,http://www.food.com
4657,4657,47988,lemon basil spaghetti,2115699d75,http://www.food.com/recipe/lemon-basil-spaghetti-330129,tablespoon,2,"cheese, parmesan, hard",18.588273,"cheese , parmesan , hard","parmesan cheese , cow milk","parmesan cheese , cow milk",1110.000000,lemon basil spaghetti,http://www.food.com
4658,4658,47989,lemon basil spaghetti,2115699d75,http://www.food.com/recipe/lemon-basil-spaghetti-330129,tablespoon,2,"spices, basil, dried",4.200000,"spice , basil , dried","basil , dried","basil , dried",120.000000,lemon basil spaghetti,http://www.food.com
25964,25964,272310,lemon butter parmesan linguini,be1c535016,http://tastykitchen.com/recipes/sidedishes/lemon-butter-parmesan-linguini/,ounce,6,"noodles, egg, dry, unenriched",170.097000,"noodle , egg , dry ,","egg ,","egg ,",320.000000,lemon butter parmesan linguini,http://tastykitchen.com
25965,25965,272311,lemon butter parmesan linguini,be1c535016,http://tastykitchen.com/recipes/sidedishes/lemon-butter-parmesan-linguini/,tablespoon,4,"butter, without salt",56.800000,"butter , salt",cocoa butter,cocoa butter,-1.000000,lemon butter parmesan linguini,http://tastykitchen.com
25966,25966,272312,lemon butter parmesan linguini,be1c535016,http://tastykitchen.com/recipes/sidedishes/lemon-butter-parmesan-linguini/,cup,2022-04-01 00:00:00,"cheese, parmesan, hard",37.176546,"cheese , parmesan , hard","parmesan cheese , cow milk","parmesan cheese , cow milk",1110.000000,lemon butter parmesan linguini,http://tastykitchen.com
25967,25967,272313,lemon butter parmesan linguini,be1c535016,http://tastykitchen.com/recipes/sidedishes/lemon-butter-parmesan-linguini/,cup,13,"lemon juice, raw",3172.000000,"lemon juice ,","lemon , pulp ,","lemon , pulp ,",460.000000,lemon butter parmesan linguini,http://tastykitchen.com
25968,25968,272314,lemon butter parmesan linguini,be1c535016,http://tastykitchen.com/recipes/sidedishes/lemon-butter-parmesan-linguini/,tablespoon,1,"spices, basil, dried",2.100000,"spice , basil , dried","basil , dried","basil , dried",120.000000,lemon butter parmesan linguini,http://tastykitchen.com


In [349]:
menu_data.loc[59, 'Closest 7t recipe by hand'] = ['lemon basil spaghetti', 'lemon butter parmesan linguini']

Next recipe

In [350]:
menu_data[menu_data.index==60]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand,Closest 7t recipe by hand
60,menu_bobby.jpg,Pasta Fresca,Norma,"Tagliatelle, sauce tomate et aubergines frites, ricotta salée fumée",Pasta alla Norma,Not yet handled


In [351]:
set(seven_data["title_raw"][seven_data["title_raw"].str.contains("pasta") & seven_data["ingredient_raw"].str.contains("tomat")].values.tolist())

set()

In [352]:
set(seven_data["title_raw"][seven_data["title_raw"].str.contains("pasta") & seven_data["ingredient_raw"].str.contains("ricot")].values.tolist())

{'creamy spinach pasta'}

In [353]:
set(seven_data["title_raw"][seven_data["title_raw"].str.contains("pasta") & seven_data["ingredient_raw"].str.contains("eggplant")].values.tolist())

set()

In [354]:
seven_data[seven_data['title_raw'].isin(['creamy spinach pasta'])]

,Unnamed: 0,Unnamed: 0.1,title_raw,id,url,unit,quantity_raw,ingredient_raw,weight_per_ingr,ingredient_clean_comma,matched_ingredient,food_clean_comma,est_impact,title_clean,source_website
3562,3562,37915,creamy spinach pasta,19b9f1ca2d,http://www.food.com/recipe/creamy-spinach-pasta-56710,ounce,6,"spinach, raw",170.097000,"spinach ,","spinach ,","spinach ,",80.0,creamy spinach pasta,http://www.food.com
3563,3563,37916,creamy spinach pasta,19b9f1ca2d,http://www.food.com/recipe/creamy-spinach-pasta-56710,tablespoon,2,"cheese, parmesan, hard",18.588273,"cheese , parmesan , hard","parmesan cheese , cow milk","parmesan cheese , cow milk",1110.0,creamy spinach pasta,http://www.food.com
3564,3564,37917,creamy spinach pasta,19b9f1ca2d,http://www.food.com/recipe/creamy-spinach-pasta-56710,cup,12,"cheese, ricotta, whole milk",2976.000000,"cheese , ricotta , whole milk",ricotta cheese,ricotta cheese,1110.0,creamy spinach pasta,http://www.food.com
3565,3565,37918,creamy spinach pasta,19b9f1ca2d,http://www.food.com/recipe/creamy-spinach-pasta-56710,cup,14,"yogurt, greek, plain, nonfat",3430.000000,"yogurt , greek , plain ,","yogurt , greek style , plain","yogurt , greek style , plain",240.0,creamy spinach pasta,http://www.food.com
3566,3566,37919,creamy spinach pasta,19b9f1ca2d,http://www.food.com/recipe/creamy-spinach-pasta-56710,tablespoon,1,"chives, raw",3.000000,"chive ,","chive spring onion , fresh","chive spring onion , fresh",-1.0,creamy spinach pasta,http://www.food.com
3567,3567,37920,creamy spinach pasta,19b9f1ca2d,http://www.food.com/recipe/creamy-spinach-pasta-56710,tablespoon,1,"nuts, walnuts, english",7.312500,"nut , walnut , english",walnut oil,walnut oil,-1.0,creamy spinach pasta,http://www.food.com


In [355]:
menu_data.loc[60, 'Closest 7t recipe by hand'] = 'no match'

Next recipe

In [356]:
menu_data[menu_data.index==61]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand,Closest 7t recipe by hand
61,menu_bobby.jpg,Pasta Fresca,Carbobby,"Spaghetti, crème d'œuf bio et de pecorino, guanciale, poivre",Pasta alla carbonara,Not yet handled


In [357]:
set(seven_data["title_raw"][seven_data["title_raw"].str.contains("pasta") & seven_data["ingredient_raw"].str.contains("ham")].values.tolist())

set()

In [358]:
set(seven_data["title_raw"][seven_data["title_raw"].str.contains("carbo")].values.tolist())

set()

In [359]:
set(seven_data["title_raw"][seven_data["title_raw"].str.contains("pasta") & seven_data["ingredient_raw"].str.contains("bacon")].values.tolist())

set()

In [360]:
set(seven_data["title_raw"][seven_data["title_raw"].str.contains("pasta") & seven_data["ingredient_raw"].str.contains("lardon")].values.tolist())

set()

In [361]:
set(seven_data["title_raw"][seven_data["title_raw"].str.contains("spaghet")].values.tolist())

{'chicken spaghetti for a crowd recipe', 'lemon basil spaghetti'}

In [362]:
menu_data.loc[61, 'Closest 7t recipe by hand'] = 'no match'

Next recipe

In [363]:
menu_data[menu_data.index==66]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand,Closest 7t recipe by hand
66,menu-PNY-BURGER-GAITE.jpg,Hamburgers de qualité,Vegan Forest Nirvana,"Pain vegan, steak végétal ""Les nouveaux Fermiers"", cheddar vegan, ketchup, moutarde, pickles",Not yet handled,Not yet handled


In [364]:
set(seven_data["title_raw"][seven_data["title_raw"].str.contains("burger")].values.tolist())

{'amara veggie burgers',
 'asian turkey burgers',
 'awesome turkey burgers',
 'barbecue burger topping recipe',
 'blue cheese stuffed turkey burgers',
 'chicken & broccoli burgers',
 'copycat burger king creamy apple cider vinaigrette',
 'greek burgers',
 "honey slaw from harold cohen's ultimate burger",
 'peanut burgers',
 'red, white & blue burger topper',
 'spicy turkey burgers',
 'spinach feta turkey burgers',
 'the woodward veggie burger',
 'tofu burgers',
 'turkey curry burgers',
 'turkey gyro burgers'}

In [365]:
seven_data[seven_data['title_raw'].isin(['tofu burgers'])]

,Unnamed: 0,Unnamed: 0.1,title_raw,id,url,unit,quantity_raw,ingredient_raw,weight_per_ingr,ingredient_clean_comma,matched_ingredient,food_clean_comma,est_impact,title_clean,source_website
13596,13596,142692,tofu burgers,648632b76d,http://www.food.com/recipe/tofu-burgers-204379,ounce,8,"tofu, raw, regular, prepared with calcium sulfate",226.796,"tofu , , , calcium sulfate","tofu , plain","tofu , plain",-1.000000,tofu burgers,http://www.food.com
13597,13597,142693,tofu burgers,648632b76d,http://www.food.com/recipe/tofu-burgers-204379,cup,2,oats,312.000,oat,"oat ,","oat ,",-1.000000,tofu burgers,http://www.food.com
13598,13598,142694,tofu burgers,648632b76d,http://www.food.com/recipe/tofu-burgers-204379,tablespoon,1,soy sauce made from soy (tamari),18.000,soy sauce soy,"soy sauce ,","soy sauce ,",120.000000,tofu burgers,http://www.food.com
13599,13599,142695,tofu burgers,648632b76d,http://www.food.com/recipe/tofu-burgers-204379,tablespoon,2,"oil, olive, salad or cooking",27.000,"oil , olive , salad cooking","albacore , olive oil , , drained","albacore , olive oil , , drained",329.411765,tofu burgers,http://www.food.com
30614,30614,317736,tofu burgers,dd77fc0278,http://www.food.com/recipe/tofu-burgers-442282,pound,1,"tofu, raw, regular, prepared with calcium sulfate",453.592,"tofu , , , calcium sulfate","tofu , plain","tofu , plain",-1.000000,tofu burgers,NaN
30615,30615,317737,tofu burgers,dd77fc0278,http://www.food.com/recipe/tofu-burgers-442282,cup,1.5,oats,234.000,oat,"oat ,","oat ,",-1.000000,tofu burgers,NaN
30616,30616,317738,tofu burgers,dd77fc0278,http://www.food.com/recipe/tofu-burgers-442282,tablespoon,2,soy sauce made from soy (tamari),36.000,soy sauce soy,"soy sauce ,","soy sauce ,",120.000000,tofu burgers,NaN
30617,30617,317739,tofu burgers,dd77fc0278,http://www.food.com/recipe/tofu-burgers-442282,tablespoon,2,"sauce, barbecue",34.000,"sauce , barbecue","barbecue sauce ,","barbecue sauce ,",460.000000,tofu burgers,NaN
30618,30618,317740,tofu burgers,dd77fc0278,http://www.food.com/recipe/tofu-burgers-442282,tablespoon,1,"mustard, prepared, yellow",15.000,"mustard , , yellow",mustard,mustard,120.000000,tofu burgers,NaN
30619,30619,317741,tofu burgers,dd77fc0278,http://www.food.com/recipe/tofu-burgers-442282,teaspoon,14,"spices, pepper, black",32.200,"spice , pepper , black","black pepper , powder","black pepper , powder",-1.000000,tofu burgers,NaN


In [366]:
menu_data.loc[66, 'Closest 7t recipe by hand'] = 'tofu burgers'

Next recipe

In [367]:
menu_data[menu_data.index==67]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand,Closest 7t recipe by hand
67,pasta.jpg,Pasta,Penne Al Pomodoro E Basilico,"Tomate, basilic",Not yet handled,Not yet handled


In [368]:
set(seven_data["title_raw"][seven_data["title_raw"].str.contains("pasta") & seven_data["ingredient_raw"].str.contains("tomat")].values.tolist())

set()

In [369]:
set(seven_data["title_raw"][seven_data["title_raw"].str.contains("pasta") & seven_data["title_raw"].str.contains("tomat")].values.tolist())

set()

In [370]:
set(seven_data["title_raw"][seven_data["title_raw"].str.contains("pomodoro")].values.tolist())

set()

In [371]:
set(seven_data["title_raw"][seven_data["title_raw"].str.contains("spagh") & seven_data["title_raw"].str.contains("tomat")].values.tolist())

set()

In [372]:
set(seven_data["title_raw"][seven_data["title_raw"].str.contains("spagh")].values.tolist())

{'chicken spaghetti for a crowd recipe', 'lemon basil spaghetti'}

In [373]:
set(seven_data["title_raw"][seven_data["title_raw"].str.contains("penne")].values.tolist())

{'lemon penne & shrimp medley'}

In [374]:
set(seven_data["title_raw"][seven_data["title_raw"].str.contains("pasta") & seven_data["ingredient_raw"].str.contains("basil")].values.tolist())

{'easy creamy pesto pasta', 'pasta blend', 'pasta seasoning mix'}

In [375]:
menu_data.loc[67, 'Closest 7t recipe by hand'] = 'no match'

Next recipe

In [376]:
menu_data[menu_data.index==68]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand,Closest 7t recipe by hand
68,pasta.jpg,Pasta,Penne Arrabiatta,"Tomate, piment",Not yet handled,Not yet handled


In [377]:
set(seven_data["title_raw"][seven_data["title_raw"].str.contains("arrabia")].values.tolist())

set()

In [378]:
menu_data.loc[68, 'Closest 7t recipe by hand'] = 'no match'

Next recipe

In [379]:
menu_data[menu_data.index==69]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand,Closest 7t recipe by hand
69,pasta.jpg,Pasta,Spaghetti Alla Bolognese,"Tomate, viande de bœuf",Not yet handled,Not yet handled


In [380]:
set(seven_data["title_raw"][seven_data["title_raw"].str.contains("bologn")].values.tolist())

{'deer camp hot mustard dip, (for bologna)'}

In [381]:
set(seven_data["title_raw"][seven_data["title_raw"].str.contains("pasta") & seven_data["ingredient_raw"].str.contains("beef")].values.tolist())

set()

In [382]:
set(seven_data["title_raw"][seven_data["title_raw"].str.contains("pasta") & seven_data["ingredient_raw"].str.contains("meat")].values.tolist())

{'chicken pesto pasta'}

In [383]:
set(seven_data["title_raw"][seven_data["ingredient_raw"].str.contains("bologn")].values.tolist())

set()

In [384]:
menu_data.loc[69, 'Closest 7t recipe by hand'] = 'no match'

Next recipe

In [385]:
menu_data[menu_data.index==71]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand,Closest 7t recipe by hand
71,pasta.jpg,Pasta,Linguine Al Salmone,"Crème fraîche, saumon fumé",Not yet handled,Not yet handled


In [386]:
set(seven_data["title_raw"][seven_data["title_raw"].str.contains("salmon")].values.tolist())

{'apricot - horseradish salmon',
 'asian glazed salmon',
 'asian grilled salmon',
 'asian sesame salmon',
 'asian-inspired alaska wild salmon',
 'baked marinated salmon',
 'baked orange salmon',
 'baked salmon patties',
 'baked salmon with mustard-dill sauce',
 'baked sesame ginger salmon',
 'barbecue salmon',
 'bbq plank salmon',
 'christmas salmon party roll',
 'cinnamon pecan crusted salmon',
 'coriander coated salmon',
 'delicious soy-ginger salmon marinade',
 'dill smoked-salmon hash',
 'dilled salmon over couscous',
 'double-salmon dip',
 'easy baked honey glazed salmon',
 'easy baked salmon',
 'easy balsamic-honey-mustard glazed salmon',
 'easy lemon and dill salmon',
 'fennel, apple and salmon salad',
 'garlic ginger crust salmon',
 'garlic ginger salmon',
 'george foreman simple grilled salmon',
 'ginger dijon salmon',
 'goat cheese potato cakes with smoked salmon',
 'grilled honey-balsamic salmon',
 'grilled mustard dill salmon',
 'grilled salmon sauce no. 2',
 'grilled salmo

In [387]:
seven_data[seven_data['title_raw'].isin(['smoked salmon pate'])]

,Unnamed: 0,Unnamed: 0.1,title_raw,id,url,unit,quantity_raw,ingredient_raw,weight_per_ingr,ingredient_clean_comma,matched_ingredient,food_clean_comma,est_impact,title_clean,source_website
20448,20448,213878,smoked salmon pate,94f5efa822,http://www.foodgeeks.com/recipes/4247,fl. oz,8,"cheese, parmesan, hard",148.706184,"cheese , parmesan , hard","parmesan cheese , cow milk","parmesan cheese , cow milk",1110.0,smoked salmon pate,http://www.foodgeeks.com
20449,20449,213879,smoked salmon pate,94f5efa822,http://www.foodgeeks.com/recipes/4247,fl. oz,4,"fish, salmon, atlantic, wild, raw",68.000000,"fish , salmon , atlantic , wild ,","salmon , , wild","salmon , , wild",467.0,smoked salmon pate,http://www.foodgeeks.com
20450,20450,213880,smoked salmon pate,94f5efa822,http://www.foodgeeks.com/recipes/4247,dash,1,"spices, pepper, red or cayenne",0.225000,"spice , pepper , red cayenne",cayenne pepper,cayenne pepper,120.0,smoked salmon pate,http://www.foodgeeks.com
20451,20451,213881,smoked salmon pate,94f5efa822,http://www.foodgeeks.com/recipes/4247,teaspoon,1,"lemon juice, raw",5.083333,"lemon juice ,","lemon , pulp ,","lemon , pulp ,",460.0,smoked salmon pate,http://www.foodgeeks.com
20452,20452,213882,smoked salmon pate,94f5efa822,http://www.foodgeeks.com/recipes/4247,tablespoon,1,"parsley, fresh",3.750000,"parsley , fresh","parsley , fresh","parsley , fresh",-1.0,smoked salmon pate,http://www.foodgeeks.com
20453,20453,213883,smoked salmon pate,94f5efa822,http://www.foodgeeks.com/recipes/4247,cup,1,"nuts, walnuts, english",117.000000,"nut , walnut , english",walnut oil,walnut oil,-1.0,smoked salmon pate,http://www.foodgeeks.com


In [388]:
set(seven_data["title_raw"][seven_data["title_raw"].str.contains("linguin")].values.tolist())

{'lemon butter parmesan linguini'}

In [389]:
menu_data.loc[71, 'Closest 7t recipe by hand'] = 'no match'

Next recipe

In [390]:
menu_data[menu_data.index==72]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand,Closest 7t recipe by hand
72,pasta.jpg,Pasta,Penne Al Formaggio,"Gorgonzola, parmigiano",Not yet handled,Not yet handled


In [391]:
set(seven_data["title_raw"][seven_data["title_raw"].str.contains("pasta")].values.tolist())

{"brigitte sealing's sausage pasta supper recipe",
 'carrot nutmeg pasta',
 'chicken pesto pasta',
 'cream cheese pasta sauce',
 'creamy bacon pasta salad',
 'creamy spinach pasta',
 'easy creamy pesto pasta',
 'homemade (fat free) pasta',
 'pasta blend',
 'pasta florentine',
 'pasta salad dressing',
 'pasta seasoning mix',
 'pasta shrimp parmesan',
 'pesto butter for vegetables and pasta',
 'roquefort pasta',
 'shrimp and pasta primavera',
 'simple cheese pasta sauce'}

In [392]:
seven_data[seven_data['title_raw'].isin(['roquefort pasta'])]

,Unnamed: 0,Unnamed: 0.1,title_raw,id,url,unit,quantity_raw,ingredient_raw,weight_per_ingr,ingredient_clean_comma,matched_ingredient,food_clean_comma,est_impact,title_clean,source_website
23355,23355,244800,roquefort pasta,aa97b703e9,http://www.food.com/recipe/roquefort-pasta-257644,ounce,16,"egg substitute, powder",453.592,"egg , powder","egg , powder","egg , powder",-1.0,roquefort pasta,http://www.food.com
23356,23356,244801,roquefort pasta,aa97b703e9,http://www.food.com/recipe/roquefort-pasta-257644,tablespoon,2,"butter, without salt",28.400,"butter , salt",cocoa butter,cocoa butter,-1.0,roquefort pasta,http://www.food.com
23357,23357,244802,roquefort pasta,aa97b703e9,http://www.food.com/recipe/roquefort-pasta-257644,tablespoon,2,"wheat flour, white, all-purpose, unenriched",15.625,"wheat flour , white , ,","wheat flour ,","wheat flour ,",-1.0,roquefort pasta,http://www.food.com
23358,23358,244806,roquefort pasta,aa97b703e9,http://www.food.com/recipe/roquefort-pasta-257644,cup,2,"milk, fluid, 1% fat, without added vitamin a and vitamin d",488.000,"milk , , fat , added vitamin vitamin","yogurt , fermented milk dairy specialty , fruit , sweetener , fat free , fortified vitamin","yogurt , fermented milk dairy specialty , fruit , sweetener , fat free , fortified vitamin",-1.0,roquefort pasta,http://www.food.com
23359,23359,244807,roquefort pasta,aa97b703e9,http://www.food.com/recipe/roquefort-pasta-257644,teaspoon,14,"spices, pepper, black",32.200,"spice , pepper , black","black pepper , powder","black pepper , powder",-1.0,roquefort pasta,http://www.food.com
23360,23360,244808,roquefort pasta,aa97b703e9,http://www.food.com/recipe/roquefort-pasta-257644,ounce,6,"cheese, roquefort",170.097,"cheese , roquefort","roquefort blue cheese sauce ,","roquefort blue cheese sauce ,",-1.0,roquefort pasta,http://www.food.com


In [393]:
set(seven_data["title_raw"][seven_data["title_raw"].str.contains("gorgonzola")].values.tolist())

{'gorgonzola butter', 'gorgonzola butter (for steaks)'}

In [394]:
set(seven_data["title_raw"][seven_data["ingredient_raw"].str.contains("gorgonzola")].values.tolist())

set()

In [395]:
menu_data.loc[72, 'Closest 7t recipe by hand'] = 'roquefort pasta'

Next recipe

In [396]:
menu_data[menu_data.index==78]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand,Closest 7t recipe by hand
78,fabbrezza_202084_665.jpg,Plat au choix,"""Petit"" Lasagne de Tonia",NaN,Not yet handled,Not yet handled


In [397]:
set(seven_data["title_raw"][seven_data["title_raw"].str.contains("lasagn")].values.tolist())

{'super easy microwave cheese sauce for lasagna',
 'sweet and savory polenta lasagna',
 'tofu lasagna filling (ricotta substitute)'}

In [398]:
set(seven_data["title_raw"][seven_data["ingredient_raw"].str.contains("lasagn")].values.tolist())

set()

In [399]:
menu_data.loc[78, 'Closest 7t recipe by hand'] = 'no match'

Next recipe

In [400]:
menu_data[menu_data.index==79]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand,Closest 7t recipe by hand
79,fabbrezza_202084_665.jpg,Dessert,Baba au Rhum,NaN,Not yet handled,Not yet handled


In [401]:
set(seven_data["title_raw"][seven_data["title_raw"].str.contains("rum")].values.tolist())

{'another crumble topping',
 'apple and pear crumble',
 'apple crumble',
 'apple crumble (a very simple recipe for a sunday afternoon treat)',
 'apple crumble in a jar',
 'apple crumble pie',
 'apricot and pecan crumble recipe',
 'baked honey glazed chicken drumsticks',
 'balsamic strawberry crumble',
 'berry almond crumble oatmeal',
 'berry crumble bake',
 'bread crumbs',
 'cinnamon sugar crumb topping',
 'crumb topping for pie',
 'crumb topping for pies',
 'crumble topping',
 'del monte fruit crumble',
 'drumstick torte recipe',
 'easy honey drummies',
 'fruit crumble',
 'gingered apricot-apple crumble',
 'hot buttered rum',
 'hot buttered rum batter mix',
 'hot buttered rum mix',
 'hot buttered whiskey (or rum) batter',
 'hot rum milk (virgin and sugar-free)',
 'kale salad with lemon, parmesan and breadcrumbs',
 'make-ahead hot buttered rum mix',
 'oatmeal crumb topping',
 'peach and hazelnut crumbles',
 'peach crumble',
 'raspberry and pear crumble',
 'rumbledethumps',
 'rump roast

In [402]:
set(seven_data["title_raw"][seven_data["ingredient_raw"].str.contains("rum")].values.tolist())

{'chicken pesto pasta', 'healthy mac & cheese', 'spicy chinese noodles'}

In [403]:
set(seven_data["title_raw"][seven_data["title_raw"].str.contains("baba")].values.tolist())

set()

In [404]:
menu_data.loc[79, 'Closest 7t recipe by hand'] = 'no match'

Next recipe

In [405]:
menu_data[menu_data.index==3]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand,Closest 7t recipe by hand
3,burgart_205074_c70.jpg,Burgers,Green Chicken,"Filet de poulet, Tomme de Savoie IGP, Roquette, Mortadelle, Parmesan, Sauce verte maison",Not yet handled,Not yet handled


In [406]:
set(seven_data["title_raw"][seven_data["title_raw"].str.contains("burger")].values.tolist())

{'amara veggie burgers',
 'asian turkey burgers',
 'awesome turkey burgers',
 'barbecue burger topping recipe',
 'blue cheese stuffed turkey burgers',
 'chicken & broccoli burgers',
 'copycat burger king creamy apple cider vinaigrette',
 'greek burgers',
 "honey slaw from harold cohen's ultimate burger",
 'peanut burgers',
 'red, white & blue burger topper',
 'spicy turkey burgers',
 'spinach feta turkey burgers',
 'the woodward veggie burger',
 'tofu burgers',
 'turkey curry burgers',
 'turkey gyro burgers'}

In [407]:
set(seven_data["title_raw"][seven_data["title_raw"].str.contains("burger") & seven_data["ingredient_raw"].str.contains("chicken")].values.tolist())

{'chicken & broccoli burgers'}

In [408]:
set(seven_data["title_raw"][seven_data["title_raw"].str.contains("burger") & seven_data["ingredient_raw"].str.contains("turkey")].values.tolist())

{'asian turkey burgers',
 'awesome turkey burgers',
 'blue cheese stuffed turkey burgers',
 'greek burgers',
 'spicy turkey burgers',
 'spinach feta turkey burgers',
 'turkey curry burgers',
 'turkey gyro burgers'}

In [409]:
set(seven_data["title_raw"][(seven_data["title_raw"].str.contains("burger")) & (seven_data["ingredient_raw"].str.contains("cheese"))].values.tolist())

{'awesome turkey burgers',
 'blue cheese stuffed turkey burgers',
 'chicken & broccoli burgers',
 'greek burgers',
 'red, white & blue burger topper',
 'spinach feta turkey burgers'}

In [410]:
seven_data[seven_data['title_raw'].isin(['awesome turkey burgers', 'blue cheese stuffed turkey burgers', \
                                        'chicken & broccoli burgers', 'greek burgers', 'spinach feta turkey burgers'])]

,Unnamed: 0,Unnamed: 0.1,title_raw,id,url,unit,quantity_raw,ingredient_raw,weight_per_ingr,ingredient_clean_comma,matched_ingredient,food_clean_comma,est_impact,title_clean,source_website
2858,2858,30082,awesome turkey burgers,14e1d21f5a,http://www.food.com/recipe/awesome-turkey-burgers-444519,pound,1,"ground turkey, raw",453.592000,"ground turkey ,","turkey , meat ,","turkey , meat ,",420.000000,awesome turkey burgers,http://www.food.com
2859,2859,30083,awesome turkey burgers,14e1d21f5a,http://www.food.com/recipe/awesome-turkey-burgers-444519,ounce,2022-10-08 00:00:00,"cheese, feta",510.291000,"cheese , feta",feta cheese cow milk,feta cheese cow milk,1110.000000,awesome turkey burgers,http://www.food.com
2860,2860,30084,awesome turkey burgers,14e1d21f5a,http://www.food.com/recipe/awesome-turkey-burgers-444519,ounce,8,"cranberries, dried, sweetened",226.796000,"cranberry , dried ,","cranberry ,","cranberry ,",-1.000000,awesome turkey burgers,http://www.food.com
7496,7496,78305,chicken & broccoli burgers,36860f32c3,http://www.food.com/recipe/chicken-broccoli-burgers-359006,pound,2,"chicken, broiler or fryers, breast, skinless, boneless, meat only, raw",907.184000,"chicken , broiler fryer , breast , skinless , boneless , meat ,","chicken , meat ,","chicken , meat ,",580.000000,chicken broccoli burgers,http://www.food.com
7497,7497,78306,chicken & broccoli burgers,36860f32c3,http://www.food.com/recipe/chicken-broccoli-burgers-359006,ounce,10,"broccoli, raw",283.495000,"broccoli ,","broccoli ,","broccoli ,",80.000000,chicken broccoli burgers,http://www.food.com
7498,7498,78307,chicken & broccoli burgers,36860f32c3,http://www.food.com/recipe/chicken-broccoli-burgers-359006,cup,34,"cheese, cheddar",4488.000000,"cheese , cheddar","cheddar cheese , cow milk","cheddar cheese , cow milk",1110.000000,chicken broccoli burgers,http://www.food.com
7499,7499,78308,chicken & broccoli burgers,36860f32c3,http://www.food.com/recipe/chicken-broccoli-burgers-359006,tablespoon,2022-03-02 00:00:00,"chicken, broiler or fryers, breast, skinless, boneless, meat only, raw",43.750000,"chicken , broiler fryer , breast , skinless , boneless , meat ,","chicken , meat ,","chicken , meat ,",580.000000,chicken broccoli burgers,http://www.food.com
23721,23721,247854,blue cheese stuffed turkey burgers,aca99276d0,https://cookpad.com/us/recipes/331769-blue-cheese-stuffed-turkey-burgers,pound,1,"ground turkey, raw",453.592000,"ground turkey ,","turkey , meat ,","turkey , meat ,",420.000000,blue cheese stuffed turkey burgers,https://cookpad.com
23722,23722,247855,blue cheese stuffed turkey burgers,aca99276d0,https://cookpad.com/us/recipes/331769-blue-cheese-stuffed-turkey-burgers,cup,1,"cheese, blue",245.437391,"cheese , blue","roquefort blue cheese sauce ,","roquefort blue cheese sauce ,",-1.000000,blue cheese stuffed turkey burgers,https://cookpad.com
23723,23723,247856,blue cheese stuffed turkey burgers,aca99276d0,https://cookpad.com/us/recipes/331769-blue-cheese-stuffed-turkey-burgers,teaspoon,1,"oil, olive, salad or cooking",4.500000,"oil , olive , salad cooking","albacore , olive oil , , drained","albacore , olive oil , , drained",329.411765,blue cheese stuffed turkey burgers,https://cookpad.com


In [411]:
menu_data.loc[3, 'Closest 7t recipe by hand'] = ['awesome turkey burgers', 'blue cheese stuffed turkey burgers', \
                                        'chicken & broccoli burgers', 'greek burgers', 'spinach feta turkey burgers']

## Let's now save the file

In [412]:
menu_data[menu_data['Closest 7t recipe by hand']!='Not yet handled'].to_excel(data_path+"data/Tools/MenuTo7tRecipes_v1.xlsx")

Save to json to keep lists as lists

In [413]:
menu_data[menu_data['Closest 7t recipe by hand']!='Not yet handled'].to_json(data_path+"data/Tools/MenuTo7tRecipes_v1.json")

## Enlarge the database and prepare a list of menu recipes matched with JOW and 7t

Let's first finish the work that has been started ie treat recipes that have been tried to be matched with at least JOW or 7t

In [414]:
menu_data[(menu_data['Closest JOW recipe by hand']=='Not yet handled') & (menu_data['Closest 7t recipe by hand']!='Not yet handled')]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand,Closest 7t recipe by hand
3,burgart_205074_c70.jpg,Burgers,Green Chicken,"Filet de poulet, Tomme de Savoie IGP, Roquette, Mortadelle, Parmesan, Sauce verte maison",Not yet handled,"[awesome turkey burgers, blue cheese stuffed turkey burgers, chicken & broccoli burgers, greek burgers, spinach feta turkey burgers]"
25,Carte-in-seoul.png,Nouilles,Japchae - bœuf ou tofu -,"Nouilles de papate douce, légumes, boeuf ou tofu, accompagné de 3 banchan",Not yet handled,no match
66,menu-PNY-BURGER-GAITE.jpg,Hamburgers de qualité,Vegan Forest Nirvana,"Pain vegan, steak végétal ""Les nouveaux Fermiers"", cheddar vegan, ketchup, moutarde, pickles",Not yet handled,tofu burgers
67,pasta.jpg,Pasta,Penne Al Pomodoro E Basilico,"Tomate, basilic",Not yet handled,no match
68,pasta.jpg,Pasta,Penne Arrabiatta,"Tomate, piment",Not yet handled,no match
69,pasta.jpg,Pasta,Spaghetti Alla Bolognese,"Tomate, viande de bœuf",Not yet handled,no match
71,pasta.jpg,Pasta,Linguine Al Salmone,"Crème fraîche, saumon fumé",Not yet handled,no match
72,pasta.jpg,Pasta,Penne Al Formaggio,"Gorgonzola, parmigiano",Not yet handled,roquefort pasta
78,fabbrezza_202084_665.jpg,Plat au choix,"""Petit"" Lasagne de Tonia",NaN,Not yet handled,no match
79,fabbrezza_202084_665.jpg,Dessert,Baba au Rhum,NaN,Not yet handled,no match


In [415]:
menu_data[menu_data.index==3]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand,Closest 7t recipe by hand
3,burgart_205074_c70.jpg,Burgers,Green Chicken,"Filet de poulet, Tomme de Savoie IGP, Roquette, Mortadelle, Parmesan, Sauce verte maison",Not yet handled,"[awesome turkey burgers, blue cheese stuffed turkey burgers, chicken & broccoli burgers, greek burgers, spinach feta turkey burgers]"


In [416]:
jow_data[jow_data["recettes"].str.contains("burger")]

,cuisson,ingredients,recettes,only_ings
Blue Burger,40 minutes,"[['1 càc', ""confits d'oignons""], ['30 g', 'bleu'], ['1', 'boeuf (steak haché)'], ['150 g', 'patate douce'], ['1 poignée', 'salade (roquette)'], ['1', 'pain burger'], ['1/2 càc', ""moutarde à l'ancienne""]]",blue burger,"confits d'oignons, bleu, boeuf, patate douce, salade, pain burger, moutarde à l'ancienne"
French Burger,40 minutes,"[['2 càc', ""confits d'oignons""], ['30 g', 'comté'], ['1', 'boeuf (steak haché)'], ['150 g', 'pomme de terre'], ['10 g', 'salade (coeur de laitue)'], ['1', 'pain burger'], ['1/2 càc', ""moutarde à l'ancienne""]]",french burger,"confits d'oignons, comté, boeuf, pomme de terre, salade, pain burger, moutarde à l'ancienne"
Italian burger,40 minutes,"[['1/2 càs', 'sauce pesto'], ['1/2', 'mozzarella (boule)'], ['1', 'boeuf (steak haché)'], ['150 g', 'pomme de terre'], ['1', 'pain burger'], ['1/4', 'tomate'], ['facultatif', '1 pinc.', 'origan (séché)'], ['1 poignée', 'salade (roquette)']]",italian burger,"sauce pesto, mozzarella, boeuf, pomme de terre, pain burger, tomate, origan, salade"
Salmon burger,40 minutes,"[['1/2 càs', 'crème fraîche épaisse'], ['1/8', 'citron jaune'], ['1 poignée', 'epinard (frais)'], ['1/2', 'avocat'], ['1', 'saumon (frais)'], ['1', 'pain burger'], ['150 g', 'pomme de terre'], ['facultatif', '1/10 bou.', 'ciboulette']]",salmon burger,"crème fraîche épaisse, citron jaune, epinard, avocat, saumon, pain burger, pomme de terre, ciboulette"
Classic Cheeseburger,40 minutes,"[['1/2 càc', 'yellow mustard'], ['1 càc', 'ketchup'], ['1/8', 'oignon rouge'], ['150 g', 'pomme de terre'], ['10 g', 'salade (coeur de laitue)'], ['1', 'pain burger'], ['1 tran.', 'cheddar (tranches)'], ['1', 'boeuf (steak haché)']]",classic cheeseburger,"yellow mustard, ketchup, oignon rouge, pomme de terre, salade, pain burger, cheddar, boeuf"
Super Bacon Burger,40 minutes,"[['1/8', 'oignon rouge'], ['1 tran.', 'cheddar (tranches)'], ['1', 'boeuf (steak haché)'], ['150 g', 'pomme de terre'], ['10 g', 'salade (coeur de laitue)'], ['1', 'pain burger'], ['1 càc', 'moutarde'], ['1 càs', ""sirop d'érable""]]",super bacon burger,"oignon rouge, cheddar, boeuf, pomme de terre, salade, pain burger, moutarde, sirop d'érable"
Tom burger,40 minutes,"[['1 càc', ""confits d'oignons""], ['1/2 càs', 'crème fraîche épaisse'], ['1/4', 'échalote'], ['150 g', 'pomme de terre'], ['1', 'pain burger'], ['30 g', 'tomme'], ['1', 'boeuf (steak haché)'], ['1/10 bou.', 'persil (frais)']]",tom burger,"confits d'oignons, crème fraîche épaisse, échalote, pomme de terre, pain burger, tomme, boeuf, persil"
Brunch Burger,3 minutes,"[['1', 'pain burger'], ['2', 'oeuf'], ['15 g', 'beurre'], ['facultatif', '1/10 bou.', 'ciboulette'], ['1 tran.', 'jambon blanc']]",brunch burger,"pain burger, oeuf, beurre, ciboulette, jambon blanc"
Cheeseburger veggie,4 minutes,"[['1', 'pain burger'], ['1', 'steak végétal'], ['1/2 poignée', 'salade (coeur de laitue)'], ['1 càc', 'yellow mustard'], ['1 càs', 'ketchup'], ['1/8', 'oignon rouge'], ['1 tran.', 'cheddar (tranches)']]",cheeseburger veggie,"pain burger, steak végétal, salade, yellow mustard, ketchup, oignon rouge, cheddar"
Burger raclette,5 minutes,"[['1', 'pain burger (brasserie)'], ['1', 'boeuf (steak haché)'], ['2 càc', ""confits d'oignons""], ['1 càc', ""moutarde à l'ancienne""], ['20 g', 'raclette']]",burger raclette,"pain burger, boeuf, confits d'oignons, moutarde à l'ancienne, raclette"


In [417]:
jow_data[(jow_data["recettes"].str.contains("burger")) & (jow_data["only_ings"].str.contains("poulet"))]

,cuisson,ingredients,recettes,only_ings


In [418]:
jow_data[(jow_data["recettes"].str.contains("sandwich")) & (jow_data["only_ings"].str.contains("poulet"))]

,cuisson,ingredients,recettes,only_ings
Club sandwich,10 minutes,"[['3 tran.', 'pain de mie'], ['50 g', 'poulet émincé (cuit)'], ['25 g', 'bacon'], ['1', 'oeuf'], ['1/2', 'tomate'], ['15 g', 'emmental'], ['20 g', 'salade (coeur de laitue)'], ['1/2 càs', 'mayonnaise']]",club sandwich,"pain de mie, poulet émincé, bacon, oeuf, tomate, emmental, salade, mayonnaise"


In [419]:
menu_data.loc[3, 'Closest JOW recipe by hand'] = 'no match'

Next recipe

In [420]:
menu_data[menu_data.index==25]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand,Closest 7t recipe by hand
25,Carte-in-seoul.png,Nouilles,Japchae - bœuf ou tofu -,"Nouilles de papate douce, légumes, boeuf ou tofu, accompagné de 3 banchan",Not yet handled,no match


In [421]:
jow_data[jow_data["recettes"].str.contains("nouille")]

,cuisson,ingredients,recettes,only_ings
Nouilles sautées aux légumes d'été,7 minutes,"[['70 g', 'nouilles chinoises (aux oeufs)'], ['50 g', 'tomate cerise'], ['50 g', 'courgette'], ['1/2 càs', 'sauce soja salée'], ['1/4 gou.', 'ail']]",nouilles sautées aux légumes d'été,"nouilles chinoises, tomate cerise, courgette, sauce soja salée, ail"
"Salade de nouilles, courgette & sésame",10 minutes,"[['80 g', 'nouilles de riz'], ['1', 'courgette'], ['1 càs', 'huile de sésame'], ['1 càc', 'sauce soja salée'], ['1/5 bou.', 'coriandre (frais)'], ['facultatif', '1/4', 'échalote']]","salade de nouilles, courgette & sésame","nouilles de riz, courgette, huile de sésame, sauce soja salée, coriandre, échalote"
Nouilles champignons & sésame,10 minutes,"[['80 g', 'nouilles de riz'], ['100 g', 'champignons de paris (frais)'], ['1 càc', 'huile de sésame'], ['1 càc', 'sauce soja salée'], ['1/4 càc'], ['facultatif', '1 cm']]",nouilles champignons & sésame,"nouilles de riz, champignons de paris, huile de sésame, sauce soja salée"
Nouilles sautées au poulet,12 minutes,"[['70 g', 'nouilles chinoises (aux oeufs)'], ['80 g', 'brocoli (frais)'], ['1/2', 'poivron rouge'], ['1/2 càs', 'sauce soja salée'], ['1/4 gou.', 'ail'], ['facultatif', '1/2 càc', 'gingembre (moulu)'], ['1', 'poulet (escalope)']]",nouilles sautées au poulet,"nouilles chinoises, brocoli, poivron rouge, sauce soja salée, ail, gingembre, poulet"
Nouilles sautées aux crevettes,6 minutes,"[['70 g', 'nouilles chinoises (aux oeufs)'], ['70 g', 'haricot vert (frais)'], ['1/2', 'carotte (frais)'], ['1/2 càs', 'sauce soja salée'], ['1/4 gou.', 'ail'], ['facultatif', '1/2 càc', 'gingembre (moulu)'], ['80 g', 'crevette (cuite)']]",nouilles sautées aux crevettes,"nouilles chinoises, haricot vert, carotte, sauce soja salée, ail, gingembre, crevette"
Nouilles sautées au bœuf,10 minutes,"[['70 g', 'nouilles chinoises (aux oeufs)'], ['80 g', 'brocoli (frais)'], ['40 g', 'pousses de soja'], ['2 càs', 'sauce soja salée'], ['1/2 gou.', 'ail'], ['facultatif', '1/2 càc', 'gingembre (moulu)'], ['1', 'boeuf (bavette)']]",nouilles sautées au boeuf,"nouilles chinoises, brocoli, pousses de soja, sauce soja salée, ail, gingembre, boeuf"
Nouilles sautées légumes & œuf,14 minutes,"[['70 g', 'nouilles chinoises (aux oeufs)'], ['70 g', 'champignons de paris (frais)'], ['80 g', 'haricot vert (frais)'], ['2 càs', 'sauce soja salée'], ['1/4 gou.', 'ail'], ['1', 'oeuf']]",nouilles sautées légumes & oeuf,"nouilles chinoises, champignons de paris, haricot vert, sauce soja salée, ail, oeuf"
Nouilles de riz & porc sauté,7 minutes,"[['80 g', 'nouilles de riz'], ['80 g', 'chair à saucisse'], ['50 g', 'concombre'], ['20 g', 'cacahuète'], ['1 pinc.', ""piment d'espelette""], ['2 càs', 'sauce soja salée'], ['facultatif', '1/10 bou.', 'menthe (feuilles)']]",nouilles de riz & porc sauté,"nouilles de riz, chair à saucisse, concombre, cacahuète, piment d'espelette, sauce soja salée, menthe"
Nouilles sautées poulet & champignons,15 minutes,"[['75 g', 'nouilles chinoises (aux oeufs)'], ['1/4', 'oignon jaune'], ['1/2', 'carotte (frais)'], ['1/2', 'poulet (escalope)'], ['1/2 gou.'], ['70 g'], ['5 g'], ['1 càs']]",nouilles sautées poulet & champignons,"nouilles chinoises, oignon jaune, carotte, poulet"
Poulet & nouilles sautées aux légumes,12 minutes,"[['70 g', 'nouilles chinoises (aux oeufs)'], ['1', 'poulet (escalope)'], ['250 g', 'julienne de légumes (surgelés)'], ['1 càs', 'sauce soja salée']]",poulet & nouilles sautées aux légumes,"nouilles chinoises, poulet, julienne de légumes, sauce soja salée"


In [422]:
jow_data[(jow_data["recettes"].str.contains("nouille")) & (jow_data["only_ings"].str.contains("tofu"))]

,cuisson,ingredients,recettes,only_ings


In [423]:
menu_data.loc[25, 'Closest JOW recipe by hand'] = jow_data[jow_data["recettes"]=='nouilles sautées au boeuf'].index[0]

Next recipe

In [424]:
menu_data[menu_data.index==66]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand,Closest 7t recipe by hand
66,menu-PNY-BURGER-GAITE.jpg,Hamburgers de qualité,Vegan Forest Nirvana,"Pain vegan, steak végétal ""Les nouveaux Fermiers"", cheddar vegan, ketchup, moutarde, pickles",Not yet handled,tofu burgers


In [425]:
jow_data[(jow_data["recettes"].str.contains("burger")) & (jow_data["only_ings"].str.contains("végé"))]

,cuisson,ingredients,recettes,only_ings
Cheeseburger veggie,4 minutes,"[['1', 'pain burger'], ['1', 'steak végétal'], ['1/2 poignée', 'salade (coeur de laitue)'], ['1 càc', 'yellow mustard'], ['1 càs', 'ketchup'], ['1/8', 'oignon rouge'], ['1 tran.', 'cheddar (tranches)']]",cheeseburger veggie,"pain burger, steak végétal, salade, yellow mustard, ketchup, oignon rouge, cheddar"


In [426]:
jow_data[(jow_data["recettes"].str.contains("burger")) & (jow_data["only_ings"].str.contains("tofu"))]

,cuisson,ingredients,recettes,only_ings


In [427]:
menu_data.loc[66, 'Closest JOW recipe by hand'] = jow_data[jow_data["recettes"]=='cheeseburger veggie'].index[0]

Next recipe

In [428]:
menu_data[menu_data.index==67]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand,Closest 7t recipe by hand
67,pasta.jpg,Pasta,Penne Al Pomodoro E Basilico,"Tomate, basilic",Not yet handled,no match


In [429]:
jow_data[(jow_data["recettes"].str.contains("pâte")) & (jow_data["only_ings"].str.contains("tomate"))]

,cuisson,ingredients,recettes,only_ings
Pâtes à la Sicilienne,9 minutes,"[['100 g', 'pâtes (conchiglioni)'], ['1 càs', 'câpres'], ['100 g', 'tomate cerise'], ['90 g', 'thon (conserve)'], ['1 càc']]",pâtes à la sicilienne,"pâtes, câpres, tomate cerise, thon"
Pâtes au pesto rosso,10 minutes,"[['100 g', 'pâtes (rigatoni)'], ['2 cl', ""huile d'olive""], ['10 g', 'pignons de pin'], ['1/4 gou.', 'ail'], ['20 g', 'parmesan (morceaux)'], ['50 g', 'tomate séchée']]",pâtes au pesto rosso,"pâtes, huile d'olive, pignons de pin, ail, parmesan, tomate séchée"
"Salade de pâtes, pesto, mozza",10 minutes,"[['100 g', 'pâtes (pipe rigate)'], ['40 g', 'tomate séchée'], ['1 càs', 'sauce pesto'], ['1 càs', 'vinaigre balsamique'], ['60 g', 'mozzarella (mini)']]","salade de pâtes, pesto, mozza","pâtes, tomate séchée, sauce pesto, vinaigre balsamique, mozzarella"
Pâtes chorizo & ricotta,15 minutes,"[['100 g', 'pâtes (rigatoni)'], ['30 g', 'chorizo (entier)'], ['1 càs', 'crème fraîche épaisse'], ['1 càs', 'ricotta'], ['80 g', 'sauce tomate (basilic)'], ['20 g', 'parmesan (râpé)']]",pâtes chorizo & ricotta,"pâtes, chorizo, crème fraîche épaisse, ricotta, sauce tomate, parmesan"
"Salade pâtes, tomates, mozzarella & chorizo",10 minutes,"[['80 g', 'pâtes (fusilli)'], ['100 g', 'tomate cerise'], ['1/4', 'concombre'], ['60 g', 'mozzarella (mini)']]","salade pâtes, tomates, mozzarella & chorizo","pâtes, tomate cerise, concombre, mozzarella"


In [430]:
jow_data[(jow_data["recettes"].str.contains("spaghetti")) & (jow_data["only_ings"].str.contains("tomate"))]

,cuisson,ingredients,recettes,only_ings
Spaghetti alla puttanesca,plat froid,"[['100 g', 'pâtes (spaghetti)'], ['100 g', 'tomate (chair)'], ['20 g', 'olive noire'], ['1/2 càs', 'câpres']]",spaghetti alla puttanesca,"pâtes, tomate, olive noire, câpres"
Spaghetti alla bolognese express,plat froid,"[['1/4', 'oignon jaune'], ['1/2', 'carotte (frais)'], ['100 g', 'tomate (purée)'], ['1/4 gou.', 'ail']]",spaghetti alla bolognese express,"oignon jaune, carotte, tomate, ail"
Cherry tomatoes spaghetti,10 minutes,"[['120 g', 'tomate cerise'], ['100 g', 'pâtes (spaghetti)'], ['15 g', 'parmesan (morceaux)'], ['1/10 bou.', 'basilic (frais)'], ['1/4 gou.', 'ail']]",cherry tomatoes spaghetti,"tomate cerise, pâtes, parmesan, basilic, ail"


In [431]:
jow_data[(jow_data["recettes"].str.contains("pasta")) & (jow_data["only_ings"].str.contains("tomate"))]

,cuisson,ingredients,recettes,only_ings
Greek pasta salad,8 minutes,"[['70 g', 'pâtes (fusilli)'], ['1', 'tomate'], ['15 g', 'olive noire'], ['130 g', 'concombre'], ['30 g', 'feta'], ['facultatif', '1/8', 'oignon rouge'], ['1 pinc.', 'origan (séché)']]",greek pasta salad,"pâtes, tomate, olive noire, concombre, feta, oignon rouge, origan"
Pasta al ragu d'anatra,plat froid,"[['100 g', 'pâtes (pappardelle)'], ['1/2', 'canard (confit)'], ['70 g', 'tomate pelée'], ['1/2', 'échalote']]",pasta al ragu d'anatra,"pâtes, canard, tomate pelée, échalote"
Tomato vegan pasta,15 minutes,"[['150 g', 'pâtes (rigatoni)'], ['100 g', 'tomate (purée)'], ['1 pinc.', 'origan (séché)'], ['1/4', 'bouillon de légumes (cube)'], ['1/2 gou.', 'ail'], ['1/2 càs', 'chapelure']]",tomato vegan pasta,"pâtes, tomate, origan, bouillon de légumes, ail, chapelure"
Pasta alla vodka,10 minutes,"[['120 g', 'pâtes (pipe rigate)'], ['15 g', 'parmesan (morceaux)'], ['1 càs', 'vodka'], ['5 cl', 'crème liquide'], ['30 g', 'tomate (concentré)'], ['1/4 gou.', 'ail']]",pasta alla vodka,"pâtes, parmesan, vodka, crème liquide, tomate, ail"
Pasta alla Norma,plat froid,"[['100 g', 'pâtes (rigatoni)'], ['100 g', 'tomate (chair)'], ['1/2 gou.', 'ail'], ['1/2', 'aubergine'], ['30 g', 'ricotta'], ['facultatif', '1/10 bou.', 'basilic (frais)']]",pasta alla norma,"pâtes, tomate, ail, aubergine, ricotta, basilic"
Pasta sauce marinara,15 minutes,"[['1/4 gou.', 'ail'], ['1/4', 'oignon jaune'], ['80 g', 'tomate'], ['1 pinc.', 'origan (séché)'], ['100 g', 'pâtes (pappardelle)'], ['20 g', 'parmesan (morceaux)']]",pasta sauce marinara,"ail, oignon jaune, tomate, origan, pâtes, parmesan"
Pasta tomates & crevettes,10 minutes,"[['150 g', 'tomate pelée'], ['100 g', 'pâtes (linguine)'], ['50 g', 'crevette (cuite)'], ['1/4 càc', 'mélange pour pâtes']]",pasta tomates & crevettes,"tomate pelée, pâtes, crevette, mélange pour pâtes"
Pasta al tonno,10 minutes,"[['60 g', 'thon (conserve)'], ['100 g', 'pâtes (linguine)'], ['40 g', 'tomate séchée'], ['1/10 bou.', 'persil (frais)'], ['1/4 gou.', 'ail']]",pasta al tonno,"thon, pâtes, tomate séchée, persil, ail"
Pasta & ragu,plat froid,"[['100 g', 'pâtes (pappardelle)'], ['100 g', 'chair à saucisse'], ['80 g', 'tomate pelée'], ['1/10 bou.', 'basilic (frais)'], ['1/4 gou.', 'ail'], ['20 g', 'parmesan (râpé)']]",pasta & ragu,"pâtes, chair à saucisse, tomate pelée, basilic, ail, parmesan"
Burrata pasta,9 minutes,"[['100 g', 'pâtes (rigatoni)'], ['100 g', 'tomate cerise'], ['70 g', 'burrata'], ['2 tran.', 'jambon cru (prosciutto)'], ['1/4 gou.', 'ail'], ['facultatif', '1/10 bou.', 'basilic (frais)']]",burrata pasta,"pâtes, tomate cerise, burrata, jambon cru, ail, basilic"


In [432]:

tmp = []
tmp.append(jow_data[jow_data["recettes"]=='cherry tomatoes spaghetti'].index[0])
tmp.append(jow_data[jow_data["recettes"]=='tomato vegan pasta'].index[0])

menu_data.loc[67, 'Closest JOW recipe by hand'] = tmp

Next recipe

In [433]:
menu_data[menu_data.index==68]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand,Closest 7t recipe by hand
68,pasta.jpg,Pasta,Penne Arrabiatta,"Tomate, piment",Not yet handled,no match


In [434]:
jow_data[jow_data["recettes"].str.contains("arrabiat")]

,cuisson,ingredients,recettes,only_ings
Gnocchis pique all'arrabiata,15 minutes,"[['150 g', 'gnocchi'], ['30 g', 'chorizo (tranches)'], ['1 càs', 'parmesan (râpé)'], ['80 g', 'sauce arrabbiata']]",gnocchis pique all'arrabiata,"gnocchi, chorizo, parmesan, sauce arrabbiata"


In [435]:
jow_data[(jow_data["recettes"].str.contains("pasta")) & (jow_data["only_ings"].str.contains("piment"))]

,cuisson,ingredients,recettes,only_ings
Creamy green pasta,10 minutes,"[['100 g', 'pâtes (pipe rigate)'], ['80 g', 'petits pois (surgelés)'], ['80 g', 'courgette'], ['1 càs', 'crème fraîche épaisse'], ['1/4', 'citron jaune'], ['1 pinc.', ""piment d'espelette""]]",creamy green pasta,"pâtes, petits pois, courgette, crème fraîche épaisse, citron jaune, piment d'espelette"


In [436]:
menu_data.loc[68, 'Closest JOW recipe by hand'] = 'no match'

Next recipe

In [437]:
menu_data[menu_data.index==69]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand,Closest 7t recipe by hand
69,pasta.jpg,Pasta,Spaghetti Alla Bolognese,"Tomate, viande de bœuf",Not yet handled,no match


In [438]:
jow_data[jow_data["recettes"].str.contains("bologn")]

,cuisson,ingredients,recettes,only_ings
Boulettes alla bolognese,15 minutes,"[['100 g', 'pâtes (spaghetti)'], ['1 càs', 'parmesan (râpé)'], ['1', 'boeuf (steak haché)'], ['1/4', 'échalote'], ['80 g', 'sauce tomate (napolitaine)'], ['facultatif', '1/10 bou.', 'basilic (frais)'], ['1 càs', 'chapelure'], ['1/4', 'oeuf']]",boulettes alla bolognese,"pâtes, parmesan, boeuf, échalote, sauce tomate, basilic, chapelure, oeuf"
Boulettes bolognese veggie,10 minutes,"[['100 g', 'pâtes (spaghetti)'], ['80 g', 'sauce tomate (napolitaine)'], ['150 g', 'haricots rouges (cuits)'], ['1/4', 'oignon jaune'], ['1/4 gou.', 'ail'], ['1 càs', 'parmesan (râpé)'], ['facultatif', '1/10 bou.', 'basilic (frais)'], ['3 càs', 'chapelure']]",boulettes bolognese veggie,"pâtes, sauce tomate, haricots rouges, oignon jaune, ail, parmesan, basilic, chapelure"
Spaghetti alla bolognese express,plat froid,"[['1/4', 'oignon jaune'], ['1/2', 'carotte (frais)'], ['100 g', 'tomate (purée)'], ['1/4 gou.', 'ail']]",spaghetti alla bolognese express,"oignon jaune, carotte, tomate, ail"
Lasagnes alla bolognese,plat froid,"[['90 g', 'lasagnes (cru)'], ['250 g', 'tomate (purée)'], ['100 g', 'mozzarella (boule)'], ['25 cl', 'lait'], ['20 g'], ['30 g'], ['1 pinc.'], ['1/2']]",lasagnes alla bolognese,"lasagnes, tomate, mozzarella, lait"
Bolognese veggie,21 minutes,"[['100 g', 'pâtes (spaghetti)'], ['50 g', 'lentilles (cuites)'], ['1/2', 'carotte (frais)'], ['100 g', 'tomate (purée)'], ['1/2 gou.', 'ail'], ['1/4', 'oignon jaune']]",bolognese veggie,"pâtes, lentilles, carotte, tomate, ail, oignon jaune"
Creamy bolognese gnocchis,6 minutes,"[['120 g', 'gnocchi'], ['100 g', 'tomate (purée)'], ['100 g', 'boeuf (haché)'], ['1/4', 'échalote'], ['1 càs', 'crème fraîche épaisse'], ['facultatif', '1/10 bou.', 'basilic (frais)']]",creamy bolognese gnocchis,"gnocchi, tomate, boeuf, échalote, crème fraîche épaisse, basilic"
Spaghetti bolognese à la ratatouille,10 minutes,"[['150 g', 'ratatouille'], ['100 g', 'boeuf (haché)'], ['1 pinc.', 'origan (séché)'], ['100 g', 'pâtes (spaghetti)'], ['1/2 càs']]",spaghetti bolognese à la ratatouille,"ratatouille, boeuf, origan, pâtes"


In [439]:
tmp = []
tmp.append(jow_data[jow_data["recettes"]=='spaghetti alla bolognese express'].index[0])
tmp.append(jow_data[jow_data["recettes"]=='spaghetti bolognese à la ratatouille'].index[0])

menu_data.loc[69, 'Closest JOW recipe by hand'] = tmp

Next recipe

In [440]:
menu_data[menu_data.index==71]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand,Closest 7t recipe by hand
71,pasta.jpg,Pasta,Linguine Al Salmone,"Crème fraîche, saumon fumé",Not yet handled,no match


In [441]:
jow_data[jow_data["recettes"].str.contains("linguine")]

,cuisson,ingredients,recettes,only_ings
"Linguine pesto maison, noix & persil",8 minutes,"[['120 g', 'pâtes (linguine)'], ['10 g', 'noix'], ['1 càs', 'parmesan (râpé)'], ['1/5 gou.', 'ail'], ['1/5 bou.', 'persil (frais)'], ['2 càs', ""huile d'olive""]]","linguine pesto maison, noix & persil","pâtes, noix, parmesan, ail, persil, huile d'olive"
Linguine saumon & petits pois,8 minutes,"[['100 g', 'pâtes (linguine)'], ['50 g', 'petits pois (frais)'], ['1 tran.', 'saumon (fumé)'], ['1 càs', 'crème fraîche épaisse'], ['1/4', 'citron jaune']]",linguine saumon & petits pois,"pâtes, petits pois, saumon, crème fraîche épaisse, citron jaune"


In [442]:
jow_data[(jow_data["recettes"].str.contains("pasta")) & (jow_data["only_ings"].str.contains("saumon"))]

,cuisson,ingredients,recettes,only_ings
Pasta saumon citron,11 minutes,"[['100 g', 'pâtes (pappardelle)'], ['1 càs', 'crème fraîche épaisse'], ['1/4', 'citron jaune'], ['1 tran.', 'saumon (fumé)'], ['10 g', 'parmesan (morceaux)']]",pasta saumon citron,"pâtes, crème fraîche épaisse, citron jaune, saumon, parmesan"


In [443]:
jow_data[(jow_data["recettes"].str.contains("pâte")) & (jow_data["only_ings"].str.contains("saumon"))]

,cuisson,ingredients,recettes,only_ings
Pâtes pesto de petits pois & saumon,10 minutes,"[['100 g', 'pâtes (penne)'], ['1', 'saumon (frais)'], ['80 g', 'petits pois (surgelés)'], ['25 g', 'feta'], ['1 càs'], ['1/4 gou.'], ['1/4']]",pâtes pesto de petits pois & saumon,"pâtes, saumon, petits pois, feta"


In [444]:
jow_data[jow_data["only_ings"].str.contains("saumon fumé")]

,cuisson,ingredients,recettes,only_ings


In [445]:
jow_data[(jow_data["recettes"].str.contains("spaghetti")) & (jow_data["only_ings"].str.contains("saumon"))]

,cuisson,ingredients,recettes,only_ings


In [446]:
tmp = []
tmp.append(jow_data[jow_data["recettes"]=='linguine saumon & petits pois'].index[0])
tmp.append(jow_data[jow_data["recettes"]=='pasta saumon citron'].index[0])
tmp.append(jow_data[jow_data["recettes"]=='pâtes pesto de petits pois & saumon'].index[0])

menu_data.loc[71, 'Closest JOW recipe by hand'] = tmp

Next recipe

In [447]:
menu_data[menu_data.index==72]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand,Closest 7t recipe by hand
72,pasta.jpg,Pasta,Penne Al Formaggio,"Gorgonzola, parmigiano",Not yet handled,roquefort pasta


In [448]:
jow_data[jow_data["recettes"].str.contains("pasta")]

,cuisson,ingredients,recettes,only_ings
Pasta alla carbonara,9 minutes,"[['100 g', 'pâtes (spaghetti)'], ['3 tran.', 'pancetta'], ['1', 'oeuf'], ['40 g', 'parmesan (morceaux)']]",pasta alla carbonara,"pâtes, pancetta, oeuf, parmesan"
Pasta Alfredo,12 minutes,"[['100 g', 'pâtes (tagliatelles)'], ['1', 'poulet (escalope)'], ['10 cl', 'crème liquide'], ['1/2 gou.', 'ail'], ['1 càs', 'parmesan (râpé)']]",pasta alfredo,"pâtes, poulet, crème liquide, ail, parmesan"
Creamy avocado pasta,8 minutes,"[['120 g', 'pâtes (linguine)'], ['1/4', 'avocat'], ['1/5 bou.', 'basilic (frais)'], ['10 g', 'amandes (effilées)'], ['15 g', 'parmesan (morceaux)']]",creamy avocado pasta,"pâtes, avocat, basilic, amandes, parmesan"
Pasta petits pois lardons,10 minutes,"[['100 g', 'pâtes (pipe rigate)'], ['100 g', 'petits pois (frais)'], ['75 g', 'lardons'], ['2 cl', 'crème liquide'], ['2 càs'], ['facultatif', '1/50 bou.']]",pasta petits pois lardons,"pâtes, petits pois, lardons, crème liquide"
Pasta Rocchini,15 minutes,"[['100 g', 'pâtes (trofie)'], ['2', 'saucisse (chipolata)'], ['1 poignée', 'salade (roquette)'], ['1 càs', 'parmesan (râpé)'], ['1/2', 'oignon rouge'], ['1/2', 'fenouil']]",pasta rocchini,"pâtes, saucisse, salade, parmesan, oignon rouge, fenouil"
Pasta poulet citron,10 minutes,"[['1', 'poulet (escalope)'], ['100 g', 'pâtes (spaghetti)'], ['8 cl', 'crème liquide'], ['1 poignée', 'epinard (frais)'], ['1 càs', 'parmesan (râpé)'], ['1/2', 'citron jaune']]",pasta poulet citron,"poulet, pâtes, crème liquide, epinard, parmesan, citron jaune"
Creamy Eggs & Prosciutto pasta,8 minutes,"[['100 g', 'pâtes (pappardelle)'], ['2 tran.', 'jambon cru (prosciutto)'], ['5 cl', 'crème liquide'], ['1/2 càs', 'ricotta'], ['1', 'oeuf'], ['facultatif', '1/10 bou.', 'basilic (frais)']]",creamy eggs & prosciutto pasta,"pâtes, jambon cru, crème liquide, ricotta, oeuf, basilic"
Greek pasta salad,8 minutes,"[['70 g', 'pâtes (fusilli)'], ['1', 'tomate'], ['15 g', 'olive noire'], ['130 g', 'concombre'], ['30 g', 'feta'], ['facultatif', '1/8', 'oignon rouge'], ['1 pinc.', 'origan (séché)']]",greek pasta salad,"pâtes, tomate, olive noire, concombre, feta, oignon rouge, origan"
Pasta pistacchio e olive,plat froid,"[['15 g', 'pistaches (grillées)'], ['100 g', 'pâtes (pipe rigate)'], ['15 g', 'olive noire'], ['1/4 gou.', 'ail']]",pasta pistacchio e olive,"pistaches, pâtes, olive noire, ail"
Pasta al ragu d'anatra,plat froid,"[['100 g', 'pâtes (pappardelle)'], ['1/2', 'canard (confit)'], ['70 g', 'tomate pelée'], ['1/2', 'échalote']]",pasta al ragu d'anatra,"pâtes, canard, tomate pelée, échalote"


In [449]:
jow_data[jow_data["recettes"].str.contains("penne")]

,cuisson,ingredients,recettes,only_ings
"Gratin de penne, petit pois & lard",20 minutes,"[['100 g', 'pâtes (penne)'], ['80 g', 'petits pois (frais)'], ['30 g', 'chèvre frais'], ['2 tran.', 'lard (tranches)'], ['1 càs', 'crème liquide']]","gratin de penne, petit pois & lard","pâtes, petits pois, chèvre frais, lard, crème liquide"


In [450]:
jow_data[jow_data["recettes"].str.contains("pâte")]

,cuisson,ingredients,recettes,only_ings
Pâtes aux aubergines & parmesan,10 minutes,"[['100 g', 'pâtes (linguine)'], ['1/4', 'aubergine'], ['2 tran.', 'jambon cru'], ['20 g', 'parmesan (morceaux)']]",pâtes aux aubergines & parmesan,"pâtes, aubergine, jambon cru, parmesan"
Gratin de pâtes au jambon,25 minutes,"[['100 g', 'pâtes (macaroni)'], ['40 g', 'fromage râpé'], ['1 tran.', 'jambon blanc'], ['1 càs', 'crème fraîche épaisse']]",gratin de pâtes au jambon,"pâtes, fromage râpé, jambon blanc, crème fraîche épaisse"
Pâtes à la Sicilienne,9 minutes,"[['100 g', 'pâtes (conchiglioni)'], ['1 càs', 'câpres'], ['100 g', 'tomate cerise'], ['90 g', 'thon (conserve)'], ['1 càc']]",pâtes à la sicilienne,"pâtes, câpres, tomate cerise, thon"
Salade de pâtes printanière,10 minutes,"[['2 tran.', 'jambon cru'], ['80 g', 'pâtes (pipe rigate)'], ['1/2', 'salade (sucrine)'], ['70 g', 'petits pois (frais)'], ['1/4', 'citron jaune'], ['15 g', 'parmesan (morceaux)']]",salade de pâtes printanière,"jambon cru, pâtes, salade, petits pois, citron jaune, parmesan"
Pâtes au pesto de coriandre,8 minutes,"[['120 g', 'pâtes (pipe rigate)'], ['1/5 bou.', 'coriandre (frais)'], ['15 g', 'amandes (entières)'], ['1/4 gou.', 'ail'], ['20 g', 'parmesan (râpé)']]",pâtes au pesto de coriandre,"pâtes, coriandre, amandes, ail, parmesan"
Pâte à tartiner aux speculoos,plat froid,"[['200 g', 'speculoos'], ['1 càc', 'cannelle (en poudre)'], ['1 càc', 'miel (liquide)'], ['300 g', 'lait concentré sucré']]",pâte à tartiner aux speculoos,"speculoos, cannelle, miel, lait concentré sucré"
Pâte à tartiner au chocolat,plat froid,"[['50 g', 'noisette'], ['50 g', 'chocolat à cuire'], ['7 cl', 'crème liquide'], ['8 g', 'sucre (en poudre)'], ['5 ml', 'huile de tournesol']]",pâte à tartiner au chocolat,"noisette, chocolat à cuire, crème liquide, sucre, huile de tournesol"
Pâtes au brocoli,20 minutes,"[['100 g', 'brocoli (frais)'], ['100 g', 'pâtes (orecchiette)'], ['2', 'anchois'], ['1/4 gou.', 'ail'], ['10 g', 'parmesan (morceaux)']]",pâtes au brocoli,"brocoli, pâtes, anchois, ail, parmesan"
Pâtes au beaufort & jambon,10 minutes,"[['130 g', 'pâtes (fusilli)'], ['1 tran.', 'jambon blanc'], ['5 cl', 'crème liquide'], ['40 g', 'beaufort']]",pâtes au beaufort & jambon,"pâtes, jambon blanc, crème liquide, beaufort"
"Pâtes thon, citron & ricotta",12 minutes,"[['100 g', 'pâtes (linguine)'], ['1 càs', 'ricotta'], ['50 g', 'thon au citron'], ['1/4', 'citron jaune'], ['1 càs', 'crème liquide']]","pâtes thon, citron & ricotta","pâtes, ricotta, thon au citron, citron jaune, crème liquide"


In [451]:
jow_data[jow_data["only_ings"].str.contains("gorgonzola")]

,cuisson,ingredients,recettes,only_ings
Gnocchis Epinards Gorgonzola,5 minutes,"[['150 g', 'gnocchi'], ['100 g', 'epinard (frais)'], ['30 g', 'gorgonzola'], ['5 cl', 'crème liquide']]",gnocchis epinards gorgonzola,"gnocchi, epinard, gorgonzola, crème liquide"
Pizza poire gorgonzola,10 minutes,"[['1', 'poire'], ['1', 'pâte à pizza'], ['100 g', 'gorgonzola'], ['facultatif', '20 g', 'noisette'], ['2 càs', 'crème fraîche épaisse']]",pizza poire gorgonzola,"poire, pâte à pizza, gorgonzola, noisette, crème fraîche épaisse"
Gnocchis gratinés au gorgonzola,14 minutes,"[['150 g', 'gnocchi (à poêler)'], ['40 g', 'gorgonzola'], ['facultatif', '10 g', 'noix'], ['2 cl', 'crème liquide']]",gnocchis gratinés au gorgonzola,"gnocchi, gorgonzola, noix, crème liquide"
Polenta 4 fromages,8 minutes,"[['60 g', 'polenta'], ['10 g', 'comté'], ['10 g', 'chèvre frais'], ['10 g', 'gorgonzola'], ['10 g', 'parmesan (morceaux)'], ['1 poignée', 'salade (roquette)'], ['25 cl', 'lait']]",polenta 4 fromages,"polenta, comté, chèvre frais, gorgonzola, parmesan, salade, lait"
Salade de lentilles à l'italienne,plat froid,"[['150 g', 'lentilles (cuites)'], ['50 g', 'tomate séchée'], ['1/4', 'oignon rouge'], ['60 g', 'gorgonzola']]",salade de lentilles à l'italienne,"lentilles, tomate séchée, oignon rouge, gorgonzola"
Salade d'endives & figues,plat froid,"[['2', 'endives'], ['1', 'figue'], ['1 càc', 'vinaigre de cidre'], ['10 g', 'noix'], ['20 g', 'gorgonzola']]",salade d'endives & figues,"endives, figue, vinaigre de cidre, noix, gorgonzola"
Gratin de chou-fleur au gorgonzola,plat froid,"[['250 g', 'chou-fleur (frais)'], ['5 cl', 'crème liquide'], ['30 g', 'gorgonzola'], ['15 g', 'noix']]",gratin de chou-fleur au gorgonzola,"chou-fleur, crème liquide, gorgonzola, noix"
"Cake courge, lard & gorgonzola",1 heure,"[['1', 'potimarron'], ['3', 'oeuf'], ['150 g', 'farine de blé'], ['10 cl', 'huile de tournesol'], ['100 g', 'lard (tranches)'], ['100 g', 'gorgonzola'], ['1 càc', 'thym (feuilles)'], ['1 sac.', 'levure chimique']]","cake courge, lard & gorgonzola","potimarron, oeuf, farine de blé, huile de tournesol, lard, gorgonzola, thym, levure chimique"
"Gnocchis, brocoli & gorgonzola",6 minutes,"[['100 g', 'gnocchi'], ['80 g', 'brocoli (frais)'], ['50 g', 'champignons de paris (frais)'], ['1 càs', 'crème liquide'], ['30 g', 'gorgonzola']]","gnocchis, brocoli & gorgonzola","gnocchi, brocoli, champignons de paris, crème liquide, gorgonzola"
Gratin de gnocchis pesto & gorgonzola,25 minutes,"[['150 g', 'gnocchi'], ['100 g', 'tomate (purée)'], ['30 g', 'gorgonzola'], ['1 càc', 'sauce pesto']]",gratin de gnocchis pesto & gorgonzola,"gnocchi, tomate, gorgonzola, sauce pesto"


In [452]:
jow_data[jow_data["recettes"].str.contains("formagg")]

,cuisson,ingredients,recettes,only_ings


In [453]:
menu_data.loc[72, 'Closest JOW recipe by hand'] = 'no match'

Next recipe

In [454]:
menu_data[menu_data.index==78]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand,Closest 7t recipe by hand
78,fabbrezza_202084_665.jpg,Plat au choix,"""Petit"" Lasagne de Tonia",NaN,Not yet handled,no match


In [455]:
jow_data[jow_data["recettes"].str.contains("lasagne")]

,cuisson,ingredients,recettes,only_ings
Lasagnes courgette ricotta,plat froid,"[['90 g', 'lasagnes (cru)'], ['500 g', 'courgette'], ['100 g', 'mozzarella (boule)'], ['100 g', 'ricotta']]",lasagnes courgette ricotta,"lasagnes, courgette, mozzarella, ricotta"
Lasagnes saumon & épinard,45 minutes,"[['90 g', 'lasagnes (cru)'], ['4 tran.', 'saumon (fumé)'], ['20 g', 'parmesan (morceaux)'], ['20 cl', 'lait'], ['60 g', 'beurre'], ['1 càs', 'farine de blé'], ['400 g', 'epinards (surgelés)']]",lasagnes saumon & épinard,"lasagnes, saumon, parmesan, lait, beurre, farine de blé, epinards"
Lasagnes alla bolognese,plat froid,"[['90 g', 'lasagnes (cru)'], ['250 g', 'tomate (purée)'], ['100 g', 'mozzarella (boule)'], ['25 cl', 'lait'], ['20 g'], ['30 g'], ['1 pinc.'], ['1/2']]",lasagnes alla bolognese,"lasagnes, tomate, mozzarella, lait"
Lasagne poireaux & ricotta,50 minutes,"[['90 g', 'lasagnes (cru)'], ['1', 'poireau'], ['100 g', 'ricotta'], ['15 cl', 'crème liquide'], ['2 càs', 'parmesan (râpé)']]",lasagne poireaux & ricotta,"lasagnes, poireau, ricotta, crème liquide, parmesan"
Lasagnes express à la ratatouille,40 minutes,"[['90 g', 'lasagnes (cru)'], ['500 g', 'ratatouille'], ['16 cl', 'béchamel'], ['150 g', 'mozzarella (à cuire)'], ['2 càs', 'parmesan (râpé)']]",lasagnes express à la ratatouille,"lasagnes, ratatouille, béchamel, mozzarella, parmesan"


In [456]:
menu_data.loc[78, 'Closest JOW recipe by hand'] = jow_data[jow_data["recettes"]=='lasagnes alla bolognese'].index[0]


Next recipe

In [457]:
menu_data[menu_data.index==79]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand,Closest 7t recipe by hand
79,fabbrezza_202084_665.jpg,Dessert,Baba au Rhum,NaN,Not yet handled,no match


In [458]:
jow_data[jow_data["recettes"].str.contains("baba")]

,cuisson,ingredients,recettes,only_ings


In [459]:
jow_data[jow_data["recettes"].str.contains("rhum")]

,cuisson,ingredients,recettes,only_ings


In [460]:
jow_data[jow_data["only_ings"].str.contains("rhum")]

,cuisson,ingredients,recettes,only_ings
Piña Colada,plat froid,"[['5 cl', 'rhum (blanc)'], ['4 cl', 'lait de coco'], ['12 cl', ""jus d'ananas""]]",piña colada,"rhum, lait de coco, jus d'ananas"
Cuba libre,plat froid,"[['15 cl', 'cola'], ['6 cl', 'rhum (blanc)'], ['1', 'citron vert']]",cuba libre,"cola, rhum, citron vert"
Mojito à la fraise,plat froid,"[['4 cl', 'rhum (blanc)'], ['1/10 bou.', 'menthe (feuilles)'], ['1/2', 'citron vert'], ['2 càc', 'sucre de canne'], ['5 cl', 'eau gazeuse'], ['40 g', 'fraise']]",mojito à la fraise,"rhum, menthe, citron vert, sucre de canne, eau gazeuse, fraise"
Mojito,plat froid,"[['4 cl', 'rhum (blanc)'], ['1/10 bou.', 'menthe (feuilles)'], ['1/2', 'citron vert'], ['2 càc', 'sucre de canne'], ['5 cl', 'eau gazeuse']]",mojito,"rhum, menthe, citron vert, sucre de canne, eau gazeuse"


In [461]:
menu_data.loc[79, 'Closest JOW recipe by hand'] = 'no match'

In [462]:
menu_data_7tjow = menu_data[(menu_data['Closest JOW recipe by hand']!='Not yet handled') & (menu_data['Closest 7t recipe by hand']!='Not yet handled')]
print('Number of handled recipes = ', menu_data_7tjow.shape[0])

menu_data_7tjow.to_json(data_path+"data/Tools/MenuTo7tJOWRecipes_v1.json")
menu_data_7tjow.to_excel(data_path+"data/Tools/MenuTo7tJOWRecipes_v1.xlsx")

Number of handled recipes =  38


### Let's continue with recipes that have been tried to be matched with Jow but not 7t

In [463]:
menu_data[(menu_data['Closest JOW recipe by hand']!='Not yet handled') & (menu_data['Closest 7t recipe by hand']=='Not yet handled')]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand,Closest 7t recipe by hand
9,burgart_205074_c70.jpg,Desserts,Financier artisan boulanger,NaN,"[Financier cerises noires & pistache, Financier à la framboise]",Not yet handled
10,burgart_205074_c70.jpg,Boissons,Expresso,NaN,no match,Not yet handled
11,burgart_205074_c70.jpg,Boissons,Café long,NaN,no match,Not yet handled
12,burgart_205074_c70.jpg,Boissons,Chocolat chaud,NaN,Chocolat chaud,Not yet handled
13,burgart_205074_c70.jpg,Boissons,Ice Tea,NaN,no match,Not yet handled
14,burgart_205074_c70.jpg,Boissons,Elixia,Limonade Bio,no match,Not yet handled
15,burgart_205074_c70.jpg,Boissons,Alain Milliat,"Orange, Mangue, Pomme",no match,Not yet handled
16,burgart_205074_c70.jpg,Boissons,Bière Corona,NaN,no match,Not yet handled
29,Carte-Restaurant-Nelsons-Paris-Jour-2-1024x724.jpg,Salades,Végé Bowl,"avocat, choux kale frit, concombre, edamame, radis, tomate et tofu mariné aux sésames",Vegan poke bowl,Not yet handled
33,Carte-Restaurant-Nelsons-Paris-Jour-2-1024x724.jpg,Végé pour tous,Truffade des montagnes,salade,Tartiflette express,Not yet handled


In [464]:
seven_data.head()

,Unnamed: 0,Unnamed: 0.1,title_raw,id,url,unit,quantity_raw,ingredient_raw,weight_per_ingr,ingredient_clean_comma,matched_ingredient,food_clean_comma,est_impact,title_clean,source_website
0,0,36,natural peanut butter chocolate bon bons,0006ca31f4,http://www.food.com/recipe/natural-peanut-butter-chocolate-bon-bons-449265,cup,12,"cocoa, dry powder, unsweetened",1032.0,"cocoa , dry powder , unsweetened","cocoa powder , sugar , powder , instant","cocoa powder , sugar , powder , instant",933.0,natural peanut butter chocolate bon bons,http://www.food.com
1,1,37,natural peanut butter chocolate bon bons,0006ca31f4,http://www.food.com/recipe/natural-peanut-butter-chocolate-bon-bons-449265,cup,12,honey,4068.0,honey,honey,honey,38.0,natural peanut butter chocolate bon bons,http://www.food.com
2,2,38,natural peanut butter chocolate bon bons,0006ca31f4,http://www.food.com/recipe/natural-peanut-butter-chocolate-bon-bons-449265,cup,1,"peanut butter, smooth style, without salt",256.0,"peanut butter , style , salt",peanut butter peanut paste,peanut butter peanut paste,316.7,natural peanut butter chocolate bon bons,http://www.food.com
3,3,170,cilantro-mustard mayo (dip or sauce),00185a60a8,http://www.food.com/recipe/cilantro-mustard-mayo-dip-or-sauce-136524,cup,12,"mustard, prepared, yellow",2880.0,"mustard , , yellow",mustard,mustard,120.0,cilantro mustard mayo,http://www.food.com
4,4,171,cilantro-mustard mayo (dip or sauce),00185a60a8,http://www.food.com/recipe/cilantro-mustard-mayo-dip-or-sauce-136524,cup,12,"salad dressing, mayonnaise, regular",2649.6,"salad dressing , mayonnaise ,","salad dressing ,","salad dressing ,",120.0,cilantro mustard mayo,http://www.food.com


In [465]:
'''
set(seven_data["title_raw"][seven_data["title_raw"].str.contains("prawn")].values.tolist())
seven_data[seven_data['title_raw'].isin(['amara veggie burgers', 'the woodward veggie burger'])]
set(seven_data["ingredient_raw"][seven_data["ingredient_raw"].str.contains("raviol")].values.tolist())
'''

'\nset(seven_data["title_raw"][seven_data["title_raw"].str.contains("prawn")].values.tolist())\nseven_data[seven_data[\'title_raw\'].isin([\'amara veggie burgers\', \'the woodward veggie burger\'])]\nset(seven_data["ingredient_raw"][seven_data["ingredient_raw"].str.contains("raviol")].values.tolist())\n'

In [466]:
menu_data[menu_data.index==9]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand,Closest 7t recipe by hand
9,burgart_205074_c70.jpg,Desserts,Financier artisan boulanger,NaN,"[Financier cerises noires & pistache, Financier à la framboise]",Not yet handled


In [467]:
set(seven_data["title_raw"][seven_data["title_raw"].str.contains("financier")].values.tolist())

set()

In [468]:
set(seven_data["title_raw"][(seven_data["title_raw"].str.contains("cake")) & (seven_data["ingredient_raw"].str.contains("nut"))].values.tolist())

{'apricot glaze for rose garden wedding cake recipe',
 'black forest cake',
 'blockbuster chocolate chip cheesecake recipe',
 'cake',
 'carrot cake granita',
 'cherry-almond ice cream cake with milk chocolate sauce',
 'chocolate broiler cake frosting',
 'chocolate cake boiling icing',
 'coconut brittle cake topping',
 "dorothy's blueberry cheesecake dessert recipe",
 'easy german chocolate cake icing',
 'fudge sundae ice cream cake',
 'german chocolate cake icing',
 'goat cheese cakes (to top your salad)',
 'honey butter syrup (goes great on pancakes!)',
 'lady baltimore layered cake filling',
 'layered sorbet and ice-cream cake',
 'low sugar coconut-almond pie crust or cheesecake crust',
 'my dump cake',
 'peachy cake',
 'pecan cheesecake bars recipe',
 'pecan pancake syrup',
 'praline topping for cheesecake or apple pie',
 'pumpkin cheesecake dip',
 'pumpkin cheesecake frozen yogurt',
 'raisin cake from the pantry',
 'rice-crusted peanut sauce coconut tofu cakes',
 'rich coconut cake

In [469]:
set(seven_data["title_raw"][(seven_data["title_raw"].str.contains("cake")) & (seven_data["ingredient_raw"].str.contains("almond"))].values.tolist())

{'apricot glaze for rose garden wedding cake recipe',
 'black forest cake',
 'blockbuster chocolate chip cheesecake recipe',
 'cherry-almond ice cream cake with milk chocolate sauce',
 'goat cheese cakes (to top your salad)',
 'low sugar coconut-almond pie crust or cheesecake crust',
 'spiced cake'}

In [470]:
set(seven_data["title_raw"][seven_data["ingredient_raw"].str.contains("financier")].values.tolist())

set()

In [471]:
menu_data.loc[9, 'Closest 7t recipe by hand'] = 'no match'

Next recipe

In [472]:
menu_data[menu_data.index==29]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand,Closest 7t recipe by hand
29,Carte-Restaurant-Nelsons-Paris-Jour-2-1024x724.jpg,Salades,Végé Bowl,"avocat, choux kale frit, concombre, edamame, radis, tomate et tofu mariné aux sésames",Vegan poke bowl,Not yet handled


In [473]:
set(seven_data["title_raw"][(seven_data["title_raw"].str.contains("bowl")) & (seven_data["ingredient_raw"].str.contains("kale"))].values.tolist())

set()

In [474]:
set(seven_data["title_raw"][seven_data["ingredient_raw"].str.contains("kale")].values.tolist())

{'"morning shake"',
 'carrots and kale',
 'chicken pesto pasta',
 'crispy soy roasted kale chips',
 'green smoothie',
 'greens and craisins',
 'kale salad with lemon, parmesan and breadcrumbs',
 'lemony quinoa and kale salad',
 'quick and easy kale',
 'sesame veggie',
 'tofu, kale, and raisins'}

In [475]:
seven_data[seven_data['title_raw'].isin(['carrots and kale', 'kale salad with lemon, parmesan and breadcrumbs', \
                                         'lemony quinoa and kale salad', 'quick and easy kale', \
                                         'sesame veggie', 'tofu, kale, and raisins'])]

,Unnamed: 0,Unnamed: 0.1,title_raw,id,url,unit,quantity_raw,ingredient_raw,weight_per_ingr,ingredient_clean_comma,matched_ingredient,food_clean_comma,est_impact,title_clean,source_website
9875,9875,102956,carrots and kale,48399dd87d,http://www.delish.com/recipefinder/carrots-kale-3101,tablespoon,1,"oil, corn, peanut, and olive",14.000000,"oil , corn , peanut , olive",maize corn oil,maize corn oil,-1.000000,carrots kale,http://www.delish.com
9876,9876,102957,carrots and kale,48399dd87d,http://www.delish.com/recipefinder/carrots-kale-3101,pound,01-fŽvr,"carrots, raw",226.796000,"carrot ,","carrot ,","carrot ,",-1.000000,carrots kale,http://www.delish.com
9877,9877,102958,carrots and kale,48399dd87d,http://www.delish.com/recipefinder/carrots-kale-3101,pound,2022-04-03 00:00:00,"kale, raw",340.194000,"kale ,","curly kale ,","curly kale ,",80.000000,carrots kale,http://www.delish.com
9878,9878,102959,carrots and kale,48399dd87d,http://www.delish.com/recipefinder/carrots-kale-3101,tablespoon,1,"rice vinegar, upc: 4979435030332",15.000000,"rice vinegar ,",vinegar,vinegar,167.000000,carrots kale,http://www.delish.com
9879,9879,102960,carrots and kale,48399dd87d,http://www.delish.com/recipefinder/carrots-kale-3101,tablespoon,1,soy sauce made from soy (tamari),18.000000,soy sauce soy,"soy sauce ,","soy sauce ,",120.000000,carrots kale,http://www.delish.com
9880,9880,102961,carrots and kale,48399dd87d,http://www.delish.com/recipefinder/carrots-kale-3101,tablespoon,2,"oil, corn, peanut, and olive",28.000000,"oil , corn , peanut , olive",maize corn oil,maize corn oil,-1.000000,carrots kale,http://www.delish.com
10834,10834,113183,quick and easy kale,4f7bd50607,http://tastykitchen.com/recipes/sidedishes/quick-and-easy-kale/,cup,6,"kale, raw",96.000000,"kale ,","curly kale ,","curly kale ,",80.000000,quick kale,http://tastykitchen.com
10835,10835,113184,quick and easy kale,4f7bd50607,http://tastykitchen.com/recipes/sidedishes/quick-and-easy-kale/,cup,1,"onions, raw",160.000000,"onion ,","onion ,","onion ,",1270.000000,quick kale,http://tastykitchen.com
10836,10836,113185,quick and easy kale,4f7bd50607,http://tastykitchen.com/recipes/sidedishes/quick-and-easy-kale/,teaspoon,1,"spices, garlic powder",3.100000,"spice , garlic powder","garlic , powder , dried","garlic , powder , dried",123.200000,quick kale,http://tastykitchen.com
10837,10837,113186,quick and easy kale,4f7bd50607,http://tastykitchen.com/recipes/sidedishes/quick-and-easy-kale/,cup,1,"soup, vegetable broth, ready to serve",221.000000,"soup , vegetable broth ,","soup , vegetable cheese , , reheated","soup , vegetable cheese , , reheated",-1.000000,quick kale,http://tastykitchen.com


In [476]:
set(seven_data["title_raw"][seven_data["title_raw"].str.contains("poke")].values.tolist())

set()

In [477]:
menu_data.loc[29, 'Closest 7t recipe by hand'] = 'tofu, kale, and raisins'

Next recipe

In [478]:
menu_data[menu_data.index==33]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand,Closest 7t recipe by hand
33,Carte-Restaurant-Nelsons-Paris-Jour-2-1024x724.jpg,Végé pour tous,Truffade des montagnes,salade,Tartiflette express,Not yet handled


In [479]:
set(seven_data["title_raw"][(seven_data["title_raw"].str.contains("gratin")) & (seven_data["ingredient_raw"].str.contains("potato"))].values.tolist())

{'blue cheese potato gratin recipe',
 'buttermilk potatoes au gratin',
 'creamed potato gratin with chevre #5fix',
 'mashed potato, cheese and chive gratin',
 'mashed potato, cheese, and chive gratin',
 'mini potato gratins',
 'potato and parmesan gratin',
 'potato gratin "boome-style"',
 'potato, caramelized onion, and goat cheese gratin',
 'potato-tarragon gratin',
 'root vegetable gratin',
 'sweet potato & parsnip gratin',
 'warm onion-potato gratin'}

In [480]:
set(seven_data["title_raw"][(seven_data["title_raw"].str.contains("potato")) & (seven_data["ingredient_raw"].str.contains("cheese"))].values.tolist())

{'aji de gallina peruvian potatoes #sp5',
 'baked mashed potatoes',
 'baked mashed potatoes with sour cream and garlic recipe',
 'baked potato topping',
 'baked sweet potato wedges',
 'blue cheese and chive mashed potatoes',
 'blue cheese and rosemary mashed potatoes',
 'blue cheese potato gratin recipe',
 'broccoli and cheese stuffed potato recipe',
 'broccoli chedder stuffed potato recipe',
 'chantilly potatoes',
 'cheesy bbq garlic mashed potatoes recipe',
 'cheesy bbq mash potatoes',
 'cheesy chicken & potato casserole',
 'cheesy chicken and potato casserole',
 'cheesy chive mashed potatoes',
 'cheesy smashed potatoes',
 'cheesy smashed potatoes & cauliflower',
 'cheesy sour cream & chives golden mashed potatoes',
 'chive and onion mashed potatoes',
 'company mashed potatoes',
 "copycat: boston pizza's cactus cut potatoes dip",
 'creamed chicken & potatoes for babies',
 'creamed potato gratin with chevre #5fix',
 'creamy cheesy mashed potatoes',
 'creamy mashed potato bake',
 'crea

In [481]:
seven_data[seven_data['title_raw'].isin(['blue cheese potato gratin recipe', 'mini potato gratins', \
                                        'potato and parmesan gratin', 'potato gratin "boome-style"', 'potato-tarragon gratin'])]

,Unnamed: 0,Unnamed: 0.1,title_raw,id,url,unit,quantity_raw,ingredient_raw,weight_per_ingr,ingredient_clean_comma,matched_ingredient,food_clean_comma,est_impact,title_clean,source_website
989,989,10172,potato and parmesan gratin,06eb83c16f,http://www.epicurious.com/recipes/food/views/potato-and-parmesan-gratin-4041,pound,2.5,"potatoes, raw, skin",1133.980000,"potato , , skin","potato salad ,","potato salad ,",-1.0,potato parmesan gratin,http://www.epicurious.com
990,990,10173,potato and parmesan gratin,06eb83c16f,http://www.epicurious.com/recipes/food/views/potato-and-parmesan-gratin-4041,cup,2022-04-01 00:00:00,"butter, without salt",56.800000,"butter , salt",cocoa butter,cocoa butter,-1.0,potato parmesan gratin,http://www.epicurious.com
991,991,10174,potato and parmesan gratin,06eb83c16f,http://www.epicurious.com/recipes/food/views/potato-and-parmesan-gratin-4041,cup,2,"cheese, parmesan, hard",297.412368,"cheese , parmesan , hard","parmesan cheese , cow milk","parmesan cheese , cow milk",1110.0,potato parmesan gratin,http://www.epicurious.com
992,992,10175,potato and parmesan gratin,06eb83c16f,http://www.epicurious.com/recipes/food/views/potato-and-parmesan-gratin-4041,cup,2,"milk, fluid, 1% fat, without added vitamin a and vitamin d",488.000000,"milk , , fat , added vitamin vitamin","yogurt , fermented milk dairy specialty , fruit , sweetener , fat free , fortified vitamin","yogurt , fermented milk dairy specialty , fruit , sweetener , fat free , fortified vitamin",-1.0,potato parmesan gratin,http://www.epicurious.com
5083,5083,53050,"potato gratin ""boome-style""",249f6385ac,http://www.foodnetwork.com/recipes/danny-boome/potato-gratin-boome-style-recipe2.html,pound,1,"potatoes, raw, skin",453.592000,"potato , , skin","potato salad ,","potato salad ,",-1.0,potato gratin boome style,http://www.foodnetwork.com
5084,5084,53051,"potato gratin ""boome-style""",249f6385ac,http://www.foodnetwork.com/recipes/danny-boome/potato-gratin-boome-style-recipe2.html,cup,2,"cheese, gruyere",433.124808,"cheese , gruyere","gruyere cheese , cow milk","gruyere cheese , cow milk",1110.0,potato gratin boome style,http://www.foodnetwork.com
5085,5085,53052,"potato gratin ""boome-style""",249f6385ac,http://www.foodnetwork.com/recipes/danny-boome/potato-gratin-boome-style-recipe2.html,cup,1,"cream, fluid, heavy whipping",120.000000,"cream , ,",cream sauce,cream sauce,670.0,potato gratin boome style,http://www.foodnetwork.com
5086,5086,53053,"potato gratin ""boome-style""",249f6385ac,http://www.foodnetwork.com/recipes/danny-boome/potato-gratin-boome-style-recipe2.html,tablespoon,1,"butter, without salt",14.200000,"butter , salt",cocoa butter,cocoa butter,-1.0,potato gratin boome style,http://www.foodnetwork.com
5087,5087,53054,"potato gratin ""boome-style""",249f6385ac,http://www.foodnetwork.com/recipes/danny-boome/potato-gratin-boome-style-recipe2.html,teaspoon,1,"spices, pepper, black",2.300000,"spice , pepper , black","black pepper , powder","black pepper , powder",-1.0,potato gratin boome style,http://www.foodnetwork.com
5401,5401,56558,potato-tarragon gratin,2704c194cc,http://www.food.com/recipe/potato-tarragon-gratin-217379,pound,2.5,"potatoes, raw, skin",1133.980000,"potato , , skin","potato salad ,","potato salad ,",-1.0,potato tarragon gratin,http://www.food.com


In [482]:
menu_data.loc[33, 'Closest 7t recipe by hand'] = ['blue cheese potato gratin recipe', \
                                                  'potato and parmesan gratin', 'potato gratin "boome-style"', 'potato-tarragon gratin']

Next recipe

In [483]:
menu_data[menu_data.index==37]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand,Closest 7t recipe by hand
37,feaa6c7b-9b16-4d51-b71d-75518f16cb08.jpg,Starters,"Gaspacho tomate, pastèque et crabe","Tomato, watermelon, coriander and chive oil",Gaspacho repas crevette & avocat,Not yet handled


In [484]:
set(seven_data["title_raw"][(seven_data["title_raw"].str.contains("gaspac")) & (seven_data["ingredient_raw"].str.contains("crab"))].values.tolist())

set()

In [485]:
set(seven_data["title_raw"][(seven_data["title_raw"].str.contains("gaspac")) & (seven_data["ingredient_raw"].str.contains("shrimp"))].values.tolist())

set()

In [486]:
set(seven_data["title_raw"][(seven_data["title_raw"].str.contains("soup")) & (seven_data["ingredient_raw"].str.contains("crab"))].values.tolist())

set()

In [487]:
set(seven_data["title_raw"][seven_data["title_raw"].str.contains("pach")])

{'fruit gazpacho', 'quick and easy gazpacho'}

In [488]:
menu_data.loc[37, 'Closest 7t recipe by hand'] = 'no match'

Next recipe

In [489]:
menu_data[menu_data.index==38]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand,Closest 7t recipe by hand
38,feaa6c7b-9b16-4d51-b71d-75518f16cb08.jpg,Starters,Carpaccio Thon Sésame,"Tuna carpaccio, miso, mango and lime salsa, chilli",no match,Not yet handled


In [490]:
set(seven_data["title_raw"][(seven_data["title_raw"].str.contains("carpac")) & (seven_data["ingredient_raw"].str.contains("tuna"))].values.tolist())

set()

In [491]:
set(seven_data["title_raw"][seven_data["title_raw"].str.contains("carpac")])

{'carpaccio dressing'}

In [492]:
menu_data.loc[38, 'Closest 7t recipe by hand'] = 'no match'

Next recipe

In [493]:
menu_data[menu_data.index==40]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand,Closest 7t recipe by hand
40,feaa6c7b-9b16-4d51-b71d-75518f16cb08.jpg,Mains,Paris London Rumsteak,"(240g) French beef, onion rings, café de Paris butter, pepper sauce",no match,Not yet handled


In [494]:
set(seven_data["title_raw"][(seven_data["title_raw"].str.contains("steak")) & (seven_data["ingredient_raw"].str.contains("beef"))].values.tolist())

{'asian steak recipe',
 'balsamic steak recipe',
 'barbecued flank steak',
 'best marinated flank steak recipe',
 'flank steak marinade',
 'gingered flank steak',
 'italian flank steak',
 "ken's flank steak marinade",
 'marinated flank steak',
 'mustard coated grilled flank steak',
 'rolled steak stuffed with asparagus',
 'spicy flank steak',
 'steak marinade',
 'tahitian grilled steak'}

In [495]:
set(seven_data["title_raw"][(seven_data["title_raw"].str.contains("steak")) & (seven_data["ingredient_raw"].str.contains("onion"))].values.tolist())

{'asian flank steak',
 'barbecued flank steak',
 'honey -ginger steak marinade',
 'marinated flank steak',
 'marinated flank steaks or chicken recipe',
 'no-salt steakhouse spice rub',
 'outback steakhouse bleu cheese dressing',
 'rub for grilled steak',
 'savory steak rub',
 'steakhouse garlic butter potato, pan grill'}

In [496]:
seven_data[seven_data['title_raw'].isin(['barbecued flank steak', 'marinated flank steak'])]

,Unnamed: 0,Unnamed: 0.1,title_raw,id,url,unit,quantity_raw,ingredient_raw,weight_per_ingr,ingredient_clean_comma,matched_ingredient,food_clean_comma,est_impact,title_clean,source_website
4805,4805,49898,barbecued flank steak,22683b83c4,http://www.food.com/recipe/barbecued-flank-steak-449773,cup,12,soy sauce made from soy (tamari),3456.0000,soy sauce soy,"soy sauce ,","soy sauce ,",120.000000,barbecued flank steak,http://www.food.com
4806,4806,49899,barbecued flank steak,22683b83c4,http://www.food.com/recipe/barbecued-flank-steak-449773,tablespoon,2,honey,42.3750,honey,honey,honey,38.000000,barbecued flank steak,http://www.food.com
4807,4807,49900,barbecued flank steak,22683b83c4,http://www.food.com/recipe/barbecued-flank-steak-449773,tablespoon,2,"vinegar, distilled",29.8000,"vinegar ,",vinegar,vinegar,167.000000,barbecued flank steak,http://www.food.com
4808,4808,49901,barbecued flank steak,22683b83c4,http://www.food.com/recipe/barbecued-flank-steak-449773,teaspoon,1.5,"spices, garlic powder",4.6500,"spice , garlic powder","garlic , powder , dried","garlic , powder , dried",123.200000,barbecued flank steak,http://www.food.com
4809,4809,49902,barbecued flank steak,22683b83c4,http://www.food.com/recipe/barbecued-flank-steak-449773,teaspoon,1.5,"spices, ginger, ground",2.7000,"spice , ginger , ground","ginger ,","ginger ,",80.000000,barbecued flank steak,http://www.food.com
4810,4810,49903,barbecued flank steak,22683b83c4,http://www.food.com/recipe/barbecued-flank-steak-449773,cup,12,"fresh red onions, upc: 888670013229",1800.0000,"fresh red onion ,","onion ,","onion ,",1270.000000,barbecued flank steak,http://www.food.com
4811,4811,49904,barbecued flank steak,22683b83c4,http://www.food.com/recipe/barbecued-flank-steak-449773,cup,34,"oil, olive, salad or cooking",7344.0000,"oil , olive , salad cooking","albacore , olive oil , , drained","albacore , olive oil , , drained",329.411765,barbecued flank steak,http://www.food.com
4812,4812,49905,barbecued flank steak,22683b83c4,http://www.food.com/recipe/barbecued-flank-steak-449773,pound,2022-04-02 00:00:00,"beef, flank, steak, separable lean and fat, trimmed to 0"" fat, all grades, cooked, broiled",2721.5520,"beef , flank , steak , separable lean fat , trimmed fat , , cooked , broiled","beef , flank steak ,","beef , flank steak ,",2090.000000,barbecued flank steak,http://www.food.com
5891,5891,61883,marinated flank steak,2ada9e7c1e,http://www.food.com/recipe/marinated-flank-steak-102769,pound,2,"beef, flank, steak, separable lean and fat, trimmed to 0"" fat, all grades, cooked, broiled",907.1840,"beef , flank , steak , separable lean fat , trimmed fat , , cooked , broiled","beef , flank steak ,","beef , flank steak ,",2090.000000,marinated flank steak,http://www.food.com
5892,5892,61884,marinated flank steak,2ada9e7c1e,http://www.food.com/recipe/marinated-flank-steak-102769,cup,1,"vinegar, red wine",238.4000,"vinegar , red wine","wine , red , Â°","wine , red , Â°",120.000000,marinated flank steak,http://www.food.com


In [497]:
set(seven_data["title_raw"][(seven_data["ingredient_raw"].str.contains("beef")) & (seven_data["ingredient_raw"].str.contains("onion"))].values.tolist())

set()

In [498]:
menu_data.loc[40, 'Closest 7t recipe by hand'] = 'barbecued flank steak'

Next recipe

In [499]:
menu_data[menu_data.index==41]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand,Closest 7t recipe by hand
41,feaa6c7b-9b16-4d51-b71d-75518f16cb08.jpg,Mains,Miso Salmon,"Raw and cooked white asparagus, artichoke mousse",Saumon & asperges rôtis,Not yet handled


In [500]:
set(seven_data["title_raw"][(seven_data["ingredient_raw"].str.contains("salmon")) & (seven_data["ingredient_raw"].str.contains("asparag"))].values.tolist())

set()

In [501]:
seven_data2["title_raw"][(seven_data2["ingredient_raw"].str.contains("salmon")) & (seven_data2["ingredient_raw"].str.contains("asparag"))].values.tolist()

[]

In [502]:
set(seven_data["title_raw"][(seven_data["title_raw"].str.contains("miso")) & (seven_data["ingredient_raw"].str.contains("salmon"))].values.tolist())

set()

In [503]:
set(seven_data["title_raw"][seven_data["title_raw"].str.contains("miso")])

{'a dip for raw crudites! miso sesame mayonnaise sauce',
 'creamy ginger miso dressing',
 'creamy miso dressing',
 'dill and orange miso spread',
 'ginger miso glazed tilapia',
 'miso butterscotch',
 'miso dressing',
 'miso mayonnaise',
 'miso mushrooms',
 'miso sesame dressing',
 'miso soup with tofu',
 'miso spice',
 'miso stock',
 'miso wet rub',
 'miso, carrot, and sesame dressing - japanese',
 'miso-cured vegetables',
 'miso-honey mustard',
 'nearly instant miso soup with tofu',
 'peanut-miso dressing',
 'steamed chicken with mayo-miso sauce'}

In [504]:
menu_data.loc[41, 'Closest 7t recipe by hand'] = 'no match'

Next recipe

In [505]:
menu_data[menu_data.index==49]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand,Closest 7t recipe by hand
49,in-bocca-al-lupo_42356_293.jpg,Pizza Bambini (pour les petits),Prosciutto,"sauce tomate, mozzarella, jambon",Pizza Parma,Not yet handled


In [506]:
set(seven_data["title_raw"][seven_data["title_raw"].str.contains("pizz")])

{'10 minute pizza dough!',
 '2 ingredient pizza crust (the slow roasted italian)',
 '2-ingredient pizza dough',
 '5 minute pizza base. no yeast!',
 'basic pizza sauce',
 'buffalo style chicken pizza recipe',
 "california chef's pizza blend recipe",
 "copycat: boston pizza's cactus cut potatoes dip",
 'homemade pizza sauce',
 "papa john's garlic butter for pizza dipping",
 'pizza cheese',
 'pizza popcorn',
 'pizza spice blend',
 'pizza spices',
 'simple pizza sauce',
 'spicy adobo for pizza',
 'ww 2 points - pizza nachos'}

In [507]:
menu_data.loc[49, 'Closest 7t recipe by hand'] = 'no match'

Next recipe

In [508]:
menu_data[menu_data.index==58]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand,Closest 7t recipe by hand
58,menu_bobby.jpg,Pizz',Zucchine,"Crème de courgette, mozza fior di latte IDF, provolone del monaco, chips de courgette, basilico",Pizza verde à la courgette,Not yet handled


In [509]:
menu_data.loc[58, 'Closest 7t recipe by hand'] = 'no match'

Next recipe

In [510]:
menu_data[menu_data.index==55]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand,Closest 7t recipe by hand
55,menu_bobby.jpg,Petites assiettes à partager,Gros raviolo,aux champis d'une ferme urbaine du 18è siècle,"[Raviolis à la crème de champignons, Raviolis aux champignons, chèvre frais & noisette]",Not yet handled


In [511]:
set(seven_data["title_raw"][seven_data["title_raw"].str.contains("raviol")])

{'gluten free ravioli filling',
 'potato and ricotta ravioli #5fix',
 'ravioli primavera'}

In [512]:
seven_data[seven_data['title_raw'].isin(['gluten free ravioli filling', 'ravioli primavera'])]

,Unnamed: 0,Unnamed: 0.1,title_raw,id,url,unit,quantity_raw,ingredient_raw,weight_per_ingr,ingredient_clean_comma,matched_ingredient,food_clean_comma,est_impact,title_clean,source_website
9378,9378,97833,ravioli primavera,448a000e76,http://www.food.com/recipe/ravioli-primavera-360597,pound,1,"cheese, parmesan, hard",453.592000,"cheese , parmesan , hard","parmesan cheese , cow milk","parmesan cheese , cow milk",1110.0,ravioli primavera,http://www.food.com
9379,9379,97834,ravioli primavera,448a000e76,http://www.food.com/recipe/ravioli-primavera-360597,pound,1,"shortening, vegetable, household, composite",453.592000,", vegetable , household ,",vegetable fritter,vegetable fritter,-1.0,ravioli primavera,http://www.food.com
9380,9380,97835,ravioli primavera,448a000e76,http://www.food.com/recipe/ravioli-primavera-360597,ounce,16,"sauce, pasta, spaghetti/marinara, ready-to-serve",453.592000,"sauce , pasta , spaghetti marinara ,","pasta cheese sauce spaghetti , tagliatelleâ€¦","pasta cheese sauce spaghetti , tagliatelleâ€¦",120.0,ravioli primavera,http://www.food.com
9381,9381,97836,ravioli primavera,448a000e76,http://www.food.com/recipe/ravioli-primavera-360597,cup,12,"cheese, parmesan, hard",1784.474208,"cheese , parmesan , hard","parmesan cheese , cow milk","parmesan cheese , cow milk",1110.0,ravioli primavera,http://www.food.com
24842,24842,260481,gluten free ravioli filling,b598be154e,http://www.food.com/recipe/gluten-free-ravioli-filling-263927,cup,23,"cheese, ricotta, whole milk",5704.000000,"cheese , ricotta , whole milk",ricotta cheese,ricotta cheese,1110.0,gluten ravioli filling,http://www.food.com
24843,24843,260482,gluten free ravioli filling,b598be154e,http://www.food.com/recipe/gluten-free-ravioli-filling-263927,cup,12,"leeks, (bulb and lower leaf-portion), raw",1068.000000,"leek , ,","leek ,","leek ,",-1.0,gluten ravioli filling,http://www.food.com
24844,24844,260483,gluten free ravioli filling,b598be154e,http://www.food.com/recipe/gluten-free-ravioli-filling-263927,cup,12,"tomatoes, sun-dried",648.000000,"tomato , sun dried","tomato , dried","tomato , dried",150.0,gluten ravioli filling,http://www.food.com
24845,24845,260484,gluten free ravioli filling,b598be154e,http://www.food.com/recipe/gluten-free-ravioli-filling-263927,tablespoon,1,"butter, without salt",14.200000,"butter , salt",cocoa butter,cocoa butter,-1.0,gluten ravioli filling,http://www.food.com


In [513]:
menu_data.loc[55, 'Closest 7t recipe by hand'] = 'no match'

Next recipe

In [514]:
menu_data[menu_data.index==63]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand,Closest 7t recipe by hand
63,menu_bobby.jpg,Dolci,Boozy tiramisù,Tiramisù classique + Amaretto (liqueur d'amande),Tiramisu,Not yet handled


In [515]:
set(seven_data["title_raw"][seven_data["title_raw"].str.contains("tiram")])

set()

In [516]:
menu_data.loc[63, 'Closest 7t recipe by hand'] = 'no match'

Next recipe

In [517]:
menu_data[menu_data.index==76]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand,Closest 7t recipe by hand
76,fabbrezza_202084_665.jpg,Plat au choix,Pizza Margherita,NaN,Pizza Margherita,Not yet handled


In [518]:
set(seven_data["title_raw"][seven_data["title_raw"].str.contains("pizz")])

{'10 minute pizza dough!',
 '2 ingredient pizza crust (the slow roasted italian)',
 '2-ingredient pizza dough',
 '5 minute pizza base. no yeast!',
 'basic pizza sauce',
 'buffalo style chicken pizza recipe',
 "california chef's pizza blend recipe",
 "copycat: boston pizza's cactus cut potatoes dip",
 'homemade pizza sauce',
 "papa john's garlic butter for pizza dipping",
 'pizza cheese',
 'pizza popcorn',
 'pizza spice blend',
 'pizza spices',
 'simple pizza sauce',
 'spicy adobo for pizza',
 'ww 2 points - pizza nachos'}

In [519]:
set(seven_data["title_raw"][seven_data["title_raw"].str.contains("marghe")])

set()

In [520]:
seven_data[seven_data['title_raw'].isin(['2 ingredient pizza crust (the slow roasted italian)', 'pizza cheese'])]

,Unnamed: 0,Unnamed: 0.1,title_raw,id,url,unit,quantity_raw,ingredient_raw,weight_per_ingr,ingredient_clean_comma,matched_ingredient,food_clean_comma,est_impact,title_clean,source_website
23978,23978,250494,pizza cheese,ae65e4d58c,http://www.food.com/recipe/pizza-cheese-346970,cup,14,"cheese, parmesan, hard",2081.886576,"cheese , parmesan , hard","parmesan cheese , cow milk","parmesan cheese , cow milk",1110.0,pizza cheese,http://www.food.com
23979,23979,250495,pizza cheese,ae65e4d58c,http://www.food.com/recipe/pizza-cheese-346970,cup,14,"cheese, mozzarella, low sodium",1848.000000,"cheese , mozzarella , low sodium","mozzarella cheese , cow milk","mozzarella cheese , cow milk",1110.0,pizza cheese,http://www.food.com
26952,26952,282034,2 ingredient pizza crust (the slow roasted italian),c4db91233f,http://www.food.com/recipe/2-ingredient-pizza-crust-the-slow-roasted-italian-523628,cup,1,"yogurt, greek, plain, nonfat",245.000000,"yogurt , greek , plain ,","yogurt , greek style , plain","yogurt , greek style , plain",240.0,2 ingredient pizza crust,http://www.food.com
26953,26953,282035,2 ingredient pizza crust (the slow roasted italian),c4db91233f,http://www.food.com/recipe/2-ingredient-pizza-crust-the-slow-roasted-italian-523628,cup,1.5,"wheat flour, white, all-purpose, unenriched",187.500000,"wheat flour , white , ,","wheat flour ,","wheat flour ,",-1.0,2 ingredient pizza crust,http://www.food.com


In [521]:
menu_data.loc[76, 'Closest 7t recipe by hand'] = 'no match'

Save data

In [522]:
menu_data_7tjow = menu_data[(menu_data['Closest JOW recipe by hand']!='Not yet handled') & (menu_data['Closest 7t recipe by hand']!='Not yet handled')]
print('Number of handled recipes = ', menu_data_7tjow.shape[0])

menu_data_7tjow.to_json(data_path+"data/Tools/MenuTo7tJOWRecipes_v1.json")
menu_data_7tjow.to_excel(data_path+"data/Tools/MenuTo7tJOWRecipes_v1.xlsx")

Number of handled recipes =  50


### Let's continue with new recipes that have not yet been tried to be matched, neither with Jow nor with 7t

In [523]:
menu_data[(menu_data['Closest JOW recipe by hand']=='Not yet handled') & (menu_data['Closest 7t recipe by hand']=='Not yet handled')].tail(15)

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand,Closest 7t recipe by hand
113,IMG_7930.PNG,Les entrées,"Pithiviers perdrix & cochon, jus de perdrix au porto",NaN,Not yet handled,Not yet handled
114,IMG_7930.PNG,Les entrées,"Tortellini de potimarron & butternut, agrumes & tagette",NaN,Not yet handled,Not yet handled
115,IMG_7930.PNG,Les plats,"Daurade royale de ligne, épinards, céleri & chou-fleur rôtis, coques et crèmes d'arêtes",NaN,Not yet handled,Not yet handled
116,IMG_7930.PNG,Les plats,"Canard, purée de cresson, pommes boulangères, trompettes de la mort & jus de volaille à l'argousier",NaN,Not yet handled,Not yet handled
117,IMG_7930.PNG,Les Desserts,"Baba à la farine de chataignes, kiwano, rhum Clément VSOP & mousse de marrons",NaN,Not yet handled,Not yet handled
118,IMG_7930.PNG,Les Desserts,"Crémeux de mandarine satsuma, biscuit joconde aux noisettes & meringue",NaN,Not yet handled,Not yet handled
119,IMG_7934.PNG,Entrées,Gougères Comté 12 mois,Fourrées à la crème de Brillat-Savarin et herbes fraîches,Not yet handled,Not yet handled
120,IMG_7934.PNG,Entrées,Tête à Tête,"Tête de champignon Portobello, Tête de Moine, crème de champignons",Not yet handled,Not yet handled
121,IMG_7934.PNG,Entrées,Croquetas jambon fromage,super Cheesy,Not yet handled,Not yet handled
122,IMG_7934.PNG,Plats,Gnocchi,"Fourme d'Ambert, chutney de poire, noix",Not yet handled,Not yet handled


In [524]:
menu_data[menu_data.index==0]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand,Closest 7t recipe by hand
0,burgart_205074_c70.jpg,Burgers,Superfin,"Bœuf (VBF Metzger), Comté AOC, Iceberg, Bacon, Compotée d'oignons, Sauce Burgart",Not yet handled,Not yet handled


In [525]:
set(seven_data["title_raw"][(seven_data["title_raw"].str.contains("burger")) & (seven_data["ingredient_raw"].str.contains("steak"))].values.tolist())

set()

In [526]:
set(seven_data["title_raw"][seven_data["title_raw"].str.contains("burger")])

{'amara veggie burgers',
 'asian turkey burgers',
 'awesome turkey burgers',
 'barbecue burger topping recipe',
 'blue cheese stuffed turkey burgers',
 'chicken & broccoli burgers',
 'copycat burger king creamy apple cider vinaigrette',
 'greek burgers',
 "honey slaw from harold cohen's ultimate burger",
 'peanut burgers',
 'red, white & blue burger topper',
 'spicy turkey burgers',
 'spinach feta turkey burgers',
 'the woodward veggie burger',
 'tofu burgers',
 'turkey curry burgers',
 'turkey gyro burgers'}

In [527]:
menu_data.loc[0, 'Closest 7t recipe by hand'] = 'no match'

In [528]:
jow_data[(jow_data["recettes"].str.contains("burger")) & (jow_data["only_ings"].str.contains("boeuf"))]

,cuisson,ingredients,recettes,only_ings
Blue Burger,40 minutes,"[['1 càc', ""confits d'oignons""], ['30 g', 'bleu'], ['1', 'boeuf (steak haché)'], ['150 g', 'patate douce'], ['1 poignée', 'salade (roquette)'], ['1', 'pain burger'], ['1/2 càc', ""moutarde à l'ancienne""]]",blue burger,"confits d'oignons, bleu, boeuf, patate douce, salade, pain burger, moutarde à l'ancienne"
French Burger,40 minutes,"[['2 càc', ""confits d'oignons""], ['30 g', 'comté'], ['1', 'boeuf (steak haché)'], ['150 g', 'pomme de terre'], ['10 g', 'salade (coeur de laitue)'], ['1', 'pain burger'], ['1/2 càc', ""moutarde à l'ancienne""]]",french burger,"confits d'oignons, comté, boeuf, pomme de terre, salade, pain burger, moutarde à l'ancienne"
Italian burger,40 minutes,"[['1/2 càs', 'sauce pesto'], ['1/2', 'mozzarella (boule)'], ['1', 'boeuf (steak haché)'], ['150 g', 'pomme de terre'], ['1', 'pain burger'], ['1/4', 'tomate'], ['facultatif', '1 pinc.', 'origan (séché)'], ['1 poignée', 'salade (roquette)']]",italian burger,"sauce pesto, mozzarella, boeuf, pomme de terre, pain burger, tomate, origan, salade"
Classic Cheeseburger,40 minutes,"[['1/2 càc', 'yellow mustard'], ['1 càc', 'ketchup'], ['1/8', 'oignon rouge'], ['150 g', 'pomme de terre'], ['10 g', 'salade (coeur de laitue)'], ['1', 'pain burger'], ['1 tran.', 'cheddar (tranches)'], ['1', 'boeuf (steak haché)']]",classic cheeseburger,"yellow mustard, ketchup, oignon rouge, pomme de terre, salade, pain burger, cheddar, boeuf"
Super Bacon Burger,40 minutes,"[['1/8', 'oignon rouge'], ['1 tran.', 'cheddar (tranches)'], ['1', 'boeuf (steak haché)'], ['150 g', 'pomme de terre'], ['10 g', 'salade (coeur de laitue)'], ['1', 'pain burger'], ['1 càc', 'moutarde'], ['1 càs', ""sirop d'érable""]]",super bacon burger,"oignon rouge, cheddar, boeuf, pomme de terre, salade, pain burger, moutarde, sirop d'érable"
Tom burger,40 minutes,"[['1 càc', ""confits d'oignons""], ['1/2 càs', 'crème fraîche épaisse'], ['1/4', 'échalote'], ['150 g', 'pomme de terre'], ['1', 'pain burger'], ['30 g', 'tomme'], ['1', 'boeuf (steak haché)'], ['1/10 bou.', 'persil (frais)']]",tom burger,"confits d'oignons, crème fraîche épaisse, échalote, pomme de terre, pain burger, tomme, boeuf, persil"
Burger raclette,5 minutes,"[['1', 'pain burger (brasserie)'], ['1', 'boeuf (steak haché)'], ['2 càc', ""confits d'oignons""], ['1 càc', ""moutarde à l'ancienne""], ['20 g', 'raclette']]",burger raclette,"pain burger, boeuf, confits d'oignons, moutarde à l'ancienne, raclette"
Summer burger,4 minutes,"[['1', 'pain burger'], ['1', 'boeuf (steak haché)'], ['1/2 càs', 'mayonnaise'], ['1 càc', 'ketchup'], ['30 g', 'tomate séchée'], ['20 g', 'tomme'], ['1/10 bou.', 'persil (frais)']]",summer burger,"pain burger, boeuf, mayonnaise, ketchup, tomate séchée, tomme, persil"


In [529]:
tmp = []
tmp.append(jow_data[jow_data["recettes"]=='blue burger'].index[0])
tmp.append(jow_data[jow_data["recettes"]=='french burger'].index[0])
tmp.append(jow_data[jow_data["recettes"]=='classic cheeseburger'].index[0])
tmp.append(jow_data[jow_data["recettes"]=='super bacon burger'].index[0])
tmp.append(jow_data[jow_data["recettes"]=='tom burger'].index[0])
tmp.append(jow_data[jow_data["recettes"]=='burger raclette'].index[0])

menu_data.loc[0, 'Closest JOW recipe by hand'] = tmp

Next recipe

In [530]:
menu_data[menu_data.index==1]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand,Closest 7t recipe by hand
1,burgart_205074_c70.jpg,Burgers,Bon & Beau,"Bœuf (VBF Metzger), Cheddar, Sucrine, Bacon, Tomate Confite, Sauce poivre maison",Not yet handled,Not yet handled


In [531]:
tmp = []
tmp.append(jow_data[jow_data["recettes"]=='super bacon burger'].index[0])
tmp.append(jow_data[jow_data["recettes"]=='summer burger'].index[0])

menu_data.loc[1, 'Closest JOW recipe by hand'] = tmp

In [532]:
menu_data.loc[1, 'Closest 7t recipe by hand'] = 'no match'

Next recipe

In [533]:
menu_data[menu_data.index==4]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand,Closest 7t recipe by hand
4,burgart_205074_c70.jpg,Poke Bowls,Black bowl,"Thon, Riz noir, Avocat, Oignon rouge, Cébette, Concombre, Sésame, Coriandre, Grenade, Sauce douce",Not yet handled,Not yet handled


In [534]:
jow_data[(jow_data["recettes"].str.contains("bowl")) & (jow_data["only_ings"].str.contains("thon"))]

,cuisson,ingredients,recettes,only_ings
Tuna bowl,10 minutes,"[['70 g', 'riz'], ['50 g', 'thon (conserve)'], ['80 g', 'brocoli (frais)'], ['80 g', 'patate douce'], ['2 càs', 'sauce soja sucrée']]",tuna bowl,"riz, thon, brocoli, patate douce, sauce soja sucrée"


In [535]:
jow_data[(jow_data["recettes"].str.contains("bowl")) & (jow_data["only_ings"].str.contains("avocat"))]

,cuisson,ingredients,recettes,only_ings
Primavera Poke bowl,10 minutes,"[['1', 'saumon (cru)'], ['1/2', 'avocat'], ['70 g', 'riz'], ['2 càs', 'sauce soja salée'], ['2 càs', 'vinaigre de riz'], ['10 g', 'radis'], ['facultatif', '1/5 bou.', 'coriandre (frais)'], ['20 g', 'concombre']]",primavera poke bowl,"saumon, avocat, riz, sauce soja salée, vinaigre de riz, radis, coriandre, concombre"
Rainbowl,plat froid,"[['1/2', 'avocat'], ['100 g', 'maïs (conserve)'], ['1/5 bou.', 'coriandre (frais)'], ['1/5', 'oignon rouge'], ['100 g', 'tomate cerise'], ['1/4', 'citron jaune']]",rainbowl,"avocat, maïs, coriandre, oignon rouge, tomate cerise, citron jaune"
Rice Power Bowl,10 minutes,"[['70 g', 'riz'], ['1/2', 'avocat'], ['1', 'oeuf'], ['facultatif', '1/10 bou.', 'ciboulette'], ['1 càc', 'vinaigre de vin rouge']]",rice power bowl,"riz, avocat, oeuf, ciboulette, vinaigre de vin rouge"
Vegan poke bowl,10 minutes,"[['70 g', 'riz (complet)'], ['70 g', 'tofu (nature)'], ['1 càs', 'sauce soja salée'], ['1/2', 'avocat'], ['50 g', 'concombre'], ['1 càs', 'yaourt végétal (coco)']]",vegan poke bowl,"riz, tofu, sauce soja salée, avocat, concombre, yaourt végétal"
Mexican bowl,10 minutes,"[['1', 'poulet (escalope)'], ['1 càc', 'mélange épices mexicaines'], ['1/4', 'avocat'], ['60 g', 'maïs (conserve)'], ['60 g', 'haricots rouges (cuits)'], ['70 g', 'riz'], ['facultatif', '1/10 bou.', 'coriandre (frais)']]",mexican bowl,"poulet, mélange épices mexicaines, avocat, maïs, haricots rouges, riz, coriandre"
Bowl crevettes & avocat,4 minutes,"[['50 g', 'vermicelles de riz'], ['1/2', 'avocat'], ['60 g', 'crevette (cuite)'], ['1 càs', 'sauce soja sucrée'], ['1/4', 'citron vert'], ['facultatif', '50 g', 'cacahuète'], ['1/10 bou.', 'coriandre (frais)']]",bowl crevettes & avocat,"vermicelles de riz, avocat, crevette, sauce soja sucrée, citron vert, cacahuète, coriandre"
Quinoa green bowl,10 minutes,"[['60 g', 'quinoa (cru)'], ['1/2', 'avocat'], ['30 g', 'pois chiches (cuits)'], ['1/4', 'citron jaune'], ['1/4 gou.', 'ail'], ['2 càs', 'yaourt amande'], ['1/10 bou.', 'menthe (feuilles)']]",quinoa green bowl,"quinoa, avocat, pois chiches, citron jaune, ail, yaourt amande, menthe"
Quinoa & hummus bowl,2 minutes,"[['75 g', 'quinoa (cuit)'], ['80 g', 'houmous'], ['1/4', 'concombre'], ['1/2', 'avocat'], ['facultatif', '1/10 bou.', 'persil (frais)']]",quinoa & hummus bowl,"quinoa, houmous, concombre, avocat, persil"


In [536]:
tmp = []
tmp.append(jow_data[jow_data["recettes"]=='tuna bowl'].index[0])
tmp.append(jow_data[jow_data["recettes"]=='primavera poke bowl'].index[0])

menu_data.loc[4, 'Closest JOW recipe by hand'] = tmp

In [537]:
set(seven_data["title_raw"][seven_data["title_raw"].str.contains("bowl")])

{'blueberry almond quinoa smoothie bowl',
 'bread bowls ii',
 'crispy parmesan bowls',
 'festive nut bowl',
 'fruit bowl',
 'orange chicken and rice bowl',
 'orange-chicken rice bowl',
 'parmesan bowl',
 'parmesan salad bowl',
 'peanut butter granola and mixed berry bowl',
 'quick oatmeal raisin bowl',
 'regular old fruit bowl',
 'rice bowl'}

In [538]:
set(seven_data["title_raw"][(seven_data["title_raw"].str.contains("bowl")) & (seven_data["ingredient_raw"].str.contains("tuna"))].values.tolist())

set()

In [539]:
seven_data["title_raw"][(seven_data["ingredient_raw"].str.contains("rice")) & (seven_data["ingredient_raw"].str.contains("onion"))]

Series([], Name: title_raw, dtype: object)

Oh Oh there is a mistake here

In [540]:
def intersection(lst1, lst2):
    return list(set(lst1) & set(lst2))

l1 = list(seven_data[seven_data["ingredient_raw"].str.contains("rice")].index)
l2 = list(seven_data[seven_data["ingredient_raw"].str.contains("onion")].index)
intersection(l1, l2)

[]

In [541]:
df_tmp = seven_data[seven_data["title_raw"]=='wild rice, brown rice and dried fruit pilaf']
df_tmp


,Unnamed: 0,Unnamed: 0.1,title_raw,id,url,unit,quantity_raw,ingredient_raw,weight_per_ingr,ingredient_clean_comma,matched_ingredient,food_clean_comma,est_impact,title_clean,source_website
5815,5815,61144,"wild rice, brown rice and dried fruit pilaf",2a56940ff1,http://www.food.com/recipe/wild-rice-brown-rice-and-dried-fruit-pilaf-514490,cup,34,"wild rice, raw",5440.000000,"wild rice ,","wild rice ,","wild rice ,",467.000000,wild rice brown rice dried fruit pilaf,http://www.food.com
5816,5816,61145,"wild rice, brown rice and dried fruit pilaf",2a56940ff1,http://www.food.com/recipe/wild-rice-brown-rice-and-dried-fruit-pilaf-514490,cup,34,"rice, white, long-grain, regular, unenriched, cooked without salt",5372.000000,"rice , white , long grain , , , cooked salt","white rice , cooked , chicken","white rice , cooked , chicken",467.000000,wild rice brown rice dried fruit pilaf,http://www.food.com
5817,5817,61146,"wild rice, brown rice and dried fruit pilaf",2a56940ff1,http://www.food.com/recipe/wild-rice-brown-rice-and-dried-fruit-pilaf-514490,cup,4,"shortening, vegetable, household, composite",819.200000,", vegetable , household ,",vegetable fritter,vegetable fritter,-1.000000,wild rice brown rice dried fruit pilaf,http://www.food.com
5818,5818,61147,"wild rice, brown rice and dried fruit pilaf",2a56940ff1,http://www.food.com/recipe/wild-rice-brown-rice-and-dried-fruit-pilaf-514490,cup,12,"nuts, pecans",1308.000000,"nut , pecan",pecan nut,pecan nut,300.000000,wild rice brown rice dried fruit pilaf,http://www.food.com
5819,5819,61148,"wild rice, brown rice and dried fruit pilaf",2a56940ff1,http://www.food.com/recipe/wild-rice-brown-rice-and-dried-fruit-pilaf-514490,cup,13,"onions, spring or scallions (includes tops and bulb), raw",1300.000000,"onion , spring scallion ,","onion ,","onion ,",1270.000000,wild rice brown rice dried fruit pilaf,http://www.food.com
5820,5820,61149,"wild rice, brown rice and dried fruit pilaf",2a56940ff1,http://www.food.com/recipe/wild-rice-brown-rice-and-dried-fruit-pilaf-514490,cup,13,"cranberries, dried, sweetened",2080.000000,"cranberry , dried ,","cranberry ,","cranberry ,",-1.000000,wild rice brown rice dried fruit pilaf,http://www.food.com
5821,5821,61150,"wild rice, brown rice and dried fruit pilaf",2a56940ff1,http://www.food.com/recipe/wild-rice-brown-rice-and-dried-fruit-pilaf-514490,cup,13,"apricots, dried, sulfured, uncooked",1690.000000,"apricot , dried , sulfured , uncooked","apricot , pitted , dried","apricot , pitted , dried",-1.000000,wild rice brown rice dried fruit pilaf,http://www.food.com
5822,5822,61151,"wild rice, brown rice and dried fruit pilaf",2a56940ff1,http://www.food.com/recipe/wild-rice-brown-rice-and-dried-fruit-pilaf-514490,cup,13,"parsley, fresh",780.000000,"parsley , fresh","parsley , fresh","parsley , fresh",-1.000000,wild rice brown rice dried fruit pilaf,http://www.food.com
5823,5823,61152,"wild rice, brown rice and dried fruit pilaf",2a56940ff1,http://www.food.com/recipe/wild-rice-brown-rice-and-dried-fruit-pilaf-514490,tablespoon,1,"oil, olive, salad or cooking",13.500000,"oil , olive , salad cooking","albacore , olive oil , , drained","albacore , olive oil , , drained",329.411765,wild rice brown rice dried fruit pilaf,http://www.food.com
5824,5824,61153,"wild rice, brown rice and dried fruit pilaf",2a56940ff1,http://www.food.com/recipe/wild-rice-brown-rice-and-dried-fruit-pilaf-514490,tablespoon,1,"orange juice, raw",15.500000,"orange juice ,","orange juice ,","orange juice ,",1270.000000,wild rice brown rice dried fruit pilaf,http://www.food.com


In [542]:
l1 = list(df_tmp[df_tmp["ingredient_raw"].str.contains("rice")].index)
l2 = list(df_tmp[df_tmp["ingredient_raw"].str.contains("onion")].index)
print(l1)
print(l2)
print(intersection(l1, l2))

[5815, 5816]
[5819]
[]


In [543]:
df_tmp[(df_tmp["ingredient_raw"].str.contains("rice")) | (df_tmp["ingredient_raw"].str.contains("onion"))]

,Unnamed: 0,Unnamed: 0.1,title_raw,id,url,unit,quantity_raw,ingredient_raw,weight_per_ingr,ingredient_clean_comma,matched_ingredient,food_clean_comma,est_impact,title_clean,source_website
5815,5815,61144,"wild rice, brown rice and dried fruit pilaf",2a56940ff1,http://www.food.com/recipe/wild-rice-brown-rice-and-dried-fruit-pilaf-514490,cup,34,"wild rice, raw",5440.0,"wild rice ,","wild rice ,","wild rice ,",467.0,wild rice brown rice dried fruit pilaf,http://www.food.com
5816,5816,61145,"wild rice, brown rice and dried fruit pilaf",2a56940ff1,http://www.food.com/recipe/wild-rice-brown-rice-and-dried-fruit-pilaf-514490,cup,34,"rice, white, long-grain, regular, unenriched, cooked without salt",5372.0,"rice , white , long grain , , , cooked salt","white rice , cooked , chicken","white rice , cooked , chicken",467.0,wild rice brown rice dried fruit pilaf,http://www.food.com
5819,5819,61148,"wild rice, brown rice and dried fruit pilaf",2a56940ff1,http://www.food.com/recipe/wild-rice-brown-rice-and-dried-fruit-pilaf-514490,cup,13,"onions, spring or scallions (includes tops and bulb), raw",1300.0,"onion , spring scallion ,","onion ,","onion ,",1270.0,wild rice brown rice dried fruit pilaf,http://www.food.com


In [544]:
seven_data2[(seven_data2["ingredient_raw"].str.contains("rice")) & (seven_data2["ingredient_raw"].str.contains("onion"))]["title_raw"].values.tolist()

['artichoke rice salad recipe',
 'asian flavored coleslaw with rice vinegar and ginger recipe',
 'asian ginger grill marinade',
 'basic sweet and sour sauce',
 'basil cashew chicken salad',
 'brown rice and cheese',
 'brown rice royal',
 'chicken curry in a hurry',
 'chicken in frypan',
 'country rice',
 'cream of tomato soup with rice and basil',
 'creamy ginger miso dressing',
 'crunchy baked vegetables',
 'dry middle east rice and bulgur mix',
 'eastchase sweet onion rice casserole recipe',
 'easy shrimp fried rice',
 'fiesta turkey with cream sauce',
 'fish steamed in packets',
 'fried rice',
 'fruited rice mix',
 'ginger black rice',
 'ginger-scallion soy sauce',
 'grecian tofu balls',
 'green herb rice',
 'herbed rice in a jar',
 'herbed rice pilaf',
 'honey sesame dipping sauce',
 'hurry curry',
 'indian rice',
 'lime-peanut rice pilaf',
 'marinated flank steak',
 'melon chicken salad',
 'mexican rice',
 "mona's onion casserole",
 'napa cabbage slaw',
 'palouse soup mix',
 'quic

In [545]:
seven_data2.head()

,title_raw,ingredient_raw
0,""" child's play"" sour pops","lemon juice, raw, lime juice, raw"
1,""" fried egg sundaes""","ice creams, vanilla, spices, pumpkin pie spice, cream, whipped, cream topping, pressurized"
2,""" world's best ""( and easiest ) teriyaki chicken wings","chicken, broilers or fryers, wing, meat and skin, raw, soy sauce made from soy (tamari), sugars, brown"
3,"""apple crisp"" peanut butter snack bites","peanut butter, smooth style, without salt, honey, oats, apples, raw, with skin, nuts, walnuts, english, spices, cinnamon, ground"
4,"""berry good"" smoothie","apple juice, canned or bottled, unsweetened, without added ascorbic acid, raspberries, raw, blackberries, raw, strawberries, raw, blueberries, raw"


OK the error is corrected. Let's see the 7t recipes with rica and tuna

In [546]:
seven_data2[(seven_data2["ingredient_raw"].str.contains("rice")) & (seven_data2["ingredient_raw"].str.contains("tuna"))]["title_raw"].values.tolist()

[]

In [547]:
menu_data.loc[4, 'Closest 7t recipe by hand'] = 'no match'

Next recipe

In [548]:
menu_data[menu_data.index==5]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand,Closest 7t recipe by hand
5,burgart_205074_c70.jpg,Salades,Crispy Caesar,"Croquettes de poulet, Parmesan, Champignons bruns, Salade Romaine, Ciboulette",Not yet handled,Not yet handled


In [549]:
jow_data[(jow_data["recettes"].str.contains("césar"))]

,cuisson,ingredients,recettes,only_ings
Salade César,13 minutes,"[['1', 'poulet (escalope)'], ['2 poignées', 'salade (coeur de laitue)'], ['20 g', 'parmesan (morceaux)'], ['1 tran.', 'pain de campagne (tranché)'], ['1/4', 'oeuf'], ['1/4 càs', 'moutarde'], ['1/4 gou.', 'ail'], ['4 cl', 'huile de tournesol']]",salade césar,"poulet, salade, parmesan, pain de campagne, oeuf, moutarde, ail, huile de tournesol"


In [550]:
jow_data[(jow_data["recettes"].str.contains("salade")) & (jow_data["only_ings"].str.contains("poulet"))]

,cuisson,ingredients,recettes,only_ings
Poulet & salade d'été,8 minutes,"[['130 g', 'tomate cerise'], ['1/2', 'avocat'], ['1', 'poulet (escalope)'], ['1/4', 'citron vert'], ['2 poignées', 'salade (roquette)'], ['1/10 bou.', 'basilic (frais)']]",poulet & salade d'été,"tomate cerise, avocat, poulet, citron vert, salade, basilic"
Salade César,13 minutes,"[['1', 'poulet (escalope)'], ['2 poignées', 'salade (coeur de laitue)'], ['20 g', 'parmesan (morceaux)'], ['1 tran.', 'pain de campagne (tranché)'], ['1/4', 'oeuf'], ['1/4 càs', 'moutarde'], ['1/4 gou.', 'ail'], ['4 cl', 'huile de tournesol']]",salade césar,"poulet, salade, parmesan, pain de campagne, oeuf, moutarde, ail, huile de tournesol"
Poulet croustillant & salade,8 minutes,"[['1', 'poulet (escalope)'], ['1', 'oeuf'], ['2 càs', 'farine de blé'], ['2 càs', 'chapelure'], ['5 cl', 'huile de tournesol'], ['2 poignées', 'salade (mélange)'], ['facultatif', '1 pinc.', 'piment (doux)'], ['facultatif', '1/2', 'citron jaune']]",poulet croustillant & salade,"poulet, oeuf, farine de blé, chapelure, huile de tournesol, salade, piment, citron jaune"
"Salade poulet, menthe & semoule",6 minutes,"[['2', 'poulet (escalope)'], ['120 g', 'semoule'], ['160 g', 'concombre'], ['2 càs', 'yaourt grec'], ['1/2 gou.', 'ail'], ['1/5 bou.', 'menthe (feuilles)'], ['1', 'oeuf']]","salade poulet, menthe & semoule","poulet, semoule, concombre, yaourt grec, ail, menthe, oeuf"
Salade printanière au poulet,10 minutes,"[['1', 'poulet (escalope)'], ['20 g', 'feta'], ['50 g', 'radis'], ['80 g', 'petits pois (surgelés)'], ['60 g', 'mélange céréales'], ['1 càc', 'vinaigre balsamique']]",salade printanière au poulet,"poulet, feta, radis, petits pois, mélange céréales, vinaigre balsamique"
"Salade poulet, avocat & lard",12 minutes,"[['1', 'poulet (escalope)'], ['1', 'tomate'], ['1/2', 'avocat'], ['2 poignées', 'salade (mélange)'], ['2 tran.', 'lard (tranches)'], ['1/2 càs', 'vinaigre balsamique']]","salade poulet, avocat & lard","poulet, tomate, avocat, salade, lard, vinaigre balsamique"
Poulet parma & salade,15 minutes,"[['1', 'poulet (pané)'], ['50 g', 'sauce tomate (basilic)'], ['1 càs', 'parmesan (râpé)'], ['40 g', 'mozzarella (à cuire)'], ['2 poignées', 'salade (mélange)'], ['facultatif', '1/10 bou.', 'basilic (frais)']]",poulet parma & salade,"poulet, sauce tomate, parmesan, mozzarella, salade, basilic"
"Salade poulet, roquette & feta",10 minutes,"[['1', 'poulet (escalope)'], ['2 poignées', 'salade (roquette)'], ['40 g', 'feta'], ['1/2 càc', ""moutarde à l'ancienne""], ['1/2', 'avocat'], ['facultatif', '10 g', 'pignons de pin']]","salade poulet, roquette & feta","poulet, salade, feta, moutarde à l'ancienne, avocat, pignons de pin"
"Salade poulet, blé & pesto",12 minutes,"[['1', 'poulet (escalope)'], ['2 poignées', 'salade (mélange)'], ['30 g', 'petits pois (surgelés)'], ['40 g', 'blé (à cuire)'], ['1 càs', 'sauce pesto'], ['20 g', 'feta']]","salade poulet, blé & pesto","poulet, salade, petits pois, blé, sauce pesto, feta"
"Salade poulet, tzatziki & betterave",10 minutes,"[['1', 'poulet (escalope)'], ['80 g', 'lentilles (cuites)'], ['1/4', 'brocoli (frais)'], ['60 g', 'betterave (cuite)'], ['50 g'], ['1/2 càc']]","salade poulet, tzatziki & betterave","poulet, lentilles, brocoli, betterave"


In [551]:
tmp = []
tmp.append(jow_data[jow_data["recettes"]=='salade césar'].index[0])
tmp.append(jow_data[jow_data["recettes"]=='salade au poulet pané'].index[0])

menu_data.loc[5, 'Closest JOW recipe by hand'] = tmp

In [552]:
seven_data2[(seven_data2["title_raw"].str.contains("caesar")) & (seven_data2["ingredient_raw"].str.contains("chicken"))]

,title_raw,ingredient_raw
819,better-for-you grilled chicken caesar salad,"salad dressing, italian dressing, commercial, regular, cheese, parmesan, hard, lettuce, cos or romaine, raw, croutons, plain, chicken, broiler or fryers, breast, skinless, boneless, meat only, raw"
1564,chicken caesar tortellini with toasted almonds,"cheese, parmesan, hard, nuts, almonds, spices, pepper, black, spices, basil, dried, salad dressing, ranch dressing, regular, chicken, broiler or fryers, breast, skinless, boneless, meat only, raw"
2735,favorite grilled chicken caesar salad,"salad dressing, italian dressing, commercial, regular, cheese, parmesan, hard, lettuce, cos or romaine, raw, croutons, plain, chicken, broiler or fryers, breast, skinless, boneless, meat only, raw"
3207,grilled chicken a la caesar,"salad dressing, italian dressing, commercial, regular, cheese, parmesan, hard, lettuce, cos or romaine, raw, croutons, plain, chicken, broiler or fryers, breast, skinless, boneless, meat only, raw"
3208,grilled chicken caesar salad,"salad dressing, italian dressing, commercial, regular, cheese, parmesan, hard, lettuce, cos or romaine, raw, croutons, plain, chicken, broiler or fryers, breast, skinless, boneless, meat only, raw"


In [553]:
menu_data.loc[5, 'Closest 7t recipe by hand'] = ['better-for-you grilled chicken caesar salad', 'favorite grilled chicken caesar salad', \
                                                'grilled chicken a la caesar', 'grilled chicken caesar salad']

Next recipe

In [554]:
menu_data[menu_data.index==6]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand,Closest 7t recipe by hand
6,burgart_205074_c70.jpg,Desserts,Cheesecake maison,NaN,Not yet handled,Not yet handled


In [555]:
jow_data[(jow_data["recettes"].str.contains("cheesecake"))]

,cuisson,ingredients,recettes,only_ings
Cheesecake,40 minutes,"[['3', 'oeuf'], ['1 càs', ""arôme fleur d'oranger""], ['200 g', 'sucre (en poudre)'], ['100 g', 'speculoos'], ['100 g', 'palet breton'], ['80 g', 'beurre'], ['40 g', 'crème fraîche épaisse'], ['1', 'vanille (gousse)']]",cheesecake,"oeuf, arôme fleur d'oranger, sucre, speculoos, palet breton, beurre, crème fraîche épaisse, vanille"
Cheesecake feta courgette menthe,50 minutes,"[['200 g', 'cracker'], ['2 càs', 'parmesan (râpé)'], ['80 g', 'beurre'], ['250 g', 'feta'], ['3', 'oeuf'], ['1 gou.', 'ail'], ['2', 'courgette'], ['1/2 bou.', 'menthe (feuilles)']]",cheesecake feta courgette menthe,"cracker, parmesan, beurre, feta, oeuf, ail, courgette, menthe"
Cheesecake coco & fraises,4 heures,"[['200 g', 'palet breton'], ['600 g', 'fromage frais'], ['50 cl', 'boisson coco'], ['8', 'gélatine'], ['250 g', 'sucre (en poudre)'], ['125 g', 'fraise'], ['80 g', 'beurre']]",cheesecake coco & fraises,"palet breton, fromage frais, boisson coco, gélatine, sucre, fraise, beurre"


In [556]:
jow_data[(jow_data["recettes"].str.contains("gâteau au fromage"))]

,cuisson,ingredients,recettes,only_ings


In [557]:
menu_data.loc[6, 'Closest JOW recipe by hand'] = jow_data[jow_data["recettes"]=='cheesecake'].index[0]

In [558]:
seven_data2[seven_data2["title_raw"].str.contains("cheesecake")]

,title_raw,ingredient_raw
896,blockbuster chocolate chip cheesecake recipe,"oats, butter, without salt, nuts, almonds, sugars, brown"
1135,brown sugar crust for cheesecakes,"butter, without salt, wheat flour, white, all-purpose, unenriched, sugars, brown"
1489,cheesecake for breakfast,"honey, yogurt, greek, plain, nonfat"
1659,chocolate cheesecake mousse,"honey, cheese, parmesan, hard, cocoa, dry powder, unsweetened"
2438,dorothy's blueberry cheesecake dessert recipe,"nuts, pecans, butter, without salt, wheat flour, white, all-purpose, unenriched"
2872,frozen lemon cheesecake ice cream,"milk, fluid, 1% fat, without added vitamin a and vitamin d, cheese, parmesan, hard, lemon juice, raw"
4014,lemon cheesecake smoothie,"sherbet, orange, milk, buttermilk, fluid, cultured, lowfat, cheese, parmesan, hard"
4131,low sugar coconut-almond pie crust or cheesecake crust,"nuts, almonds, sugars, brown, shortening confectionery, coconut (hydrogenated) and or palm kernel (hydrogenated), vanilla extract, butter, without salt"
5223,pecan cheesecake bars recipe,"nuts, pecans, butter, without salt, wheat flour, white, all-purpose, unenriched, sugars, brown"
5470,praline topping for cheesecake or apple pie,"nuts, pecans, vanilla extract, cream, whipped, cream topping, pressurized, sugars, brown"


In [559]:
menu_data.loc[6, 'Closest 7t recipe by hand'] = 'no match'

Next recipe

In [560]:
menu_data[menu_data.index==26]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand,Closest 7t recipe by hand
26,Carte-in-seoul.png,Nouilles,Jjajan myun,"Nouille à la sauce noir jjajan, légume accomagné de radis jaune",Not yet handled,Not yet handled


In [561]:
jow_data[jow_data["recettes"].str.contains("corée")]

,cuisson,ingredients,recettes,only_ings


In [562]:
jow_data[(jow_data["recettes"].str.contains("nouille")) & (jow_data["only_ings"].str.contains("soja"))]

,cuisson,ingredients,recettes,only_ings
Nouilles sautées aux légumes d'été,7 minutes,"[['70 g', 'nouilles chinoises (aux oeufs)'], ['50 g', 'tomate cerise'], ['50 g', 'courgette'], ['1/2 càs', 'sauce soja salée'], ['1/4 gou.', 'ail']]",nouilles sautées aux légumes d'été,"nouilles chinoises, tomate cerise, courgette, sauce soja salée, ail"
"Salade de nouilles, courgette & sésame",10 minutes,"[['80 g', 'nouilles de riz'], ['1', 'courgette'], ['1 càs', 'huile de sésame'], ['1 càc', 'sauce soja salée'], ['1/5 bou.', 'coriandre (frais)'], ['facultatif', '1/4', 'échalote']]","salade de nouilles, courgette & sésame","nouilles de riz, courgette, huile de sésame, sauce soja salée, coriandre, échalote"
Nouilles champignons & sésame,10 minutes,"[['80 g', 'nouilles de riz'], ['100 g', 'champignons de paris (frais)'], ['1 càc', 'huile de sésame'], ['1 càc', 'sauce soja salée'], ['1/4 càc'], ['facultatif', '1 cm']]",nouilles champignons & sésame,"nouilles de riz, champignons de paris, huile de sésame, sauce soja salée"
Nouilles sautées au poulet,12 minutes,"[['70 g', 'nouilles chinoises (aux oeufs)'], ['80 g', 'brocoli (frais)'], ['1/2', 'poivron rouge'], ['1/2 càs', 'sauce soja salée'], ['1/4 gou.', 'ail'], ['facultatif', '1/2 càc', 'gingembre (moulu)'], ['1', 'poulet (escalope)']]",nouilles sautées au poulet,"nouilles chinoises, brocoli, poivron rouge, sauce soja salée, ail, gingembre, poulet"
Nouilles sautées aux crevettes,6 minutes,"[['70 g', 'nouilles chinoises (aux oeufs)'], ['70 g', 'haricot vert (frais)'], ['1/2', 'carotte (frais)'], ['1/2 càs', 'sauce soja salée'], ['1/4 gou.', 'ail'], ['facultatif', '1/2 càc', 'gingembre (moulu)'], ['80 g', 'crevette (cuite)']]",nouilles sautées aux crevettes,"nouilles chinoises, haricot vert, carotte, sauce soja salée, ail, gingembre, crevette"
Nouilles sautées au bœuf,10 minutes,"[['70 g', 'nouilles chinoises (aux oeufs)'], ['80 g', 'brocoli (frais)'], ['40 g', 'pousses de soja'], ['2 càs', 'sauce soja salée'], ['1/2 gou.', 'ail'], ['facultatif', '1/2 càc', 'gingembre (moulu)'], ['1', 'boeuf (bavette)']]",nouilles sautées au boeuf,"nouilles chinoises, brocoli, pousses de soja, sauce soja salée, ail, gingembre, boeuf"
Nouilles sautées légumes & œuf,14 minutes,"[['70 g', 'nouilles chinoises (aux oeufs)'], ['70 g', 'champignons de paris (frais)'], ['80 g', 'haricot vert (frais)'], ['2 càs', 'sauce soja salée'], ['1/4 gou.', 'ail'], ['1', 'oeuf']]",nouilles sautées légumes & oeuf,"nouilles chinoises, champignons de paris, haricot vert, sauce soja salée, ail, oeuf"
Nouilles de riz & porc sauté,7 minutes,"[['80 g', 'nouilles de riz'], ['80 g', 'chair à saucisse'], ['50 g', 'concombre'], ['20 g', 'cacahuète'], ['1 pinc.', ""piment d'espelette""], ['2 càs', 'sauce soja salée'], ['facultatif', '1/10 bou.', 'menthe (feuilles)']]",nouilles de riz & porc sauté,"nouilles de riz, chair à saucisse, concombre, cacahuète, piment d'espelette, sauce soja salée, menthe"
Poulet & nouilles sautées aux légumes,12 minutes,"[['70 g', 'nouilles chinoises (aux oeufs)'], ['1', 'poulet (escalope)'], ['250 g', 'julienne de légumes (surgelés)'], ['1 càs', 'sauce soja salée']]",poulet & nouilles sautées aux légumes,"nouilles chinoises, poulet, julienne de légumes, sauce soja salée"


In [563]:
tmp = []
tmp.append(jow_data[jow_data["recettes"]=="nouilles sautées aux légumes d'été"].index[0])
tmp.append(jow_data[jow_data["recettes"]=='nouilles champignons & sésame'].index[0])
tmp.append(jow_data[jow_data["recettes"]=='nouilles sautées légumes & oeuf'].index[0])

menu_data.loc[26, 'Closest JOW recipe by hand'] = tmp

In [564]:
seven_data2[seven_data2["title_raw"].str.contains("korea")]

,title_raw,ingredient_raw
738,beef bulgogi(korean marinated beef),"spices, garlic powder, sugars, brown, spices, pepper, black, oil, sesame, salad or cooking, onions, spring or scallions (includes tops and bulb), raw, beef, flank, steak, separable lean and fat, trimmed to 0"" fat, all grades, cooked, broiled, seeds, sesame seeds, whole, dried, soy sauce made from soy (tamari)"
2321,dak bulgogi (korean chicken bulgogi),"spices, garlic powder, soup, swanson chicken broth 99% fat free, honey, spices, ginger, ground, spices, pepper, black, oil, sesame, salad or cooking, rice, white, long-grain, regular, unenriched, cooked without salt, sugars, brown, seeds, sesame seeds, whole, dried, soy sauce made from soy (tamari), lemon juice, raw"
3961,korean bbq chicken,"spices, garlic powder, honey, spices, ginger, ground, spices, pepper, black, oil, sesame, salad or cooking, rice, white, long-grain, regular, unenriched, cooked without salt, sugars, brown, seeds, sesame seeds, whole, dried, soy sauce made from soy (tamari), chicken, broiler or fryers, breast, skinless, boneless, meat only, raw, lemon juice, raw"
3962,korean dipping sauce,"spices, garlic powder, spices, pepper, red or cayenne, oil, sesame, salad or cooking, seeds, sesame seeds, whole, dried, onions, spring or scallions (includes tops and bulb), raw, soy sauce made from soy (tamari)"
3963,korean greens,"spinach, raw, oil, sesame, salad or cooking, seeds, sesame seeds, whole, dried, oil, olive, salad or cooking, vinegar, distilled, soy sauce made from soy (tamari)"
3964,korean sesame seed cookies,"syrup, maple, canadian, seeds, sesame seeds, whole, dried, sugars, brown"
3965,korean spinach,"spices, garlic powder, spinach, raw, oil, sesame, salad or cooking, onions, spring or scallions (includes tops and bulb), raw, seeds, sesame seeds, whole, dried, soy sauce made from soy (tamari)"
6181,simple spicy korean hot red pepper paste dip for cucumbers,"spices, pepper, red or cayenne, oil, sesame, salad or cooking, seeds, sesame seeds, whole, dried"


In [565]:
seven_data2[(seven_data2["title_raw"].str.contains("noodle")) & (seven_data2["ingredient_raw"].str.contains("soja"))]

,title_raw,ingredient_raw


In [566]:
seven_data2[(seven_data2["title_raw"].str.contains("pasta")) & (seven_data2["ingredient_raw"].str.contains("soja"))]

,title_raw,ingredient_raw


In [567]:
menu_data.loc[26, 'Closest 7t recipe by hand'] = 'no match'

Next recipe

In [568]:
menu_data[menu_data.index==124]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand,Closest 7t recipe by hand
124,IMG_7934.PNG,Plats,Tartiflette,"Reblochon, pommes de terre, oignons, poitrine fumée, mesclun (20' de cuisson)",Not yet handled,Not yet handled


In [569]:
jow_data[jow_data["recettes"].str.contains("tartiflette")]

,cuisson,ingredients,recettes,only_ings
Tartiflette express,35 minutes,"[['100 g', 'reblochon'], ['250 g', 'pomme de terre'], ['50 g', 'lardons'], ['1/2', 'oignon jaune'], ['2 cl', 'vin blanc'], ['1 càs', 'crème fraîche épaisse']]",tartiflette express,"reblochon, pomme de terre, lardons, oignon jaune, vin blanc, crème fraîche épaisse"


In [570]:
menu_data.loc[124, 'Closest JOW recipe by hand'] = jow_data[jow_data["recettes"]=='tartiflette express'].index[0]

In [571]:
menu_data.loc[124, 'Closest 7t recipe by hand'] = ['blue cheese potato gratin recipe', \
                                                  'potato and parmesan gratin', 'potato gratin "boome-style"', 'potato-tarragon gratin']

Next recipe

In [572]:
menu_data[menu_data.index==125]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand,Closest 7t recipe by hand
125,IMG_7934.PNG,Plats,Saucisse - Purée truffée,"Saucisse d'Auvergne, purée à la truffe, crème d'Abondance",Not yet handled,Not yet handled


In [573]:
jow_data[(jow_data["only_ings"].str.contains("saucisse")) & (jow_data["only_ings"].str.contains("pomme de terre"))]

,cuisson,ingredients,recettes,only_ings
Chipo & légumes rôtis,45 minutes,"[['2', 'saucisse (chipolata)'], ['200 g', 'pomme de terre (petites)'], ['1/4', 'oignon jaune'], ['1/2 gou.', 'ail'], ['2 cl', 'vin blanc'], ['1/2', 'poivron rouge'], ['1 pinc.', 'thym (feuilles)']]",chipo & légumes rôtis,"saucisse, pomme de terre, oignon jaune, ail, vin blanc, poivron rouge, thym"
Saucisse purée & épinards,plat froid,"[['200 g', 'pomme de terre'], ['100 g', 'saucisse (fumée)'], ['80 g', 'epinards (surgelés)'], ['1 càs', 'crème fraîche épaisse']]",saucisse purée & épinards,"pomme de terre, saucisse, epinards, crème fraîche épaisse"
Saucisse fumée & choux de Bruxelles rôtis,30 minutes,"[['1', 'saucisse (fumée)'], ['150 g', 'chou de bruxelles'], ['100 g', 'pomme de terre'], ['1 càc', 'miel (liquide)'], ['1 càc', ""moutarde à l'ancienne""]]",saucisse fumée & choux de bruxelles rôtis,"saucisse, chou de bruxelles, pomme de terre, miel, moutarde à l'ancienne"
Pommes de terre & saucisses rôties,30 minutes,"[['1', 'saucisse (fumée)'], ['250 g', 'pomme de terre'], ['1 pinc.', 'romarin (séché)'], ['1 càc', 'moutarde']]",pommes de terre & saucisses rôties,"saucisse, pomme de terre, romarin, moutarde"
Saucisse aux oignons,plat froid,"[['250 g', 'pomme de terre'], ['5 cl', 'lait'], ['40 g', 'beurre'], ['1', 'saucisse (fumée)']]",saucisse aux oignons,"pomme de terre, lait, beurre, saucisse"
Saucisse & purée maison,plat froid,"[['250 g', 'pomme de terre'], ['5 cl', 'lait'], ['20 g', 'beurre'], ['1', 'saucisse (fumée)'], ['1/2 càc']]",saucisse & purée maison,"pomme de terre, lait, beurre, saucisse"
One pan chipolatas & légumes rôtis,45 minutes,"[['2', 'saucisse (chipolata)'], ['1', 'carotte (frais)'], ['1/2', 'courgette'], ['200 g', 'pomme de terre'], ['1/2', 'oignon rouge'], ['facultatif', '1 pinc.', 'herbes de provence']]",one pan chipolatas & légumes rôtis,"saucisse, carotte, courgette, pomme de terre, oignon rouge, herbes de provence"


In [574]:
jow_data[(jow_data["only_ings"].str.contains("saucisse")) & (jow_data["recettes"].str.contains("purée"))]

,cuisson,ingredients,recettes,only_ings
Saucisse purée & épinards,plat froid,"[['200 g', 'pomme de terre'], ['100 g', 'saucisse (fumée)'], ['80 g', 'epinards (surgelés)'], ['1 càs', 'crème fraîche épaisse']]",saucisse purée & épinards,"pomme de terre, saucisse, epinards, crème fraîche épaisse"
Saucisse & purée maison,plat froid,"[['250 g', 'pomme de terre'], ['5 cl', 'lait'], ['20 g', 'beurre'], ['1', 'saucisse (fumée)'], ['1/2 càc']]",saucisse & purée maison,"pomme de terre, lait, beurre, saucisse"
Boulettes & purée de courge,plat froid,"[['150 g', 'chair à saucisse'], ['1/4', 'échalote'], ['1/4 bou.', 'persil (frais)'], ['1/4', 'potimarron'], ['1/4'], ['1 càs']]",boulettes & purée de courge,"chair à saucisse, échalote, persil, potimarron"
Saucisse & purée de potimarron,20 minutes,"[['1', 'saucisse de toulouse'], ['1/4', 'potimarron'], ['1/2', 'oignon jaune'], ['1 càs', 'crème fraîche épaisse'], ['1 noisette', 'beurre']]",saucisse & purée de potimarron,"saucisse de toulouse, potimarron, oignon jaune, crème fraîche épaisse, beurre"


In [575]:
jow_data[(jow_data["only_ings"].str.contains("saucisse")) & (jow_data["only_ings"].str.contains("patate"))]

,cuisson,ingredients,recettes,only_ings


In [576]:
jow_data[(jow_data["only_ings"].str.contains("knack")) & (jow_data["only_ings"].str.contains("pomme de terre"))]

,cuisson,ingredients,recettes,only_ings
Knack & purée,plat froid,"[['250 g', 'pomme de terre'], ['2', 'knack (porc)'], ['5 cl', 'lait'], ['20 g', 'beurre']]",knack & purée,"pomme de terre, knack, lait, beurre"


In [577]:
tmp = []
tmp.append(jow_data[jow_data["recettes"]=="saucisse & purée maison"].index[0])
tmp.append(jow_data[jow_data["recettes"]=='saucisse purée & épinards'].index[0])
tmp.append(jow_data[jow_data["recettes"]=='pommes de terre & saucisses rôties'].index[0])
tmp.append(jow_data[jow_data["recettes"]=='knack & purée'].index[0])

menu_data.loc[125, 'Closest JOW recipe by hand'] = tmp

In [578]:
seven_data2[(seven_data2["title_raw"].str.contains("sausag")) & (seven_data2["ingredient_raw"].str.contains("potato"))]

,title_raw,ingredient_raw
6739,sweet potatoes with pecans & sausage,"sweet potato, raw, unprepared, sugars, brown, nuts, pecans, pork, fresh, loin, tenderloin, separable lean only, raw, spices, cinnamon, ground, butter, without salt"
6740,sweet potatoes with pecans and sausage,"sweet potato, raw, unprepared, sugars, brown, nuts, pecans, pork, fresh, loin, tenderloin, separable lean only, raw, spices, cinnamon, ground, butter, without salt"


In [579]:
seven_data2[(seven_data2["title_raw"].str.contains("knack")) & (seven_data2["ingredient_raw"].str.contains("potato"))]

,title_raw,ingredient_raw


In [580]:
menu_data.loc[125, 'Closest 7t recipe by hand'] = ['sweet potatoes with pecans & sausage', \
                                                  'sweet potatoes with pecans and sausage']

Next recipe

In [581]:
menu_data[menu_data.index==123]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand,Closest 7t recipe by hand
123,IMG_7934.PNG,Plats,Hot-Duck,"Pain brioché, confit de canard, ketchup de pomme, coleslaw, Tomme des Aravis, frites",Not yet handled,Not yet handled


In [582]:
jow_data[(jow_data["only_ings"].str.contains("pain")) & (jow_data["only_ings"].str.contains("canard"))]

,cuisson,ingredients,recettes,only_ings
Salade périgourdine,4 minutes,"[['2 poignées', 'salade (mélange)'], ['40 g', 'abricots (secs)'], ['50 g', 'canard (magret séché)'], ['15 g', 'noix'], ['1 tran.', 'pain de campagne (tranché)'], ['1/4 gou.', 'ail']]",salade périgourdine,"salade, abricots, canard, noix, pain de campagne, ail"


In [583]:
jow_data[(jow_data["recettes"].str.contains("sandwich")) & (jow_data["only_ings"].str.contains("canard"))]

,cuisson,ingredients,recettes,only_ings


In [584]:
jow_data[jow_data["recettes"].str.contains("canard")]

,cuisson,ingredients,recettes,only_ings
Parmentier de canard,50 minutes,"[['1/2', 'canard (confit)'], ['10 g', 'noisette'], ['1/2', 'échalote'], ['200 g', 'pomme de terre'], ['5 cl', 'lait'], ['1 noisette', 'beurre']]",parmentier de canard,"canard, noisette, échalote, pomme de terre, lait, beurre"
Magret de canard à l'orange,19 minutes,"[['1/2', 'canard (magret)'], ['1/2', 'orange'], ['1/2 càs', 'miel (liquide)'], ['1 poignée', 'salade (mélange)']]",magret de canard à l'orange,"canard, orange, miel, salade"
Magret de canard & purée,20 minutes,"[['1/2', 'canard (magret)'], ['1 càs', 'vinaigre de vin rouge'], ['1/4', 'échalote'], ['250 g', 'pomme de terre'], ['5 cl', 'lait'], ['20 g', 'beurre']]",magret de canard & purée,"canard, vinaigre de vin rouge, échalote, pomme de terre, lait, beurre"
Parmentier de canard aux patates douces,plat froid,"[['1', 'canard (confit)'], ['200 g', 'patate douce'], ['1/2', 'échalote'], ['3 cl', 'lait']]",parmentier de canard aux patates douces,"canard, patate douce, échalote, lait"
Canard & ravioles forestières,15 minutes,"[['1/2', 'canard (magret)'], ['5 ml', 'vin blanc'], ['10 g', 'parmesan (morceaux)'], ['125 g', 'ravioli (aux champignons)']]",canard & ravioles forestières,"canard, vin blanc, parmesan, ravioli"
Poêlée de chou-fleur & canard,10 minutes,"[['1/2', 'canard (magret)'], ['200 g', 'chou-fleur (frais)'], ['1/2', 'oignon rouge'], ['1 càs', 'vinaigre de cidre'], ['1/4 càc', 'curcuma (poudre)']]",poêlée de chou-fleur & canard,"canard, chou-fleur, oignon rouge, vinaigre de cidre, curcuma"
Canard à l'orange & patate douce,21 minutes,"[['1/2', 'canard (magret)'], ['150 g', 'patate douce'], ['1/2', 'orange'], ['1/2 càs', 'miel (liquide)']]",canard à l'orange & patate douce,"canard, patate douce, orange, miel"
Magret de canard & endives braisées,19 minutes,"[['1/2', 'canard (magret)'], ['1/2 càs', 'miel (liquide)'], ['1', 'endives']]",magret de canard & endives braisées,"canard, miel, endives"


In [585]:
jow_data[jow_data["only_ings"].str.contains("canard")]

,cuisson,ingredients,recettes,only_ings
Parmentier de canard,50 minutes,"[['1/2', 'canard (confit)'], ['10 g', 'noisette'], ['1/2', 'échalote'], ['200 g', 'pomme de terre'], ['5 cl', 'lait'], ['1 noisette', 'beurre']]",parmentier de canard,"canard, noisette, échalote, pomme de terre, lait, beurre"
Magret de canard à l'orange,19 minutes,"[['1/2', 'canard (magret)'], ['1/2', 'orange'], ['1/2 càs', 'miel (liquide)'], ['1 poignée', 'salade (mélange)']]",magret de canard à l'orange,"canard, orange, miel, salade"
Magret de canard & purée,20 minutes,"[['1/2', 'canard (magret)'], ['1 càs', 'vinaigre de vin rouge'], ['1/4', 'échalote'], ['250 g', 'pomme de terre'], ['5 cl', 'lait'], ['20 g', 'beurre']]",magret de canard & purée,"canard, vinaigre de vin rouge, échalote, pomme de terre, lait, beurre"
Parmentier de canard aux patates douces,plat froid,"[['1', 'canard (confit)'], ['200 g', 'patate douce'], ['1/2', 'échalote'], ['3 cl', 'lait']]",parmentier de canard aux patates douces,"canard, patate douce, échalote, lait"
Pasta al ragu d'anatra,plat froid,"[['100 g', 'pâtes (pappardelle)'], ['1/2', 'canard (confit)'], ['70 g', 'tomate pelée'], ['1/2', 'échalote']]",pasta al ragu d'anatra,"pâtes, canard, tomate pelée, échalote"
Canard & ravioles forestières,15 minutes,"[['1/2', 'canard (magret)'], ['5 ml', 'vin blanc'], ['10 g', 'parmesan (morceaux)'], ['125 g', 'ravioli (aux champignons)']]",canard & ravioles forestières,"canard, vin blanc, parmesan, ravioli"
"Magret, abricot, miel & romarin",25 minutes,"[['1/2', 'canard (magret)'], ['50 g', 'abricot (frais)'], ['1 càc', 'miel (liquide)'], ['1 pinc.', 'romarin (séché)'], ['2 poignées', 'salade (mélange)']]","magret, abricot, miel & romarin","canard, abricot, miel, romarin, salade"
Poêlée de chou-fleur & canard,10 minutes,"[['1/2', 'canard (magret)'], ['200 g', 'chou-fleur (frais)'], ['1/2', 'oignon rouge'], ['1 càs', 'vinaigre de cidre'], ['1/4 càc', 'curcuma (poudre)']]",poêlée de chou-fleur & canard,"canard, chou-fleur, oignon rouge, vinaigre de cidre, curcuma"
Canard à l'orange & patate douce,21 minutes,"[['1/2', 'canard (magret)'], ['150 g', 'patate douce'], ['1/2', 'orange'], ['1/2 càs', 'miel (liquide)']]",canard à l'orange & patate douce,"canard, patate douce, orange, miel"
Salade périgourdine,4 minutes,"[['2 poignées', 'salade (mélange)'], ['40 g', 'abricots (secs)'], ['50 g', 'canard (magret séché)'], ['15 g', 'noix'], ['1 tran.', 'pain de campagne (tranché)'], ['1/4 gou.', 'ail']]",salade périgourdine,"salade, abricots, canard, noix, pain de campagne, ail"


In [586]:
menu_data.loc[123, 'Closest JOW recipe by hand'] = 'no match'

In [587]:
seven_data2[(seven_data2["title_raw"].str.contains("burger")) & (seven_data2["ingredient_raw"].str.contains("duck"))]

,title_raw,ingredient_raw


In [588]:
seven_data2[(seven_data2["ingredient_raw"].str.contains("bread")) & (seven_data2["ingredient_raw"].str.contains("duck"))]

,title_raw,ingredient_raw


In [589]:
seven_data2[(seven_data2["title_raw"].str.contains("duck"))]

,title_raw,ingredient_raw
2475,duck sauce,"soy sauce made from soy (tamari), apricots, dried, sulfured, uncooked, vinegar, cider"
7032,turducken seasoning,"spices, onion powder, spices, garlic powder, spices, pepper, red or cayenne, spices, thyme, dried, spices, basil, dried, spices, pepper, black, spices, paprika, spices, oregano, dried"


In [590]:
menu_data.loc[123, 'Closest 7t recipe by hand'] = 'no match'

Next recipe

In [591]:
menu_data[menu_data.index==116]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand,Closest 7t recipe by hand
116,IMG_7930.PNG,Les plats,"Canard, purée de cresson, pommes boulangères, trompettes de la mort & jus de volaille à l'argousier",NaN,Not yet handled,Not yet handled


In [592]:
jow_data[(jow_data["only_ings"].str.contains("pomme")) & (jow_data["only_ings"].str.contains("canard"))]

,cuisson,ingredients,recettes,only_ings
Parmentier de canard,50 minutes,"[['1/2', 'canard (confit)'], ['10 g', 'noisette'], ['1/2', 'échalote'], ['200 g', 'pomme de terre'], ['5 cl', 'lait'], ['1 noisette', 'beurre']]",parmentier de canard,"canard, noisette, échalote, pomme de terre, lait, beurre"
Magret de canard & purée,20 minutes,"[['1/2', 'canard (magret)'], ['1 càs', 'vinaigre de vin rouge'], ['1/4', 'échalote'], ['250 g', 'pomme de terre'], ['5 cl', 'lait'], ['20 g', 'beurre']]",magret de canard & purée,"canard, vinaigre de vin rouge, échalote, pomme de terre, lait, beurre"


In [593]:
tmp = []
tmp.append(jow_data[jow_data["recettes"]=="parmentier de canard"].index[0])
tmp.append(jow_data[jow_data["recettes"]=='magret de canard & purée'].index[0])

menu_data.loc[116, 'Closest JOW recipe by hand'] = tmp

In [594]:
seven_data2[(seven_data2["ingredient_raw"].str.contains("duck"))]

,title_raw,ingredient_raw


In [595]:
seven_data2[(seven_data2["title_raw"].str.contains("duck"))]

,title_raw,ingredient_raw
2475,duck sauce,"soy sauce made from soy (tamari), apricots, dried, sulfured, uncooked, vinegar, cider"
7032,turducken seasoning,"spices, onion powder, spices, garlic powder, spices, pepper, red or cayenne, spices, thyme, dried, spices, basil, dried, spices, pepper, black, spices, paprika, spices, oregano, dried"


In [596]:
menu_data.loc[116, 'Closest 7t recipe by hand'] = 'no match'

In [597]:
menu_data_7tjow = menu_data[(menu_data['Closest JOW recipe by hand']!='Not yet handled') & (menu_data['Closest 7t recipe by hand']!='Not yet handled')]
print('Number of handled recipes = ', menu_data_7tjow.shape[0])

menu_data_7tjow.to_json(data_path+"data/Tools/MenuTo7tJOWRecipes_v1.json")
menu_data_7tjow.to_excel(data_path+"data/Tools/MenuTo7tJOWRecipes_v1.xlsx")

Number of handled recipes =  60
